In [4]:
import sys
print(sys.executable)

c:\Users\ICSA\eleni\ORCID\orcid-venv\Scripts\python.exe


In [5]:
!pip install pymysql pandas matplotlib sqlalchemy scikit-learn
import pymysql
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy

import pymysql
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy

In [3]:
import sys
print(sys.executable)


c:\Users\ICSA\eleni\ORCID\orcid-venv\Scripts\python.exe


In [6]:
import pandas as pd

# Load merged ORCID dataset
df = pd.read_csv(r"C:\Users\ICSA\eleni\ORCID\pythonProject11\data\merged_affil_journals.csv")

# Concatenate all journal titles per researcher
grouped = df.groupby("OID")["Jtitle"].apply(
    lambda titles: " ".join(str(t).lower() for t in titles if pd.notna(t))
).reset_index()

# Add cleaned department info per researcher
grouped = grouped.merge(df[["OID", "clean_dept"]].drop_duplicates(), on="OID", how="left")


Clean grouped data

In [7]:
# Remove entries with empty journal titles or missing department
grouped["Jtitle"] = grouped["Jtitle"].str.strip()
grouped = grouped[grouped["Jtitle"].str.len() > 0]  # remove empty title strings
grouped = grouped.dropna(subset=["clean_dept"])     # drop rows with missing department

# Optional: reset index
grouped = grouped.reset_index(drop=True)

# Show basic stats
print("Remaining researchers:", len(grouped))
grouped.head()


Remaining researchers: 415315


,OID,Jtitle,clean_dept
0,0000-0001-5000-0162,apha 2018 annual meeting,biology
1,0000-0001-5000-0218,open science framework proceedings of the huma...,psychology
2,0000-0001-5000-0584,international review of social psychology soci...,psychology
3,0000-0001-5000-0648,revista inter ação women and birth revista est...,psychology
4,0000-0001-5000-0648,revista inter ação women and birth revista est...,education


In [8]:
print("Original rows:", len(df["OID"].unique()))
print("After cleaning:", len(grouped))


Original rows: 398025
After cleaning: 415315


In [ ]:
print(df["locale"].value_counts())

locale
en       14135986
pt         613137
es         597933
it         194913
fr         155856
zh_CN      136928
ja          97257
ru          94066
ko          70044
zh_TW       56815
cs           5442
ar           4485
Name: count, dtype: int64


PAPERS CLASSIFICATION 

In [10]:
from collections import Counter
import pandas as pd
import re

from deep_translator import GoogleTranslator
import json
import time

# Sample domain keyword dictionary expanded with more fields and terms
domain_keywords = {
    'medicine': [
        # General medical terms
        'medicine', 'medical', 'clinical', 'patient', 'patients', 'health', 'disease', 'diseases', 
        'disorder', 'disorders', 'illness', 'condition', 'conditions', 'symptom', 'symptoms', 
        'treatment', 'therapy', 'therapies', 'diagnosis', 'diagnoses', 'pathology', 'care',

        # Common single medical terms and fields
        'cardio', 'cardiology', 'heart', 'vascular', 'cardiovascular', 'neuro', 'neurology', 
        'brain', 'cognitive', 'psychiatry', 'mental', 'psychological', 'psych', 'psychosis',
        'oncology', 'cancer', 'tumor', 'tumors', 'neoplasm', 'malignant', 'benign',
        'stroke', 'diabetes', 'glucose', 'obesity', 'hypertension', 'pressure', 'cholesterol',
        'infection', 'infections', 'pathogen', 'virus', 'viral', 'bacteria', 'bacterial', 
        'fungal', 'parasite', 'sepsis', 'inflammation', 'fever', 'fatigue',

        # Medical specialties
        'pediatrics', 'paediatrics', 'children', 'child', 'obstetrics', 'gynecology', 'gyn', 
        'fertility', 'birth', 'reproductive', 'anesthesiology', 'anesthesia', 'orthopedics', 
        'bones', 'spine', 'joint', 'fracture', 'surgery', 'surgical', 'dermatology', 'skin',
        'endocrinology', 'hormone', 'thyroid', 'insulin', 'hematology', 'blood', 'hemoglobin',
        'urology', 'renal', 'kidney', 'dialysis', 'nephrology', 'liver', 'hepatitis', 'hepatology',
        'gastroenterology', 'intestine', 'colon', 'digestive', 'abdomen', 'abdomenal',

        # Diagnostics and procedures
        'scan', 'mri', 'ct', 'xray', 'x-ray', 'ultrasound', 'ecg', 'eeg', 'monitoring', 'test',
        'screening', 'biopsy', 'lab', 'labs', 'radiology', 'diagnostic', 'histopathology',

        # Pharmacology and drugs
        'drug', 'drugs', 'medication', 'medications', 'vaccine', 'vaccines', 'immunization',
        'chemotherapy', 'chemo', 'dose', 'dosing', 'pill', 'tablet', 'antibiotic', 'antiviral',
        'opioid', 'prescription', 'analgesic', 'painkiller', 'side effect', 'interaction',

        # Public health and research
        'epidemiology', 'biomedical', 'biomed', 'meta-analysis', 'systematic', 'case', 'cohort',
        'randomized', 'clinical trial', 'trials', 'mortality', 'morbidity', 'risk', 'exposure',
        'prevalence', 'incidence', 'pandemic', 'covid', 'sars', 'corona', 'lockdown',

        # Institutions and professionals
        'hospital', 'clinic', 'nurse', 'doctor', 'physician', 'surgeon', 'staff', 'resident',
        'caregiver', 'paramedic', 'ambulance', 'emergency', 'icu', 'ward', 'unit', 'hospice',

        # Mental health
        'depression', 'anxiety', 'bipolar', 'adhd', 'autism', 'alzheimer', 'dementia',
        'suicide', 'stress', 'trauma', 'addiction', 'substance', 'abuse', 'rehabilitation',

        # Other common conditions
        'asthma', 'allergy', 'arthritis', 'hiv', 'aids', 'std', 'infertility', 'migraine',
        'parkinson', 'ms', 'multiple sclerosis', 'eczema', 'psoriasis', 'sleep', 'fatigue',

        # Medical tech / bioscience
        'biomarker', 'genome', 'gene', 'genetic', 'mutation', 'dna', 'rna', 'sequencing',
        'omics', 'proteomics', 'bioinformatics', 'metabolism', 'cell', 'organ', 'tissue',

        # Lifestyle & prevention
        'nutrition', 'diet', 'exercise', 'fitness', 'smoking', 'alcohol', 'screening',
        'lifestyle', 'prevention', 'wellness', 'well-being', 'public health', 'vaccination',


        # Términos médicos generales
    'medicina', 'médico', 'clínico', 'paciente', 'pacientes', 'salud', 'enfermedad', 'enfermedades',
    'trastorno', 'trastornos', 'dolencia', 'condición', 'condiciones', 'síntoma', 'síntomas',
    'tratamiento', 'terapia', 'terapias', 'diagnóstico', 'diagnósticos', 'patología', 'cuidado',

    # Términos y campos médicos comunes
    'cardio', 'cardiología', 'corazón', 'vascular', 'cardiovascular', 'neuro', 'neurología',
    'cerebro', 'cognitivo', 'psiquiatría', 'mental', 'psicológico', 'psico', 'psicosis',
    'oncología', 'cáncer', 'tumor', 'tumores', 'neoplasia', 'maligno', 'benigno',
    'derrame cerebral', 'diabetes', 'glucosa', 'obesidad', 'hipertensión', 'presión', 'colesterol',
    'infección', 'infecciones', 'patógeno', 'virus', 'viral', 'bacteria', 'bacteriano',
    'fúngico', 'parásito', 'sepsis', 'inflamación', 'fiebre', 'fatiga',

    # Especialidades médicas
    'pediatría', 'pediatría', 'niños', 'niño', 'obstetricia', 'ginecología', 'gine',
    'fertilidad', 'nacimiento', 'reproductivo', 'anestesiología', 'anestesia', 'ortopedia',
    'huesos', 'columna', 'articulación', 'fractura', 'cirugía', 'quirúrgico', 'dermatología', 'piel',
    'endocrinología', 'hormona', 'tiroides', 'insulina', 'hematología', 'sangre', 'hemoglobina',
    'urología', 'renal', 'riñón', 'diálisis', 'nefrología', 'hígado', 'hepatitis', 'hepatología',
    'gastroenterología', 'intestino', 'colon', 'digestivo', 'abdomen', 'abdominal',

    # Diagnóstico y procedimientos
    'escaneo', 'resonancia', 'tomografía', 'radiografía', 'rayos x', 'ultrasonido', 'ecg', 'eeg', 'monitoreo', 'prueba',
    'cribado', 'biopsia', 'laboratorio', 'laboratorios', 'radiología', 'diagnóstico', 'histopatología',

    # Farmacología y medicamentos
    'fármaco', 'fármacos', 'medicación', 'medicaciones', 'vacuna', 'vacunas', 'inmunización',
    'quimioterapia', 'quimio', 'dosis', 'dosificación', 'píldora', 'tableta', 'antibiótico', 'antiviral',
    'opioide', 'prescripción', 'analgésico', 'calmante', 'efecto secundario', 'interacción',

    # Salud pública e investigación
    'epidemiología', 'biomédico', 'biomedicina', 'meta-análisis', 'sistemático', 'caso', 'cohorte',
    'aleatorizado', 'ensayo clínico', 'ensayos', 'mortalidad', 'morbilidad', 'riesgo', 'exposición',
    'prevalencia', 'incidencia', 'pandemia', 'covid', 'sars', 'corona', 'confinamiento',

    # Instituciones y profesionales
    'hospital', 'clínica', 'enfermera', 'médico', 'facultativo', 'cirujano', 'personal', 'residente',
    'cuidador', 'paramédico', 'ambulancia', 'emergencia', 'uci', 'sala', 'unidad', 'hospicio',

    # Salud mental
    'depresión', 'ansiedad', 'bipolar', 'tdah', 'autismo', 'alzheimer', 'demencia',
    'suicidio', 'estrés', 'trauma', 'adicción', 'sustancia', 'abuso', 'rehabilitación',

    # Otras condiciones comunes
    'asma', 'alergia', 'artritis', 'vih', 'sida', 'ets', 'infertilidad', 'migraña',
    'parkinson', 'esclerosis múltiple', 'esclerosis múltiple', 'eccema', 'psoriasis', 'sueño', 'fatiga',

    # Tecnología médica / biociencia
    'biomarcador', 'genoma', 'gen', 'genético', 'mutación', 'adn', 'arn', 'secuenciación',
    'ómicas', 'proteómica', 'bioinformática', 'metabolismo', 'célula', 'órgano', 'tejido',

    # Estilo de vida y prevención
    'nutrición', 'dieta', 'ejercicio', 'fitness', 'tabaquismo', 'alcohol', 'cribado',
    'estilo de vida', 'prevención', 'bienestar', 'bienestar', 'salud pública', 'vacunación',

    # Termos médicos gerais
    'medicina', 'médico', 'clínico', 'paciente', 'pacientes', 'saúde', 'doença', 'doenças',
    'transtorno', 'transtornos', 'enfermidade', 'condição', 'condições', 'sintoma', 'sintomas',
    'tratamento', 'terapia', 'terapias', 'diagnóstico', 'diagnósticos', 'patologia', 'cuidado',

    # Termos e áreas médicas comuns
    'cardio', 'cardiologia', 'coração', 'vascular', 'cardiovascular', 'neuro', 'neurologia',
    'cérebro', 'cognitivo', 'psiquiatria', 'mental', 'psicológico', 'psic', 'psicose',
    'oncologia', 'câncer', 'tumor', 'tumores', 'neoplasia', 'maligno', 'benigno',
    'acidente vascular cerebral', 'diabetes', 'glicose', 'obesidade', 'hipertensão', 'pressão', 'colesterol',
    'infecção', 'infecções', 'patógeno', 'vírus', 'viral', 'bactéria', 'bacteriano',
    'fúngico', 'parasita', 'sepse', 'inflamação', 'febre', 'fadiga',

    # Especialidades médicas
    'pediatria', 'pediatria', 'crianças', 'criança', 'obstetrícia', 'ginecologia', 'gineco',
    'fertilidade', 'nascimento', 'reprodutivo', 'anestesiologia', 'anestesia', 'ortopedia',
    'ossos', 'coluna', 'articulação', 'fratura', 'cirurgia', 'cirúrgico', 'dermatologia', 'pele',
    'endocrinologia', 'hormônio', 'tireoide', 'insulina', 'hematologia', 'sangue', 'hemoglobina',
    'urologia', 'renal', 'rim', 'diálise', 'nefrologia', 'fígado', 'hepatite', 'hepatologia',
    'gastroenterologia', 'intestino', 'cólon', 'digestivo', 'abdômen', 'abdominal',

    # Diagnósticos e procedimentos
    'escaneamento', 'ressonância', 'tomografia', 'raio-x', 'raio x', 'ultrassom', 'ecg', 'eeg', 'monitoramento', 'teste',
    'triagem', 'biópsia', 'laboratório', 'laboratórios', 'radiologia', 'diagnóstico', 'histopatologia',

    # Farmacologia e medicamentos
    'droga', 'drogas', 'medicação', 'medicações', 'vacina', 'vacinas', 'imunização',
    'quimioterapia', 'quimio', 'dose', 'dosagem', 'comprimido', 'tablete', 'antibiótico', 'antiviral',
    'opioide', 'prescrição', 'analgésico', 'remédio para dor', 'efeito colateral', 'interação',

    # Saúde pública e pesquisa
    'epidemiologia', 'biomédico', 'biomedicina', 'meta-análise', 'sistemático', 'caso', 'coorte',
    'randomizado', 'ensaio clínico', 'ensaios', 'mortalidade', 'morbidade', 'risco', 'exposição',
    'prevalência', 'incidência', 'pandemia', 'covid', 'sars', 'corona', 'confinamento',

    # Instituições e profissionais
    'hospital', 'clínica', 'enfermeiro', 'médico', 'fisiatra', 'cirurgião', 'equipe', 'residente',
    'cuidador', 'paramédico', 'ambulância', 'emergência', 'uti', 'ala', 'unidade', 'hospice',

    # Saúde mental
    'depressão', 'ansiedade', 'bipolar', 'tdah', 'autismo', 'alzheimer', 'demência',
    'suicídio', 'estresse', 'trauma', 'dependência', 'substância', 'abuso', 'reabilitação',

    # Outras condições comuns
    'asma', 'alergia', 'artrite', 'hiv', 'aids', 'dst', 'infertilidade', 'enxaqueca',
    'parkinson', 'esclerose múltipla', 'esclerose múltipla', 'eczema', 'psoríase', 'sono', 'fadiga',

    # Tecnologia médica / biociência
    'biomarcador', 'genoma', 'gene', 'genético', 'mutação', 'dna', 'rna', 'sequenciamento',
    'ômicas', 'proteômica', 'bioinformática', 'metabolismo', 'célula', 'órgão', 'tecido',

    # Estilo de vida e prevenção
    'nutrição', 'dieta', 'exercício', 'fitness', 'tabagismo', 'álcool', 'triagem',
    'estilo de vida', 'prevenção', 'bem-estar', 'bem-estar', 'saúde pública', 'vacinação',


    # Termini medici generali
    'medicina', 'medico', 'clinico', 'paziente', 'pazienti', 'salute', 'malattia', 'malattie',
    'disturbo', 'disturbi', 'infermità', 'condizione', 'condizioni', 'sintomo', 'sintomi',
    'trattamento', 'terapia', 'terapie', 'diagnosi', 'diagnosi', 'patologia', 'cura',

    # Termini medici e aree comuni
    'cardio', 'cardiologia', 'cuore', 'vascolare', 'cardiovascolare', 'neuro', 'neurologia',
    'cervello', 'cognitivo', 'psichiatria', 'mentale', 'psicologico', 'psic', 'psicosi',
    'oncologia', 'cancro', 'tumore', 'tumori', 'neoplasia', 'maligno', 'benigno',
    'ictus', 'diabete', 'glucosio', 'obesità', 'ipertensione', 'pressione', 'colesterolo',
    'infezione', 'infezioni', 'patogeno', 'virus', 'virale', 'batteri', 'batterico',
    'fungino', 'parassita', 'sepsi', 'infiammazione', 'febbre', 'affaticamento',

    # Specialità mediche
    'pediatria', 'pediatria', 'bambini', 'bambino', 'ostetricia', 'ginecologia', 'gine',
    'fertilità', 'nascita', 'riproduttivo', 'anestesiologia', 'anestesia', 'ortopedia',
    'ossa', 'colonna', 'articolazione', 'frattura', 'chirurgia', 'chirurgico', 'dermatologia', 'pelle',
    'endocrinologia', 'ormone', 'tiroide', 'insulina', 'ematologia', 'sangue', 'emoglobina',
    'urologia', 'renale', 'rene', 'dialisi', 'nefrologia', 'fegato', 'epatite', 'epatologia',
    'gastroenterologia', 'intestino', 'colon', 'digestivo', 'addome', 'addominale',

    # Diagnostica e procedure
    'scansione', 'risonanza', 'tc', 'raggi x', 'radiografia', 'ecografia', 'ecg', 'eeg', 'monitoraggio', 'test',
    'screening', 'biopsia', 'laboratorio', 'laboratori', 'radiologia', 'diagnostico', 'istopatologia',

    # Farmacologia e farmaci
    'farmaco', 'farmaci', 'medicinale', 'medicinali', 'vaccino', 'vaccini', 'immunizzazione',
    'chemioterapia', 'chemio', 'dose', 'dosaggio', 'pillola', 'compressa', 'antibiotico', 'antivirale',
    'oppioide', 'prescrizione', 'analgesico', 'antidolorifico', 'effetto collaterale', 'interazione',

    # Salute pubblica e ricerca
    'epidemiologia', 'biomedico', 'biomedicina', 'meta-analisi', 'sistematico', 'caso', 'coorte',
    'randomizzato', 'studio clinico', 'studi', 'mortalità', 'morbidità', 'rischio', 'esposizione',
    'prevalenza', 'incidenza', 'pandemia', 'covid', 'sars', 'coronavirus', 'confinamento',

    # Istituzioni e professionisti
    'ospedale', 'clinica', 'infermiere', 'medico', 'fisiatra', 'chirurgo', 'personale', 'residente',
    'assistente', 'paramedico', 'ambulanza', 'emergenza', 'rianimazione', 'reparto', 'unità', 'hospice',

    # Salute mentale
    'depressione', 'ansia', 'bipolare', 'adhd', 'autismo', 'alzheimer', 'demenza',
    'suicidio', 'stress', 'trauma', 'dipendenza', 'sostanza', 'abuso', 'riabilitazione',

    # Altre condizioni comuni
    'asma', 'allergia', 'artrite', 'hiv', 'aids', 'std', 'infertilità', 'emicrania',
    'parkinson', 'sclerosi multipla', 'sclerosi multipla', 'eczema', 'psoriasi', 'sonno', 'affaticamento',

    # Tecnologia medica / bioscienza
    'biomarcatore', 'genoma', 'gene', 'genetico', 'mutazione', 'dna', 'rna', 'sequenziamento',
    'omiche', 'proteomica', 'bioinformatica', 'metabolismo', 'cellula', 'organo', 'tessuto',

    # Stile di vita e prevenzione
    'nutrizione', 'dieta', 'esercizio', 'fitness', 'fumo', 'alcol', 'screening',
    'stile di vita', 'prevenzione', 'benessere', 'benessere', 'salute pubblica', 'vaccinazione',


    # Termes médicaux généraux
    'médecine', 'médical', 'clinique', 'patient', 'patients', 'santé', 'maladie', 'maladies',
    'trouble', 'troubles', 'mal', 'condition', 'conditions', 'symptôme', 'symptômes',
    'traitement', 'thérapie', 'thérapies', 'diagnostic', 'diagnostics', 'pathologie', 'soin',

    # Termes médicaux simples et domaines
    'cardio', 'cardiologie', 'cœur', 'vasculaire', 'cardiovasculaire', 'neuro', 'neurologie',
    'cerveau', 'cognitif', 'psychiatrie', 'mental', 'psychologique', 'psy', 'psychose',
    'oncologie', 'cancer', 'tumeur', 'tumeurs', 'néoplasme', 'malin', 'bénin',
    'avc', 'diabète', 'glucose', 'obésité', 'hypertension', 'pression', 'cholestérol',
    'infection', 'infections', 'agent pathogène', 'virus', 'viral', 'bactérie', 'bactérien',
    'fongique', 'parasite', 'septicémie', 'inflammation', 'fièvre', 'fatigue',

    # Spécialités médicales
    'pédiatrie', 'pédiatrie', 'enfants', 'enfant', 'obstétrique', 'gynécologie', 'gynéco',
    'fertilité', 'naissance', 'reproductif', 'anesthésiologie', 'anesthésie', 'orthopédie',
    'os', 'colonne vertébrale', 'articulation', 'fracture', 'chirurgie', 'chirurgical', 'dermatologie', 'peau',
    'endocrinologie', 'hormone', 'thyroïde', 'insuline', 'hématologie', 'sang', 'hémoglobine',
    'urologie', 'rénal', 'rein', 'dialyse', 'néphrologie', 'foie', 'hépatite', 'hépatologie',
    'gastroentérologie', 'intestin', 'côlon', 'digestif', 'abdomen', 'abdominal',

    # Diagnostic et procédures
    'scanner', 'irm', 'tdm', 'radiographie', 'rayon x', 'échographie', 'ecg', 'eeg', 'surveillance', 'test',
    'dépistage', 'biopsie', 'labo', 'labos', 'radiologie', 'diagnostique', 'histopathologie',

    # Pharmacologie et médicaments
    'médicament', 'médicaments', 'médication', 'médications', 'vaccin', 'vaccins', 'immunisation',
    'chimiothérapie', 'chimio', 'dose', 'dosage', 'pilule', 'comprimé', 'antibiotique', 'antiviral',
    'opiacé', 'ordonnance', 'analgésique', 'antidouleur', 'effet secondaire', 'interaction',

    # Santé publique et recherche
    'épidémiologie', 'biomédical', 'biomédecine', 'méta-analyse', 'systématique', 'cas', 'cohorte',
    'randomisé', 'essai clinique', 'essais', 'mortalité', 'morbidité', 'risque', 'exposition',
    'prévalence', 'incidence', 'pandémie', 'covid', 'sras', 'corona', 'confinement',

    # Institutions et professionnels
    'hôpital', 'clinique', 'infirmier', 'médecin', 'docteur', 'chirurgien', 'personnel', 'interne',
    'soignant', 'paramédic', 'ambulance', 'urgence', 'soins intensifs', 'service', 'unité', 'hospice',

    # Santé mentale
    'dépression', 'anxiété', 'bipolaire', 'tdah', 'autisme', 'alzheimer', 'démence',
    'suicide', 'stress', 'traumatisme', 'addiction', 'substance', 'abus', 'réhabilitation',

    # Autres pathologies courantes
    'asthme', 'allergie', 'arthrite', 'vih', 'sida', 'mst', 'infertilité', 'migraine',
    'parkinson', 'sep', 'sclérose en plaques', 'eczéma', 'psoriasis', 'sommeil', 'fatigue',

    # Technologie médicale / biosciences
    'biomarqueur', 'génome', 'gène', 'génétique', 'mutation', 'adn', 'arn', 'séquençage',
    'omiques', 'protéomique', 'bioinformatique', 'métabolisme', 'cellule', 'organe', 'tissu',

    # Mode de vie et prévention
    'nutrition', 'régime', 'exercice', 'forme physique', 'tabagisme', 'alcool', 'dépistage',
    'mode de vie', 'prévention', 'bien-être', 'santé mentale', 'santé publique', 'vaccination',

    # 一般医学术语
    '医学', '医疗', '临床', '病人', '病人们', '健康', '疾病', '疾病',
    '障碍', '障碍', '病症', '状况', '状况', '症状', '症状',
    '治疗', '疗法', '疗法', '诊断', '诊断', '病理学', '护理',

    # 常见医学术语和领域
    '心脏', '心脏病学', '心脏', '血管', '心血管', '神经', '神经病学',
    '大脑', '认知', '精神病学', '心理', '心理学', '心理', '精神病',
    '肿瘤学', '癌症', '肿瘤', '肿瘤', '肿瘤', '恶性', '良性',
    '中风', '糖尿病', '葡萄糖', '肥胖', '高血压', '压力', '胆固醇',
    '感染', '感染', '病原体', '病毒', '病毒性', '细菌', '细菌性',
    '真菌性', '寄生虫', '败血症', '炎症', '发烧', '疲劳',

    # 医学专科
    '儿科学', '儿科学', '儿童', '小孩', '产科学', '妇科', '妇科',
    '生育', '出生', '生殖', '麻醉学', '麻醉', '骨科',
    '骨头', '脊柱', '关节', '骨折', '外科', '外科手术', '皮肤科', '皮肤',
    '内分泌学', '激素', '甲状腺', '胰岛素', '血液学', '血液', '血红蛋白',
    '泌尿科', '肾脏', '肾', '透析', '肾病学', '肝脏', '肝炎', '肝病学',
    '胃肠病学', '肠道', '结肠', '消化', '腹部', '腹部的',

    # 诊断与程序
    '扫描', '核磁共振', 'CT', 'X光', 'X-光', '超声波', '心电图', '脑电图', '监测', '测试',
    '筛查', '活检', '实验室', '实验室', '放射科', '诊断', '组织病理学',

    # 药理学和药物
    '药物', '药物', '药品', '药物治疗', '疫苗', '疫苗', '免疫接种',
    '化疗', '化疗', '剂量', '剂量调整', '药丸', '药片', '抗生素', '抗病毒药',
    '阿片类药物', '处方药', '镇痛药', '止痛药', '副作用', '相互作用',

    # 公共卫生与研究
    '流行病学', '生物医学', '生物医药', '荟萃分析', '系统性', '病例', '队列',
    '随机', '临床试验', '试验', '死亡率', '发病率', '风险', '暴露',
    '患病率', '发病率', '大流行', '新冠', '非典', '冠状病毒', '封锁',

    # 医疗机构与人员
    '医院', '诊所', '护士', '医生', '医师', '外科医生', '员工', '住院医师',
    '护理人员', '护理人员', '救护车', '急诊', '重症监护室', '病房', '科室', '临终关怀',

    # 精神健康
    '抑郁症', '焦虑', '躁郁症', '多动症', '自闭症', '阿尔茨海默病', '痴呆',
    '自杀', '压力', '创伤', '上瘾', '物质', '滥用', '康复',

    # 其他常见疾病
    '哮喘', '过敏', '关节炎', '艾滋病毒', '艾滋病', '性传播疾病', '不育症', '偏头痛',
    '帕金森', '多发性硬化症', '多发性硬化症', '湿疹', '银屑病', '睡眠', '疲劳',

    # 医疗科技与生物科学
    '生物标志物', '基因组', '基因', '遗传', '突变', 'DNA', 'RNA', '测序',
    '组学', '蛋白质组学', '生物信息学', '代谢', '细胞', '器官', '组织',

    # 生活方式与预防
    '营养', '饮食', '锻炼', '健身', '吸烟', '酒精', '筛查',
    '生活方式', '预防', '健康', '幸福感', '公共卫生', '疫苗接种',

    # Общие медицинские термины
    'медицина', 'медицинский', 'клинический', 'пациент', 'пациенты', 'здоровье', 'болезнь', 'болезни',
    'расстройство', 'расстройства', 'недуг', 'состояние', 'состояния', 'симптом', 'симптомы',
    'лечение', 'терапия', 'терапии', 'диагноз', 'диагнозы', 'патология', 'уход',

    # Отдельные медицинские термины и направления
    'кардио', 'кардиология', 'сердце', 'сосудистый', 'сердечно-сосудистый', 'нейро', 'неврология',
    'мозг', 'когнитивный', 'психиатрия', 'психический', 'психологический', 'псих', 'психоз',
    'онкология', 'рак', 'опухоль', 'опухоли', 'новообразование', 'злокачественный', 'доброкачественный',
    'инсульт', 'диабет', 'глюкоза', 'ожирение', 'гипертония', 'давление', 'холестерин',
    'инфекция', 'инфекции', 'патоген', 'вирус', 'вирусный', 'бактерия', 'бактериальный',
    'грибковый', 'паразит', 'сепсис', 'воспаление', 'лихорадка', 'усталость',

    # Медицинские специальности
    'педиатрия', 'педиатрия', 'дети', 'ребёнок', 'акушерство', 'гинекология', 'гинеколог',
    'фертильность', 'роды', 'репродуктивный', 'анестезиология', 'анестезия', 'ортопедия',
    'кости', 'позвоночник', 'сустав', 'перелом', 'хирургия', 'хирургический', 'дерматология', 'кожа',
    'эндокринология', 'гормон', 'щитовидка', 'инсулин', 'гематология', 'кровь', 'гемоглобин',
    'урология', 'почечный', 'почка', 'диализ', 'нефрология', 'печень', 'гепатит', 'гепатология',
    'гастроэнтерология', 'кишечник', 'толстая кишка', 'пищеварительный', 'живот', 'абдоминальный',

    # Диагностика и процедуры
    'сканирование', 'мрт', 'кт', 'рентген', 'рентгеновский', 'ультразвук', 'экг', 'ээг', 'мониторинг', 'тест',
    'скрининг', 'биопсия', 'лаборатория', 'анализы', 'радиология', 'диагностический', 'гистопатология',

    # Фармакология и лекарства
    'лекарство', 'лекарства', 'медикамент', 'медикаменты', 'вакцина', 'вакцины', 'иммунизация',
    'химиотерапия', 'химио', 'доза', 'дозировка', 'таблетка', 'пилюля', 'антибиотик', 'противовирусный',
    'опиоид', 'рецепт', 'анальгетик', 'обезболивающее', 'побочный эффект', 'взаимодействие',

    # Общественное здравоохранение и исследования
    'эпидемиология', 'биомедицина', 'биомед', 'мета-анализ', 'систематический', 'случай', 'когорта',
    'рандомизированный', 'клиническое испытание', 'испытания', 'смертность', 'заболеваемость', 'риск', 'воздействие',
    'распространенность', 'заболеваемость', 'пандемия', 'ковид', 'sars', 'коронавирус', 'локдаун',

    # Учреждения и специалисты
    'больница', 'клиника', 'медсестра', 'врач', 'терапевт', 'хирург', 'персонал', 'ординатор',
    'опекун', 'парамедик', 'скорая', 'экстренный', 'реанимация', 'палата', 'отделение', 'хоспис',

    # Психическое здоровье
    'депрессия', 'тревожность', 'биполярное', 'сдвав', 'аутизм', 'альцгеймер', 'деменция',
    'суицид', 'стресс', 'травма', 'зависимость', 'вещество', 'злоупотребление', 'реабилитация',

    # Прочие распространённые заболевания
    'астма', 'аллергия', 'артрит', 'вич', 'спид', 'ппп', 'бесплодие', 'мигрень',
    'паркинсон', 'рассеянный склероз', 'рассеянный склероз', 'экзема', 'псориаз', 'сон', 'усталость',

    # Медицинские технологии и бионауки
    'биомаркер', 'геном', 'ген', 'генетический', 'мутация', 'днк', 'рнк', 'секвенирование',
    'омика', 'протеомика', 'биоинформатика', 'метаболизм', 'клетка', 'орган', 'ткань',

    # Образ жизни и профилактика
    'питание', 'диета', 'упражнения', 'фитнес', 'курение', 'алкоголь', 'скрининг',
    'образ жизни', 'профилактика', 'здоровье', 'благополучие', 'общественное здравоохранение', 'вакцинация',

    # 一般的な医療用語
    '医学', '医療', '臨床', '患者', '患者たち', '健康', '病気', '疾患',
    '障害', '障害群', '病', '状態', '症状', '症候',
    '治療', '療法', '治療法', '診断', '診断群', '病理学', 'ケア',

    # 医学用語と分野
    '心臓', '循環器科', '心臓', '血管', '心血管', '神経', '神経学',
    '脳', '認知', '精神医学', '精神的', '心理的', '精神', '精神病',
    '腫瘍学', 'がん', '腫瘍', '腫瘍群', '新生物', '悪性', '良性',
    '脳卒中', '糖尿病', 'グルコース', '肥満', '高血圧', '血圧', 'コレステロール',
    '感染症', '感染', '病原体', 'ウイルス', 'ウイルス性', '細菌', '細菌性',
    '真菌性', '寄生虫', '敗血症', '炎症', '発熱', '倦怠感',

    # 医療専門分野
    '小児科', '小児科学', '子供', '児童', '産科', '婦人科', '婦人',
    '不妊治療', '出産', '生殖', '麻酔科', '麻酔', '整形外科',
    '骨', '脊椎', '関節', '骨折', '外科', '外科的', '皮膚科', '皮膚',
    '内分泌学', 'ホルモン', '甲状腺', 'インスリン', '血液学', '血液', 'ヘモグロビン',
    '泌尿器科', '腎臓の', '腎臓', '透析', '腎臓病学', '肝臓', '肝炎', '肝臓病学',
    '消化器科', '腸', '結腸', '消化', '腹部', '腹部の',

    # 診断と処置
    'スキャン', 'MRI', 'CT', 'レントゲン', 'X線', '超音波', '心電図', '脳波', 'モニタリング', '検査',
    'スクリーニング', '生検', '検査室', 'ラボ', '放射線科', '診断', '組織病理学',

    # 薬理学と薬
    '薬', '薬剤', '薬物治療', '医薬品', 'ワクチン', 'ワクチン群', '免疫接種',
    '化学療法', '抗がん剤', '投与量', '投与', '錠剤', 'タブレット', '抗生物質', '抗ウイルス薬',
    'オピオイド', '処方箋', '鎮痛剤', '痛み止め', '副作用', '相互作用',

    # 公衆衛生と研究
    '疫学', '生物医学', 'バイオメッド', 'メタ解析', '系統的', '症例', 'コホート',
    '無作為化', '臨床試験', '試験', '死亡率', '罹患率', 'リスク', '曝露',
    '有病率', '発症率', 'パンデミック', 'コロナ', 'SARS', 'コロナウイルス', 'ロックダウン',

    # 医療機関と専門職
    '病院', 'クリニック', '看護師', '医師', '内科医', '外科医', 'スタッフ', '研修医',
    '介護者', '救急救命士', '救急車', '緊急', 'ICU', '病棟', 'ユニット', 'ホスピス',

    # 精神的健康
    'うつ病', '不安障害', '双極性障害', 'ADHD', '自閉症', 'アルツハイマー', '認知症',
    '自殺', 'ストレス', 'トラウマ', '依存症', '薬物', '乱用', 'リハビリテーション',

    # その他の一般的な疾患
    '喘息', 'アレルギー', '関節炎', 'HIV', 'エイズ', '性病', '不妊症', '偏頭痛',
    'パーキンソン病', '多発性硬化症', 'MS', '湿疹', '乾癬', '睡眠', '疲労',

    # 医療技術とバイオサイエンス
    'バイオマーカー', 'ゲノム', '遺伝子', '遺伝的', '突然変異', 'DNA', 'RNA', 'シーケンシング',
    'オミクス', 'プロテオミクス', 'バイオインフォマティクス', '代謝', '細胞', '臓器', '組織',

    # 生活習慣と予防
    '栄養', '食事', '運動', 'フィットネス', '喫煙', 'アルコール', 'スクリーニング',
    'ライフスタイル', '予防', '健康', '幸福', '公衆衛生', '予防接種',

    # 일반 의학 용어
    '의학', '의료', '임상', '환자', '환자들', '건강', '질병', '질환',
    '장애', '장애들', '질환', '상태', '상태들', '증상', '증상들',
    '치료', '요법', '치료법', '진단', '진단들', '병리학', '돌봄',

    # 일반 의학 용어 및 분야
    '심장', '심장학', '심장', '혈관', '심혈관', '신경', '신경학',
    '뇌', '인지', '정신의학', '정신', '심리적', '정신', '정신병',
    '종양학', '암', '종양', '종양들', '신생물', '악성', '양성',
    '뇌졸중', '당뇨병', '포도당', '비만', '고혈압', '혈압', '콜레스테롤',
    '감염', '감염들', '병원체', '바이러스', '바이러스성', '박테리아', '세균성',
    '진균성', '기생충', '패혈증', '염증', '열', '피로',

    # 의학 전문 분야
    '소아과', '소아과학', '어린이', '아이', '산부인과', '부인과', '여성의학',
    '불임', '출산', '생식', '마취과', '마취', '정형외과',
    '뼈', '척추', '관절', '골절', '수술', '외과적인', '피부과', '피부',
    '내분비학', '호르몬', '갑상선', '인슐린', '혈액학', '혈액', '헤모글로빈',
    '비뇨기과', '신장', '신장', '투석', '신장학', '간', '간염', '간질환학',
    '소화기학', '장', '대장', '소화', '복부', '복부의',

    # 진단 및 시술
    '스캔', 'MRI', 'CT', '엑스레이', 'X-선', '초음파', '심전도', '뇌파검사', '모니터링', '검사',
    '선별검사', '생검', '실험실', '랩', '방사선과', '진단', '조직병리학',

    # 약리학 및 약물
    '약', '약물', '약물치료', '의약품', '백신', '백신들', '예방접종',
    '화학요법', '항암치료', '복용량', '투약', '알약', '정제', '항생제', '항바이러스제',
    '오피오이드', '처방전', '진통제', '진통약', '부작용', '상호작용',

    # 공중보건 및 연구
    '역학', '생의학', '바이오메드', '메타분석', '체계적', '사례', '코호트',
    '무작위화', '임상시험', '시험들', '사망률', '이환율', '위험', '노출',
    '유병률', '발생률', '팬데믹', '코로나', '사스', '코로나바이러스', '봉쇄',

    # 기관 및 전문가
    '병원', '클리닉', '간호사', '의사', '내과의', '외과의', '직원', '레지던트',
    '간병인', '구급대원', '구급차', '응급', '중환자실', '병동', '유닛', '호스피스',

    # 정신 건강
    '우울증', '불안', '양극성 장애', 'ADHD', '자폐증', '알츠하이머', '치매',
    '자살', '스트레스', '외상', '중독', '물질', '남용', '재활',

    # 기타 일반 질환
    '천식', '알레르기', '관절염', 'HIV', '에이즈', '성병', '불임증', '편두통',
    '파킨슨병', '다발성경화증', '다발성 경화증', '습진', '건선', '수면', '피로',

    # 의료 기술 및 생명과학
    '바이오마커', '게놈', '유전자', '유전적', '돌연변이', 'DNA', 'RNA', '염기서열 분석',
    '오믹스', '프로테오믹스', '생물정보학', '신진대사', '세포', '기관', '조직',

    # 생활습관 및 예방
    '영양', '식이요법', '운동', '피트니스', '흡연', '알코올', '검진',
    '라이프스타일', '예방', '건강 증진', '웰빙', '공중보건', '예방접종',

    # 一般醫學術語
    '醫學', '醫療', '臨床', '病人', '病患', '健康', '疾病', '病症',
    '失調', '紊亂', '病痛', '狀況', '情況', '症狀', '症候',
    '治療', '療法', '治療方式', '診斷', '診斷結果', '病理學', '照護',

    # 一般醫學詞彙與領域
    '心臟', '心臟病學', '心', '血管', '心血管', '神經', '神經學',
    '大腦', '認知', '精神醫學', '心理', '心理學的', '心理', '精神病',
    '腫瘤學', '癌症', '腫瘤', '腫瘤們', '新生物', '惡性', '良性',
    '中風', '糖尿病', '葡萄糖', '肥胖', '高血壓', '壓力', '膽固醇',
    '感染', '感染症', '病原體', '病毒', '病毒性', '細菌', '細菌性',
    '真菌', '寄生蟲', '敗血症', '發炎', '發燒', '疲勞',

    # 醫學專科
    '小兒科', '兒科', '兒童', '孩子', '產科', '婦科', '婦產科',
    '生育', '出生', '生殖', '麻醉學', '麻醉', '骨科',
    '骨頭', '脊椎', '關節', '骨折', '手術', '外科手術', '皮膚科', '皮膚',
    '內分泌學', '荷爾蒙', '甲狀腺', '胰島素', '血液學', '血液', '血紅素',
    '泌尿科', '腎臟的', '腎臟', '透析', '腎臟病學', '肝臟', '肝炎', '肝病學',
    '胃腸科', '腸道', '結腸', '消化', '腹部', '腹部的',

    # 診斷與檢查
    '掃描', '核磁共振', '電腦斷層', 'X光', 'X-射線', '超音波', '心電圖', '腦電圖', '監控', '檢驗',
    '篩檢', '切片檢查', '實驗室', '實驗', '放射科', '診斷', '組織病理學',

    # 藥理學與藥物
    '藥物', '藥品', '藥物治療', '藥物們', '疫苗', '疫苗們', '免疫接種',
    '化學療法', '化療', '劑量', '劑量控制', '藥丸', '錠劑', '抗生素', '抗病毒藥',
    '鴉片類藥物', '處方藥', '止痛藥', '止痛劑', '副作用', '交互作用',

    # 公共衛生與研究
    '流行病學', '生物醫學', '生醫', '統合分析', '系統性', '案例', '隊列研究',
    '隨機分配', '臨床試驗', '試驗', '死亡率', '罹患率', '風險', '暴露',
    '盛行率', '發生率', '大流行', '新冠', '嚴重急性呼吸道症候群', '冠狀病毒', '封鎖',

    # 機構與專業人士
    '醫院', '診所', '護士', '醫師', '內科醫師', '外科醫師', '人員', '住院醫師',
    '照護者', '救護人員', '救護車', '急診', '加護病房', '病房', '單位', '安寧病房',

    # 精神健康
    '憂鬱症', '焦慮', '躁鬱症', '注意力缺陷過動症', '自閉症', '阿茲海默症', '失智症',
    '自殺', '壓力', '創傷', '成癮', '物質', '濫用', '復健',

    # 其他常見病況
    '氣喘', '過敏', '關節炎', '愛滋病毒', '愛滋病', '性病', '不孕症', '偏頭痛',
    '帕金森氏症', '多發性硬化症', '多發性硬化', '濕疹', '乾癬', '睡眠', '疲勞',

    # 醫療科技與生科
    '生物標記', '基因組', '基因', '遺傳', '突變', '去氧核糖核酸', '核糖核酸', '定序',
    '組學', '蛋白質體學', '生物資訊學', '新陳代謝', '細胞', '器官', '組織',

    # 生活方式與預防
    '營養', '飲食', '運動', '健身', '吸菸', '酒精', '篩檢',
    '生活方式', '預防', '健康促進', '福祉', '公共衛生', '疫苗接種',

    # Obecné lékařské pojmy
    'medicína', 'lékařský', 'klinický', 'pacient', 'pacienti', 'zdraví', 'nemoc', 'nemoci',
    'porucha', 'poruchy', 'onemocnění', 'stav', 'stavy', 'příznak', 'příznaky',
    'léčba', 'terapie', 'terapie', 'diagnóza', 'diagnózy', 'patologie', 'péče',

    # Běžné lékařské termíny a obory
    'kardio', 'kardiologie', 'srdce', 'cévní', 'kardiovaskulární', 'neuro', 'neurologie',
    'mozek', 'kognitivní', 'psychiatrie', 'duševní', 'psychologický', 'psych', 'psychóza',
    'onkologie', 'rakovina', 'nádor', 'nádory', 'neoplazie', 'maligní', 'benigní',
    'mrtvice', 'cukrovka', 'glukóza', 'obezita', 'hypertenze', 'tlak', 'cholesterol',
    'infekce', 'infekce', 'patogen', 'virus', 'virový', 'bakterie', 'bakteriální',
    'plísňový', 'parazit', 'sepse', 'zánět', 'horečka', 'únava',

    # Lékařské specializace
    'pediatrie', 'pediatrie', 'děti', 'dítě', 'porodnictví', 'gynekologie', 'gynekolog',
    'plodnost', 'porod', 'reprodukční', 'anesteziologie', 'anestezie', 'ortopedie',
    'kosti', 'páteř', 'kloub', 'zlomenina', 'chirurgie', 'chirurgický', 'dermatologie', 'kůže',
    'endokrinologie', 'hormon', 'štítná žláza', 'inzulin', 'hematologie', 'krev', 'hemoglobin',
    'urologie', 'renální', 'ledvina', 'dialýza', 'nefrologie', 'játra', 'hepatitida', 'hepatologie',
    'gastroenterologie', 'střevo', 'tlusté střevo', 'trávení', 'břicho', 'abdominální',

    # Diagnostika a procedury
    'sken', 'mri', 'ct', 'rentgen', 'rentgen', 'ultrazvuk', 'ekg', 'eeg', 'monitorování', 'test',
    'screening', 'biopsie', 'laboratoř', 'laboratoře', 'radiologie', 'diagnostika', 'histopatologie',

    # Farmakologie a léky
    'lék', 'léky', 'medikace', 'léčiva', 'vakcína', 'vakcíny', 'očkování',
    'chemoterapie', 'chemo', 'dávka', 'dávkování', 'pilulka', 'tableta', 'antibiotikum', 'antivirotikum',
    'opioid', 'předpis', 'analgetikum', 'lék proti bolesti', 'vedlejší účinek', 'interakce',

    # Veřejné zdraví a výzkum
    'epidemiologie', 'biomedicína', 'biomed', 'metaanalýza', 'systematický', 'případ', 'kohorta',
    'randomizovaný', 'klinická studie', 'studie', 'úmrtnost', 'nemocnost', 'riziko', 'expozice',
    'prevalence', 'incidence', 'pandemie', 'covid', 'sars', 'koronavirus', 'lockdown',

    # Instituce a profesionálové
    'nemocnice', 'klinika', 'zdravotní sestra', 'doktor', 'lékař', 'chirurg', 'personál', 'rezident',
    'pečovatel', 'záchranář', 'sanitka', 'pohotovost', 'JIP', 'oddělení', 'jednotka', 'hospic',

    # Duševní zdraví
    'deprese', 'úzkost', 'bipolární', 'adhd', 'autismus', 'alzheimer', 'demence',
    'sebevražda', 'stres', 'trauma', 'závislost', 'návyková látka', 'zneužívání', 'rehabilitace',

    # Další běžné stavy
    'astma', 'alergie', 'artritida', 'hiv', 'aids', 'std', 'neplodnost', 'migréna',
    'parkinson', 'rs', 'roztroušená skleróza', 'ekzém', 'psoriáza', 'spánek', 'únava',

    # Zdravotnické technologie a biologické vědy
    'biomarker', 'genom', 'gen', 'genetický', 'mutace', 'dna', 'rna', 'sekvenování',
    'omika', 'proteomika', 'bioinformatika', 'metabolismus', 'buňka', 'orgán', 'tkáň',

    # Životní styl a prevence
    'výživa', 'dieta', 'cvičení', 'fitness', 'kouření', 'alkohol', 'screening',
    'životní styl', 'prevence', 'zdravý životní styl', 'pohoda', 'veřejné zdraví', 'očkování',

    # المصطلحات الطبية العامة
    'الطب', 'طبي', 'سريري', 'مريض', 'مرضى', 'صحة', 'مرض', 'أمراض',
    'اضطراب', 'اضطرابات', 'علّة', 'حالة', 'حالات', 'عرض', 'أعراض',
    'علاج', 'معالجة', 'معالجات', 'تشخيص', 'تشخيصات', 'علم الأمراض', 'رعاية',

    # المصطلحات الطبية والمجالات الشائعة
    'قلبي', 'طب القلب', 'قلب', 'أوعية دموية', 'قلبي وعائي', 'عصبي', 'علم الأعصاب',
    'دماغ', 'معرفي', 'طب نفسي', 'نفسي', 'نفسي/عقلي', 'سايكو', 'ذهان',
    'طب الأورام', 'سرطان', 'ورم', 'أورام', 'ورم جديد', 'خبيث', 'حميد',
    'سكتة دماغية', 'سكري', 'جلوكوز', 'سمنة', 'ارتفاع ضغط الدم', 'ضغط', 'كوليسترول',
    'عدوى', 'عدوى', 'مُمْرِض', 'فيروس', 'فيروسي', 'بكتيريا', 'بكتيري',
    'فطري', 'طفيلي', 'إنتان', 'التهاب', 'حمى', 'إرهاق',

    # التخصصات الطبية
    'طب الأطفال', 'طب الأطفال', 'أطفال', 'طفل', 'توليد', 'أمراض نسائية', 'نسائي',
    'خصوبة', 'ولادة', 'إنجابي', 'تخدير', 'تخدير', 'جراحة العظام',
    'عظام', 'عمود فقري', 'مفصل', 'كسر', 'جراحة', 'جراحي', 'أمراض جلدية', 'جلد',
    'طب الغدد الصماء', 'هرمون', 'غدة درقية', 'أنسولين', 'أمراض الدم', 'دم', 'هيموغلوبين',
    'المسالك البولية', 'كلوي', 'كلية', 'غسيل الكلى', 'أمراض الكلى', 'كبد', 'التهاب الكبد', 'طب الكبد',
    'طب الجهاز الهضمي', 'أمعاء', 'قولون', 'هضمي', 'بطن', 'بطني',

    # التشخيص والإجراءات
    'فحص', 'تصوير بالرنين المغناطيسي', 'تصوير مقطعي', 'أشعة سينية', 'أشعة سينية', 'موجات فوق صوتية', 'تخطيط القلب', 'تخطيط الدماغ الكهربائي', 'مراقبة', 'اختبار',
    'فحص مبكر', 'خزعة', 'مختبر', 'مختبرات', 'أشعة', 'تشخيصي', 'تشريح مرضي',

    # علم الأدوية والأدوية
    'دواء', 'أدوية', 'علاج دوائي', 'علاجات دوائية', 'لقاح', 'لقاحات', 'تطعيم',
    'علاج كيميائي', 'كيمو', 'جرعة', 'توزيع الجرعة', 'حبوب', 'قرص', 'مضاد حيوي', 'مضاد فيروسي',
    'أفيوني', 'وصفة طبية', 'مسكن', 'مسكن للألم', 'آثار جانبية', 'تفاعل',

    # الصحة العامة والبحث
    'علم الوبائيات', 'الطب الحيوي', 'الطب الحيوي', 'تحليل شمولي', 'منهجي', 'حالة', 'مجموعة',
    'عشوائي', 'تجربة سريرية', 'تجارب', 'وفيات', 'مراضة', 'خطر', 'تعرض',
    'الانتشار', 'الحدوث', 'جائحة', 'كوفيد', 'سارس', 'كورونا', 'إغلاق',

    # المؤسسات والمهنيون
    'مستشفى', 'عيادة', 'ممرضة', 'طبيب', 'طبيب', 'جراح', 'طاقم', 'مقيم',
    'مُقدم رعاية', 'مسعف', 'إسعاف', 'طوارئ', 'العناية المركزة', 'جناح', 'وحدة', 'رعاية ملطفة',

    # الصحة النفسية
    'اكتئاب', 'قلق', 'اضطراب ثنائي القطب', 'فرط الحركة وتشتت الانتباه', 'توحد', 'ألزهايمر', 'خرف',
    'انتحار', 'ضغط نفسي', 'صدمة', 'إدمان', 'مواد', 'إساءة', 'تأهيل',

    # حالات شائعة أخرى
    'ربو', 'حساسية', 'التهاب المفاصل', 'فيروس نقص المناعة', 'الإيدز', 'أمراض منقولة جنسياً', 'عقم', 'صداع نصفي',
    'باركنسون', 'التصلب المتعدد', 'التصلب اللويحي', 'أكزيما', 'صدفية', 'نوم', 'إرهاق',

    # التكنولوجيا الطبية / العلوم الحيوية
    'مؤشر حيوي', 'جينوم', 'جين', 'وراثي', 'طفرة', 'حمض نووي', 'رنا', 'تسلسل جيني',
    'أوميكس', 'بروتيوميات', 'معلوماتية حيوية', 'التمثيل الغذائي', 'خلية', 'عضو', 'نسيج',

    # أسلوب الحياة والوقاية
    'تغذية', 'حمية', 'تمارين', 'لياقة', 'تدخين', 'كحول', 'فحص',
    'أسلوب حياة', 'وقاية', 'رفاهية', 'رفاه نفسي', 'الصحة العامة', 'تطعيم',
    ],


    'informatics': [
        # Core and general CS terms
        'informatics', 'computing', 'computation', 'information', 'it', 'ict',
        'digital', 'system', 'systems', 'technology', 'technologies', 'cyber',

        # Data science & machine learning
        'data', 'dataset', 'datasets', 'databases', 'mining', 'clustering', 'classification',
        'regression', 'prediction', 'analytics', 'analysis', 'model', 'models', 'modeling',
        'statistics', 'statistical', 'estimation', 'preprocessing', 'cleaning', 'normalization',
        'correlation', 'causality', 'outliers', 'transformation', 'dimensionality',

        # Artificial Intelligence
        'ai', 'ml', 'dl', 'llm', 'gpt', 'chatgpt', 'bert', 'transformer', 'neural', 'network',
        'networks', 'cnn', 'rnn', 'gan', 'autoencoder', 'supervised', 'unsupervised',
        'reinforcement', 'robot', 'robotics', 'agent', 'agents', 'learning', 'intelligence',

        # Programming and coding
        'programming', 'code', 'coding', 'script', 'scripts', 'developer', 'engineering',
        'software', 'hardware', 'compiler', 'debug', 'debugging', 'deployment', 'devops',
        'ide', 'api', 'backend', 'frontend', 'fullstack', 'repository', 'git', 'github',
        'bitbucket', 'vscode', 'intellij', 'java', 'python', 'r', 'cpp', 'csharp', 'javascript',
        'typescript', 'matlab', 'bash', 'shell', 'notebook', 'jupyter',

        # Databases and storage
        'database', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle', 'mongodb', 'redis',
        'elasticsearch', 'query', 'queries', 'schema', 'tables', 'transaction', 'index',
        'indexing', 'view', 'trigger', 'join', 'etl', 'warehouse', 'lake', 'dataflow',

        # Software development methodologies
        'oop', 'object', 'objects', 'agile', 'scrum', 'devops', 'sprint', 'workflow',
        'version', 'control', 'ci', 'cd', 'pipeline', 'release', 'issue', 'testing',
        'unit', 'integration', 'coverage', 'refactoring',

        # Web, mobile, UI/UX
        'interface', 'interfaces', 'ui', 'ux', 'frontend', 'react', 'vue', 'html', 'css',
        'bootstrap', 'tailwind', 'mobile', 'android', 'ios', 'flutter', 'design', 'wireframe',
        'prototype', 'accessibility', 'usability', 'user', 'navigation',

        # HCI & cognitive computing
        'hci', 'interaction', 'interactions', 'gesture', 'touch', 'eye-tracking',
        'bci', 'brain-computer', 'voice', 'speech', 'natural', 'nlp', 'text', 'language',
        'sentiment', 'dialog', 'chatbot', 'multimodal', 'cognitive', 'attention',

        # Cybersecurity & blockchain
        'security', 'cybersecurity', 'encryption', 'decryption', 'cryptography', 'crypto',
        'hash', 'token', 'authentication', 'authorization', 'phishing', 'malware',
        'blockchain', 'bitcoin', 'ethereum', 'smartcontract', 'smart', 'contract', 'wallet',

        # Cloud computing and infrastructure
        'cloud', 'aws', 'azure', 'gcp', 'virtualization', 'kubernetes', 'docker',
        'container', 'serverless', 'lambda', 'infrastructure', 'scaling', 'availability',
        'latency', 'throughput', 'loadbalancer', 'microservices', 'api-gateway',

        # Networking
        'network', 'networks', 'tcp', 'ip', 'tcp/ip', 'udp', 'protocol', 'latency', 'bandwidth',
        'router', 'dns', 'dnssec', 'firewall', 'vpn', 'socket', 'internet', 'web', 'cloudflare',

        # Interdisciplinary informatics applications
        'bioinformatics', 'cheminformatics', 'healthinformatics', 'clinicalinformatics',
        'neuroinformatics', 'fintech', 'legaltech', 'govtech', 'ehealth', 'egov', 'smartcity',
        'iot', 'sensor', 'wearable', 'blockchain', 'edge', 'fog', 'industry4.0',

        # Ethics, regulation, societal impact
        'bias', 'fairness', 'accountability', 'privacy', 'gdpr', 'explainable', 'transparency',
        'trustworthy', 'safety', 'disinformation', 'deepfake', 'surveillance', 'ethics',

        # Theoretical & formal
        'automata', 'turing', 'machine', 'graph', 'graphs', 'theory', 'logic', 'boolean',
        'proof', 'theorem', 'discrete', 'algebra', 'complexity', 'np', 'p', 'approximation',
        'reduction', 'optimization', 'combinatorics', 'mathematics', 'probability'

        # Términos principales y generales de informática
    'informática', 'computación', 'cómputo', 'información', 'ti', 'tic',
    'digital', 'sistema', 'sistemas', 'tecnología', 'tecnologías', 'ciber',

    # Ciencia de datos y aprendizaje automático
    'datos', 'conjunto de datos', 'conjuntos de datos', 'bases de datos', 'minería',
    'agrupamiento', 'clasificación', 'regresión', 'predicción', 'analítica', 'análisis',
    'modelo', 'modelos', 'modelado', 'estadísticas', 'estadístico', 'estimación',
    'preprocesamiento', 'limpieza', 'normalización', 'correlación', 'causalidad',
    'valores atípicos', 'transformación', 'dimensionalidad',

    # Inteligencia Artificial
    'ia', 'am', 'ap', 'mlg', 'gpt', 'chatgpt', 'bert', 'transformador', 'neuronal',
    'red', 'redes', 'cnn', 'rnn', 'gan', 'autoencoder', 'supervisado', 'no supervisado',
    'refuerzo', 'robot', 'robótica', 'agente', 'agentes', 'aprendizaje', 'inteligencia',

    # Programación y codificación
    'programación', 'código', 'codificación', 'script', 'scripts', 'desarrollador',
    'ingeniería', 'software', 'hardware', 'compilador', 'depurar', 'depuración',
    'despliegue', 'devops', 'ide', 'api', 'backend', 'frontend', 'fullstack',
    'repositorio', 'git', 'github', 'bitbucket', 'vscode', 'intellij', 'java',
    'python', 'r', 'cpp', 'csharp', 'javascript', 'typescript', 'matlab',
    'bash', 'shell', 'notebook', 'jupyter',

    # Bases de datos y almacenamiento
    'base de datos', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle', 'mongodb',
    'redis', 'elasticsearch', 'consulta', 'consultas', 'esquema', 'tablas',
    'transacción', 'índice', 'indexación', 'vista', 'disparador', 'unión',
    'etl', 'almacén de datos', 'lago de datos', 'flujo de datos',

    # Metodologías de desarrollo de software
    'poo', 'objeto', 'objetos', 'ágil', 'scrum', 'devops', 'sprint', 'flujo de trabajo',
    'versión', 'control', 'ci', 'cd', 'pipeline', 'lanzamiento', 'incidencia',
    'pruebas', 'unitaria', 'integración', 'cobertura', 'refactorización',

    # Web, móvil, UI/UX
    'interfaz', 'interfaces', 'ui', 'ux', 'frontend', 'react', 'vue', 'html', 'css',
    'bootstrap', 'tailwind', 'móvil', 'android', 'ios', 'flutter', 'diseño',
    'prototipo', 'boceto', 'accesibilidad', 'usabilidad', 'usuario', 'navegación',

    # IHC y computación cognitiva
    'ihc', 'interacción', 'interacciones', 'gesto', 'tacto', 'seguimiento ocular',
    'bci', 'interfaz cerebro-computadora', 'voz', 'habla', 'natural', 'pln',
    'texto', 'lenguaje', 'sentimiento', 'diálogo', 'chatbot', 'multimodal',
    'cognitivo', 'atención',

    # Ciberseguridad y blockchain
    'seguridad', 'ciberseguridad', 'encriptación', 'desencriptación',
    'criptografía', 'cripto', 'hash', 'token', 'autenticación', 'autorización',
    'phishing', 'malware', 'blockchain', 'bitcoin', 'ethereum', 'contratointeligente',
    'inteligente', 'contrato', 'billetera',

    # Computación en la nube e infraestructura
    'nube', 'aws', 'azure', 'gcp', 'virtualización', 'kubernetes', 'docker',
    'contenedor', 'sin servidor', 'lambda', 'infraestructura', 'escalabilidad',
    'disponibilidad', 'latencia', 'rendimiento', 'balanceador de carga',
    'microservicios', 'puerta de enlace api',

    # Redes
    'red', 'redes', 'tcp', 'ip', 'tcp/ip', 'udp', 'protocolo', 'latencia',
    'ancho de banda', 'enrutador', 'dns', 'dnssec', 'firewall', 'vpn', 'socket',
    'internet', 'web', 'cloudflare',

    # Aplicaciones interdisciplinarias de la informática
    'bioinformática', 'quimioinformática', 'informática de la salud',
    'informática clínica', 'neuroinformática', 'fintech', 'legaltech', 'govtech',
    'esalud', 'egobierno', 'ciudadinteligente', 'iot', 'sensor', 'wearable',
    'blockchain', 'borde', 'niebla', 'industria4.0',

    # Ética, regulación, impacto social
    'sesgo', 'equidad', 'responsabilidad', 'privacidad', 'rgpd', 'explicable',
    'transparencia', 'confiable', 'seguridad', 'desinformación', 'deepfake',
    'vigilancia', 'ética',

    # Teórico y formal
    'autómata', 'turing', 'máquina', 'grafo', 'grafos', 'teoría', 'lógica',
    'booleano', 'prueba', 'teorema', 'discreto', 'álgebra', 'complejidad',
    'np', 'p', 'aproximación', 'reducción', 'optimización', 'combinatoria',
    'matemáticas', 'probabilidad'

    # Termos principais e gerais de ciência da computação
    'informática', 'computação', 'cálculo', 'informação', 'ti', 'tic',
    'digital', 'sistema', 'sistemas', 'tecnologia', 'tecnologias', 'cibernético',

    # Ciência de dados e aprendizado de máquina
    'dados', 'conjunto de dados', 'conjuntos de dados', 'bancos de dados', 'mineração',
    'agrupamento', 'classificação', 'regressão', 'previsão', 'analítica', 'análise',
    'modelo', 'modelos', 'modelagem', 'estatística', 'estatístico', 'estimação',
    'pré-processamento', 'limpeza', 'normalização', 'correlação', 'causalidade',
    'valores atípicos', 'transformação', 'dimensionalidade',

    # Inteligência Artificial
    'ia', 'am', 'ap', 'gpl', 'gpt', 'chatgpt', 'bert', 'transformador', 'neuronal',
    'rede', 'redes', 'cnn', 'rnn', 'gan', 'autoencoder', 'supervisionado',
    'não supervisionado', 'reforço', 'robô', 'robótica', 'agente', 'agentes',
    'aprendizado', 'inteligência',

    # Programação e codificação
    'programação', 'código', 'codificação', 'script', 'scripts', 'desenvolvedor',
    'engenharia', 'software', 'hardware', 'compilador', 'depurar', 'depuração',
    'implantação', 'devops', 'ide', 'api', 'backend', 'frontend', 'fullstack',
    'repositório', 'git', 'github', 'bitbucket', 'vscode', 'intellij', 'java',
    'python', 'r', 'cpp', 'csharp', 'javascript', 'typescript', 'matlab',
    'bash', 'shell', 'notebook', 'jupyter',

    # Bancos de dados e armazenamento
    'banco de dados', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle', 'mongodb',
    'redis', 'elasticsearch', 'consulta', 'consultas', 'esquema', 'tabelas',
    'transação', 'índice', 'indexação', 'visualização', 'gatilho', 'junção',
    'etl', 'armazém de dados', 'lago de dados', 'fluxo de dados',

    # Metodologias de desenvolvimento de software
    'poo', 'objeto', 'objetos', 'ágil', 'scrum', 'devops', 'sprint',
    'fluxo de trabalho', 'versão', 'controle', 'ci', 'cd', 'pipeline',
    'lançamento', 'problema', 'teste', 'unitário', 'integração',
    'cobertura', 'refatoração',

    # Web, móvel, UI/UX
    'interface', 'interfaces', 'ui', 'ux', 'frontend', 'react', 'vue', 'html', 'css',
    'bootstrap', 'tailwind', 'móvel', 'android', 'ios', 'flutter', 'design',
    'wireframe', 'protótipo', 'acessibilidade', 'usabilidade', 'usuário',
    'navegação',

    # IHC e computação cognitiva
    'ihc', 'interação', 'interações', 'gesto', 'toque', 'rastreamento ocular',
    'icc', 'cérebro-computador', 'voz', 'fala', 'natural', 'pln',
    'texto', 'linguagem', 'sentimento', 'diálogo', 'chatbot', 'multimodal',
    'cognitivo', 'atenção',

    # Cibersegurança e blockchain
    'segurança', 'cibersegurança', 'criptografia', 'descriptografia',
    'criptografia', 'cripto', 'hash', 'token', 'autenticação', 'autorização',
    'phishing', 'malware', 'blockchain', 'bitcoin', 'ethereum',
    'contrato inteligente', 'inteligente', 'contrato', 'carteira',

    # Computação em nuvem e infraestrutura
    'nuvem', 'aws', 'azure', 'gcp', 'virtualização', 'kubernetes', 'docker',
    'container', 'sem servidor', 'lambda', 'infraestrutura', 'escalabilidade',
    'disponibilidade', 'latência', 'vazão', 'balanceador de carga',
    'microsserviços', 'gateway de api',

    # Redes
    'rede', 'redes', 'tcp', 'ip', 'tcp/ip', 'udp', 'protocolo', 'latência',
    'largura de banda', 'roteador', 'dns', 'dnssec', 'firewall', 'vpn', 'socket',
    'internet', 'web', 'cloudflare',

    # Aplicações interdisciplinares da informática
    'bioinformática', 'quimioinformática', 'informática em saúde',
    'informática clínica', 'neuroinformática', 'fintech', 'legaltech', 'govtech',
    'esaúde', 'egov', 'cidade inteligente', 'iot', 'sensor', 'vestível',
    'blockchain', 'borda', 'névoa', 'indústria4.0',

    # Ética, regulação e impacto social
    'viés', 'justiça', 'responsabilidade', 'privacidade', 'lgpd',
    'explicável', 'transparência', 'confiável', 'segurança', 'desinformação',
    'deepfake', 'vigilância', 'ética',

    # Teórico e formal
    'autômato', 'turing', 'máquina', 'grafo', 'grafos', 'teoria', 'lógica',
    'booleano', 'prova', 'teorema', 'discreto', 'álgebra', 'complexidade',
    'np', 'p', 'aproximação', 'redução', 'otimização', 'combinatória',
    'matemática', 'probabilidade'

    # Termini principali e generali dell'informatica
    'informatica', 'calcolo', 'computazione', 'informazione', 'it', 'ict',
    'digitale', 'sistema', 'sistemi', 'tecnologia', 'tecnologie', 'cyber',

    # Data science e machine learning
    'dati', 'insieme di dati', 'insiemi di dati', 'basi di dati', 'estrazione',
    'clustering', 'classificazione', 'regressione', 'previsione', 'analitica',
    'analisi', 'modello', 'modelli', 'modellazione', 'statistica', 'statistico',
    'stima', 'preprocessing', 'pulizia', 'normalizzazione', 'correlazione',
    'causalità', 'valori anomali', 'trasformazione', 'dimensionalità',

    # Intelligenza Artificiale
    'ia', 'ml', 'dl', 'llm', 'gpt', 'chatgpt', 'bert', 'trasformatore', 'neurale',
    'rete', 'reti', 'cnn', 'rnn', 'gan', 'autoencoder', 'supervisionato',
    'non supervisionato', 'rinforzo', 'robot', 'robotica', 'agente', 'agenti',
    'apprendimento', 'intelligenza',

    # Programmazione e codifica
    'programmazione', 'codice', 'codifica', 'script', 'scripts', 'sviluppatore',
    'ingegneria', 'software', 'hardware', 'compilatore', 'debug', 'debugging',
    'distribuzione', 'devops', 'ide', 'api', 'backend', 'frontend', 'fullstack',
    'repository', 'git', 'github', 'bitbucket', 'vscode', 'intellij', 'java',
    'python', 'r', 'cpp', 'csharp', 'javascript', 'typescript', 'matlab',
    'bash', 'shell', 'notebook', 'jupyter',

    # Basi di dati e archiviazione
    'base di dati', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle', 'mongodb',
    'redis', 'elasticsearch', 'query', 'querys', 'schema', 'tabelle',
    'transazione', 'indice', 'indicizzazione', 'vista', 'trigger', 'join',
    'etl', 'magazzino dati', 'data lake', 'flusso di dati',

    # Metodologie di sviluppo software
    'oop', 'oggetto', 'oggetti', 'agile', 'scrum', 'devops', 'sprint',
    'workflow', 'versione', 'controllo', 'ci', 'cd', 'pipeline', 'rilascio',
    'problema', 'test', 'unità', 'integrazione', 'copertura', 'refactoring',

    # Web, mobile, UI/UX
    'interfaccia', 'interfacce', 'ui', 'ux', 'frontend', 'react', 'vue', 'html',
    'css', 'bootstrap', 'tailwind', 'mobile', 'android', 'ios', 'flutter',
    'design', 'wireframe', 'prototipo', 'accessibilità', 'usabilità', 'utente',
    'navigazione',

    # IHC e calcolo cognitivo
    'ihc', 'interazione', 'interazioni', 'gesto', 'tocco', 'eye-tracking',
    'bci', 'cervello-computer', 'voce', 'linguaggio parlato', 'naturale',
    'pln', 'testo', 'lingua', 'sentimento', 'dialogo', 'chatbot', 'multimodale',
    'cognitivo', 'attenzione',

    # Cybersecurity e blockchain
    'sicurezza', 'cybersicurezza', 'crittografia', 'decrittazione',
    'crittografia', 'cripto', 'hash', 'token', 'autenticazione',
    'autorizzazione', 'phishing', 'malware', 'blockchain', 'bitcoin',
    'ethereum', 'smartcontract', 'smart', 'contratto', 'portafoglio',

    # Cloud computing e infrastruttura
    'cloud', 'aws', 'azure', 'gcp', 'virtualizzazione', 'kubernetes', 'docker',
    'contenitore', 'serverless', 'lambda', 'infrastruttura', 'scalabilità',
    'disponibilità', 'latenza', 'throughput', 'bilanciamento carico',
    'microservizi', 'gateway-api',

    # Reti
    'rete', 'reti', 'tcp', 'ip', 'tcp/ip', 'udp', 'protocollo', 'latenza',
    'larghezza di banda', 'router', 'dns', 'dnssec', 'firewall', 'vpn',
    'socket', 'internet', 'web', 'cloudflare',

    # Applicazioni interdisciplinari dell'informatica
    'bioinformatica', 'chemiinformatica', 'informatica sanitaria',
    'informatica clinica', 'neuroinformatica', 'fintech', 'legaltech',
    'govtech', 'esanità', 'egov', 'città intelligente', 'iot', 'sensore',
    'wearable', 'blockchain', 'edge', 'fog', 'industria4.0',

    # Etica, regolamentazione, impatto sociale
    'pregiudizio', 'equità', 'responsabilità', 'privacy', 'gdpr',
    'spiegabile', 'trasparenza', 'affidabile', 'sicurezza', 'disinformazione',
    'deepfake', 'sorveglianza', 'etica',

    # Teorico e formale
    'automa', 'turing', 'macchina', 'grafo', 'grafi', 'teoria', 'logica',
    'booleano', 'dimostrazione', 'teorema', 'discreto', 'algebra',
    'complessità', 'np', 'p', 'approssimazione', 'riduzione',
    'ottimizzazione', 'combinatoria', 'matematica', 'probabilità'

    # Termes principaux et généraux en informatique
    'informatique', 'informatique', 'calcul', 'information', 'it', 'tic',
    'numérique', 'système', 'systèmes', 'technologie', 'technologies', 'cyber',

    # Science des données et apprentissage automatique
    'données', 'ensemble de données', 'ensembles de données', 'bases de données',
    'extraction', 'regroupement', 'classification', 'régression', 'prédiction',
    'analytique', 'analyse', 'modèle', 'modèles', 'modélisation',
    'statistiques', 'statistique', 'estimation', 'prétraitement', 'nettoyage',
    'normalisation', 'corrélation', 'causalité', 'valeurs aberrantes',
    'transformation', 'dimensionnalité',

    # Intelligence artificielle
    'ia', 'am', 'aa', 'gdl', 'gpt', 'chatgpt', 'bert', 'transformeur', 'neuronal',
    'réseau', 'réseaux', 'cnn', 'rnn', 'gan', 'autoencodeur', 'supervisé',
    'non supervisé', 'renforcement', 'robot', 'robotique', 'agent', 'agents',
    'apprentissage', 'intelligence',

    # Programmation et codage
    'programmation', 'code', 'codage', 'script', 'scripts', 'développeur',
    'ingénierie', 'logiciel', 'matériel', 'compilateur', 'déboguer', 'débogage',
    'déploiement', 'devops', 'ide', 'api', 'backend', 'frontend', 'fullstack',
    'référentiel', 'git', 'github', 'bitbucket', 'vscode', 'intellij', 'java',
    'python', 'r', 'cpp', 'csharp', 'javascript', 'typescript', 'matlab',
    'bash', 'shell', 'notebook', 'jupyter',

    # Bases de données et stockage
    'base de données', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle', 'mongodb',
    'redis', 'elasticsearch', 'requête', 'requêtes', 'schéma', 'tables',
    'transaction', 'index', 'indexation', 'vue', 'déclencheur', 'jointure',
    'etl', 'entrepôt de données', 'lac de données', 'flux de données',

    # Méthodologies de développement logiciel
    'poo', 'objet', 'objets', 'agile', 'scrum', 'devops', 'sprint',
    'flux de travail', 'version', 'contrôle', 'ci', 'cd', 'pipeline',
    'version', 'problème', 'tests', 'unité', 'intégration', 'couverture',
    'refactorisation',

    # Web, mobile, UI/UX
    'interface', 'interfaces', 'ui', 'ux', 'frontend', 'react', 'vue', 'html',
    'css', 'bootstrap', 'tailwind', 'mobile', 'android', 'ios', 'flutter',
    'design', 'maquette', 'prototype', 'accessibilité', 'utilisabilité',
    'utilisateur', 'navigation',

    # IHM et informatique cognitive
    'ihm', 'interaction', 'interactions', 'geste', 'toucher', 'suivi oculaire',
    'icm', 'cerveau-ordinateur', 'voix', 'parole', 'naturel', 'taln',
    'texte', 'langage', 'sentiment', 'dialogue', 'chatbot', 'multimodal',
    'cognitif', 'attention',

    # Cybersécurité et blockchain
    'sécurité', 'cybersécurité', 'chiffrement', 'déchiffrement',
    'cryptographie', 'crypto', 'hachage', 'jeton', 'authentification',
    'autorisation', 'phishing', 'malware', 'blockchain', 'bitcoin',
    'ethereum', 'contratintelligent', 'intelligent', 'contrat', 'portefeuille',

    # Informatique en nuage et infrastructure
    'cloud', 'aws', 'azure', 'gcp', 'virtualisation', 'kubernetes', 'docker',
    'conteneur', 'sans serveur', 'lambda', 'infrastructure', 'scalabilité',
    'disponibilité', 'latence', 'débit', 'équilibreur de charge',
    'microservices', 'passerelle-api',

    # Réseautique
    'réseau', 'réseaux', 'tcp', 'ip', 'tcp/ip', 'udp', 'protocole', 'latence',
    'bande passante', 'routeur', 'dns', 'dnssec', 'pare-feu', 'vpn',
    'socket', 'internet', 'web', 'cloudflare',

    # Applications interdisciplinaires de l'informatique
    'bioinformatique', 'chimioinformatique', 'informatiquesanté',
    'informatiqueclinique', 'neuroinformatique', 'fintech', 'legaltech',
    'govtech', 'esanté', 'egouv', 'villeintelligente', 'iot', 'capteur',
    'portable', 'blockchain', 'edge', 'brouillard', 'industrie4.0',

    # Éthique, réglementation et impact sociétal
    'biais', 'équité', 'responsabilité', 'vie privée', 'rgpd',
    'explicable', 'transparence', 'fiable', 'sécurité', 'désinformation',
    'deepfake', 'surveillance', 'éthique',

    # Théorique et formel
    'automate', 'turing', 'machine', 'graphe', 'graphes', 'théorie',
    'logique', 'booléen', 'preuve', 'théorème', 'discret', 'algèbre',
    'complexité', 'np', 'p', 'approximation', 'réduction', 'optimisation',
    'combinatoire', 'mathématiques', 'probabilité',

    # 计算机科学核心和通用术语
    '信息学', '计算', '运算', '信息', '信息技术', '信息通信技术',
    '数字', '系统', '系统们', '技术', '技术们', '网络安全',

    # 数据科学与机器学习
    '数据', '数据集', '数据集们', '数据库', '挖掘', '聚类', '分类',
    '回归', '预测', '分析学', '分析', '模型', '模型们', '建模',
    '统计', '统计的', '估计', '预处理', '清洗', '归一化',
    '相关性', '因果关系', '离群值', '变换', '维度',

    # 人工智能
    '人工智能', '机器学习', '深度学习', '大语言模型', 'gpt', 'chatgpt',
    'bert', 'transformer', '神经', '网络', '网络们', '卷积神经网络',
    '循环神经网络', '生成对抗网络', '自动编码器', '监督学习', '无监督学习',
    '强化学习', '机器人', '机器人学', '代理', '代理们', '学习', '智能',

    # 编程与编码
    '编程', '代码', '编码', '脚本', '脚本们', '开发者', '工程',
    '软件', '硬件', '编译器', '调试', '调试中', '部署', 'devops',
    'ide', 'api', '后端', '前端', '全栈', '代码库', 'git', 'github',
    'bitbucket', 'vscode', 'intellij', 'java', 'python', 'r', 'cpp',
    'csharp', 'javascript', 'typescript', 'matlab', 'bash', 'shell',
    '笔记本', 'jupyter',

    # 数据库与存储
    '数据库', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle', 'mongodb',
    'redis', 'elasticsearch', '查询', '查询们', '模式', '表格',
    '事务', '索引', '索引化', '视图', '触发器', '连接', 'etl',
    '数据仓库', '数据湖', '数据流',

    # 软件开发方法论
    '面向对象', '对象', '对象们', '敏捷', 'scrum', 'devops', '冲刺',
    '工作流', '版本', '控制', 'ci', 'cd', '管道', '发布', '问题',
    '测试', '单元', '集成', '覆盖率', '重构',

    # Web、移动开发、UI/UX
    '界面', '界面们', 'ui', 'ux', '前端', 'react', 'vue', 'html', 'css',
    'bootstrap', 'tailwind', '移动端', 'android', 'ios', 'flutter',
    '设计', '线框图', '原型', '可访问性', '可用性', '用户', '导航',

    # 人机交互与认知计算
    '人机交互', '交互', '交互们', '手势', '触摸', '眼动追踪',
    '脑机接口', '脑-计算机', '语音', '语音', '自然', '自然语言处理',
    '文本', '语言', '情感', '对话', '聊天机器人', '多模态', '认知',
    '注意力',

    # 网络安全与区块链
    '安全', '网络安全', '加密', '解密', '密码学', '加密货币',
    '哈希', '令牌', '身份验证', '授权', '网络钓鱼', '恶意软件',
    '区块链', '比特币', '以太坊', '智能合约', '智能', '合约', '钱包',

    # 云计算与基础设施
    '云', 'aws', 'azure', 'gcp', '虚拟化', 'kubernetes', 'docker',
    '容器', '无服务器', 'lambda', '基础设施', '可扩展性', '可用性',
    '延迟', '吞吐量', '负载均衡器', '微服务', 'api网关',

    # 网络
    '网络', '网络们', 'tcp', 'ip', 'tcp/ip', 'udp', '协议', '延迟',
    '带宽', '路由器', 'dns', 'dnssec', '防火墙', 'vpn', '套接字',
    '互联网', '网页', 'cloudflare',

    # 跨学科信息学应用
    '生物信息学', '化学信息学', '健康信息学', '临床信息学',
    '神经信息学', '金融科技', '法律科技', '政务科技',
    '电子健康', '电子政务', '智慧城市', '物联网', '传感器',
    '可穿戴设备', '区块链', '边缘计算', '雾计算', '工业4.0',

    # 伦理、法规与社会影响
    '偏差', '公平性', '责任', '隐私', '通用数据保护条例',
    '可解释性', '透明度', '可信', '安全性', '虚假信息',
    '深度伪造', '监控', '伦理',

    # 理论与形式方法
    '自动机', '图灵', '机器', '图', '图们', '理论', '逻辑', '布尔',
    '证明', '定理', '离散', '代数', '复杂性', 'np', 'p', '近似',
    '归约', '优化', '组合数学', '数学', '概率',

    # Основные и общие термины информатики
    'информатика', 'вычисления', 'компьютинг', 'информация', 'ит', 'икт',
    'цифровой', 'система', 'системы', 'технология', 'технологии', 'кибер',

    # Data science и машинное обучение
    'данные', 'набор данных', 'наборы данных', 'базы данных', 'добыча данных',
    'кластеризация', 'классификация', 'регрессия', 'прогнозирование', 'аналитика',
    'анализ', 'модель', 'модели', 'моделирование', 'статистика', 'статистический',
    'оценка', 'предобработка', 'очистка', 'нормализация', 'корреляция',
    'причинность', 'выбросы', 'преобразование', 'размерность',

    # Искусственный интеллект
    'ии', 'мл', 'гл', 'бяп', 'гпт', 'чатгпт', 'берт', 'трансформер', 'нейронный',
    'сеть', 'сети', 'снс', 'рнс', 'ган', 'автокодировщик', 'супервизируемое',
    'несупервизируемое', 'обучение с подкреплением', 'робот', 'робототехника',
    'агент', 'агенты', 'обучение', 'интеллект',

    # Программирование и кодирование
    'программирование', 'код', 'кодирование', 'скрипт', 'скрипты', 'разработчик',
    'инжиниринг', 'программное обеспечение', 'аппаратное обеспечение',
    'компилятор', 'отладка', 'отлаживание', 'развертывание', 'девопс',
    'иде', 'апи', 'бэкенд', 'фронтенд', 'фуллстек', 'репозиторий', 'гит',
    'гитхаб', 'битбакет', 'vscode', 'intellij', 'java', 'python', 'r', 'cpp',
    'csharp', 'javascript', 'typescript', 'matlab', 'bash', 'shell', 'ноутбук',
    'jupyter',

    # Базы данных и хранилища
    'база данных', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle', 'mongodb',
    'redis', 'elasticsearch', 'запрос', 'запросы', 'схема', 'таблицы',
    'транзакция', 'индекс', 'индексация', 'представление', 'триггер', 'соединение',
    'etl', 'хранилище данных', 'озеро данных', 'поток данных',

    # Методологии разработки ПО
    'ооп', 'объект', 'объекты', 'agile', 'scrum', 'devops', 'спринт', 'рабочий процесс',
    'версия', 'контроль', 'ci', 'cd', 'конвейер', 'релиз', 'проблема', 'тестирование',
    'юнит', 'интеграция', 'покрытие', 'рефакторинг',

    # Веб, мобильная разработка, UI/UX
    'интерфейс', 'интерфейсы', 'ui', 'ux', 'фронтенд', 'react', 'vue', 'html', 'css',
    'bootstrap', 'tailwind', 'мобильный', 'android', 'ios', 'flutter', 'дизайн',
    'каркас', 'прототип', 'доступность', 'удобство использования', 'пользователь',
    'навигация',

    # Взаимодействие человека и компьютера, когнитивные технологии
    'ввк', 'взаимодействие', 'взаимодействия', 'жест', 'прикосновение',
    'отслеживание взгляда', 'вчи', 'мозг-компьютер', 'голос', 'речь', 'естественный',
    'нлп', 'текст', 'язык', 'эмоции', 'диалог', 'чат-бот', 'мультимодальный',
    'когнитивный', 'внимание',

    # Кибербезопасность и блокчейн
    'безопасность', 'кибербезопасность', 'шифрование', 'дешифрование',
    'криптография', 'крипто', 'хеш', 'токен', 'аутентификация', 'авторизация',
    'фишинг', 'вредоносное ПО', 'блокчейн', 'биткойн', 'эфириум',
    'смартконтракт', 'смарт', 'контракт', 'кошелек',

    # Облачные вычисления и инфраструктура
    'облако', 'aws', 'azure', 'gcp', 'виртуализация', 'kubernetes', 'docker',
    'контейнер', 'безсерверный', 'lambda', 'инфраструктура', 'масштабируемость',
    'доступность', 'задержка', 'пропускная способность', 'балансировщик нагрузки',
    'микросервисы', 'api-шлюз',

    # Сетевые технологии
    'сеть', 'сети', 'tcp', 'ip', 'tcp/ip', 'udp', 'протокол', 'задержка',
    'пропускная способность', 'маршрутизатор', 'dns', 'dnssec', 'фаервол', 'vpn',
    'сокет', 'интернет', 'веб', 'cloudflare',

    # Междисциплинарные применения информатики
    'биоинформатика', 'хемоинформатика', 'медицинская информатика',
    'клиническая информатика', 'нейроинформатика', 'финтех', 'легалтех',
    'говтех', 'электронное здравоохранение', 'электронное правительство',
    'умный город', 'iot', 'сенсор', 'носимое устройство', 'блокчейн',
    'периферия', 'туманные вычисления', 'индустрия4.0',

    # Этика, регулирование, социальное воздействие
    'предвзятость', 'справедливость', 'ответственность', 'конфиденциальность',
    'gdpr', 'объяснимый', 'прозрачность', 'доверие', 'безопасность',
    'дезинформация', 'дипфейк', 'наблюдение', 'этика',

    # Теория и формальные методы
    'автомат', 'тюринг', 'машина', 'граф', 'графы', 'теория', 'логика',
    'булев', 'доказательство', 'теорема', 'дискретный', 'алгебра',
    'сложность', 'np', 'p', 'аппроксимация', 'сведение', 'оптимизация',
    'комбинаторика', 'математика', 'вероятность',

    # 情報科学の基本および一般用語
    '情報学', 'コンピューティング', '計算', '情報', 'it', 'ict',
    'デジタル', 'システム', 'システム群', '技術', '技術群', 'サイバー',

    # データサイエンスと機械学習
    'データ', 'データセット', 'データセット群', 'データベース', 'マイニング',
    'クラスタリング', '分類', '回帰', '予測', 'アナリティクス', '分析',
    'モデル', 'モデル群', 'モデリング', '統計', '統計的', '推定',
    '前処理', 'クリーニング', '正規化', '相関', '因果関係', '外れ値',
    '変換', '次元性',

    # 人工知能
    'ai', 'ml', 'dl', 'llm', 'gpt', 'chatgpt', 'bert', 'トランスフォーマー',
    'ニューラル', 'ネットワーク', 'ネットワーク群', 'cnn', 'rnn', 'gan',
    'オートエンコーダー', '教師あり', '教師なし', '強化学習',
    'ロボット', 'ロボティクス', 'エージェント', 'エージェント群',
    '学習', 'インテリジェンス',

    # プログラミングとコーディング
    'プログラミング', 'コード', 'コーディング', 'スクリプト', 'スクリプト群',
    '開発者', 'エンジニアリング', 'ソフトウェア', 'ハードウェア', 'コンパイラ',
    'デバッグ', 'デバッグ中', 'デプロイメント', 'devops', 'ide', 'api',
    'バックエンド', 'フロントエンド', 'フルスタック', 'リポジトリ',
    'git', 'github', 'bitbucket', 'vscode', 'intellij', 'java',
    'python', 'r', 'cpp', 'csharp', 'javascript', 'typescript',
    'matlab', 'bash', 'シェル', 'ノートブック', 'jupyter',

    # データベースとストレージ
    'データベース', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle',
    'mongodb', 'redis', 'elasticsearch', 'クエリ', 'クエリ群', 'スキーマ',
    'テーブル', 'トランザクション', 'インデックス', 'インデキシング',
    'ビュー', 'トリガー', 'ジョイン', 'etl', 'データウェアハウス',
    'データレイク', 'データフロー',

    # ソフトウェア開発手法
    'oop', 'オブジェクト', 'オブジェクト群', 'アジャイル', 'スクラム', 'devops',
    'スプリント', 'ワークフロー', 'バージョン', 'コントロール', 'ci', 'cd',
    'パイプライン', 'リリース', '課題', 'テスト', 'ユニット', '統合', 'カバレッジ',
    'リファクタリング',

    # Web・モバイル・UI/UX
    'インターフェース', 'インターフェース群', 'ui', 'ux', 'フロントエンド',
    'react', 'vue', 'html', 'css', 'bootstrap', 'tailwind',
    'モバイル', 'android', 'ios', 'flutter', 'デザイン', 'ワイヤーフレーム',
    'プロトタイプ', 'アクセシビリティ', 'ユーザビリティ', 'ユーザー', 'ナビゲーション',

    # HCIと認知コンピューティング
    'hci', 'インタラクション', 'インタラクション群', 'ジェスチャー',
    'タッチ', 'アイトラッキング', 'bci', 'ブレインコンピュータ',
    '音声', 'スピーチ', '自然', 'nlp', 'テキスト', '言語',
    '感情分析', '対話', 'チャットボット', 'マルチモーダル', '認知',
    '注意',

    # サイバーセキュリティとブロックチェーン
    'セキュリティ', 'サイバーセキュリティ', '暗号化', '復号化',
    '暗号技術', '暗号資産', 'ハッシュ', 'トークン', '認証',
    '認可', 'フィッシング', 'マルウェア', 'ブロックチェーン',
    'ビットコイン', 'イーサリアム', 'スマートコントラクト', 'スマート',
    '契約', 'ウォレット',

    # クラウドコンピューティングとインフラ
    'クラウド', 'aws', 'azure', 'gcp', '仮想化', 'kubernetes', 'docker',
    'コンテナ', 'サーバーレス', 'lambda', 'インフラストラクチャ',
    'スケーリング', '可用性', 'レイテンシ', 'スループット',
    'ロードバランサー', 'マイクロサービス', 'apiゲートウェイ',

    # ネットワーキング
    'ネットワーク', 'ネットワーク群', 'tcp', 'ip', 'tcp/ip', 'udp',
    'プロトコル', 'レイテンシ', '帯域幅', 'ルーター', 'dns', 'dnssec',
    'ファイアウォール', 'vpn', 'ソケット', 'インターネット', 'ウェブ', 'cloudflare',

    # 学際的応用
    'バイオインフォマティクス', 'ケモインフォマティクス', 'ヘルスインフォマティクス',
    'クリニカルインフォマティクス', 'ニューロインフォマティクス', 'フィンテック',
    'リーガルテック', 'ガブテック', 'eヘルス', 'eガバメント', 'スマートシティ',
    'iot', 'センサー', 'ウェアラブル', 'ブロックチェーン', 'エッジ', 'フォグ',
    'インダストリー4.0',

    # 倫理・規制・社会的影響
    'バイアス', '公正性', '説明責任', 'プライバシー', 'gdpr',
    '説明可能', '透明性', '信頼性', '安全性', '誤報', 'ディープフェイク',
    '監視', '倫理',

    # 理論と形式的手法
    'オートマトン', 'チューリング', 'マシン', 'グラフ', 'グラフ群',
    '理論', '論理', 'ブール', '証明', '定理', '離散', '代数',
    '計算複雑性', 'np', 'p', '近似', '還元', '最適化', '組合せ論',
    '数学', '確率',

    # 컴퓨터 과학의 핵심 및 일반 용어
    '정보학', '컴퓨팅', '연산', '정보', 'it', 'ict',
    '디지털', '시스템', '시스템들', '기술', '기술들', '사이버',

    # 데이터 과학 및 머신러닝
    '데이터', '데이터셋', '데이터셋들', '데이터베이스', '마이닝',
    '클러스터링', '분류', '회귀', '예측', '분석학', '분석',
    '모델', '모델들', '모델링', '통계', '통계적', '추정',
    '전처리', '정리', '정규화', '상관관계', '인과관계',
    '이상치', '변환', '차원',

    # 인공지능
    'ai', 'ml', 'dl', 'llm', 'gpt', 'chatgpt', 'bert',
    '트랜스포머', '신경망', '네트워크', '네트워크들', 'cnn',
    'rnn', 'gan', '오토인코더', '지도학습', '비지도학습',
    '강화학습', '로봇', '로보틱스', '에이전트', '에이전트들',
    '학습', '지능',

    # 프로그래밍 및 코딩
    '프로그래밍', '코드', '코딩', '스크립트', '스크립트들',
    '개발자', '엔지니어링', '소프트웨어', '하드웨어', '컴파일러',
    '디버그', '디버깅', '배포', 'devops', 'ide', 'api',
    '백엔드', '프론트엔드', '풀스택', '저장소', 'git', 'github',
    'bitbucket', 'vscode', 'intellij', 'java', 'python', 'r',
    'cpp', 'csharp', 'javascript', 'typescript', 'matlab',
    'bash', '셸', '노트북', 'jupyter',

    # 데이터베이스 및 저장소
    '데이터베이스', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle',
    'mongodb', 'redis', 'elasticsearch', '쿼리', '쿼리들', '스키마',
    '테이블', '트랜잭션', '인덱스', '인덱싱', '뷰', '트리거',
    '조인', 'etl', '데이터웨어하우스', '데이터레이크', '데이터플로우',

    # 소프트웨어 개발 방법론
    'oop', '객체', '객체들', '애자일', '스크럼', 'devops',
    '스프린트', '워크플로우', '버전', '버전관리', 'ci', 'cd',
    '파이프라인', '릴리스', '이슈', '테스트', '단위', '통합',
    '커버리지', '리팩토링',

    # 웹, 모바일, UI/UX
    '인터페이스', '인터페이스들', 'ui', 'ux', '프론트엔드',
    'react', 'vue', 'html', 'css', 'bootstrap', 'tailwind',
    '모바일', 'android', 'ios', 'flutter', '디자인', '와이어프레임',
    '프로토타입', '접근성', '사용성', '사용자', '내비게이션',

    # 인간-컴퓨터 상호작용 및 인지 컴퓨팅
    'hci', '상호작용', '상호작용들', '제스처', '터치', '시선 추적',
    'bci', '브레인컴퓨터', '음성', '음성합성', '자연어', 'nlp',
    '텍스트', '언어', '감성', '대화', '챗봇', '멀티모달',
    '인지', '주의',

    # 사이버보안 및 블록체인
    '보안', '사이버보안', '암호화', '복호화', '암호학', '암호화폐',
    '해시', '토큰', '인증', '인가', '피싱', '악성코드',
    '블록체인', '비트코인', '이더리움', '스마트계약', '스마트',
    '계약', '지갑',

    # 클라우드 컴퓨팅 및 인프라
    '클라우드', 'aws', 'azure', 'gcp', '가상화', 'kubernetes',
    'docker', '컨테이너', '서버리스', 'lambda', '인프라',
    '스케일링', '가용성', '지연시간', '처리량',
    '로드밸런서', '마이크로서비스', 'api게이트웨이',

    # 네트워킹
    '네트워크', '네트워크들', 'tcp', 'ip', 'tcp/ip', 'udp',
    '프로토콜', '지연', '대역폭', '라우터', 'dns', 'dnssec',
    '방화벽', 'vpn', '소켓', '인터넷', '웹', 'cloudflare',

    # 융합 정보학 응용
    '생물정보학', '화학정보학', '건강정보학', '임상정보학',
    '신경정보학', '핀테크', '리걸테크', '거버먼트테크',
    'e헬스', '전자정부', '스마트시티', 'iot', '센서',
    '웨어러블', '블록체인', '엣지', '포그', '산업4.0',

    # 윤리, 규제, 사회적 영향
    '편향', '공정성', '책임', '개인정보', 'gdpr', '설명가능한',
    '투명성', '신뢰성', '안전성', '허위정보', '딥페이크', '감시',
    '윤리',

    # 이론 및 형식적 방법
    '오토마타', '튜링', '기계', '그래프', '그래프들', '이론',
    '논리', '불리언', '증명', '정리', '이산', '대수',
    '복잡도', 'np', 'p', '근사화', '환원', '최적화',
    '조합론', '수학', '확률',

    # 資訊科學核心與通用術語
    '資訊學', '運算', '計算', '資訊', 'it', 'ict',
    '數位', '系統', '系統們', '技術', '技術們', '網路安全',

    # 數據科學與機器學習
    '資料', '資料集', '資料集們', '資料庫', '資料探勘', '分群', '分類',
    '迴歸', '預測', '分析', '分析', '模型', '模型們', '建模',
    '統計', '統計的', '估計', '前處理', '清理', '正規化',
    '相關性', '因果關係', '離群值', '轉換', '維度',

    # 人工智慧
    'ai', 'ml', 'dl', 'llm', 'gpt', 'chatgpt', 'bert',
    'transformer', '神經', '網路', '網路們', 'cnn', 'rnn', 'gan',
    '自編碼器', '監督式', '非監督式', '強化學習', '機器人', '機器人技術',
    '代理人', '代理們', '學習', '智慧',

    # 程式設計與編碼
    '程式設計', '程式碼', '編碼', '腳本', '腳本們', '開發者',
    '工程', '軟體', '硬體', '編譯器', '除錯', '偵錯', '部署',
    'devops', 'ide', 'api', '後端', '前端', '全端', '程式庫',
    'git', 'github', 'bitbucket', 'vscode', 'intellij', 'java',
    'python', 'r', 'cpp', 'csharp', 'javascript', 'typescript',
    'matlab', 'bash', 'shell', '筆記本', 'jupyter',

    # 資料庫與儲存
    '資料庫', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle',
    'mongodb', 'redis', 'elasticsearch', '查詢', '查詢們', '結構描述',
    '表格', '交易', '索引', '索引化', '檢視', '觸發器',
    '連接', 'etl', '數據倉儲', '資料湖', '資料流',

    # 軟體開發方法
    'oop', '物件', '物件們', '敏捷', 'scrum', 'devops', '衝刺',
    '工作流程', '版本', '控制', 'ci', 'cd', '管線', '發佈',
    '問題', '測試', '單元', '整合', '涵蓋率', '重構',

    # 網頁、行動、UI/UX
    '介面', '介面們', 'ui', 'ux', '前端', 'react', 'vue', 'html',
    'css', 'bootstrap', 'tailwind', '行動', 'android', 'ios',
    'flutter', '設計', '線框圖', '原型', '無障礙', '可用性',
    '使用者', '導覽',

    # 人機互動與認知運算
    'hci', '互動', '互動們', '手勢', '觸控', '眼動追蹤',
    'bci', '腦機介面', '語音', '語音', '自然', 'nlp', '文字',
    '語言', '情感', '對話', '聊天機器人', '多模態', '認知',
    '注意力',

    # 資安與區塊鏈
    '安全', '資安', '加密', '解密', '密碼學', '加密貨幣',
    '雜湊', '代幣', '身份驗證', '授權', '網路釣魚', '惡意程式',
    '區塊鏈', '比特幣', '以太幣', '智慧合約', '智慧', '合約', '錢包',

    # 雲端運算與基礎設施
    '雲端', 'aws', 'azure', 'gcp', '虛擬化', 'kubernetes', 'docker',
    '容器', '無伺服器', 'lambda', '基礎設施', '擴展性', '可用性',
    '延遲', '吞吐量', '負載平衡器', '微服務', 'api閘道',

    # 網路技術
    '網路', '網路們', 'tcp', 'ip', 'tcp/ip', 'udp', '協定',
    '延遲', '頻寬', '路由器', 'dns', 'dnssec', '防火牆',
    'vpn', 'socket', '網際網路', '網頁', 'cloudflare',

    # 跨領域資訊應用
    '生物資訊學', '化學資訊學', '健康資訊學', '臨床資訊學',
    '神經資訊學', '金融科技', '法律科技', '政府科技',
    '電子健康', '電子政務', '智慧城市', 'iot', '感測器',
    '穿戴裝置', '區塊鏈', '邊緣運算', '霧運算', '工業4.0',

    # 倫理、法規與社會影響
    '偏誤', '公平性', '責任', '隱私', 'gdpr', '可解釋的',
    '透明度', '可信任', '安全性', '錯誤資訊', '深偽', '監控',
    '倫理',

    # 理論與形式方法
    '自動機', '圖靈', '機器', '圖形', '圖形們', '理論', '邏輯',
    '布林', '證明', '定理', '離散', '代數', '複雜度',
    'np', 'p', '近似', '化簡', '最佳化', '組合數學',
    '數學', '機率',

    # Základní a obecné pojmy z informatiky
    'informatika', 'výpočetní technika', 'výpočty', 'informace', 'it', 'ikt',
    'digitální', 'systém', 'systémy', 'technologie', 'technologie (mn.)', 'kyber',

    # Data science a strojové učení
    'data', 'datová sada', 'datové sady', 'databáze', 'dolování dat',
    'shlukování', 'klasifikace', 'regrese', 'predikce', 'analytika',
    'analýza', 'model', 'modely', 'modelování', 'statistika', 'statistický',
    'odhad', 'předzpracování', 'čištění', 'normalizace', 'korelace',
    'kauzalita', 'odlehlé hodnoty', 'transformace', 'dimenzionalita',

    # Umělá inteligence
    'ai', 'ml', 'dl', 'llm', 'gpt', 'chatgpt', 'bert', 'transformátor',
    'neuronová', 'síť', 'sítě', 'cnn', 'rnn', 'gan', 'autoenkodér',
    'supervidované', 'ne-supervidované', 'posilované učení',
    'robot', 'robotika', 'agent', 'agenti', 'učení', 'inteligence',

    # Programování a kódování
    'programování', 'kód', 'kódování', 'skript', 'skripty', 'vývojář',
    'inženýrství', 'software', 'hardware', 'kompilátor', 'ladění',
    'odlaďování', 'nasazení', 'devops', 'ide', 'api', 'backend',
    'frontend', 'fullstack', 'repozitář', 'git', 'github', 'bitbucket',
    'vscode', 'intellij', 'java', 'python', 'r', 'cpp', 'csharp',
    'javascript', 'typescript', 'matlab', 'bash', 'shell', 'notebook',
    'jupyter',

    # Databáze a úložiště
    'databáze', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle', 'mongodb',
    'redis', 'elasticsearch', 'dotaz', 'dotazy', 'schéma', 'tabulky',
    'transakce', 'index', 'indexování', 'zobrazení', 'spouštěč',
    'spojení', 'etl', 'datový sklad', 'datové jezero', 'tok dat',

    # Metodiky vývoje softwaru
    'oop', 'objekt', 'objekty', 'agilní', 'scrum', 'devops', 'sprint',
    'pracovní tok', 'verze', 'řízení', 'ci', 'cd', 'pipeline',
    'vydání', 'problém', 'testování', 'jednotka', 'integrace',
    'pokrytí', 'refaktoring',

    # Web, mobilní vývoj, UI/UX
    'rozhraní', 'rozhraní (mn.)', 'ui', 'ux', 'frontend', 'react',
    'vue', 'html', 'css', 'bootstrap', 'tailwind', 'mobilní',
    'android', 'ios', 'flutter', 'design', 'wireframe', 'prototyp',
    'přístupnost', 'použitelnost', 'uživatel', 'navigace',

    # HCI a kognitivní výpočty
    'hci', 'interakce', 'interakce (mn.)', 'gesto', 'dotek',
    'sledování očí', 'bci', 'mozek-počítač', 'hlas', 'řeč', 'přirozený',
    'nlp', 'text', 'jazyk', 'sentiment', 'dialog', 'chatbot',
    'multimodální', 'kognitivní', 'pozornost',

    # Kyberbezpečnost a blockchain
    'bezpečnost', 'kyberbezpečnost', 'šifrování', 'dešifrování',
    'kryptografie', 'krypto', 'hash', 'token', 'autentizace',
    'autorizace', 'phishing', 'malware', 'blockchain', 'bitcoin',
    'ethereum', 'chytrý kontrakt', 'chytrý', 'kontrakt', 'peněženka',

    # Cloud computing a infrastruktura
    'cloud', 'aws', 'azure', 'gcp', 'virtualizace', 'kubernetes',
    'docker', 'kontejner', 'bezserverové', 'lambda', 'infrastruktura',
    'škálování', 'dostupnost', 'latence', 'propustnost',
    'vyvažování zátěže', 'mikroslužby', 'api-brána',

    # Sítě a komunikace
    'síť', 'sítě', 'tcp', 'ip', 'tcp/ip', 'udp', 'protokol',
    'latence', 'šířka pásma', 'router', 'dns', 'dnssec',
    'firewall', 'vpn', 'socket', 'internet', 'web', 'cloudflare',

    # Interdisciplinární aplikace informatiky
    'bioinformatika', 'chemoformatika', 'zdravotní informatika',
    'klinická informatika', 'neuroinformatika', 'fintech',
    'legaltech', 'govtech', 'ehealth', 'egov', 'chytré město',
    'iot', 'senzor', 'nositelné zařízení', 'blockchain',
    'edge', 'mlha', 'průmysl4.0',

    # Etika, regulace, společenský dopad
    'zkreslení', 'spravedlnost', 'odpovědnost', 'soukromí', 'gdpr',
    'vysvětlitelný', 'transparentnost', 'důvěryhodný',
    'bezpečnost', 'dezinformace', 'deepfake', 'dohled', 'etika',

    # Teoretické a formální přístupy
    'automaty', 'turing', 'stroj', 'graf', 'grafy', 'teorie',
    'logika', 'booleovský', 'důkaz', 'věta', 'diskrétní',
    'algebra', 'složitost', 'np', 'p', 'aproximace', 'redukce',
    'optimalizace', 'kombinatorika', 'matematika', 'pravděpodobnost',

    # المصطلحات الأساسية والعامة في علوم الحاسوب
    'المعلوماتية', 'الحوسبة', 'الاحتساب', 'المعلومات', 'تكنولوجيا المعلومات', 'تكنولوجيا المعلومات والاتصالات',
    'رقمي', 'نظام', 'أنظمة', 'تكنولوجيا', 'تكنولوجيات', 'الأمن السيبراني',

    # علم البيانات وتعلم الآلة
    'بيانات', 'مجموعة بيانات', 'مجموعات بيانات', 'قواعد بيانات', 'التنقيب', 'التجميع', 'التصنيف',
    'الانحدار', 'التنبؤ', 'التحليلات', 'التحليل', 'نموذج', 'نماذج', 'النمذجة',
    'إحصاء', 'إحصائي', 'تقدير', 'المعالجة المسبقة', 'تنظيف', 'التطبيع',
    'الارتباط', 'السببية', 'القيم الشاذة', 'التحول', 'البعدية',

    # الذكاء الاصطناعي
    'ذكاء اصطناعي', 'تعلم آلي', 'تعلم عميق', 'نماذج لغوية كبيرة', 'gpt', 'chatgpt', 'bert',
    'محول', 'عصبي', 'شبكة', 'شبكات', 'cnn', 'rnn', 'gan',
    'المشفّر التلقائي', 'خاضع للإشراف', 'غير خاضع للإشراف', 'تعلم التعزيز',
    'روبوت', 'روبوتات', 'عامل', 'عوامل', 'تعلم', 'ذكاء',

    # البرمجة والترميز
    'برمجة', 'كود', 'ترميز', 'سكريبت', 'سكريبتات', 'مطور',
    'هندسة', 'برمجيات', 'أجهزة', 'مُصرّف', 'تصحيح الأخطاء', 'تصحيح', 'نشر',
    'devops', 'بيئة تطوير', 'واجهة برمجة التطبيقات', 'الخلفية', 'الواجهة الأمامية', 'شامل',
    'مستودع', 'git', 'github', 'bitbucket', 'vscode', 'intellij', 'java',
    'python', 'r', 'cpp', 'csharp', 'javascript', 'typescript',
    'matlab', 'bash', 'شل', 'دفتر ملاحظات', 'jupyter',

    # قواعد البيانات والتخزين
    'قاعدة بيانات', 'sql', 'nosql', 'postgresql', 'mysql', 'oracle', 'mongodb',
    'redis', 'elasticsearch', 'استعلام', 'استعلامات', 'مخطط', 'جداول',
    'معاملة', 'فهرس', 'فهرسة', 'عرض', 'مشغل', 'انضمام', 'etl',
    'مخزن البيانات', 'بحيرة البيانات', 'تدفق البيانات',

    # منهجيات تطوير البرمجيات
    'البرمجة الكائنية', 'كائن', 'كائنات', 'أجايل', 'scrum', 'devops', 'سبرينت',
    'سير العمل', 'إصدار', 'تحكم', 'تكامل مستمر', 'تسليم مستمر', 'خط أنابيب',
    'إصدار', 'قضية', 'اختبار', 'وحدة', 'تكامل', 'تغطية', 'إعادة هيكلة',

    # الويب، الهاتف المحمول، واجهات المستخدم
    'واجهة', 'واجهات', 'ui', 'ux', 'الواجهة الأمامية', 'react', 'vue', 'html',
    'css', 'bootstrap', 'tailwind', 'الهاتف المحمول', 'android', 'ios',
    'flutter', 'تصميم', 'سلكي تخطيطي', 'نموذج أولي', 'إمكانية الوصول',
    'قابلية الاستخدام', 'مستخدم', 'التنقل',

    # التفاعل بين الإنسان والحاسوب والحوسبة المعرفية
    'hci', 'تفاعل', 'تفاعلات', 'إيماءة', 'لمس', 'تتبع العين',
    'bci', 'واجهة الدماغ والحاسوب', 'صوت', 'كلام', 'طبيعي', 'nlp',
    'نص', 'لغة', 'مشاعر', 'حوار', 'روبوت محادثة', 'متعدد الوسائط',
    'معرفي', 'الاهتمام',

    # الأمن السيبراني والبلوك تشين
    'أمان', 'الأمن السيبراني', 'تشفير', 'فك التشفير', 'علم التشفير', 'العملات الرقمية',
    'تجزئة', 'رمز', 'مصادقة', 'تفويض', 'تصيد', 'برمجيات خبيثة',
    'سلسلة الكتل', 'بيتكوين', 'إيثيريوم', 'عقد ذكي', 'ذكي', 'عقد', 'محفظة',

    # الحوسبة السحابية والبنية التحتية
    'سحابة', 'aws', 'azure', 'gcp', 'الافتراضية', 'kubernetes', 'docker',
    'حاوية', 'بدون خادم', 'lambda', 'البنية التحتية', 'التحجيم', 'التوافر',
    'الكمون', 'معدل النقل', 'موازن التحميل', 'الخدمات المصغرة', 'بوابة api',

    # الشبكات
    'شبكة', 'شبكات', 'tcp', 'ip', 'tcp/ip', 'udp', 'بروتوكول',
    'كمون', 'عرض النطاق', 'موجه', 'dns', 'dnssec', 'جدار حماية',
    'vpn', 'مقبس', 'الإنترنت', 'ويب', 'cloudflare',

    # التطبيقات متعددة التخصصات في المعلوماتية
    'المعلوماتية الحيوية', 'الكيمياء المعلوماتية', 'معلوماتية صحية',
    'معلوماتية سريرية', 'معلوماتية عصبية', 'فينتك', 'ليجالتك',
    'جوفتيك', 'الصحة الإلكترونية', 'الحكومة الإلكترونية',
    'المدينة الذكية', 'انترنت الأشياء', 'مستشعر', 'قابل للارتداء',
    'بلوك تشين', 'الحوسبة الطرفية', 'الحوسبة الضبابية', 'الصناعة4.0',

    # الأخلاقيات، التنظيم، الأثر المجتمعي
    'تحيز', 'العدالة', 'المساءلة', 'الخصوصية', 'gdpr', 'قابل للتفسير',
    'الشفافية', 'موثوق', 'السلامة', 'المعلومات المضللة',
    'التزييف العميق', 'مراقبة', 'أخلاقيات',

    # النظري والشكلاني
    'الآلات المحدودة', 'تورنج', 'آلة', 'رسم بياني', 'رسوم بيانية',
    'نظرية', 'منطق', 'قيمي منطقي', 'إثبات', 'مبرهنة',
    'منفصل', 'جبر', 'تعقيد', 'np', 'p', 'تقريب', 'اختزال',
    'تحسين', 'التوافقيات', 'رياضيات', 'احتمالية'

    ],

    'psychology': [
    # General terms
    'psychology', 'psychological', 'psychologist', 'mental', 'mind', 'behavior', 'behaviour',
    'emotion', 'emotions', 'affect', 'affective', 'mood', 'cognition', 'cognitive',
    'thinking', 'thought', 'reasoning', 'intelligence', 'iq', 'perception', 'awareness',
    'consciousness', 'unconscious', 'learning', 'memory', 'attention', 'focus',

    # Disorders & conditions
    'depression', 'anxiety', 'stress', 'bipolar', 'ptsd', 'ocd', 'adhd', 'autism',
    'disorder', 'disorders', 'phobia', 'schizophrenia', 'delusion', 'hallucination',
    'paranoia', 'insomnia', 'burnout', 'trauma', 'self-harm', 'suicidal', 'addiction',
    'dependence', 'abuse', 'eating', 'anorexia', 'bulimia', 'dysregulation',

    # Clinical & therapeutic
    'therapy', 'therapist', 'psychotherapy', 'counseling', 'diagnosis', 'clinical',
    'psychiatry', 'psychiatric', 'intervention', 'treatment', 'session', 'mentalhealth',
    'care', 'support', 'healing', 'recovery',

    # Social psychology
    'social', 'group', 'groups', 'identity', 'influence', 'prejudice', 'bias',
    'conformity', 'obedience', 'aggression', 'altruism', 'empathy', 'interaction',
    'interpersonal', 'relationship', 'trust', 'cooperation', 'conflict',

    # Cognitive psychology & neuroscience
    'neuroscience', 'neural', 'brain', 'cortex', 'neuron', 'synapse', 'hippocampus',
    'amygdala', 'fmri', 'eeg', 'erp', 'scan', 'neuropsychology', 'processing',
    'stimulus', 'attention', 'cognitive', 'executive', 'decision', 'decisions',
    'problem', 'problem-solving', 'load', 'reaction',

    # Developmental & educational
    'development', 'developmental', 'infant', 'toddler', 'child', 'children', 'adolescence',
    'adolescent', 'youth', 'teen', 'teenager', 'youngadult', 'earlychildhood',
    'milestone', 'learningstyle', 'performance', 'motivation', 'school', 'student',

    # Personality & self
    'personality', 'trait', 'traits', 'temperament', 'character', 'self', 'identity',
    'selfesteem', 'selfconcept', 'introversion', 'extroversion', 'ego', 'shadow',
    'mbti', 'bigfive', 'openness', 'conscientiousness', 'agreeableness', 'neuroticism',

    # Work, industrial & organizational
    'work', 'job', 'employee', 'workplace', 'leader', 'leadership', 'motivation',
    'satisfaction', 'engagement', 'burnout', 'conflict', 'organizational', 'stress',
    'performance', 'productivity',

    # Positive psychology
    'wellbeing', 'resilience', 'happiness', 'mindfulness', 'coping', 'gratitude',
    'optimism', 'hope', 'meaning', 'purpose', 'flow', 'life', 'satisfaction',

    # Methods & tools
    'experiment', 'study', 'case', 'survey', 'interview', 'observation', 'scale',
    'test', 'assessment', 'measure', 'questionnaire', 'inventory', 'instrument',
    'validity', 'reliability', 'psychometrics', 'longitudinal', 'crosssectional',
    'meta', 'analysis', 'statistical', 'correlation',

    # Ethical & academic
    'ethics', 'consent', 'confidentiality', 'debriefing', 'participant',
    'bias', 'randomized', 'trial', 'rct', 'placebo', 'control', 'peerreview',

    # Términos generales
    'psicología', 'psicológico', 'psicólogo', 'mental', 'mente', 'comportamiento', 'conducta',
    'emoción', 'emociones', 'afecto', 'afectivo', 'estado de ánimo', 'cognición', 'cognitivo',
    'pensamiento', 'pensamiento', 'razonamiento', 'inteligencia', 'ci', 'percepción', 'conciencia',
    'consciencia', 'inconsciente', 'aprendizaje', 'memoria', 'atención', 'concentración',

    # Trastornos y condiciones
    'depresión', 'ansiedad', 'estrés', 'bipolaridad', 'ptsd', 'toc', 'tdah', 'autismo',
    'trastorno', 'trastornos', 'fobia', 'esquizofrenia', 'delirio', 'alucinación',
    'paranoia', 'insomnio', 'agotamiento', 'trauma', 'autolesión', 'suicida', 'adicción',
    'dependencia', 'abuso', 'alimentación', 'anorexia', 'bulimia', 'desregulación',

    # Clínico y terapéutico
    'terapia', 'terapeuta', 'psicoterapia', 'asesoría', 'diagnóstico', 'clínico',
    'psiquiatría', 'psiquiátrico', 'intervención', 'tratamiento', 'sesión', 'saludmental',
    'cuidado', 'apoyo', 'sanación', 'recuperación',

    # Psicología social
    'social', 'grupo', 'grupos', 'identidad', 'influencia', 'prejuicio', 'sesgo',
    'conformidad', 'obediencia', 'agresión', 'altruismo', 'empatía', 'interacción',
    'interpersonal', 'relación', 'confianza', 'cooperación', 'conflicto',

    # Psicología cognitiva y neurociencia
    'neurociencia', 'neuronal', 'cerebro', 'corteza', 'neurona', 'sinapsis', 'hipocampo',
    'amígdala', 'fmri', 'eeg', 'erp', 'escáner', 'neuropsicología', 'procesamiento',
    'estímulo', 'atención', 'cognitivo', 'ejecutivo', 'decisión', 'decisiones',
    'problema', 'resolución de problemas', 'carga', 'reacción',

    # Psicología del desarrollo y educativa
    'desarrollo', 'del desarrollo', 'bebé', 'niño pequeño', 'niño', 'niños', 'adolescencia',
    'adolescente', 'juventud', 'adolescente', 'adolescente', 'adultojoven', 'primerainfancia',
    'hito', 'estilo de aprendizaje', 'rendimiento', 'motivación', 'escuela', 'estudiante',

    # Personalidad y yo
    'personalidad', 'rasgo', 'rasgos', 'temperamento', 'carácter', 'yo', 'identidad',
    'autoestima', 'autoconcepto', 'introversión', 'extroversión', 'ego', 'sombra',
    'mbti', 'bigfive', 'apertura', 'responsabilidad', 'amabilidad', 'neuroticismo',

    # Psicología del trabajo y organizacional
    'trabajo', 'empleo', 'empleado', 'lugar de trabajo', 'líder', 'liderazgo', 'motivación',
    'satisfacción', 'compromiso', 'agotamiento', 'conflicto', 'organizacional', 'estrés',
    'rendimiento', 'productividad',

    # Psicología positiva
    'bienestar', 'resiliencia', 'felicidad', 'atención plena', 'afrontamiento', 'gratitud',
    'optimismo', 'esperanza', 'significado', 'propósito', 'fluidez', 'vida', 'satisfacción',

    # Métodos y herramientas
    'experimento', 'estudio', 'caso', 'encuesta', 'entrevista', 'observación', 'escala',
    'prueba', 'evaluación', 'medida', 'cuestionario', 'inventario', 'instrumento',
    'validez', 'fiabilidad', 'psicometría', 'longitudinal', 'transversal',
    'meta', 'análisis', 'estadístico', 'correlación',

    # Ética y academia
    'ética', 'consentimiento', 'confidencialidad', 'explicación', 'participante',
    'sesgo', 'aleatorizado', 'ensayo', 'eca', 'placebo', 'control', 'revisiónporpares',

    # Termos gerais
    'psicologia', 'psicológico', 'psicólogo', 'mental', 'mente', 'comportamento', 'comportamento',
    'emoção', 'emoções', 'afeto', 'afetivo', 'humor', 'cognição', 'cognitivo',
    'pensamento', 'pensamento', 'raciocínio', 'inteligência', 'qi', 'percepção', 'consciência',
    'consciência', 'inconsciente', 'aprendizagem', 'memória', 'atenção', 'foco',

    # Transtornos e condições
    'depressão', 'ansiedade', 'estresse', 'bipolar', 'ptsd', 'toc', 'tdah', 'autismo',
    'transtorno', 'transtornos', 'fobia', 'esquizofrenia', 'delírio', 'alucinação',
    'paranoia', 'insônia', 'esgotamento', 'trauma', 'automutilação', 'suicida', 'vício',
    'dependência', 'abuso', 'alimentação', 'anorexia', 'bulimia', 'desregulação',

    # Clínico e terapêutico
    'terapia', 'terapeuta', 'psicoterapia', 'aconselhamento', 'diagnóstico', 'clínico',
    'psiquiatria', 'psiquiátrico', 'intervenção', 'tratamento', 'sessão', 'saúdemental',
    'cuidados', 'apoio', 'cura', 'recuperação',

    # Psicologia social
    'social', 'grupo', 'grupos', 'identidade', 'influência', 'preconceito', 'viés',
    'conformidade', 'obediência', 'agressão', 'altruísmo', 'empatia', 'interação',
    'interpessoal', 'relacionamento', 'confiança', 'cooperação', 'conflito',

    # Psicologia cognitiva e neurociência
    'neurociência', 'neural', 'cérebro', 'córtex', 'neurônio', 'sinapse', 'hipocampo',
    'amígdala', 'fmri', 'eeg', 'erp', 'escaneamento', 'neuropsicologia', 'processamento',
    'estímulo', 'atenção', 'cognitivo', 'executivo', 'decisão', 'decisões',
    'problema', 'resolução de problemas', 'carga', 'reação',

    # Desenvolvimento e educação
    'desenvolvimento', 'desenvolvimental', 'bebê', 'criança pequena', 'criança', 'crianças', 'adolescência',
    'adolescente', 'juventude', 'adolescente', 'adolescente', 'jovemadulto', 'primeirainfância',
    'marco', 'estilo de aprendizagem', 'desempenho', 'motivação', 'escola', 'estudante',

    # Personalidade e o eu
    'personalidade', 'traço', 'traços', 'temperamento', 'caráter', 'eu', 'identidade',
    'autoestima', 'autoconceito', 'introversão', 'extroversão', 'ego', 'sombra',
    'mbti', 'bigfive', 'abertura', 'consciência', 'agradabilidade', 'neuroticismo',

    # Trabalho e organizacional
    'trabalho', 'emprego', 'empregado', 'local de trabalho', 'líder', 'liderança', 'motivação',
    'satisfação', 'engajamento', 'esgotamento', 'conflito', 'organizacional', 'estresse',
    'desempenho', 'produtividade',

    # Psicologia positiva
    'bem-estar', 'resiliência', 'felicidade', 'atenção plena', 'enfrentamento', 'gratidão',
    'otimismo', 'esperança', 'significado', 'propósito', 'fluxo', 'vida', 'satisfação',

    # Métodos e ferramentas
    'experimento', 'estudo', 'caso', 'pesquisa', 'entrevista', 'observação', 'escala',
    'teste', 'avaliação', 'medida', 'questionário', 'inventário', 'instrumento',
    'validade', 'confiabilidade', 'psicometria', 'longitudinal', 'transversal',
    'meta', 'análise', 'estatístico', 'correlação',

    # Ética e acadêmico
    'ética', 'consentimento', 'confidencialidade', 'explicação', 'participante',
    'viés', 'randomizado', 'ensaio', 'eca', 'placebo', 'controle', 'revisãopar',

    # Termini generali
    'psicologia', 'psicologico', 'psicologo', 'mentale', 'mente', 'comportamento', 'comportamento',
    'emozione', 'emozioni', 'affetto', 'affettivo', 'umore', 'cognizione', 'cognitivo',
    'pensiero', 'pensiero', 'ragionamento', 'intelligenza', 'qi', 'percezione', 'consapevolezza',
    'coscienza', 'inconscio', 'apprendimento', 'memoria', 'attenzione', 'concentrazione',

    # Disturbi e condizioni
    'depressione', 'ansia', 'stress', 'bipolare', 'ptsd', 'doc', 'adhd', 'autismo',
    'disturbo', 'disturbi', 'fobia', 'schizofrenia', 'delirio', 'allucinazione',
    'paranoia', 'insonnia', 'esaurimento', 'trauma', 'autolesionismo', 'suicida', 'dipendenza',
    'dipendenza', 'abuso', 'alimentazione', 'anoressia', 'bulimia', 'disregolazione',

    # Clinico e terapeutico
    'terapia', 'terapeuta', 'psicoterapia', 'consulenza', 'diagnosi', 'clinico',
    'psichiatria', 'psichiatrico', 'intervento', 'trattamento', 'sessione', 'salutementale',
    'cura', 'supporto', 'guarigione', 'recupero',

    # Psicologia sociale
    'sociale', 'gruppo', 'gruppi', 'identità', 'influenza', 'pregiudizio', 'pregiudizio',
    'conformità', 'obbedienza', 'aggressività', 'altruismo', 'empatia', 'interazione',
    'interpersonale', 'relazione', 'fiducia', 'cooperazione', 'conflitto',

    # Psicologia cognitiva e neuroscienze
    'neuroscienze', 'neurale', 'cervello', 'corteccia', 'neurone', 'sinapsi', 'ippocampo',
    'amigdala', 'fmri', 'eeg', 'erp', 'scansione', 'neuropsicologia', 'elaborazione',
    'stimolo', 'attenzione', 'cognitivo', 'esecutivo', 'decisione', 'decisioni',
    'problema', 'risoluzione dei problemi', 'carico', 'reazione',

    # Psicologia dello sviluppo e dell’educazione
    'sviluppo', 'dello sviluppo', 'neonato', 'bambino piccolo', 'bambino', 'bambini', 'adolescenza',
    'adolescente', 'gioventù', 'teenager', 'adolescente', 'giovane adulto', 'prima infanzia',
    'tappa evolutiva', 'stile di apprendimento', 'prestazione', 'motivazione', 'scuola', 'studente',

    # Personalità e sé
    'personalità', 'tratto', 'tratti', 'temperamento', 'carattere', 'sé', 'identità',
    'autostima', 'concetto di sé', 'introversione', 'estroversione', 'ego', 'ombra',
    'mbti', 'bigfive', 'apertura', 'coscienziosità', 'gradevolezza', 'nevroticismo',

    # Lavoro, psicologia industriale e organizzativa
    'lavoro', 'impiego', 'dipendente', 'luogo di lavoro', 'leader', 'leadership', 'motivazione',
    'soddisfazione', 'coinvolgimento', 'esaurimento', 'conflitto', 'organizzativo', 'stress',
    'prestazione', 'produttività',

    # Psicologia positiva
    'benessere', 'resilienza', 'felicità', 'consapevolezza', 'affrontare', 'gratitudine',
    'ottimismo', 'speranza', 'significato', 'scopo', 'flusso', 'vita', 'soddisfazione',

    # Metodi e strumenti
    'esperimento', 'studio', 'caso', 'sondaggio', 'intervista', 'osservazione', 'scala',
    'test', 'valutazione', 'misura', 'questionario', 'inventario', 'strumento',
    'validità', 'affidabilità', 'psicometria', 'longitudinale', 'trasversale',
    'meta', 'analisi', 'statistico', 'correlazione',

    # Etica e accademia
    'etica', 'consenso', 'riservatezza', 'debriefing', 'partecipante',
    'bias', 'randomizzato', 'studio', 'rct', 'placebo', 'controllo', 'revisioneparitaria',

    # Termes généraux
    'psychologie', 'psychologique', 'psychologue', 'mental', 'esprit', 'comportement', 'conduite',
    'émotion', 'émotions', 'affect', 'affectif', 'humeur', 'cognition', 'cognitif',
    'pensée', 'idée', 'raisonnement', 'intelligence', 'qi', 'perception', 'conscience',
    'conscience', 'inconscient', 'apprentissage', 'mémoire', 'attention', 'concentration',

    # Troubles et conditions
    'dépression', 'anxiété', 'stress', 'bipolaire', 'espt', 'toc', 'tdah', 'autisme',
    'trouble', 'troubles', 'phobie', 'schizophrénie', 'délire', 'hallucination',
    'paranoïa', 'insomnie', 'épuisement', 'traumatisme', 'automutilation', 'suicidaire', 'addiction',
    'dépendance', 'abus', 'alimentation', 'anorexie', 'boulimie', 'dérégulation',

    # Clinique et thérapeutique
    'thérapie', 'thérapeute', 'psychothérapie', 'counseling', 'diagnostic', 'clinique',
    'psychiatrie', 'psychiatrique', 'intervention', 'traitement', 'séance', 'santémentale',
    'soin', 'soutien', 'guérison', 'rétablissement',

    # Psychologie sociale
    'social', 'groupe', 'groupes', 'identité', 'influence', 'préjugé', 'biais',
    'conformité', 'obéissance', 'agression', 'altruisme', 'empathie', 'interaction',
    'interpersonnel', 'relation', 'confiance', 'coopération', 'conflit',

    # Psychologie cognitive et neurosciences
    'neurosciences', 'neuronal', 'cerveau', 'cortex', 'neurone', 'synapse', 'hippocampe',
    'amygdale', 'irmf', 'eeg', 'erp', 'scanner', 'neuropsychologie', 'traitement',
    'stimulus', 'attention', 'cognitif', 'exécutif', 'décision', 'décisions',
    'problème', 'résolution de problèmes', 'charge', 'réaction',

    # Développement et éducation
    'développement', 'développemental', 'nourrisson', 'tout-petit', 'enfant', 'enfants', 'adolescence',
    'adolescent', 'jeunesse', 'ado', 'adolescent', 'jeuneadulte', 'petiteenfance',
    'étape', 'styledapprentissage', 'performance', 'motivation', 'école', 'étudiant',

    # Personnalité et soi
    'personnalité', 'caractère', 'traits', 'tempérament', 'caractère', 'soi', 'identité',
    'estimedesoi', 'conceptdesoi', 'introversion', 'extraversion', 'ego', 'ombre',
    'mbti', 'bigfive', 'ouverture', 'conscience', 'amabilité', 'névrosisme',

    # Travail et organisation
    'travail', 'emploi', 'employé', 'lieu de travail', 'leader', 'leadership', 'motivation',
    'satisfaction', 'engagement', 'épuisement', 'conflit', 'organisationnel', 'stress',
    'performance', 'productivité',

    # Psychologie positive
    'bien-être', 'résilience', 'bonheur', 'pleine conscience', 'adaptation', 'gratitude',
    'optimisme', 'espoir', 'sens', 'objectif', 'flux', 'vie', 'satisfaction',

    # Méthodes et outils
    'expérience', 'étude', 'cas', 'enquête', 'entretien', 'observation', 'échelle',
    'test', 'évaluation', 'mesure', 'questionnaire', 'inventaire', 'instrument',
    'validité', 'fiabilité', 'psychométrie', 'longitudinale', 'transversale',
    'méta', 'analyse', 'statistique', 'corrélation',

    # Éthique et académique
    'éthique', 'consentement', 'confidentialité', 'débriefing', 'participant',
    'biais', 'randomisé', 'essai', 'ecr', 'placebo', 'contrôle', 'évaluationparlespairs',

    # 通用术语
    '心理学', '心理的', '心理学家', '心理', '心智', '行为', '行为（英式）',
    '情绪', '情感', '情感', '情绪的', '心情', '认知', '认知的',
    '思维', '想法', '推理', '智力', '智商', '感知', '意识',
    '意识', '无意识', '学习', '记忆', '注意力', '专注',

    # 疾病与状况
    '抑郁', '焦虑', '压力', '躁郁症', '创伤后应激障碍', '强迫症', '多动症', '自闭症',
    '障碍', '障碍（复数）', '恐惧症', '精神分裂症', '妄想', '幻觉',
    '偏执', '失眠', '倦怠', '创伤', '自残', '自杀', '成瘾',
    '依赖', '滥用', '饮食', '厌食症', '贪食症', '情绪失调',

    # 临床与治疗
    '治疗', '治疗师', '心理治疗', '咨询', '诊断', '临床的',
    '精神病学', '精神病的', '干预', '治疗', '会谈', '心理健康',
    '护理', '支持', '康复', '恢复',

    # 社会心理学
    '社会的', '群体', '群体（复数）', '身份', '影响', '偏见', '偏差',
    '从众', '服从', '攻击性', '利他主义', '同理心', '互动',
    '人际的', '关系', '信任', '合作', '冲突',

    # 认知心理学与神经科学
    '神经科学', '神经的', '大脑', '皮层', '神经元', '突触', '海马体',
    '杏仁核', '功能磁共振', '脑电图', '事件相关电位', '扫描', '神经心理学', '处理',
    '刺激', '注意力', '认知的', '执行的', '决策', '决策（复数）',
    '问题', '解决问题', '负荷', '反应',

    # 发展与教育心理学
    '发展', '发展的', '婴儿', '学步儿', '儿童', '儿童（复数）', '青春期',
    '青少年', '青年', '青少年', '青少年（复数）', '年轻成人', '幼儿期',
    '发展里程碑', '学习风格', '表现', '动机', '学校', '学生',

    # 人格与自我
    '人格', '特质', '特质（复数）', '气质', '性格', '自我', '身份',
    '自尊', '自我概念', '内向', '外向', '自我', '阴影',
    'MBTI', '大五人格', '开放性', '尽责性', '宜人性', '神经质',

    # 职场与组织心理学
    '工作', '职位', '员工', '职场', '领导者', '领导力', '动机',
    '满意度', '投入感', '倦怠', '冲突', '组织的', '压力',
    '表现', '生产力',

    # 积极心理学
    '幸福感', '韧性', '幸福', '正念', '应对', '感恩',
    '乐观', '希望', '意义', '目标', '心流', '生活', '满足感',

    # 方法与工具
    '实验', '研究', '案例', '调查', '访谈', '观察', '量表',
    '测试', '评估', '测量', '问卷', '清单', '工具',
    '效度', '信度', '心理测量学', '纵向研究', '横断面研究',
    '元', '分析', '统计的', '相关',

    # 伦理与学术
    '伦理', '知情同意', '保密', '事后说明', '参与者',
    '偏差', '随机', '试验', '随机对照试验', '安慰剂', '对照', '同行评审',

    # Общие термины
    'психология', 'психологический', 'психолог', 'психический', 'ум', 'поведение', 'поведение',
    'эмоция', 'эмоции', 'аффект', 'аффективный', 'настроение', 'когниция', 'когнитивный',
    'мышление', 'мысль', 'рассуждение', 'интеллект', 'айкью', 'восприятие', 'осознание',
    'сознание', 'бессознательное', 'обучение', 'память', 'внимание', 'концентрация',

    # Расстройства и состояния
    'депрессия', 'тревога', 'стресс', 'биполярное расстройство', 'птср', 'окр', 'сдвр', 'аутизм',
    'расстройство', 'расстройства', 'фобия', 'шизофрения', 'бред', 'галлюцинация',
    'паранойя', 'бессонница', 'выгорание', 'травма', 'самоповреждение', 'суицидальный', 'зависимость',
    'привязанность', 'насилие', 'пищевое поведение', 'анорексия', 'булимия', 'дисрегуляция',

    # Клиническая и терапевтическая
    'терапия', 'терапевт', 'психотерапия', 'консультирование', 'диагноз', 'клинический',
    'психиатрия', 'психиатрический', 'вмешательство', 'лечение', 'сеанс', 'психическоездоровье',
    'уход', 'поддержка', 'исцеление', 'восстановление',

    # Социальная психология
    'социальный', 'группа', 'группы', 'идентичность', 'влияние', 'предубеждение', 'предвзятость',
    'конформизм', 'послушание', 'агрессия', 'альтруизм', 'эмпатия', 'взаимодействие',
    'межличностный', 'отношение', 'доверие', 'сотрудничество', 'конфликт',

    # Когнитивная психология и нейронаука
    'нейронаука', 'нейронный', 'мозг', 'кора', 'нейрон', 'синапс', 'гиппокамп',
    'амигдала', 'фмрт', 'ээг', 'эрп', 'сканирование', 'нейропсихология', 'обработка',
    'стимул', 'внимание', 'когнитивный', 'исполнительный', 'решение', 'решения',
    'проблема', 'решение проблем', 'нагрузка', 'реакция',

    # Развитие и образование
    'развитие', 'развивающийся', 'младенец', 'малыш', 'ребёнок', 'дети', 'подростковый возраст',
    'подросток', 'молодёжь', 'тинейджер', 'подросток', 'молодойвзрослый', 'раннеедетство',
    'веха', 'стильобучения', 'успеваемость', 'мотивация', 'школа', 'ученик',

    # Личность и самость
    'личность', 'черта', 'черты', 'темперамент', 'характер', 'я', 'самоидентификация',
    'самооценка', 'самопонятие', 'интроверсия', 'экстраверсия', 'эго', 'тень',
    'mbti', 'большаяпятёрка', 'открытость', 'добросовестность', 'доброжелательность', 'нейротизм',

    # Работа и организационная психология
    'работа', 'работа', 'сотрудник', 'рабочееместо', 'лидер', 'лидерство', 'мотивация',
    'удовлетворенность', 'вовлечённость', 'выгорание', 'конфликт', 'организационный', 'стресс',
    'результативность', 'производительность',

    # Позитивная психология
    'благополучие', 'устойчивость', 'счастье', 'осознанность', 'совладание', 'благодарность',
    'оптимизм', 'надежда', 'смысл', 'цель', 'поток', 'жизнь', 'удовлетворенность',

    # Методы и инструменты
    'эксперимент', 'исследование', 'случай', 'опрос', 'интервью', 'наблюдение', 'шкала',
    'тест', 'оценка', 'измерение', 'анкета', 'инвентарь', 'инструмент',
    'валидность', 'надёжность', 'психометрика', 'лонгитюдный', 'срезовой',
    'мета', 'анализ', 'статистический', 'корреляция',

    # Этика и академия
    'этика', 'согласие', 'конфиденциальность', 'информирование', 'участник',
    'предвзятость', 'рандомизированный', 'испытание', 'ркт', 'плацебо', 'контроль', 'научнаяэкспертиза',

    # 一般用語
    '心理学', '心理的', '心理学者', '精神的', '心', '行動', '行動（英式）',
    '感情', '感情（複数）', '情動', '情動的', '気分', '認知', '認知的',
    '思考', '思考（名詞）', '推論', '知能', 'IQ', '知覚', '意識',
    '意識状態', '無意識', '学習', '記憶', '注意', '集中',

    # 障害・症状
    'うつ病', '不安', 'ストレス', '双極性障害', 'PTSD', '強迫性障害', 'ADHD', '自閉症',
    '障害', '障害（複数）', '恐怖症', '統合失調症', '妄想', '幻覚',
    '被害妄想', '不眠症', '燃え尽き症候群', 'トラウマ', '自傷行為', '自殺傾向', '依存症',
    '依存', '虐待', '摂食', '拒食症', '過食症', '調整障害',

    # 臨床・治療
    'セラピー', 'セラピスト', '心理療法', 'カウンセリング', '診断', '臨床',
    '精神医学', '精神科', '介入', '治療', 'セッション', 'メンタルヘルス',
    'ケア', '支援', '癒し', '回復',

    # 社会心理学
    '社会的', '集団', '集団（複数）', 'アイデンティティ', '影響', '偏見', 'バイアス',
    '同調', '服従', '攻撃性', '利他主義', '共感', '相互作用',
    '対人', '関係', '信頼', '協力', '対立',

    # 認知心理学・神経科学
    '神経科学', '神経的', '脳', '大脳皮質', 'ニューロン', 'シナプス', '海馬',
    '扁桃体', 'fMRI', 'EEG', 'ERP', 'スキャン', '神経心理学', '処理',
    '刺激', '注意', '認知', '実行機能', '意思決定', '意思決定（複数）',
    '問題', '問題解決', '負荷', '反応',

    # 発達・教育心理学
    '発達', '発達的', '乳児', '幼児', '子供', '子供たち', '思春期',
    '思春期の人', '若者', 'ティーン', 'ティーンエイジャー', '若年成人', '幼児期',
    '発達段階', '学習スタイル', 'パフォーマンス', '動機付け', '学校', '生徒',

    # 性格と自己
    '性格', '特性', '特性（複数）', '気質', '性格（人格）', '自己', 'アイデンティティ',
    '自尊心', '自己概念', '内向性', '外向性', '自我', 'シャドウ',
    'MBTI', 'ビッグファイブ', '開放性', '誠実性', '協調性', '神経症傾向',

    # 労働・産業・組織心理学
    '仕事', '職業', '従業員', '職場', 'リーダー', 'リーダーシップ', '動機づけ',
    '満足度', 'エンゲージメント', '燃え尽き', '対立', '組織的', 'ストレス',
    'パフォーマンス', '生産性',

    # ポジティブ心理学
    '幸福', 'レジリエンス', '幸せ', 'マインドフルネス', '対処', '感謝',
    '楽観主義', '希望', '意味', '目的', 'フロー', '人生', '満足感',

    # 方法とツール
    '実験', '研究', '事例', '調査', '面接', '観察', '尺度',
    'テスト', '評価', '測定', '質問票', 'インベントリ', '測定器具',
    '妥当性', '信頼性', '心理測定法', '縦断的', '横断的',
    'メタ', '分析', '統計的', '相関',

    # 倫理と学術
    '倫理', '同意', '機密性', '事後説明', '参加者',
    'バイアス', 'ランダム化', '試験', 'RCT', 'プラセボ', '対照', '査読',

    # 일반 용어
    '심리학', '심리적인', '심리학자', '정신적인', '마음', '행동', '행동(영국식)',
    '감정', '감정들', '정서', '정서적인', '기분', '인지', '인지적인',
    '사고', '생각', '추론', '지능', '지능지수', '지각', '자각',
    '의식', '무의식', '학습', '기억', '주의', '집중',

    # 장애 및 상태
    '우울증', '불안', '스트레스', '양극성장애', '외상후스트레스장애', '강박장애', '주의력결핍과잉행동장애', '자폐증',
    '장애', '장애들', '공포증', '조현병', '망상', '환각',
    '편집증', '불면증', '번아웃', '트라우마', '자해', '자살충동', '중독',
    '의존', '남용', '섭식', '거식증', '폭식증', '조절장애',

    # 임상 및 치료
    '치료', '치료사', '심리치료', '상담', '진단', '임상',
    '정신의학', '정신과의', '중재', '치료', '세션', '정신건강',
    '돌봄', '지원', '치유', '회복',

    # 사회심리학
    '사회적인', '집단', '집단들', '정체성', '영향력', '편견', '편향',
    '순응', '복종', '공격성', '이타주의', '공감', '상호작용',
    '대인관계', '관계', '신뢰', '협동', '갈등',

    # 인지심리학 및 신경과학
    '신경과학', '신경의', '뇌', '대뇌피질', '뉴런', '시냅스', '해마',
    '편도체', '기능적 자기공명영상', '뇌파검사', '사건관련전위', '스캔', '신경심리학', '처리',
    '자극', '주의', '인지', '집행기능', '결정', '결정들',
    '문제', '문제해결', '부하', '반응',

    # 발달 및 교육
    '발달', '발달적인', '영아', '유아', '아동', '어린이들', '청소년기',
    '청소년', '청년', '십대', '청소년(사람)', '청년기', '유아기',
    '발달이정표', '학습스타일', '수행', '동기부여', '학교', '학생',

    # 성격 및 자아
    '성격', '특성', '특성들', '기질', '성격(캐릭터)', '자아', '정체성',
    '자존감', '자기개념', '내향성', '외향성', '자기', '그림자',
    'MBTI', '빅파이브', '개방성', '성실성', '친화성', '신경증',

    # 직장, 산업 및 조직
    '직장', '직업', '직원', '직장환경', '리더', '리더십', '동기',
    '만족도', '몰입도', '번아웃', '갈등', '조직적인', '스트레스',
    '성과', '생산성',

    # 긍정심리학
    '웰빙', '회복탄력성', '행복', '마음챙김', '대처', '감사',
    '낙관주의', '희망', '의미', '목적', '몰입', '삶', '만족감',

    # 방법론 및 도구
    '실험', '연구', '사례', '설문조사', '면접', '관찰', '척도',
    '시험', '평가', '측정', '설문지', '목록', '도구',
    '타당도', '신뢰도', '심리측정학', '종단적', '횡단적',
    '메타', '분석', '통계적인', '상관관계',

    # 윤리 및 학문
    '윤리', '동의', '비밀보장', '사후설명', '참가자',
    '편향', '무작위화', '임상시험', '무작위대조시험', '플라세보', '통제', '동료심사',

    # 一般用語
    '心理學', '心理的', '心理學家', '精神的', '心智', '行為', '行為（英式）',
    '情緒', '情緒們', '情感', '情感的', '情緒狀態', '認知', '認知的',
    '思考', '想法', '推理', '智能', '智商', '知覺', '覺察',
    '意識', '意識狀態', '無意識', '學習', '記憶', '注意', '專注',

    # 疾病與症狀
    '憂鬱', '焦慮', '壓力', '雙極性', '創傷後壓力症', '強迫症', '注意力缺陷過動症', '自閉症',
    '障礙', '障礙們', '恐懼症', '精神分裂症', '妄想', '幻覺',
    '偏執', '失眠', '倦怠', '創傷', '自我傷害', '自殺傾向', '成癮',
    '依賴', '濫用', '飲食', '厭食症', '暴食症', '調節失衡',

    # 臨床與治療
    '治療', '治療師', '心理治療', '諮商', '診斷', '臨床',
    '精神醫學', '精神科', '介入', '治療', '療程', '心理健康',
    '照護', '支持', '療癒', '復原',

    # 社會心理學
    '社會的', '群體', '群體們', '身分認同', '影響', '偏見', '偏差',
    '從眾', '服從', '侵略', '利他主義', '同理', '互動',
    '人際的', '關係', '信任', '合作', '衝突',

    # 認知心理學與神經科學
    '神經科學', '神經的', '大腦', '皮質', '神經元', '突觸', '海馬迴',
    '杏仁核', '功能性磁振造影', '腦電圖', '事件相關電位', '掃描', '神經心理學', '處理',
    '刺激', '注意', '認知', '執行功能', '決策', '決策們',
    '問題', '問題解決', '負荷', '反應',

    # 發展與教育心理學
    '發展', '發展性的', '嬰兒', '幼童', '兒童', '兒童們', '青少年期',
    '青少年', '青年', '青少年', '青少年', '成年前期', '幼兒期',
    '里程碑', '學習風格', '表現', '動機', '學校', '學生',

    # 人格與自我
    '人格', '特質', '特質們', '氣質', '性格', '自我', '身分',
    '自尊', '自我概念', '內向性', '外向性', '自我', '陰影',
    'MBTI', '大五人格', '開放性', '盡責性', '宜人性', '神經質',

    # 工作、工業與組織心理學
    '工作', '職業', '員工', '職場', '領導者', '領導力', '動機',
    '滿意度', '投入', '倦怠', '衝突', '組織的', '壓力',
    '績效', '生產力',

    # 正向心理學
    '福祉', '韌性', '幸福', '正念', '因應', '感恩',
    '樂觀', '希望', '意義', '目標', '心流', '生命', '滿足感',

    # 方法與工具
    '實驗', '研究', '案例', '調查', '訪談', '觀察', '量表',
    '測驗', '評估', '衡量', '問卷', '清單', '工具',
    '效度', '信度', '心理計量學', '縱向', '橫斷',
    '統整', '分析', '統計的', '相關',

    # 倫理與學術
    '倫理', '同意', '保密', '事後說明', '參與者',
    '偏差', '隨機化', '試驗', '隨機對照試驗', '安慰劑', '對照', '同行評審',

    # Obecné pojmy
    'psychologie', 'psychologický', 'psycholog', 'mentální', 'mysl', 'chování', 'chování (britská angličtina)',
    'emoce', 'emoce (množné číslo)', 'afekt', 'afektivní', 'nálada', 'kognice', 'kognitivní',
    'myšlení', 'myšlenka', 'uvažování', 'inteligence', 'iq', 'percepce', 'uvědomění',
    'vědomí', 'nevědomí', 'učení', 'paměť', 'pozornost', 'soustředění',

    # Poruchy a stavy
    'deprese', 'úzkost', 'stres', 'bipolární porucha', 'ptsd', 'ocd', 'adhd', 'autismus',
    'porucha', 'poruchy', 'fobie', 'schizofrenie', 'blud', 'halucinace',
    'paranoia', 'insomnie', 'vyhoření', 'trauma', 'sebeublížení', 'sebevražedné', 'závislost',
    'závislost (jiný význam)', 'zneužívání', 'stravování', 'anorexie', 'bulimie', 'dysregulace',

    # Klinická a terapeutická oblast
    'terapie', 'terapeut', 'psychoterapie', 'poradenství', 'diagnóza', 'klinický',
    'psychiatrie', 'psychiatrický', 'zásah', 'léčba', 'sezení', 'duševní zdraví',
    'péče', 'podpora', 'léčení', 'uzdravení',

    # Sociální psychologie
    'sociální', 'skupina', 'skupiny', 'identita', 'vliv', 'předsudek', 'zkreslení',
    'konformita', 'poslušnost', 'agrese', 'altruismus', 'empatie', 'interakce',
    'mezilidský', 'vztah', 'důvěra', 'spolupráce', 'konflikt',

    # Kognitivní psychologie a neurovědy
    'neurověda', 'neurální', 'mozek', 'kortex', 'neuron', 'synapse', 'hippokampus',
    'amygdala', 'fmri', 'eeg', 'erp', 'skenování', 'neuropsychologie', 'zpracování',
    'stimulus', 'pozornost', 'kognitivní', 'výkonný', 'rozhodnutí', 'rozhodnutí (mn. číslo)',
    'problém', 'řešení problémů', 'zátěž', 'reakce',

    # Vývojová a vzdělávací psychologie
    'vývoj', 'vývojový', 'dítě', 'batole', 'dítě', 'děti', 'adolescence',
    'adolescent', 'mladistvý', 'teenager', 'teenager', 'mladý dospělý', 'rané dětství',
    'milník', 'styl učení', 'výkon', 'motivace', 'škola', 'student',

    # Osobnost a já
    'osobnost', 'rysy', 'rysy (mn. číslo)', 'temperament', 'charakter', 'já', 'identita',
    'sebevědomí', 'sebepojetí', 'introverze', 'extroverze', 'ego', 'stín',
    'mbti', 'velká pětka', 'otevřenost', 'svědomitost', 'přívětivost', 'neuroticismus',

    # Práce, průmysl a organizace
    'práce', 'zaměstnání', 'zaměstnanec', 'pracoviště', 'vedoucí', 'vůdcovství', 'motivace',
    'spokojenost', 'angažovanost', 'vyhoření', 'konflikt', 'organizační', 'stres',
    'výkon', 'produktivita',

    # Pozitivní psychologie
    'blaho', 'odolnost', 'štěstí', 'všímavost', 'zdolávání', 'vděčnost',
    'optimismus', 'naděje', 'smysl', 'účel', 'flow', 'život', 'spokojenost',

    # Metody a nástroje
    'experiment', 'studie', 'případ', 'průzkum', 'rozhovor', 'pozorování', 'škála',
    'test', 'hodnocení', 'měření', 'dotazník', 'inventář', 'nástroj',
    'validita', 'spolehlivost', 'psychometrie', 'longitudinální', 'průřezový',
    'meta', 'analýza', 'statistický', 'korelace',

    # Etika a akademické záležitosti
    'etika', 'souhlas', 'důvěrnost', 'briefing', 'účastník',
    'zkreslení', 'randomizovaný', 'klinická studie', 'randomizovaná kontrolovaná studie', 'placebo', 'kontrola', 'recenze',

    # المصطلحات العامة
    'علم النفس', 'نفسي', 'طبيب نفسي', 'ذهني', 'عقل', 'سلوك', 'تصرف',
    'عاطفة', 'مشاعر', 'تأثير', 'عاطفي', 'مزاج', 'إدراك', 'معرفي',
    'تفكير', 'فكر', 'استدلال', 'ذكاء', 'معدل الذكاء', 'إدراك حسي', 'وعي',
    'وعي', 'لاوعي', 'تعلم', 'ذاكرة', 'انتباه', 'تركيز',

    # الاضطرابات والحالات
    'اكتئاب', 'قلق', 'توتر', 'اضطراب ثنائي القطب', 'اضطراب ما بعد الصدمة', 'اضطراب الوسواس القهري', 'اضطراب فرط الحركة وتشتت الانتباه', 'توحد',
    'اضطراب', 'اضطرابات', 'رهاب', 'فصام', 'هذيان', 'هلوسة',
    'جنون الارتياب', 'أرق', 'احتراق نفسي', 'صدمة', 'إيذاء النفس', 'انتحاري', 'إدمان',
    'اعتماد', 'إساءة', 'أكل', 'فقدان الشهية', 'الشره المرضي', 'خلل التنظيم',

    # العيادي والعلاجي
    'علاج', 'معالج', 'العلاج النفسي', 'استشارة', 'تشخيص', 'عيادي',
    'طب نفسي', 'نفسي', 'تدخل', 'علاج', 'جلسة', 'صحة نفسية',
    'رعاية', 'دعم', 'شفاء', 'تعافي',

    # علم النفس الاجتماعي
    'اجتماعي', 'مجموعة', 'مجموعات', 'هوية', 'تأثير', 'تحامل', 'تحيز',
    'امتثال', 'طاعة', 'عدوان', 'إيثار', 'تعاطف', 'تفاعل',
    'بين الأشخاص', 'علاقة', 'ثقة', 'تعاون', 'صراع',

    # علم النفس المعرفي وعلم الأعصاب
    'علم الأعصاب', 'عصبي', 'دماغ', 'قشرة الدماغ', 'عصبون', 'تشابك عصبي', 'قرن آمون',
    'اللوزة الدماغية', 'التصوير بالرنين المغناطيسي الوظيفي', 'تخطيط الدماغ', 'الاستجابة المرتبطة بالحدث', 'مسح', 'علم النفس العصبي', 'معالجة',
    'مثير', 'انتباه', 'معرفي', 'تنفيذي', 'قرار', 'قرارات',
    'مشكلة', 'حل المشكلة', 'حمل', 'استجابة',

    # التطوري والتعليمي
    'تطور', 'تطوري', 'رضيع', 'طفل صغير', 'طفل', 'أطفال', 'مراهقة',
    'مراهق', 'شباب', 'مراهق', 'مراهق', 'شاب بالغ', 'الطفولة المبكرة',
    'معلم', 'أسلوب تعلم', 'أداء', 'دافع', 'مدرسة', 'طالب',

    # الشخصية والذات
    'شخصية', 'سمة', 'سمات', 'مزاج', 'طابع', 'ذات', 'هوية',
    'تقدير الذات', 'مفهوم الذات', 'انطوائية', 'انبساطية', 'الأنا', 'ظل',
    'MBTI', 'الخمسة الكبار', 'الانفتاح', 'الضمير', 'الموافقة', 'العصابية',

    # العمل والصناعة والتنظيم
    'عمل', 'وظيفة', 'موظف', 'مكان العمل', 'قائد', 'قيادة', 'دافع',
    'رضا', 'مشاركة', 'احتراق وظيفي', 'صراع', 'تنظيمي', 'ضغط',
    'أداء', 'إنتاجية',

    # علم النفس الإيجابي
    'رفاهية', 'مرونة', 'سعادة', 'الوعي', 'تأقلم', 'امتنان',
    'تفاؤل', 'أمل', 'معنى', 'هدف', 'تدفق', 'حياة', 'رضا',

    # الطرق والأدوات
    'تجربة', 'دراسة', 'حالة', 'مسح', 'مقابلة', 'ملاحظة', 'مقياس',
    'اختبار', 'تقييم', 'قياس', 'استبيان', 'جرد', 'أداة',
    'صلاحية', 'موثوقية', 'القياس النفسي', 'طولي', 'مقطعي',
    'ميتا', 'تحليل', 'إحصائي', 'ارتباط',

    # أخلاقيات وأكاديمي
    'أخلاقيات', 'موافقة', 'سرية', 'إحاطة', 'مشارك',
    'تحيز', 'عشوائي', 'تجربة', 'تجربة محكمة عشوائية', 'دواء وهمي', 'مراقبة', 'مراجعة الأقران'

],


    'environment': [
        # Core and general terms
    'environment', 'environmental', 'ecology', 'ecological', 'eco', 'green',
    'sustainability', 'sustainable', 'nature', 'natural', 'planet', 'earth', 'sphere',

    # Ecosystem & biodiversity
    'ecosystem', 'habitat', 'biodiversity', 'species', 'flora', 'fauna', 'conservation',
    'preservation', 'wildlife', 'endangered', 'extinction', 'marine', 'forest', 'jungle',
    'wetlands', 'reef', 'ocean', 'arctic', 'glacier', 'river', 'watershed', 'basin',
    'desert', 'soil', 'land', 'landscape', 'biome',

    # Pollution & contamination
    'pollution', 'pollutant', 'air', 'water', 'soil', 'microplastic', 'plastic',
    'emission', 'emissions', 'toxic', 'hazardous', 'contaminant', 'contamination',
    'waste', 'wastewater', 'runoff', 'discharge', 'dumping', 'e-waste', 'landfill',
    'sewage', 'smog', 'gases', 'co2', 'methane', 'ozone', 'chemical', 'chemicals',
    'heavy metals', 'particulate', 'particulates', 'ash', 'soot', 'biodegradable',
    'degradable', 'compost', 'compostable', 'decomposition',

    # Climate change
    'climate', 'warming', 'cooling', 'temperature', 'weather', 'carbon', 'carbonation',
    'greenhouse', 'climatechange', 'globalwarming', 'sea', 'flood', 'drought',
    'storm', 'cyclone', 'tornado', 'hurricane', 'resilience', 'mitigation', 'adaptation',
    'ipcc', 'floodplain', 'rise', 'melting', 'netzero', 'neutrality', 'neutral',
    'footprint', 'carboncapture', 'offset', 'emissions', 'reduction', 'energies'

    # Energy & fuels
    'energy', 'renewable', 'renewables', 'solar', 'wind', 'hydro', 'hydropower',
    'bioenergy', 'biomass', 'geothermal', 'photovoltaic', 'efficiency', 'smartgrid',
    'clean', 'cleanenergy', 'greenenergy', 'lowcarbon', 'fossil', 'fossils',
    'coal', 'gas', 'oil', 'fuel', 'fuels', 'nuclear', 'hydrogen',

    # Circular economy and waste management
    'recycle', 'recycling', 'reuse', 'reduce', 'zerowaste', 'circulareconomy',
    'wastemanagement', 'incineration', 'composting', 'sustainableconsumption',
    'life cycle', 'lifecycle', 'resource', 'resources', 'management', 'supplychain',

    # Agriculture, land and food
    'agriculture', 'agro', 'farming', 'farm', 'organic', 'crop', 'harvest', 'pesticide',
    'fertilizer', 'degradation', 'erosion', 'tillage', 'irrigation', 'landuse',
    'landcover', 'soilhealth', 'regenerative', 'agroecology', 'restoration',

    # Urban & policy
    'urban', 'urbanization', 'smartcity', 'city', 'infrastructure', 'transport',
    'mobility', 'planning', 'development', 'zoning', 'greenroof', 'greening',
    'environmentalpolicy', 'regulation', 'law', 'act', 'sdg', 'sdgs', 'justice',
    'eia', 'impact', 'assessment', 'monitoring', 'enforcement', 'penalty',

    # Water
    'water', 'aquatic', 'hydrology', 'hydrological', 'river', 'stream', 'reservoir',
    'aquifer', 'wetland', 'marsh', 'lake', 'ocean', 'groundwater', 'drainage',
    'desalination', 'runoff',

    # Miscellaneous
    'chemosphere', 'ecosphere', 'biosphere', 'atmosphere', 'geosphere', 'anthropocene',
    'environmentalhealth', 'exposure', 'toxicology', 'ecotoxicology', 'sdg', 'goal',
    'cleanerproduction', 'climateresilience', 'carbonneutral', 'transition', 'planetary',

    # Términos básicos y generales
    'medio ambiente', 'ambiental', 'ecología', 'ecológico', 'eco', 'verde',
    'sostenibilidad', 'sostenible', 'naturaleza', 'natural', 'planeta', 'tierra', 'esfera',

    # Ecosistema y biodiversidad
    'ecosistema', 'hábitat', 'biodiversidad', 'especies', 'flora', 'fauna', 'conservación',
    'preservación', 'vida silvestre', 'en peligro', 'extinción', 'marino', 'bosque', 'selva',
    'humedales', 'arrecife', 'océano', 'ártico', 'glaciar', 'río', 'cuenca hidrográfica', 'cuenca',
    'desierto', 'suelo', 'tierra', 'paisaje', 'bioma',

    # Contaminación y polución
    'contaminación', 'contaminante', 'aire', 'agua', 'suelo', 'microplástico', 'plástico',
    'emisión', 'emisiones', 'tóxico', 'peligroso', 'contaminante', 'contaminación',
    'residuos', 'aguas residuales', 'escorrentía', 'descarga', 'vertido', 'residuos electrónicos', 'vertedero',
    'alcantarillado', 'smog', 'gases', 'co2', 'metano', 'ozono', 'químico', 'químicos',
    'metales pesados', 'partículas', 'partículas', 'ceniza', 'hollín', 'biodegradable',
    'descomponible', 'compost', 'compostable', 'descomposición',

    # Cambio climático
    'clima', 'calentamiento', 'enfriamiento', 'temperatura', 'clima', 'carbono', 'carbonatación',
    'invernadero', 'cambio climático', 'calentamiento global', 'mar', 'inundación', 'sequía',
    'tormenta', 'ciclón', 'tornado', 'huracán', 'resiliencia', 'mitigación', 'adaptación',
    'ipcc', 'llanura de inundación', 'incremento', 'derretimiento', 'neutralidad neta', 'neutralidad', 'neutral',
    'huella', 'captura de carbono', 'compensación', 'emisiones', 'reducción', 'energías',

    # Energía y combustibles
    'energía', 'renovable', 'renovables', 'solar', 'eólica', 'hidroeléctrica', 'energía hidroeléctrica',
    'bioenergía', 'biomasa', 'geotérmica', 'fotovoltaica', 'eficiencia', 'red inteligente',
    'limpia', 'energía limpia', 'energía verde', 'bajo carbono', 'fósil', 'combustibles fósiles',
    'carbón', 'gas', 'petróleo', 'combustible', 'combustibles', 'nuclear', 'hidrógeno',

    # Economía circular y gestión de residuos
    'reciclar', 'reciclaje', 'reutilizar', 'reducir', 'residuos cero', 'economía circular',
    'gestión de residuos', 'incineración', 'compostaje', 'consumo sostenible',
    'ciclo de vida', 'ciclo de vida', 'recurso', 'recursos', 'gestión', 'cadena de suministro',

    # Agricultura, tierra y alimentación
    'agricultura', 'agro', 'agricultura', 'granja', 'orgánico', 'cultivo', 'cosecha', 'pesticida',
    'fertilizante', 'degradación', 'erosión', 'labranza', 'riego', 'uso de la tierra',
    'cobertura del suelo', 'salud del suelo', 'regenerativo', 'agroecología', 'restauración',

    # Urbana y política
    'urbano', 'urbanización', 'ciudad inteligente', 'ciudad', 'infraestructura', 'transporte',
    'movilidad', 'planificación', 'desarrollo', 'zonificación', 'techo verde', 'reforestación urbana',
    'política ambiental', 'regulación', 'ley', 'acto', 'ods', 'ods (plural)', 'justicia',
    'eia', 'impacto', 'evaluación', 'monitoreo', 'cumplimiento', 'sanción',

    # Agua
    'agua', 'acuático', 'hidrología', 'hidrológico', 'río', 'arroyo', 'embalse',
    'acuífero', 'humedal', 'pantano', 'lago', 'océano', 'agua subterránea', 'drenaje',
    'desalinización', 'escorrentía',

    # Misceláneos
    'quimosfera', 'ecósfera', 'biosfera', 'atmósfera', 'geósfera', 'antropoceno',
    'salud ambiental', 'exposición', 'toxicología', 'ecotoxicología', 'ods', 'meta',
    'producción más limpia', 'resiliencia climática', 'neutralidad de carbono', 'transición', 'planetario',

    # Termos básicos e gerais
    'ambiente', 'ambiental', 'ecologia', 'ecológico', 'eco', 'verde',
    'sustentabilidade', 'sustentável', 'natureza', 'natural', 'planeta', 'terra', 'esfera',

    # Ecossistema e biodiversidade
    'ecossistema', 'habitat', 'biodiversidade', 'espécies', 'flora', 'fauna', 'conservação',
    'preservação', 'vida selvagem', 'ameaçado', 'extinção', 'marinho', 'floresta', 'selva',
    'áreas úmidas', 'recife', 'oceano', 'ártico', 'geleira', 'rio', 'bacia hidrográfica', 'bacia',
    'deserto', 'solo', 'terra', 'paisagem', 'bioma',

    # Poluição e contaminação
    'poluição', 'poluente', 'ar', 'água', 'solo', 'microplástico', 'plástico',
    'emissão', 'emissões', 'tóxico', 'perigoso', 'contaminante', 'contaminação',
    'resíduos', 'água residual', 'escoamento', 'descarga', 'descarte', 'lixo eletrônico', 'aterro sanitário',
    'esgoto', 'smog', 'gases', 'co2', 'metano', 'ozônio', 'químico', 'químicos',
    'metais pesados', 'partícula', 'partículas', 'cinzas', 'fuligem', 'biodegradável',
    'degradável', 'composto', 'compostável', 'decomposição',

    # Mudança climática
    'clima', 'aquecimento', 'resfriamento', 'temperatura', 'tempo', 'carbono', 'carbonatação',
    'efeito estufa', 'mudança climática', 'aquecimento global', 'mar', 'inundação', 'seca',
    'tempestade', 'ciclone', 'tornado', 'furacão', 'resiliência', 'mitigação', 'adaptação',
    'ipcc', 'planície de inundação', 'elevação', 'derretimento', 'emissão zero', 'neutralidade', 'neutro',
    'pegada', 'captura de carbono', 'compensação', 'emissões', 'redução', 'energias',

    # Energia e combustíveis
    'energia', 'renovável', 'renováveis', 'solar', 'vento', 'hidro', 'hidrelétrica',
    'bioenergia', 'biomassa', 'geotérmica', 'fotovoltaica', 'eficiência', 'rede inteligente',
    'limpa', 'energia limpa', 'energia verde', 'baixo carbono', 'fóssil', 'combustíveis fósseis',
    'carvão', 'gás', 'óleo', 'combustível', 'combustíveis', 'nuclear', 'hidrogênio',

    # Economia circular e gestão de resíduos
    'reciclar', 'reciclagem', 'reutilizar', 'reduzir', 'resíduo zero', 'economia circular',
    'gestão de resíduos', 'incineração', 'compostagem', 'consumo sustentável',
    'ciclo de vida', 'ciclo de vida', 'recurso', 'recursos', 'gestão', 'cadeia de suprimentos',

    # Agricultura, terra e alimentação
    'agricultura', 'agro', 'agricultura', 'fazenda', 'orgânico', 'colheita', 'colheita', 'pesticida',
    'fertilizante', 'degradação', 'erosão', 'preparo do solo', 'irrigação', 'uso da terra',
    'cobertura do solo', 'saúde do solo', 'regenerativo', 'agroecologia', 'restauração',

    # Urbano e políticas
    'urbano', 'urbanização', 'cidade inteligente', 'cidade', 'infraestrutura', 'transporte',
    'mobilidade', 'planejamento', 'desenvolvimento', 'zoneamento', 'telhado verde', 'revitalização',
    'política ambiental', 'regulação', 'lei', 'ato', 'ods', 'ods (plural)', 'justiça',
    'aia', 'impacto', 'avaliação', 'monitoramento', 'cumprimento', 'penalidade',

    # Água
    'água', 'aquático', 'hidrologia', 'hidrológico', 'rio', 'riacho', 'reservatório',
    'aquífero', 'área úmida', 'pântano', 'lago', 'oceano', 'água subterrânea', 'drenagem',
    'dessalinização', 'escoamento',

    # Diversos
    'quimosfera', 'ecósfera', 'biosfera', 'atmosfera', 'geosfera', 'antropoceno',
    'saúde ambiental', 'exposição', 'toxicologia', 'ecotoxicologia', 'ods', 'meta',
    'produção mais limpa', 'resiliência climática', 'neutralidade de carbono', 'transição', 'planetário',

    # Termini base e generali
    'ambiente', 'ambientale', 'ecologia', 'ecologico', 'eco', 'verde',
    'sostenibilità', 'sostenibile', 'natura', 'naturale', 'pianeta', 'terra', 'sfera',

    # Ecosistema e biodiversità
    'ecosistema', 'habitat', 'biodiversità', 'specie', 'flora', 'fauna', 'conservazione',
    'preservazione', 'fauna selvatica', 'in pericolo', 'estinzione', 'marino', 'foresta', 'giungla',
    'zone umide', 'barriera corallina', 'oceano', 'artico', 'ghiacciaio', 'fiume', 'bacino idrografico', 'bacino',
    'deserto', 'suolo', 'terra', 'paesaggio', 'bioma',

    # Inquinamento e contaminazione
    'inquinamento', 'inquinante', 'aria', 'acqua', 'suolo', 'microplastica', 'plastica',
    'emissione', 'emissioni', 'tossico', 'pericoloso', 'contaminante', 'contaminazione',
    'rifiuti', 'acque reflue', 'deflusso', 'scarico', 'scarico illegale', 'rifiuti elettronici', 'discarica',
    'acque nere', 'smog', 'gas', 'co2', 'metano', 'ozono', 'chimico', 'sostanze chimiche',
    'metalli pesanti', 'particella', 'particelle', 'cenere', 'fuliggine', 'biodegradabile',
    'degradabile', 'compost', 'compostabile', 'decomposizione',

    # Cambiamento climatico
    'clima', 'riscaldamento', 'raffreddamento', 'temperatura', 'tempo', 'carbonio', 'carbonatazione',
    'serra', 'cambiamento climatico', 'riscaldamento globale', 'mare', 'alluvione', 'siccità',
    'tempesta', 'ciclone', 'tornado', 'uragano', 'resilienza', 'mitigazione', 'adattamento',
    'ipcc', 'pianura alluvionale', 'aumento', 'scioglimento', 'emissioni nette zero', 'neutralità', 'neutrale',
    'impronta', 'cattura del carbonio', 'compensazione', 'emissioni', 'riduzione', 'energie',

    # Energia e combustibili
    'energia', 'rinnovabile', 'energie rinnovabili', 'solare', 'vento', 'idro', 'energia idroelettrica',
    'bioenergia', 'biomassa', 'geotermica', 'fotovoltaico', 'efficienza', 'rete intelligente',
    'pulita', 'energia pulita', 'energia verde', 'basso carbonio', 'combustibili fossili', 'fossili',
    'carbone', 'gas', 'petrolio', 'combustibile', 'combustibili', 'nucleare', 'idrogeno',

    # Economia circolare e gestione dei rifiuti
    'riciclo', 'riciclaggio', 'riutilizzo', 'riduzione', 'zero rifiuti', 'economia circolare',
    'gestione dei rifiuti', 'incenerimento', 'compostaggio', 'consumo sostenibile',
    'ciclo di vita', 'ciclo di vita', 'risorsa', 'risorse', 'gestione', 'catena di approvvigionamento',

    # Agricoltura, terra e alimentazione
    'agricoltura', 'agro', 'agricoltura', 'fattoria', 'biologico', 'coltura', 'raccolto', 'pesticida',
    'fertilizzante', 'degradazione', 'erosione', 'lavorazione del suolo', 'irrigazione', 'uso del suolo',
    'copertura del suolo', 'salute del suolo', 'rigenerativo', 'agroecologia', 'restauro',

    # Urbano e politiche
    'urbano', 'urbanizzazione', 'città intelligente', 'città', 'infrastruttura', 'trasporto',
    'mobilità', 'pianificazione', 'sviluppo', 'zonizzazione', 'tetto verde', 'verdi urbani',
    'politica ambientale', 'regolamentazione', 'legge', 'atto', 'sdg', 'sdg (plurale)', 'giustizia',
    'via', 'impatto', 'valutazione', 'monitoraggio', 'applicazione', 'penalità',

    # Acqua
    'acqua', 'acquatico', 'idrologia', 'idrologico', 'fiume', 'ruscello', 'bacino idrico',
    'acquifero', 'zona umida', 'palude', 'lago', 'oceano', 'acqua sotterranea', 'drenaggio',
    'desalinizzazione', 'deflusso',

    # Varie
    'quimosfera', 'ecosphera', 'biosfera', 'atmosfera', 'geosfera', 'antropocene',
    'salute ambientale', 'esposizione', 'tossicologia', 'ecotossicologia', 'sdg', 'obiettivo',
    'produzione più pulita', 'resilienza climatica', 'neutralità carbonica', 'transizione', 'planetaria',

    # Termes de base et généraux
    'environnement', 'environnemental', 'écologie', 'écologique', 'éco', 'vert',
    'durabilité', 'durable', 'nature', 'naturel', 'planète', 'terre', 'sphère',

    # Écosystème et biodiversité
    'écosystème', 'habitat', 'biodiversité', 'espèces', 'flore', 'faune', 'conservation',
    'préservation', 'faune sauvage', 'en danger', 'extinction', 'marin', 'forêt', 'jungle',
    'zones humides', 'récif', 'océan', 'arctique', 'glacier', 'rivière', 'bassin versant', 'bassin',
    'désert', 'sol', 'terre', 'paysage', 'biome',

    # Pollution et contamination
    'pollution', 'polluant', 'air', 'eau', 'sol', 'microplastique', 'plastique',
    'émission', 'émissions', 'toxique', 'dangereux', 'contaminant', 'contamination',
    'déchets', 'eaux usées', 'ruissellement', 'rejet', 'déversement', 'déchets électroniques', 'décharge',
    'égouts', 'smog', 'gaz', 'co2', 'méthane', 'ozone', 'chimique', 'produits chimiques',
    'métaux lourds', 'particules', 'particules fines', 'cendres', 'suie', 'biodégradable',
    'dégradable', 'compost', 'compostable', 'décomposition',

    # Changement climatique
    'climat', 'réchauffement', 'refroidissement', 'température', 'météo', 'carbone', 'carbonatation',
    'effet de serre', 'changement climatique', 'réchauffement climatique', 'mer', 'inondation', 'sécheresse',
    'tempête', 'cyclone', 'tornade', 'ouragan', 'résilience', 'atténuation', 'adaptation',
    'giec', 'plaine inondable', 'élévation', 'fonte', 'zéro net', 'neutralité', 'neutre',
    'empreinte', 'capture de carbone', 'compensation', 'émissions', 'réduction', 'énergies',

    # Énergie et combustibles
    'énergie', 'renouvelable', 'énergies renouvelables', 'solaire', 'éolien', 'hydroélectrique', 'hydroélectricité',
    'bioénergie', 'biomasse', 'géothermie', 'photovoltaïque', 'efficacité', 'réseau intelligent',
    'propre', 'énergie propre', 'énergie verte', 'faible carbone', 'fossile', 'combustibles fossiles',
    'charbon', 'gaz', 'pétrole', 'carburant', 'combustibles', 'nucléaire', 'hydrogène',

    # Économie circulaire et gestion des déchets
    'recycler', 'recyclage', 'réutiliser', 'réduire', 'zéro déchet', 'économie circulaire',
    'gestion des déchets', 'incinération', 'compostage', 'consommation durable',
    'cycle de vie', 'cycle de vie', 'ressource', 'ressources', 'gestion', 'chaîne d\'approvisionnement',

    # Agriculture, terres et alimentation
    'agriculture', 'agro', 'agriculture', 'ferme', 'biologique', 'culture', 'récolte', 'pesticide',
    'engrais', 'dégradation', 'érosion', 'labour', 'irrigation', 'utilisation des terres',
    'couverture des sols', 'santé des sols', 'régénératif', 'agroécologie', 'restauration',

    # Urbain et politique
    'urbain', 'urbanisation', 'ville intelligente', 'ville', 'infrastructure', 'transport',
    'mobilité', 'planification', 'développement', 'zonage', 'toit vert', 'végétalisation',
    'politique environnementale', 'réglementation', 'loi', 'acte', 'odd', 'odds', 'justice',
    'eie', 'impact', 'évaluation', 'surveillance', 'application', 'sanction',

    # Eau
    'eau', 'aquatique', 'hydrologie', 'hydrologique', 'rivière', 'ruisseau', 'réservoir',
    'nappe phréatique', 'zone humide', 'marais', 'lac', 'océan', 'eau souterraine', 'drainage',
    'dessalement', 'ruissellement',

    # Divers
    'chémosphère', 'écosphère', 'biosphère', 'atmosphère', 'géosphère', 'anthropocène',
    'santé environnementale', 'exposition', 'toxicologie', 'écotoxicologie', 'odd', 'objectif',
    'production plus propre', 'résilience climatique', 'neutralité carbone', 'transition', 'planétaire',

    # 核心及通用术语
    '环境', '环境的', '生态学', '生态的', '生态', '绿色',
    '可持续性', '可持续的', '自然', '天然', '星球', '地球', '球体',

    # 生态系统与生物多样性
    '生态系统', '栖息地', '生物多样性', '物种', '植物群', '动物群', '保护',
    '保存', '野生动物', '濒危', '灭绝', '海洋的', '森林', '丛林',
    '湿地', '珊瑚礁', '海洋', '北极', '冰川', '河流', '流域', '盆地',
    '沙漠', '土壤', '土地', '景观', '生物群落',

    # 污染与污染物
    '污染', '污染物', '空气', '水', '土壤', '微塑料', '塑料',
    '排放', '排放物', '有毒的', '危险的', '污染物', '污染',
    '废物', '废水', '径流', '排放', '倾倒', '电子垃圾', '垃圾填埋场',
    '污水', '烟雾', '气体', '二氧化碳', '甲烷', '臭氧', '化学物质', '化学品',
    '重金属', '颗粒物', '颗粒物质', '灰烬', '烟尘', '可生物降解',
    '可降解', '堆肥', '可堆肥', '分解',

    # 气候变化
    '气候', '变暖', '变冷', '温度', '天气', '碳', '碳酸化',
    '温室', '气候变化', '全球变暖', '海洋', '洪水', '干旱',
    '风暴', '气旋', '龙卷风', '飓风', '弹性', '缓解', '适应',
    '气候变化政府间专门委员会', '洪泛区', '上升', '融化', '净零排放', '中和', '中性',
    '足迹', '碳捕获', '抵消', '排放', '减少', '能源',

    # 能源与燃料
    '能源', '可再生', '可再生能源', '太阳能', '风能', '水力', '水电',
    '生物能源', '生物质', '地热', '光伏', '效率', '智能电网',
    '清洁', '清洁能源', '绿色能源', '低碳', '化石', '化石燃料',
    '煤炭', '天然气', '石油', '燃料', '燃料（复数）', '核能', '氢能',

    # 循环经济和废物管理
    '回收', '回收利用', '再利用', '减少', '零废弃', '循环经济',
    '废物管理', '焚烧', '堆肥', '可持续消费',
    '生命周期', '生命周期', '资源', '资源（复数）', '管理', '供应链',

    # 农业、土地和食品
    '农业', '农用', '耕作', '农场', '有机', '作物', '收获', '农药',
    '化肥', '退化', '侵蚀', '耕作', '灌溉', '土地使用',
    '土地覆盖', '土壤健康', '再生', '农生态学', '修复',

    # 城市及政策
    '城市', '城市化', '智慧城市', '城市', '基础设施', '交通',
    '流动性', '规划', '发展', '区划', '绿色屋顶', '绿化',
    '环境政策', '法规', '法律', '法案', '可持续发展目标', '可持续发展目标(复数)', '正义',
    '环境影响评价', '影响', '评估', '监测', '执法', '处罚',

    # 水资源
    '水', '水生', '水文学', '水文的', '河流', '溪流', '水库',
    '含水层', '湿地', '沼泽', '湖泊', '海洋', '地下水', '排水',
    '海水淡化', '径流',

    # 其他
    '化学圈', '生态圈', '生物圈', '大气层', '地圈', '人类世',
    '环境健康', '暴露', '毒理学', '生态毒理学', '可持续发展目标', '目标',
    '清洁生产', '气候韧性', '碳中和', '转型', '行星性',

    # Основные и общие термины
    'окружающая среда', 'экологический', 'экология', 'экологический', 'эко', 'зеленый',
    'устойчивость', 'устойчивый', 'природа', 'естественный', 'планета', 'земля', 'сфера',

    # Экосистема и биоразнообразие
    'экосистема', 'местообитание', 'биоразнообразие', 'виды', 'флора', 'фауна', 'сохранение',
    'охрана', 'дикая природа', 'вымирающий', 'вымирание', 'морской', 'лес', 'джунгли',
    'водно-болотные угодья', 'риф', 'океан', 'арктика', 'ледник', 'река', 'водосбор', 'бассейн',
    'пустыня', 'почва', 'земля', 'ландшафт', 'биом',

    # Загрязнение и загрязнители
    'загрязнение', 'загрязнитель', 'воздух', 'вода', 'почва', 'микропластик', 'пластик',
    'выброс', 'выбросы', 'токсичный', 'опасный', 'загрязнитель', 'загрязнение',
    'отходы', 'сточные воды', 'сток', 'слив', 'свалка', 'электронные отходы', 'полигон',
    'канализация', 'смог', 'газы', 'co2', 'метан', 'озон', 'химический', 'химикаты',
    'тяжелые металлы', 'взвешенные частицы', 'частицы', 'зола', 'сажa', 'биоразлагаемый',
    'разлагаемый', 'компост', 'компостируемый', 'разложение',

    # Изменение климата
    'климат', 'потепление', 'охлаждение', 'температура', 'погода', 'углерод', 'карбонизация',
    'парниковый', 'изменение климата', 'глобальное потепление', 'море', 'наводнение', 'засуха',
    'шторм', 'циклон', 'торнадо', 'ураган', 'устойчивость', 'смягчение', 'адаптация',
    'ipcc', 'пойма', 'подъем', 'таяние', 'чистый ноль', 'нейтральность', 'нейтральный',
    'экологический след', 'улавливание углерода', 'компенсация', 'выбросы', 'сокращение', 'энергия',

    # Энергия и топливо
    'энергия', 'возобновляемый', 'возобновляемые источники', 'солнечная', 'ветровая', 'гидро', 'гидроэнергия',
    'биоэнергия', 'биомасса', 'геотермальная', 'фотовольтаика', 'эффективность', 'умная сеть',
    'чистая', 'чистая энергия', 'зеленая энергия', 'низкоуглеродная', 'ископаемое', 'ископаемые',
    'уголь', 'газ', 'нефть', 'топливо', 'топливо (множественное)', 'ядерная', 'водород',

    # Циркулярная экономика и управление отходами
    'переработка', 'переработка отходов', 'повторное использование', 'сокращение', 'нулевые отходы', 'циркулярная экономика',
    'управление отходами', 'сжигание', 'компостирование', 'устойчивое потребление',
    'жизненный цикл', 'жизненный цикл', 'ресурс', 'ресурсы', 'управление', 'цепочка поставок',

    # Сельское хозяйство, земля и продукты питания
    'сельское хозяйство', 'агро', 'фермерство', 'ферма', 'органический', 'урожай', 'сбор урожая', 'пестицид',
    'удобрение', 'деградация', 'эрозия', 'обработка почвы', 'орошение', 'землепользование',
    'покрытие земли', 'здоровье почвы', 'регенеративный', 'агроэкология', 'восстановление',

    # Урбанистика и политика
    'городской', 'урбанизация', 'умный город', 'город', 'инфраструктура', 'транспорт',
    'мобильность', 'планирование', 'развитие', 'зонирование', 'зеленая крыша', 'озеленение',
    'экологическая политика', 'регулирование', 'закон', 'акт', 'цели устойчивого развития', 'цели устойчивого развития (множественное число)', 'справедливость',
    'оценка воздействия на окружающую среду', 'воздействие', 'оценка', 'мониторинг', 'принуждение к исполнению', 'штраф',

    # Вода
    'вода', 'водный', 'гидрология', 'гидрологический', 'река', 'ручей', 'водохранилище',
    'водоносный горизонт', 'болото', 'топь', 'озеро', 'океан', 'подземные воды', 'дренаж',
    'опреснение', 'сток',

    # Разное
    'хемосфера', 'экосфера', 'биосфера', 'атмосфера', 'геосфера', 'антропоцен',
    'экологическое здоровье', 'воздействие', 'токсикология', 'экотоксикология', 'цели устойчивого развития', 'цель',
    'чистое производство', 'климатическая устойчивость', 'углеродная нейтральность', 'переход', 'планетарный',

    # 基本および一般用語
    '環境', '環境の', '生態学', '生態の', 'エコ', '緑',
    '持続可能性', '持続可能な', '自然', '天然', '惑星', '地球', '球体',

    # 生態系と生物多様性
    '生態系', '生息地', '生物多様性', '種', '植物相', '動物相', '保全',
    '保存', '野生生物', '絶滅危惧', '絶滅', '海洋の', '森林', 'ジャングル',
    '湿地', 'サンゴ礁', '海洋', '北極', '氷河', '川', '流域', '盆地',
    '砂漠', '土壌', '土地', '景観', 'バイオーム',

    # 汚染と汚染物質
    '汚染', '汚染物質', '空気', '水', '土壌', 'マイクロプラスチック', 'プラスチック',
    '排出', '排出物', '有毒な', '有害な', '汚染物質', '汚染',
    '廃棄物', '廃水', '流出', '放出', '投棄', '電子廃棄物', '埋め立て地',
    '下水', 'スモッグ', 'ガス', '二酸化炭素', 'メタン', 'オゾン', '化学物質', '化学物質（複数）',
    '重金属', '粒子状物質', '粒子', '灰', 'すす', '生分解性',
    '分解可能', '堆肥', '堆肥化可能', '分解',

    # 気候変動
    '気候', '温暖化', '冷却', '温度', '天気', '炭素', '炭酸化',
    '温室', '気候変動', '地球温暖化', '海', '洪水', '干ばつ',
    '嵐', 'サイクロン', '竜巻', 'ハリケーン', '回復力', '緩和', '適応',
    'IPCC', '氾濫原', '上昇', '融解', 'ネットゼロ', '中立性', '中立',
    'フットプリント', '炭素回収', '相殺', '排出', '削減', 'エネルギー',

    # エネルギーと燃料
    'エネルギー', '再生可能', '再生可能エネルギー', '太陽光', '風力', '水力', '水力発電',
    'バイオエネルギー', 'バイオマス', '地熱', '太陽光発電', '効率', 'スマートグリッド',
    'クリーン', 'クリーンエネルギー', 'グリーンエネルギー', '低炭素', '化石', '化石燃料',
    '石炭', 'ガス', '石油', '燃料', '燃料（複数）', '原子力', '水素',

    # 循環型経済と廃棄物管理
    'リサイクル', 'リサイクル活動', '再利用', '削減', 'ゼロウェイスト', '循環型経済',
    '廃棄物管理', '焼却', '堆肥化', '持続可能な消費',
    'ライフサイクル', 'ライフサイクル', '資源', '資源（複数）', '管理', 'サプライチェーン',

    # 農業、土地、食料
    '農業', '農業関連', '農業', '農場', '有機', '作物', '収穫', '農薬',
    '肥料', '劣化', '侵食', '耕作', '灌漑', '土地利用',
    '土地被覆', '土壌の健康', '再生型', '農業生態学', '回復',

    # 都市および政策
    '都市', '都市化', 'スマートシティ', '市', 'インフラ', '輸送',
    'モビリティ', '計画', '開発', 'ゾーニング', 'グリーンルーフ', '緑化',
    '環境政策', '規制', '法律', '法案', '持続可能な開発目標', '持続可能な開発目標（複数）', '正義',
    '環境影響評価', '影響', '評価', 'モニタリング', '施行', '罰則',

    # 水関連
    '水', '水生', '水文学', '水文の', '川', '小川', '貯水池',
    '帯水層', '湿地', '沼地', '湖', '海洋', '地下水', '排水',
    '淡水化', '流出',

    # その他
    'ケモスフィア', 'エコスフィア', 'バイオスフィア', '大気圏', '地圏', '人新世',
    '環境健康', '暴露', '毒性学', '環境毒性学', '持続可能な開発目標', '目標',
    'クリーン生産', '気候レジリエンス', 'カーボンニュートラル', '移行', '惑星規模',

    # 핵심 및 일반 용어
    '환경', '환경의', '생태학', '생태학적인', '에코', '녹색',
    '지속 가능성', '지속 가능한', '자연', '자연스러운', '행성', '지구', '구체',

    # 생태계 및 생물 다양성
    '생태계', '서식지', '생물 다양성', '종', '식물군', '동물군', '보존',
    '보호', '야생 동물', '멸종 위기', '멸종', '해양', '숲', '정글',
    '습지', '산호초', '바다', '북극', '빙하', '강', '유역', '분지',
    '사막', '토양', '토지', '경관', '생물 군계',

    # 오염 및 오염물
    '오염', '오염물질', '공기', '물', '토양', '미세 플라스틱', '플라스틱',
    '배출', '배출물', '독성', '위험한', '오염 물질', '오염',
    '폐기물', '폐수', '유출', '배출', '투기', '전자 폐기물', '매립지',
    '하수', '스모그', '가스', '이산화탄소', '메탄', '오존', '화학 물질', '화학 물질들',
    '중금속', '입자', '입자들', '재', '그을음', '생분해성',
    '분해 가능한', '퇴비', '퇴비화 가능한', '분해',

    # 기후 변화
    '기후', '온난화', '냉각', '온도', '날씨', '탄소', '탄산화',
    '온실', '기후 변화', '지구 온난화', '바다', '홍수', '가뭄',
    '폭풍', '사이클론', '토네이도', '허리케인', '탄력성', '완화', '적응',
    'IPCC', '홍수원', '상승', '융해', '넷제로', '중립성', '중립',
    '탄소 발자국', '탄소 포집', '상쇄', '배출', '감소', '에너지',

    # 에너지 및 연료
    '에너지', '재생 가능', '재생 가능 에너지', '태양광', '풍력', '수력', '수력 발전',
    '바이오 에너지', '바이오매스', '지열', '태양광 발전', '효율', '스마트 그리드',
    '청정', '청정 에너지', '녹색 에너지', '저탄소', '화석', '화석 연료',
    '석탄', '가스', '석유', '연료', '연료들', '핵', '수소',

    # 순환 경제 및 폐기물 관리
    '재활용', '재활용', '재사용', '감소', '제로 웨이스트', '순환 경제',
    '폐기물 관리', '소각', '퇴비화', '지속 가능한 소비',
    '수명 주기', '수명 주기', '자원', '자원들', '관리', '공급망',

    # 농업, 토지 및 식품
    '농업', '농업', '농사', '농장', '유기농', '작물', '수확', '농약',
    '비료', '퇴화', '침식', '경운', '관개', '토지 이용',
    '토지 피복', '토양 건강', '재생', '농생태학', '복원',

    # 도시 및 정책
    '도시', '도시화', '스마트 시티', '도시', '인프라', '교통',
    '모빌리티', '계획', '개발', '구역 지정', '녹색 지붕', '녹화',
    '환경 정책', '규제', '법률', '법안', '지속가능발전목표', '지속가능발전목표들', '정의',
    '환경영향평가', '영향', '평가', '모니터링', '집행', '벌칙',

    # 수자원
    '물', '수생의', '수문학', '수문학적', '강', '시내', '저수지',
    '대수층', '습지', '늪', '호수', '바다', '지하수', '배수',
    '해수 담수화', '유출',

    # 기타
    '화학권', '생태권', '생물권', '대기권', '지권', '인류세',
    '환경 건강', '노출', '독성학', '생태독성학', '지속가능발전목표', '목표',
    '청정 생산', '기후 탄력성', '탄소 중립', '전환', '행성',

    # 核心與一般詞彙
    '環境', '環境的', '生態學', '生態的', '生態', '綠色',
    '永續性', '永續的', '自然', '天然', '星球', '地球', '球體',

    # 生態系統與生物多樣性
    '生態系統', '棲息地', '生物多樣性', '物種', '植物群', '動物群', '保育',
    '保存', '野生動物', '瀕危', '絕種', '海洋的', '森林', '叢林',
    '濕地', '珊瑚礁', '海洋', '北極', '冰川', '河流', '集水區', '盆地',
    '沙漠', '土壤', '土地', '景觀', '生物群系',

    # 污染與污染物
    '污染', '污染物', '空氣', '水', '土壤', '微塑膠', '塑膠',
    '排放', '排放物', '有毒', '有害', '污染物', '污染',
    '廢棄物', '廢水', '逕流', '排放', '傾倒', '電子廢棄物', '垃圾掩埋場',
    '污水', '煙霧', '氣體', '二氧化碳', '甲烷', '臭氧', '化學物質', '化學品',
    '重金屬', '顆粒物', '顆粒', '灰燼', '煤煙', '可生物分解',
    '可降解', '堆肥', '可堆肥', '分解',

    # 氣候變遷
    '氣候', '暖化', '冷卻', '溫度', '天氣', '碳', '碳酸化',
    '溫室', '氣候變遷', '全球暖化', '海洋', '洪水', '乾旱',
    '風暴', '氣旋', '龍捲風', '颶風', '韌性', '緩解', '適應',
    '氣候變遷政府間專門委員會', '氾濫平原', '上升', '融化', '淨零', '中和', '中性',
    '足跡', '碳捕捉', '抵銷', '排放', '減少', '能源',

    # 能源與燃料
    '能源', '再生能源', '再生能源（複數）', '太陽能', '風能', '水力', '水電',
    '生物能源', '生物質', '地熱', '光伏', '效率', '智慧電網',
    '乾淨', '清潔能源', '綠色能源', '低碳', '化石', '化石燃料',
    '煤炭', '天然氣', '石油', '燃料', '燃料（複數）', '核能', '氫氣',

    # 循環經濟與廢棄物管理
    '回收', '回收利用', '再利用', '減少', '零廢棄', '循環經濟',
    '廢棄物管理', '焚化', '堆肥', '永續消費',
    '生命週期', '生命週期', '資源', '資源（複數）', '管理', '供應鏈',

    # 農業、土地與食物
    '農業', '農業', '耕作', '農場', '有機', '作物', '收穫', '農藥',
    '肥料', '退化', '侵蝕', '耕作', '灌溉', '土地利用',
    '土地覆蓋', '土壤健康', '再生', '農業生態學', '復育',

    # 城市與政策
    '城市', '都市化', '智慧城市', '城市', '基礎建設', '交通',
    '流動性', '規劃', '發展', '分區', '綠屋頂', '綠化',
    '環境政策', '規範', '法律', '法案', '永續發展目標', '永續發展目標(複數)', '正義',
    '環境影響評估', '影響', '評估', '監測', '執法', '處罰',

    # 水資源
    '水', '水生', '水文學', '水文的', '河流', '溪流', '水庫',
    '含水層', '濕地', '沼澤', '湖泊', '海洋', '地下水', '排水',
    '海水淡化', '逕流',

    # 其他
    '化學圈', '生態圈', '生物圈', '大氣層', '地圈', '人類世',
    '環境健康', '暴露', '毒理學', '生態毒理學', '永續發展目標', '目標',
    '清潔生產', '氣候韌性', '碳中和', '轉型', '行星',

    # Základní a obecné pojmy
    'životní prostředí', 'environmentální', 'ekologie', 'ekologický', 'eko', 'zelený',
    'udržitelnost', 'udržitelný', 'příroda', 'přirozený', 'planeta', 'země', 'sféra',

    # Ekosystém a biodiverzita
    'ekosystém', 'habitat', 'biodiverzita', 'druhy', 'flora', 'fauna', 'ochrana',
    'zachování', 'divoká příroda', 'ohrožený', 'vyhynutí', 'mořský', 'les', 'džungle',
    'mokřady', 'korálový útes', 'oceán', 'arktida', 'ledovec', 'řeka', 'povodí', 'pánve',
    'poušť', 'půda', 'země', 'krajina', 'biom',

    # Znečištění a kontaminace
    'znečištění', 'znečišťující látka', 'vzduch', 'voda', 'půda', 'mikroplast', 'plast',
    'emise', 'emise (množné číslo)', 'toxický', 'nebezpečný', 'kontaminant', 'kontaminace',
    'odpad', 'odpadní voda', 'odtok', 'vypouštění', 'skládka', 'elektronický odpad', 'skládka odpadu',
    'kanalizace', 'smog', 'plyny', 'co2', 'metan', 'ozón', 'chemikálie', 'chemikálie (množné číslo)',
    'těžké kovy', 'částice', 'částice (množné číslo)', 'popel', 'saze', 'biologicky rozložitelný',
    'rozložitelný', 'kompost', 'kompostovatelný', 'rozklad',

    # Klimatické změny
    'klima', 'oteplování', 'ochlazování', 'teplota', 'počasí', 'uhlík', 'karbonatace',
    'skleníkový', 'změna klimatu', 'globální oteplování', 'moře', 'povodeň', 'sucho',
    'bouře', 'cyklón', 'tornádo', 'hurikán', 'odolnost', 'zmírnění', 'adaptace',
    'ipcc', 'záplavová oblast', 'stoupání', 'tání', 'čistá nula', 'neutralita', 'neutrální',
    'uhlíková stopa', 'zachytávání uhlíku', 'kompenzace', 'emise', 'snížení', 'energie',

    # Energie a paliva
    'energie', 'obnovitelný', 'obnovitelné zdroje', 'solární', 'větrný', 'vodní', 'vodní energie',
    'bioenergie', 'biomasa', 'geotermální', 'fotovoltaický', 'účinnost', 'chytrá síť',
    'čistý', 'čistá energie', 'zelená energie', 'nízkouhlíkový', 'fosilní', 'fosilní paliva',
    'uhlí', 'plyn', 'ropa', 'palivo', 'paliva', 'jaderný', 'vodík',

    # Cirkulární ekonomika a nakládání s odpady
    'recyklace', 'recyklování', 'znovu použití', 'snížení', 'nulový odpad', 'cirkulární ekonomika',
    'nakládání s odpady', 'spalování', 'kompostování', 'udržitelná spotřeba',
    'životní cyklus', 'životní cyklus', 'zdroj', 'zdroje', 'řízení', 'dodavatelský řetězec',

    # Zemědělství, půda a potraviny
    'zemědělství', 'agro', 'zemědělství', 'farma', 'organický', 'plodina', 'sklizeň', 'pesticid',
    'hnojivo', 'degradace', 'eroze', 'orbě', 'zavlažování', 'využití půdy',
    'pokryv půdy', 'zdraví půdy', 'regenerativní', 'agroekologie', 'obnova',

    # Město a politika
    'městský', 'urbanizace', 'chytré město', 'město', 'infrastruktura', 'doprava',
    'mobilita', 'plánování', 'rozvoj', 'zónování', 'zelená střecha', 'zeleň',
    'environmentální politika', 'regulace', 'zákon', 'akt', 'sdg', 'sdgs', 'spravedlnost',
    'eia', 'dopad', 'hodnocení', 'monitorování', 'prosazování', 'pokuta',

    # Voda
    'voda', 'vodní', 'hydrologie', 'hydrologický', 'řeka', 'potok', 'nádrž',
    'zásobárna vody', 'mokřad', 'bažina', 'jezero', 'oceán', 'podzemní voda', 'odvodnění',
    'odsolování', 'odtok',

    # Různé
    'chemosféra', 'ekosféra', 'biosféra', 'atmosféra', 'geosféra', 'antropocén',
    'environmentální zdraví', 'expozice', 'toxikologie', 'ekotoxikologie', 'sdg', 'cíl',
    'čistší výroba', 'klimatická odolnost', 'uhlíková neutralita', 'přechod', 'planetární',

    # المصطلحات الأساسية والعامة
    'البيئة', 'بيئي', 'علم البيئة', 'إيكولوجي', 'إيكو', 'أخضر',
    'الاستدامة', 'مستدام', 'الطبيعة', 'طبيعي', 'الكوكب', 'الأرض', 'الكرة',

    # النظام البيئي والتنوع البيولوجي
    'النظام البيئي', 'موطن', 'التنوع البيولوجي', 'الأنواع', 'النباتات', 'الحيوانات', 'الحفظ',
    'الحفاظ', 'الحياة البرية', 'مهدد بالانقراض', 'الانقراض', 'بحري', 'غابة', 'غابة مطيرة',
    'الأراضي الرطبة', 'الحيد', 'المحيط', 'القطب الشمالي', 'الأنهار الجليدية', 'نهر', 'حوض مائي', 'حوض',
    'صحراء', 'تربة', 'أرض', 'مشهد طبيعي', 'حزام حيوي',

    # التلوث والتسمم
    'تلوث', 'مُلوث', 'هواء', 'ماء', 'تربة', 'جسيمات بلاستيكية دقيقة', 'بلاستيك',
    'انبعاث', 'انبعاثات', 'سام', 'خطير', 'مُلوث', 'تلوث',
    'نفايات', 'مياه الصرف الصحي', 'جريان سطحي', 'تصريف', 'إغراق', 'نفايات إلكترونية', 'مطمر نفايات',
    'مياه الصرف', 'ضباب الدخان', 'غازات', 'ثاني أكسيد الكربون', 'ميثان', 'أوزون', 'مادة كيميائية', 'مواد كيميائية',
    'معادن ثقيلة', 'جسيم', 'جسيمات', 'رماد', 'سخام', 'قابل للتحلل الحيوي',
    'قابل للتحلل', 'سماد عضوي', 'قابل للتسميد', 'تحلل',

    # تغير المناخ
    'مناخ', 'احترار', 'تبريد', 'درجة الحرارة', 'طقس', 'كربون', 'تكوّن الكربونات',
    'الاحتباس الحراري', 'تغير المناخ', 'الاحتباس الحراري العالمي', 'بحر', 'فيضان', 'جفاف',
    'عاصفة', 'إعصار مداري', 'إعصار', 'إعصار هوريكي', 'مرونة', 'تخفيف', 'تكيّف',
    'اللجنة الحكومية الدولية المعنية بتغير المناخ', 'سهل فيضاني', 'ارتفاع', 'ذوبان', 'صفر صافي', 'حياد', 'محايد',
    'بصمة', 'احتجاز الكربون', 'تعويض', 'انبعاثات', 'تقليل', 'طاقات',

    # الطاقة والوقود
    'طاقة', 'متجددة', 'مصادر الطاقة المتجددة', 'طاقة شمسية', 'طاقة رياح', 'طاقة مائية', 'الطاقة الكهرومائية',
    'الطاقة الحيوية', 'كتلة حيوية', 'حرارية أرضية', 'كهرباء ضوئية', 'كفاءة', 'شبكة ذكية',
    'نظيفة', 'طاقة نظيفة', 'طاقة خضراء', 'انبعاثات كربون منخفضة', 'أحفوري', 'وقود أحفوري',
    'فحم', 'غاز', 'نفط', 'وقود', 'وقود (جمع)', 'نووي', 'هيدروجين',

    # الاقتصاد الدائري وإدارة النفايات
    'إعادة التدوير', 'إعادة التدوير', 'إعادة الاستخدام', 'التقليل', 'صفر نفايات', 'الاقتصاد الدائري',
    'إدارة النفايات', 'الحرق', 'التسميد', 'الاستهلاك المستدام',
    'دورة الحياة', 'دورة الحياة', 'مورد', 'موارد', 'إدارة', 'سلسلة التوريد',

    # الزراعة والأرض والطعام
    'زراعة', 'زراعي', 'الزراعة', 'مزرعة', 'عضوي', 'محصول', 'حصاد', 'مبيد حشري',
    'سماد', 'تدهور', 'تعرية', 'حرث', 'ري', 'استخدام الأرض',
    'تغطية الأرض', 'صحة التربة', 'تجديدي', 'الزراعة الإيكولوجية', 'استعادة',

    # الحضر والسياسات
    'حضري', 'تحضر', 'مدينة ذكية', 'مدينة', 'بنية تحتية', 'نقل',
    'تنقل', 'تخطيط', 'تطوير', 'تقسيم المناطق', 'سطح أخضر', 'تخضير',
    'سياسة بيئية', 'تنظيم', 'قانون', 'تشريع', 'الأهداف التنموية المستدامة', 'الأهداف التنموية المستدامة (جمع)', 'عدالة',
    'تقييم الأثر البيئي', 'تأثير', 'تقييم', 'مراقبة', 'إنفاذ', 'عقوبة',

    # المياه
    'ماء', 'مائي', 'هيدرولوجيا', 'هيدرولوجي', 'نهر', 'جدول', 'خزان',
    'مستودع المياه الجوفية', 'أراضي رطبة', 'مستنقع', 'بحيرة', 'محيط', 'مياه جوفية', 'صرف',
    'تحلية', 'جريان سطحي',

    # متفرقات
    'الكرة الكيميائية', 'الكرة البيئية', 'الكرة الحيوية', 'الغلاف الجوي', 'الكرة الأرضية', 'العصر الأنثروبوسيني',
    'الصحة البيئية', 'تعرض', 'علم السموم', 'علم السموم البيئي', 'الأهداف التنموية المستدامة', 'هدف',
    'الإنتاج الأنظف', 'مرونة المناخ', 'حياد الكربون', 'الانتقال', 'كوكبي'
],

    'economics': [
        # Core economic terms
    'economy', 'economic', 'economics', 'macroeconomics', 'microeconomics', 'market',
    'supply', 'demand', 'scarcity', 'tradeoff', 'utility', 'cost', 'benefit', 'equilibrium',
    'elasticity', 'price', 'consumer', 'producer', 'output', 'input', 'production',

    # Economic indicators
    'gdp', 'inflation', 'deflation', 'cpi', 'ppi', 'index', 'growth', 'unemployment',
    'employment', 'labor', 'wage', 'income', 'productivity', 'rate', 'forecast', 'trend',

    # Finance and investment
    'finance', 'financial', 'capital', 'asset', 'liability', 'equity', 'stock', 'bond',
    'portfolio', 'investment', 'investor', 'return', 'dividend', 'valuation', 'hedge',
    'mutual', 'fund', 'risk', 'volatility', 'leverage', 'arbitrage', 'securities',
    'derivative', 'option', 'futures', 'equity', 'cashflow', 'alpha', 'beta', 'sharpe',

    # Banking and monetary policy
    'bank', 'banking', 'central', 'monetary', 'fiscal', 'liquidity', 'money', 'currency',
    'interest', 'policy', 'qe', 'quantitative', 'credit', 'debt', 'loan', 'deposit',
    'reserve', 'inflation targeting', 'discount rate', 'money supply',

    # Trade and globalization
    'trade', 'global', 'globalization', 'import', 'export', 'exchange', 'tariff',
    'bop', 'balance', 'deficit', 'surplus', 'fdi', 'foreign', 'cross-border', 'currency',

    # Government, taxation, and public finance
    'tax', 'taxes', 'subsidy', 'budget', 'public', 'spending', 'government',
    'expenditure', 'fiscal', 'revenue', 'national', 'debt', 'deficit', 'policy',
    'redistribution', 'stimulus', 'infrastructure',

    # Development and inequality
    'inequality', 'poverty', 'development', 'sustainable', 'sdg', 'growth', 'human capital',
    'basic income', 'education', 'health', 'nutrition', 'social protection', 'informal sector',

    # Labor and employment
    'employment', 'job', 'unemployment', 'labor', 'labour', 'force', 'wage',
    'job creation', 'jobless', 'employment rate', 'skills', 'productivity',

    # Theoretical and empirical methods
    'econometrics', 'regression', 'panel', 'cross-section', 'instrumental', 'tsls',
    'time series', 'data', 'causality', 'model', 'simulation', 'forecasting', 'structural',
    'estimation', 'hypothesis', 'null', 'control group', 'experiment', 'survey',

    # Behavioral, institutional, and applied economics
    'behavioral', 'game', 'strategy', 'decision', 'auction', 'bias', 'bounded rationality',
    'nudge', 'preference', 'psychology', 'rules', 'institutions', 'incentive', 'mechanism',
    'choice', 'rational', 'irrational', 'public choice',

    # Environmental and ESG finance
    'green', 'sustainable', 'esg', 'carbon', 'externality', 'pollution', 'climate',
    'environmental', 'pricing', 'carbon market', 'carbon tax', 'cap and trade',
    'valuation', 'natural capital', 'resources', 'renewables',

    # Global orgs and institutions
    'imf', 'world bank', 'wto', 'oecd', 'ecb', 'fed', 'eurozone', 'emerging', 'developed',
    'smc', 'sme', 'enterprise', 'entrepreneurship', 'startup', 'venture',

    # Miscellaneous real-world terms
    'recession', 'boom', 'bust', 'crisis', 'bailout', 'housing', 'bubble', 'fiscal cliff',
    'sovereign debt', 'market failure', 'externality', 'opportunity cost', 'energies',

    # Términos económicos básicos
    'economía', 'económico', 'economía (disciplina)', 'macroeconomía', 'microeconomía', 'mercado',
    'oferta', 'demanda', 'escasez', 'compensación', 'utilidad', 'costo', 'beneficio', 'equilibrio',
    'elasticidad', 'precio', 'consumidor', 'productor', 'producción', 'insumo', 'producción',

    # Indicadores económicos
    'producto interno bruto', 'inflación', 'deflación', 'índice de precios al consumidor', 'índice de precios al productor', 'índice', 'crecimiento', 'desempleo',
    'empleo', 'trabajo', 'salario', 'ingreso', 'productividad', 'tasa', 'pronóstico', 'tendencia',

    # Finanzas e inversión
    'finanzas', 'financiero', 'capital', 'activo', 'pasivo', 'patrimonio', 'acción', 'bono',
    'portafolio', 'inversión', 'inversor', 'retorno', 'dividendo', 'valoración', 'cobertura',
    'fondo mutuo', 'fondo', 'riesgo', 'volatilidad', 'apalancamiento', 'arbitraje', 'valores',
    'derivado', 'opción', 'futuros', 'patrimonio', 'flujo de caja', 'alfa', 'beta', 'sharpe',

    # Banca y política monetaria
    'banco', 'banca', 'central', 'monetario', 'fiscal', 'liquidez', 'dinero', 'moneda',
    'interés', 'política', 'expansión cuantitativa', 'cuantitativo', 'crédito', 'deuda', 'préstamo', 'depósito',
    'reserva', 'objetivo de inflación', 'tasa de descuento', 'oferta monetaria',

    # Comercio y globalización
    'comercio', 'global', 'globalización', 'importación', 'exportación', 'intercambio', 'arancel',
    'balanza de pagos', 'balanza', 'déficit', 'superávit', 'inversión extranjera directa', 'extranjero', 'transfronterizo', 'moneda',

    # Gobierno, impuestos y finanzas públicas
    'impuesto', 'impuestos', 'subsidio', 'presupuesto', 'público', 'gasto', 'gobierno',
    'gasto público', 'fiscal', 'ingresos', 'nacional', 'deuda', 'déficit', 'política',
    'redistribución', 'estímulo', 'infraestructura',

    # Desarrollo y desigualdad
    'desigualdad', 'pobreza', 'desarrollo', 'sostenible', 'objetivos de desarrollo sostenible', 'crecimiento', 'capital humano',
    'ingreso básico', 'educación', 'salud', 'nutrición', 'protección social', 'sector informal',

    # Trabajo y empleo
    'empleo', 'trabajo', 'desempleo', 'mano de obra', 'fuerza laboral', 'salario',
    'creación de empleo', 'sin empleo', 'tasa de empleo', 'habilidades', 'productividad',

    # Métodos teóricos y empíricos
    'econometría', 'regresión', 'panel', 'sección transversal', 'instrumental', 'tsls',
    'series temporales', 'datos', 'causalidad', 'modelo', 'simulación', 'pronóstico', 'estructural',
    'estimación', 'hipótesis', 'nula', 'grupo de control', 'experimento', 'encuesta',

    # Economía conductual, institucional y aplicada
    'conductual', 'juego', 'estrategia', 'decisión', 'subasta', 'sesgo', 'racionalidad limitada',
    'empujón', 'preferencia', 'psicología', 'reglas', 'instituciones', 'incentivo', 'mecanismo',
    'elección', 'racional', 'irracional', 'elección pública',

    # Finanzas ambientales y ESG
    'verde', 'sostenible', 'esg', 'carbono', 'externalidad', 'contaminación', 'clima',
    'ambiental', 'precio', 'mercado de carbono', 'impuesto al carbono', 'límite y comercio',
    'valoración', 'capital natural', 'recursos', 'renovables',

    # Organizaciones globales e instituciones
    'fmi', 'banco mundial', 'omc', 'ocde', 'bce', 'fed', 'zona euro', 'emergente', 'desarrollado',
    'pyme', 'pyme', 'empresa', 'emprendimiento', 'startup', 'riesgo',

    # Términos diversos del mundo real
    'recesión', 'auge', 'caída', 'crisis', 'rescate', 'vivienda', 'burbuja', 'precipicio fiscal',
    'deuda soberana', 'fallo de mercado', 'externalidad', 'costo de oportunidad', 'energías',

    # Termos econômicos básicos
    'economia', 'econômico', 'economia (disciplina)', 'macroeconomia', 'microeconomia', 'mercado',
    'oferta', 'demanda', 'escassez', 'compensação', 'utilidade', 'custo', 'benefício', 'equilíbrio',
    'elasticidade', 'preço', 'consumidor', 'produtor', 'produção', 'insumo', 'produção',

    # Indicadores econômicos
    'produto interno bruto', 'inflação', 'deflação', 'índice de preços ao consumidor', 'índice de preços ao produtor', 'índice', 'crescimento', 'desemprego',
    'emprego', 'trabalho', 'salário', 'renda', 'produtividade', 'taxa', 'previsão', 'tendência',

    # Finanças e investimento
    'finanças', 'financeiro', 'capital', 'ativo', 'passivo', 'patrimônio', 'ação', 'título',
    'portfólio', 'investimento', 'investidor', 'retorno', 'dividendo', 'avaliação', 'hedge',
    'fundo mútuo', 'fundo', 'risco', 'volatilidade', 'alavancagem', 'arbitragem', 'valores mobiliários',
    'derivado', 'opção', 'futuros', 'patrimônio', 'fluxo de caixa', 'alfa', 'beta', 'sharpe',

    # Bancos e política monetária
    'banco', 'bancário', 'central', 'monetário', 'fiscal', 'liquidez', 'dinheiro', 'moeda',
    'juros', 'política', 'afrouxamento quantitativo', 'quantitativo', 'crédito', 'dívida', 'empréstimo', 'depósito',
    'reserva', 'meta de inflação', 'taxa de desconto', 'oferta monetária',

    # Comércio e globalização
    'comércio', 'global', 'globalização', 'importação', 'exportação', 'câmbio', 'tarifa',
    'balança de pagamentos', 'balança', 'déficit', 'superávit', 'investimento estrangeiro direto', 'estrangeiro', 'transfronteiriço', 'moeda',

    # Governo, tributação e finanças públicas
    'imposto', 'impostos', 'subsídio', 'orçamento', 'público', 'gastos', 'governo',
    'despesa', 'fiscal', 'receita', 'nacional', 'dívida', 'déficit', 'política',
    'redistribuição', 'estímulo', 'infraestrutura',

    # Desenvolvimento e desigualdade
    'desigualdade', 'pobreza', 'desenvolvimento', 'sustentável', 'objetivos de desenvolvimento sustentável', 'crescimento', 'capital humano',
    'renda básica', 'educação', 'saúde', 'nutrição', 'proteção social', 'setor informal',

    # Trabalho e emprego
    'emprego', 'trabalho', 'desemprego', 'força de trabalho', 'força laboral', 'salário',
    'criação de empregos', 'desempregado', 'taxa de emprego', 'habilidades', 'produtividade',

    # Métodos teóricos e empíricos
    'econometria', 'regressão', 'painel', 'seção transversal', 'instrumental', 'tsls',
    'séries temporais', 'dados', 'causalidade', 'modelo', 'simulação', 'previsão', 'estrutural',
    'estimação', 'hipótese', 'nula', 'grupo controle', 'experimento', 'pesquisa',

    # Economia comportamental, institucional e aplicada
    'comportamental', 'jogo', 'estratégia', 'decisão', 'leilão', 'viés', 'racionalidade limitada',
    'empurrão', 'preferência', 'psicologia', 'regras', 'instituições', 'incentivo', 'mecanismo',
    'escolha', 'racional', 'irracional', 'escolha pública',

    # Finanças ambientais e ESG
    'verde', 'sustentável', 'esg', 'carbono', 'externalidade', 'poluição', 'clima',
    'ambiental', 'precificação', 'mercado de carbono', 'imposto sobre carbono', 'cap and trade',
    'avaliação', 'capital natural', 'recursos', 'renováveis',

    # Organizações globais e instituições
    'fmi', 'banco mundial', 'omc', 'ocde', 'bce', 'fed', 'zona do euro', 'emergente', 'desenvolvido',
    'pmc', 'pme', 'empresa', 'empreendedorismo', 'startup', 'venture',

    # Termos diversos do mundo real
    'recessão', 'boom', 'crise', 'salvamento', 'habitação', 'bolha', 'precipício fiscal',
    'dívida soberana', 'falha de mercado', 'externalidade', 'custo de oportunidade', 'energias',

    # Termini economici di base
    'economia', 'economico', 'economia (disciplina)', 'macroeconomia', 'microeconomia', 'mercato',
    'offerta', 'domanda', 'scarsità', 'compromesso', 'utilità', 'costo', 'beneficio', 'equilibrio',
    'elasticità', 'prezzo', 'consumatore', 'produttore', 'produzione', 'input', 'produzione',

    # Indicatori economici
    'pil', 'inflazione', 'deflazione', 'ipc', 'ipp', 'indice', 'crescita', 'disoccupazione',
    'occupazione', 'lavoro', 'salario', 'reddito', 'produttività', 'tasso', 'previsione', 'tendenza',

    # Finanza e investimenti
    'finanza', 'finanziario', 'capitale', 'attivo', 'passivo', 'equità', 'azione', 'obbligazione',
    'portafoglio', 'investimento', 'investitore', 'rendimento', 'dividendo', 'valutazione', 'copertura',
    'fondo comune', 'fondo', 'rischio', 'volatilità', 'leva', 'arbitraggio', 'titoli',
    'derivato', 'opzione', 'futures', 'equità', 'flusso di cassa', 'alfa', 'beta', 'sharpe',

    # Banche e politica monetaria
    'banca', 'bancario', 'centrale', 'monetario', 'fiscale', 'liquidità', 'denaro', 'valuta',
    'interesse', 'politica', 'qe', 'quantitativo', 'credito', 'debito', 'prestito', 'deposito',
    'riserva', 'targeting dell’inflazione', 'tasso di sconto', 'offerta di moneta',

    # Commercio e globalizzazione
    'commercio', 'globale', 'globalizzazione', 'importazione', 'esportazione', 'scambio', 'tariffa',
    'bop', 'bilancia', 'deficit', 'surplus', 'ide', 'estero', 'transfrontaliero', 'valuta',

    # Governo, tassazione e finanza pubblica
    'tassa', 'tasse', 'sussidio', 'bilancio', 'pubblico', 'spesa', 'governo',
    'spesa pubblica', 'fiscale', 'entrate', 'nazionale', 'debito', 'deficit', 'politica',
    'ridistribuzione', 'stimolo', 'infrastruttura',

    # Sviluppo e disuguaglianza
    'disuguaglianza', 'povertà', 'sviluppo', 'sostenibile', 'ods', 'crescita', 'capitale umano',
    'reddito di base', 'istruzione', 'salute', 'nutrizione', 'protezione sociale', 'settore informale',

    # Lavoro e occupazione
    'occupazione', 'lavoro', 'disoccupazione', 'lavoro', 'forza lavoro', 'salario',
    'creazione di posti di lavoro', 'senza lavoro', 'tasso di occupazione', 'competenze', 'produttività',

    # Metodi teorici ed empirici
    'econometria', 'regressione', 'panel', 'sezione trasversale', 'strumentale', 'tsls',
    'serie temporali', 'dati', 'causalità', 'modello', 'simulazione', 'previsione', 'strutturale',
    'stima', 'ipotesi', 'nulla', 'gruppo di controllo', 'esperimento', 'sondaggio',

    # Economia comportamentale, istituzionale e applicata
    'comportamentale', 'gioco', 'strategia', 'decisione', 'asta', 'bias', 'razionalità limitata',
    'spinta', 'preferenza', 'psicologia', 'regole', 'istituzioni', 'incentivo', 'meccanismo',
    'scelta', 'razionale', 'irrazionale', 'scelta pubblica',

    # Finanza ambientale e ESG
    'verde', 'sostenibile', 'esg', 'carbonio', 'esternalità', 'inquinamento', 'clima',
    'ambientale', 'prezzatura', 'mercato del carbonio', 'tassa sul carbonio', 'cap and trade',
    'valutazione', 'capitale naturale', 'risorse', 'rinnovabili',

    # Organizzazioni globali e istituzioni
    'fmi', 'banca mondiale', 'omc', 'ocse', 'bce', 'fed', 'zona euro', 'emergenti', 'sviluppate',
    'pmi', 'pmi', 'impresa', 'imprenditorialità', 'startup', 'venture',

    # Termini vari del mondo reale
    'recessione', 'boom', 'crisi', 'crisi', 'salvataggio', 'abitazioni', 'bolla', 'precipizio fiscale',
    'debito sovrano', 'fallimento del mercato', 'esternalità', 'costo opportunità', 'energie',

    # Termes économiques de base
    'économie', 'économique', 'économie (discipline)', 'macroéconomie', 'microéconomie', 'marché',
    'offre', 'demande', 'pénurie', 'compromis', 'utilité', 'coût', 'bénéfice', 'équilibre',
    'élasticité', 'prix', 'consommateur', 'producteur', 'production', 'intrant', 'production',

    # Indicateurs économiques
    'produit intérieur brut', 'inflation', 'déflation', 'indice des prix à la consommation', 'indice des prix à la production', 'indice', 'croissance', 'chômage',
    'emploi', 'travail', 'salaire', 'revenu', 'productivité', 'taux', 'prévision', 'tendance',

    # Finances et investissements
    'finances', 'financier', 'capital', 'actif', 'passif', 'capitaux propres', 'action', 'obligation',
    'portefeuille', 'investissement', 'investisseur', 'rendement', 'dividende', 'valorisation', 'couverture',
    'fonds commun', 'fonds', 'risque', 'volatilité', 'effet de levier', 'arbitrage', 'titres',
    'dérivé', 'option', 'futures', 'capitaux propres', 'flux de trésorerie', 'alpha', 'bêta', 'sharpe',

    # Banque et politique monétaire
    'banque', 'bancarisation', 'central', 'monétaire', 'fiscal', 'liquidité', 'argent', 'monnaie',
    'intérêt', 'politique', 'expansion quantitative', 'quantitatif', 'crédit', 'dette', 'prêt', 'dépôt',
    'réserve', 'objectif d’inflation', 'taux d’escompte', 'offre de monnaie',

    # Commerce et mondialisation
    'commerce', 'mondial', 'mondialisation', 'importation', 'exportation', 'échange', 'tarif',
    'balance des paiements', 'solde', 'déficit', 'excédent', 'investissement direct étranger', 'étranger', 'transfrontalier', 'monnaie',

    # Gouvernement, fiscalité et finances publiques
    'impôt', 'impôts', 'subvention', 'budget', 'public', 'dépenses', 'gouvernement',
    'dépenses publiques', 'fiscal', 'revenu', 'national', 'dette', 'déficit', 'politique',
    'redistribution', 'stimulus', 'infrastructure',

    # Développement et inégalité
    'inégalité', 'pauvreté', 'développement', 'durable', 'odsd', 'croissance', 'capital humain',
    'revenu de base', 'éducation', 'santé', 'nutrition', 'protection sociale', 'secteur informel',

    # Travail et emploi
    'emploi', 'travail', 'chômage', 'main d’œuvre', 'force de travail', 'salaire',
    'création d’emplois', 'sans emploi', 'taux d’emploi', 'compétences', 'productivité',

    # Méthodes théoriques et empiriques
    'économétrie', 'régression', 'panneau', 'section transversale', 'instrumental', 'tsls',
    'séries temporelles', 'données', 'causalité', 'modèle', 'simulation', 'prévision', 'structurel',
    'estimation', 'hypothèse', 'nulle', 'groupe de contrôle', 'expérience', 'enquête',

    # Économie comportementale, institutionnelle et appliquée
    'comportemental', 'jeu', 'stratégie', 'décision', 'enchère', 'biais', 'rationalité limitée',
    'incitation', 'préférence', 'psychologie', 'règles', 'institutions', 'incitation', 'mécanisme',
    'choix', 'rationnel', 'irrationnel', 'choix public',

    # Finances environnementales et ESG
    'vert', 'durable', 'esg', 'carbone', 'externalité', 'pollution', 'climat',
    'environnemental', 'tarification', 'marché du carbone', 'taxe carbone', 'plafond et échange',
    'valorisation', 'capital naturel', 'ressources', 'énergies renouvelables',

    # Organisations et institutions mondiales
    'fmi', 'bancaire mondial', 'omc', 'ocde', 'bce', 'fed', 'zone euro', 'émergente', 'développé',
    'pme', 'petites et moyennes entreprises', 'entreprise', 'entrepreneuriat', 'startup', 'capital-risque',

    # Termes divers du monde réel
    'récession', 'expansion', 'effondrement', 'crise', 'sauvetage', 'logement', 'bulle', 'falaise fiscale',
    'dette souveraine', 'échec du marché', 'externalité', 'coût d’opportunité', 'énergies',

    # 核心经济术语
    '经济', '经济的', '经济学', '宏观经济学', '微观经济学', '市场',
    '供给', '需求', '稀缺', '权衡', '效用', '成本', '收益', '均衡',
    '弹性', '价格', '消费者', '生产者', '产出', '投入', '生产',

    # 经济指标
    '国内生产总值', '通货膨胀', '通货紧缩', '消费者物价指数', '生产者物价指数', '指数', '增长', '失业',
    '就业', '劳动力', '工资', '收入', '生产率', '利率', '预测', '趋势',

    # 财务与投资
    '金融', '财务的', '资本', '资产', '负债', '股本', '股票', '债券',
    '投资组合', '投资', '投资者', '回报', '股息', '估值', '对冲',
    '共同基金', '基金', '风险', '波动性', '杠杆', '套利', '证券',
    '衍生品', '期权', '期货', '股本', '现金流', '阿尔法', '贝塔', '夏普',

    # 银行与货币政策
    '银行', '银行业务', '中央', '货币', '财政', '流动性', '货币', '货币种类',
    '利率', '政策', '量化宽松', '量化', '信用', '债务', '贷款', '存款',
    '储备', '通胀目标', '贴现率', '货币供应量',

    # 贸易与全球化
    '贸易', '全球', '全球化', '进口', '出口', '汇率', '关税',
    '国际收支', '平衡', '赤字', '盈余', '外商直接投资', '外国', '跨境', '货币',

    # 政府、税收与公共财政
    '税收', '税款', '补贴', '预算', '公共', '支出', '政府',
    '开支', '财政', '收入', '国家', '债务', '赤字', '政策',
    '再分配', '刺激', '基础设施',

    # 发展与不平等
    '不平等', '贫困', '发展', '可持续', '可持续发展目标', '增长', '人力资本',
    '基本收入', '教育', '健康', '营养', '社会保障', '非正式部门',

    # 劳动与就业
    '就业', '工作', '失业', '劳动', '劳动力', '劳动市场', '工资',
    '就业创造', '失业', '就业率', '技能', '生产力',

    # 理论与实证方法
    '计量经济学', '回归', '面板', '横截面', '工具变量', '两阶段最小二乘法',
    '时间序列', '数据', '因果关系', '模型', '模拟', '预测', '结构',
    '估计', '假设', '零假设', '控制组', '实验', '调查',

    # 行为经济学、制度经济学与应用经济学
    '行为经济学', '博弈', '策略', '决策', '拍卖', '偏差', '有限理性',
    '推力', '偏好', '心理学', '规则', '制度', '激励', '机制',
    '选择', '理性', '非理性', '公共选择',

    # 环境与ESG金融
    '绿色', '可持续', 'ESG', '碳', '外部性', '污染', '气候',
    '环境的', '定价', '碳市场', '碳税', '限额与交易',
    '估值', '自然资本', '资源', '可再生能源',

    # 全球组织与机构
    '国际货币基金组织', '世界银行', '世界贸易组织', '经济合作与发展组织', '欧洲中央银行', '美联储', '欧元区', '新兴', '发达',
    '小型企业', '中小企业', '企业', '创业', '初创企业', '风险投资',

    # 现实世界的其他术语
    '衰退', '繁荣', '萧条', '危机', '救助', '住房', '泡沫', '财政悬崖',
    '主权债务', '市场失灵', '外部性', '机会成本', '能源',

    # Основные экономические термины
    'экономика', 'экономический', 'экономика (дисциплина)', 'макроэкономика', 'микроэкономика', 'рынок',
    'предложение', 'спрос', 'дефицит', 'компромисс', 'полезность', 'стоимость', 'польза', 'равновесие',
    'эластичность', 'цена', 'потребитель', 'производитель', 'выход', 'вход', 'производство',

    # Экономические индикаторы
    'ввп', 'инфляция', 'дефляция', 'индекс потребительских цен', 'индекс цен производителей', 'индекс', 'рост', 'безработица',
    'занятость', 'труд', 'зарплата', 'доход', 'продуктивность', 'ставка', 'прогноз', 'тенденция',

    # Финансы и инвестиции
    'финансы', 'финансовый', 'капитал', 'актив', 'пассив', 'собственный капитал', 'акции', 'облигации',
    'портфель', 'инвестиции', 'инвестор', 'доходность', 'дивиденд', 'оценка', 'хеджирование',
    'паевой', 'фонд', 'риск', 'волатильность', 'плечо', 'арбитраж', 'ценные бумаги',
    'дериватив', 'опцион', 'фьючерсы', 'собственный капитал', 'денежный поток', 'альфа', 'бета', 'шарп',

    # Банковская деятельность и денежно-кредитная политика
    'банк', 'банковское дело', 'центральный', 'монетарный', 'фискальный', 'ликвидность', 'деньги', 'валюта',
    'процент', 'политика', 'количественное смягчение', 'количественный', 'кредит', 'долг', 'кредитование', 'депозит',
    'резерв', 'целевая инфляция', 'ставка дисконтирования', 'денежная масса',

    # Торговля и глобализация
    'торговля', 'глобальный', 'глобализация', 'импорт', 'экспорт', 'обмен', 'тариф',
    'баланс платежей', 'баланс', 'дефицит', 'профицит', 'прямые иностранные инвестиции', 'иностранный', 'международный', 'валюта',

    # Государство, налогообложение и государственные финансы
    'налог', 'налоги', 'субсидия', 'бюджет', 'публичный', 'расходы', 'правительство',
    'расходы государственного бюджета', 'фискальный', 'доход', 'национальный', 'долг', 'дефицит', 'политика',
    'перераспределение', 'стимулы', 'инфраструктура',

    # Развитие и неравенство
    'неравенство', 'бедность', 'развитие', 'устойчивое', 'цели устойчивого развития', 'рост', 'человеческий капитал',
    'базовый доход', 'образование', 'здоровье', 'питание', 'социальная защита', 'неформальный сектор',

    # Труд и занятость
    'занятость', 'работа', 'безработица', 'труд', 'рабочая сила', 'зарплата',
    'создание рабочих мест', 'безработный', 'уровень занятости', 'навыки', 'продуктивность',

    # Теоретические и эмпирические методы
    'эконометрика', 'регрессия', 'панель', 'сечение', 'инструментальный', 'ТСЛС',
    'временные ряды', 'данные', 'каузальность', 'модель', 'симуляция', 'прогнозирование', 'структурный',
    'оценка', 'гипотеза', 'нулевая', 'контрольная группа', 'эксперимент', 'опрос',

    # Поведенческая, институциональная и прикладная экономика
    'поведенческая', 'игра', 'стратегия', 'решение', 'аукцион', 'предвзятость', 'ограниченная рациональность',
    'подталкивание', 'предпочтение', 'психология', 'правила', 'институты', 'стимул', 'механизм',
    'выбор', 'рациональный', 'иррациональный', 'публичный выбор',

    # Экология и ESG финансы
    'зеленый', 'устойчивый', 'esg', 'углерод', 'внешний эффект', 'загрязнение', 'климат',
    'экологический', 'ценовая политика', 'углеродный рынок', 'углеродный налог', 'пределы и торговля',
    'оценка', 'природный капитал', 'ресурсы', 'возобновляемые',

    # Глобальные организации и учреждения
    'мвф', 'мировой банк', 'вто', 'оэср', 'европейский центральный банк', 'федеральная резервная система', 'еврозона', 'развивающиеся', 'развиты',
    'малые и средние предприятия', 'малый и средний бизнес', 'предприятие', 'предпринимательство', 'стартап', 'рисковый капитал',

    # Разные реальные термины
    'рецессия', 'бум', 'спад', 'кризис', 'спасение', 'жилищный рынок', 'пузырь', 'фискальная пропасть',
    'государственный долг', 'неудача рынка', 'внешний эффект', 'стоимость упущенной возможности', 'энергии',

    # 核心的な経済用語
    '経済', '経済的', '経済学', 'マクロ経済学', 'ミクロ経済学', '市場',
    '供給', '需要', '希少性', 'トレードオフ', '効用', 'コスト', '利益', '均衡',
    '弾力性', '価格', '消費者', '生産者', '生産量', '投入', '生産',

    # 経済指標
    'GDP', 'インフレ', 'デフレ', 'CPI', 'PPI', '指数', '成長', '失業',
    '雇用', '労働', '賃金', '所得', '生産性', '率', '予測', '傾向',

    # 財務と投資
    '財務', '金融', '資本', '資産', '負債', '株式', '株', '債券',
    'ポートフォリオ', '投資', '投資家', 'リターン', '配当', '評価', 'ヘッジ',
    '投資信託', 'ファンド', 'リスク', 'ボラティリティ', 'レバレッジ', 'アービトラージ', '証券',
    'デリバティブ', 'オプション', '先物', '株式', 'キャッシュフロー', 'アルファ', 'ベータ', 'シャープ',

    # 銀行と金融政策
    '銀行', '銀行業', '中央銀行', '金融', '財政', '流動性', 'お金', '通貨',
    '金利', '政策', 'QE', '量的', '信用', '負債', 'ローン', '預金',
    '準備金', 'インフレターゲティング', '割引率', 'マネーサプライ',

    # 貿易とグローバリゼーション
    '貿易', 'グローバル', 'グローバリゼーション', '輸入', '輸出', '交換', '関税',
    'BOP', 'バランス', '赤字', '黒字', 'FDI', '外国', '国境を越えた', '通貨',

    # 政府、課税、および公共財政
    '税金', '税', '補助金', '予算', '公共', '支出', '政府',
    '支出', '財政', '収入', '国の', '負債', '赤字', '政策',
    '再分配', '刺激策', 'インフラ',

    # 発展と不平等
    '不平等', '貧困', '開発', '持続可能', 'SDG', '成長', '人的資本',
    'ベーシックインカム', '教育', '健康', '栄養', '社会的保護', '非公式部門',

    # 労働と雇用
    '雇用', '仕事', '失業', '労働', '労働力', '賃金',
    '仕事創出', '失業', '雇用率', 'スキル', '生産性',

    # 理論的および実証的な方法
    '計量経済学', '回帰', 'パネル', 'クロスセクション', '計量的', 'TSLS',
    '時系列', 'データ', '因果関係', 'モデル', 'シミュレーション', '予測', '構造的',
    '推定', '仮説', '帰無', 'コントロールグループ', '実験', '調査',

    # 行動経済学、制度経済学および応用経済学
    '行動', 'ゲーム', '戦略', '意思決定', 'オークション', 'バイアス', '限定合理性',
    'ナッジ', '好み', '心理学', 'ルール', '制度', 'インセンティブ', 'メカニズム',
    '選択', '合理的', '非合理的', '公共選択',

    # 環境経済学とESGファイナンス
    'グリーン', '持続可能', 'ESG', 'カーボン', '外部性', '汚染', '気候',
    '環境的', '価格設定', 'カーボンマーケット', '炭素税', 'キャップアンドトレード',
    '評価', '自然資本', '資源', '再生可能エネルギー',

    # グローバル機関と組織
    'IMF', '世界銀行', 'WTO', 'OECD', 'ECB', 'FED', 'ユーロ圏', '新興', '先進',
    'SMC', 'SME', '企業', '起業家精神', 'スタートアップ', 'ベンチャー',

    # その他の実世界の用語
    '不況', '好況', '不況', '危機', '救済', '住宅', 'バブル', '財政の崖',
    '国債', '市場の失敗', '外部性', '機会費用', 'エネルギー',

    # 핵심 경제 용어
    '경제', '경제적인', '경제학', '거시경제학', '미시경제학', '시장',
    '공급', '수요', '희소성', '기회비용', '효용', '비용', '이익', '균형',
    '탄력성', '가격', '소비자', '생산자', '산출', '투입', '생산',

    # 경제 지표
    'GDP', '인플레이션', '디플레이션', '소비자 물가 지수', '생산자 물가 지수', '지수', '성장', '실업',
    '고용', '노동', '임금', '소득', '생산성', '비율', '예측', '트렌드',

    # 금융 및 투자
    '금융', '금융의', '자본', '자산', '부채', '자본금', '주식', '채권',
    '포트폴리오', '투자', '투자자', '수익', '배당금', '평가', '헤지',
    '뮤추얼', '펀드', '위험', '변동성', '레버리지', '차익 거래', '증권',
    '파생상품', '옵션', '선물', '자본', '현금 흐름', '알파', '베타', '샤프 지수',

    # 은행 및 통화 정책
    '은행', '은행업', '중앙', '통화', '재정', '유동성', '돈', '통화',
    '이자', '정책', '양적완화', '양적', '신용', '부채', '대출', '예금',
    '비축', '인플레이션 목표', '할인율', '통화 공급',

    # 무역 및 글로벌화
    '무역', '글로벌', '글로벌화', '수입', '수출', '교환', '관세',
    'BOP', '균형', '적자', '흑자', 'FDI', '외국', '국경을 넘은', '통화',

    # 정부, 과세 및 공공 재정
    '세금', '세금들', '보조금', '예산', '공공', '지출', '정부',
    '지출', '재정', '수익', '국가', '부채', '적자', '정책',
    '재분배', '자극', '인프라',

    # 개발 및 불평등
    '불평등', '빈곤', '개발', '지속 가능한', 'SDG', '성장', '인간 자본',
    '기본소득', '교육', '건강', '영양', '사회적 보호', '비공식 부문',

    # 노동 및 고용
    '고용', '직업', '실업', '노동', '노동력', '임금',
    '일자리 창출', '실직', '고용률', '기술', '생산성',

    # 이론적 및 실증적 방법
    '계량경제학', '회귀', '패널', '횡단면', '도구 변수', 'TSLS',
    '시계열', '데이터', '인과관계', '모델', '시뮬레이션', '예측', '구조적',
    '추정', '가설', '영가설', '통제 그룹', '실험', '설문조사',

    # 행동 경제학, 제도 경제학 및 응용 경제학
    '행동 경제학', '게임', '전략', '결정', '경매', '편향', '제한된 합리성',
    '넛지', '선호', '심리학', '규칙', '제도', '인센티브', '메커니즘',
    '선택', '합리적', '비합리적', '공공 선택',

    # 환경 및 ESG 금융
    '녹색', '지속 가능한', 'ESG', '탄소', '외부성', '오염', '기후',
    '환경적인', '가격 책정', '탄소 시장', '탄소세', '캡 앤 트레이드',
    '평가', '자연 자본', '자원', '재생 가능',

    # 글로벌 조직 및 기관
    'IMF', '세계은행', 'WTO', 'OECD', 'ECB', 'FED', '유로존', '신흥', '개발된',
    'SMC', 'SME', '기업', '기업가정신', '스타트업', '벤처',

    # 기타 실제 용어
    '침체', '호황', '불황', '위기', '구제 금융', '주택', '버블', '재정 절벽',
    '국가 부채', '시장 실패', '외부성', '기회비용', '에너지',

    # 核心經濟術語
    '經濟', '經濟的', '經濟學', '宏觀經濟學', '微觀經濟學', '市場',
    '供應', '需求', '稀缺', '權衡', '效用', '成本', '利益', '均衡',
    '彈性', '價格', '消費者', '生產者', '產出', '輸入', '生產',

    # 經濟指標
    'GDP', '通脹', '通縮', '消費者物價指數', '生產者物價指數', '指數', '增長', '失業',
    '就業', '勞動', '工資', '收入', '生產力', '比率', '預測', '趨勢',

    # 財務與投資
    '金融', '金融的', '資本', '資產', '負債', '股本', '股票', '債券',
    '投資組合', '投資', '投資者', '回報', '股息', '估值', '對沖',
    '共同', '基金', '風險', '波動性', '杠桿', '套利', '證券',
    '衍生品', '期權', '期貨', '股權', '現金流', '阿爾法', '貝塔', '夏普',

    # 銀行與貨幣政策
    '銀行', '銀行業務', '中央', '貨幣', '財政', '流動性', '貨幣', '貨幣單位',
    '利率', '政策', '量化寬鬆', '量化', '信貸', '債務', '貸款', '存款',
    '儲備', '通脹目標', '折扣率', '貨幣供應量',

    # 貿易與全球化
    '貿易', '全球', '全球化', '進口', '出口', '匯率', '關稅',
    '國際收支', '平衡', '赤字', '盈餘', '外商直接投資', '外國', '跨境', '貨幣',

    # 政府、稅收與公共財政
    '稅收', '稅金', '補貼', '預算', '公共', '支出', '政府',
    '支出', '財政', '收入', '國家', '債務', '赤字', '政策',
    '再分配', '刺激', '基礎設施',

    # 發展與不平等
    '不平等', '貧困', '發展', '可持續', '可持續發展目標', '增長', '人力資本',
    '基本收入', '教育', '健康', '營養', '社會保障', '非正式部門',

    # 勞動與就業
    '就業', '工作', '失業', '勞動', '勞動力', '工資',
    '創造就業', '失業者', '就業率', '技能', '生產力',

    # 理論與實證方法
    '計量經濟學', '回歸', '面板', '橫截面', '工具變量', 'TSLS',
    '時間序列', '數據', '因果關係', '模型', '模擬', '預測', '結構性',
    '估計', '假設', '零假設', '控制組', '實驗', '調查',

    # 行為經濟學、制度經濟學與應用經濟學
    '行為', '博弈', '策略', '決策', '拍賣', '偏見', '有限理性',
    '推動', '偏好', '心理學', '規則', '制度', '激勳', '機制',
    '選擇', '理性', '非理性', '公共選擇',

    # 環境與ESG金融
    '綠色', '可持續', 'ESG', '碳', '外部性', '污染', '氣候',
    '環境', '定價', '碳市場', '碳稅', '上限與交易',
    '估值', '自然資本', '資源', '可再生能源',

    # 全球組織與機構
    '國際貨幣基金組織', '世界銀行', '世界貿易組織', '經濟合作與發展組織', '歐洲中央銀行', '美國聯邦儲備系統', '歐元區', '新興市場', '發達市場',
    '中小企業', '中小型企業', '企業', '創業', '初創企業', '風險投資',

    # 現實世界的多樣性術語
    '衰退', '繁榮', '崩潰', '危機', '救助', '住房', '泡沫', '財政懸崖',
    '主權債務', '市場失敗', '外部性', '機會成本', '能源',

    # Základní ekonomické termíny
    'ekonomika', 'ekonomický', 'ekonomie', 'makroekonomie', 'mikroekonomie', 'trh',
    'nabídka', 'poptávka', 'nedostatek', 'kompromis', 'užitek', 'náklady', 'benefit', 'rovnováha',
    'elasticita', 'cena', 'spotřebitel', 'výrobce', 'výstup', 'vstup', 'produkce',

    # Ekonomické ukazatele
    'hdp', 'inflace', 'deflace', 'cpi', 'ppi', 'index', 'růst', 'nezaměstnanost',
    'zaměstnanost', 'práce', 'mzda', 'příjem', 'produktivita', 'míra', 'předpověď', 'trend',

    # Finance a investice
    'finance', 'finanční', 'kapitál', 'aktivum', 'pasivum', 'vlastní kapitál', 'akcie', 'dluhopis',
    'portfolia', 'investice', 'investor', 'výnos', 'dividenda', 'ocenění', 'zabezpečení',
    'podílový', 'fond', 'riziko', 'volatilita', 'páka', 'arbitráž', 'cenné papíry',
    'derivát', 'opce', 'futures', 'vlastní kapitál', 'cashflow', 'alfa', 'beta', 'sharpe',

    # Bankovnictví a měnová politika
    'banka', 'bankovnictví', 'centrální', 'měnový', 'fiskální', 'likvidita', 'peníze', 'měna',
    'úrok', 'politika', 'qe', 'kvantitativní', 'kredit', 'dluh', 'půjčka', 'vklad',
    'rezerva', 'cílení inflace', 'diskontní sazba', 'peněžní zásoba',

    # Obchod a globalizace
    'obchod', 'globální', 'globalizace', 'import', 'export', 'směna', 'tarif',
    'bop', 'bilance', 'deficit', 'přebytek', 'fdi', 'zahraniční', 'přeshraniční', 'měna',

    # Vláda, zdanění a veřejné finance
    'daň', 'daně', 'dotace', 'rozpočet', 'veřejný', 'výdaje', 'vláda',
    'výdaje veřejné', 'fiskální', 'příjmy', 'národní', 'dluh', 'deficit', 'politika',
    'přerozdělení', 'stimulus', 'infrastruktura',

    # Rozvoj a nerovnost
    'nerovnost', 'chudoba', 'rozvoj', 'udržitelný', 'sdg', 'růst', 'lidský kapitál',
    'základní příjem', 'vzdělání', 'zdraví', 'výživa', 'sociální ochrana', 'neformální sektor',

    # Práce a zaměstnanost
    'zaměstnanost', 'práce', 'nezaměstnanost', 'práce', 'pracovní síla', 'mzda',
    'vytváření pracovních míst', 'nezaměstnaný', 'míra zaměstnanosti', 'dovednosti', 'produktivita',

    # Teoretické a empirické metody
    'ekonometrie', 'regrese', 'panel', 'přesná data', 'instrumentální', 'tsls',
    'časové řady', 'data', 'kauzalita', 'model', 'simulace', 'předpověď', 'strukturální',
    'odhad', 'hypotéza', 'nulová', 'kontrolní skupina', 'experiment', 'průzkum',

    # Chování, institucionální a aplikovaná ekonomie
    'chování', 'hra', 'strategie', 'rozhodnutí', 'aukce', 'sklon', 'omezená racionalita',
    'podnět', 'preference', 'psychologie', 'pravidla', 'instituce', 'incentiva', 'mechanismus',
    'volba', 'racionální', 'iracionální', 'veřejná volba',

    # Ekologie a ESG finance
    'zelený', 'udržitelný', 'esg', 'uhlík', 'externalita', 'znečištění', 'klima',
    'environmentální', 'cena', 'uhlíkový trh', 'uhlíková daň', 'omezení a obchod',
    'ocenění', 'přírodní kapitál', 'zdroje', 'obnovitelné',

    # Globální organizace a instituce
    'imf', 'světová banka', 'wto', 'oecd', 'ecb', 'fed', 'eurozóna', 'rozvojové', 'rozvinuté',
    'smc', 'sme', 'podnik', 'podnikání', 'startup', 'rizikový',

    # Různé reálné termíny
    'recesní', 'boom', 'krach', 'krize', 'záchranný balíček', 'bydlení', 'bublina', 'fiskální útes',
    'suverénní dluh', 'selhání trhu', 'externalita', 'náklady na příležitosti', 'energie',

    # المصطلحات الاقتصادية الأساسية
    'الاقتصاد', 'اقتصادي', 'الاقتصاد (المفهوم)', 'الاقتصاد الكلي', 'الاقتصاد الجزئي', 'السوق',
    'العرض', 'الطلب', 'الندرة', 'التنازل', 'المنفعة', 'التكلفة', 'الفائدة', 'التوازن',
    'المرونة', 'السعر', 'المستهلك', 'المنتج', 'الإنتاج', 'المدخلات', 'الإنتاج',

    # المؤشرات الاقتصادية
    'الناتج المحلي الإجمالي', 'التضخم', 'الانكماش', 'مؤشر أسعار المستهلك', 'مؤشر أسعار المنتج', 'المؤشر', 'النمو', 'البطالة',
    'التوظيف', 'العمل', 'الأجر', 'الدخل', 'الإنتاجية', 'المعدل', 'التوقع', 'الاتجاه',

    # المالية والاستثمار
    'المالية', 'المالي', 'رأس المال', 'الأصول', 'الخصوم', 'الأسهم', 'السندات',
    'محفظة الاستثمار', 'الاستثمار', 'المستثمر', 'العائد', 'التوزيع', 'التقييم', 'التحوط',
    'صندوق مشترك', 'الصندوق', 'المخاطر', 'التقلبات', 'الرافعة المالية', 'المراجحة', 'الأوراق المالية',
    'المشتقات', 'الخيارات', 'العقود المستقبلية', 'رأس المال', 'التدفق النقدي', 'ألفا', 'بيتا', 'شارب',

    # البنوك والسياسة النقدية
    'البنك', 'البنوك', 'المركزي', 'النقدي', 'المالي', 'السيولة', 'المال', 'العملة',
    'الفائدة', 'السياسة', 'التيسير الكمي', 'كمية', 'الائتمان', 'الدين', 'القرض', 'الإيداع',
    'الاحتياطي', 'استهداف التضخم', 'سعر الخصم', 'عرض النقود',

    # التجارة والعولمة
    'التجارة', 'عالمي', 'العولمة', 'الاستيراد', 'التصدير', 'التبادل', 'الرسوم الجمركية',
    'ميزان المدفوعات', 'التوازن', 'العجز', 'الفائض', 'الاستثمار الأجنبي المباشر', 'أجنبي', 'عبر الحدود', 'العملة',

    # الحكومة والضرائب والمالية العامة
    'الضريبة', 'الضرائب', 'الدعم', 'الميزانية', 'العام', 'الإنفاق', 'الحكومة',
    'النفقات', 'المالي', 'الإيرادات', 'الوطنية', 'الدين', 'العجز', 'السياسة',
    'إعادة التوزيع', 'التحفيز', 'البنية التحتية',

    # التنمية وعدم المساواة
    'عدم المساواة', 'الفقر', 'التنمية', 'مستدام', 'أهداف التنمية المستدامة', 'النمو', 'رأس المال البشري',
    'الدخل الأساسي', 'التعليم', 'الصحة', 'التغذية', 'الحماية الاجتماعية', 'القطاع غير الرسمي',

    # العمل والتوظيف
    'التوظيف', 'الوظيفة', 'البطالة', 'العمل', 'القوى العاملة', 'الأجر',
    'خلق الوظائف', 'العاطلين عن العمل', 'معدل التوظيف', 'المهارات', 'الإنتاجية',

    # الأساليب النظرية والتجريبية
    'الاقتصاد القياسي', 'الانحدار', 'اللوحة', 'القطاع العرضي', 'الأدوات', 'تقديرات شاملة للوقت',
    'سلاسل زمنية', 'البيانات', 'السببية', 'النموذج', 'المحاكاة', 'التنبؤ', 'الهيكلية',
    'التقدير', 'الافتراض', 'الصفري', 'مجموعة التحكم', 'التجربة', 'الاستطلاع',

    # الاقتصاد السلوكي، المؤسساتي والتطبيقي
    'سلوكي', 'لعبة', 'استراتيجية', 'قرار', 'مزاد', 'انحياز', 'العقلانية المحدودة',
    'دفع', 'تفضيل', 'علم النفس', 'القواعد', 'المؤسسات', 'الحوافز', 'الآلية',
    'اختيار', 'عقلاني', 'غير عقلاني', 'اختيار عام',

    # المالية البيئية وESG
    'أخضر', 'مستدام', 'ESG', 'الكربون', 'الآثار الجانبية', 'التلوث', 'المناخ',
    'بيئي', 'التسعير', 'سوق الكربون', 'ضريبة الكربون', 'الحد والتجارة',
    'التقييم', 'رأس المال الطبيعي', 'الموارد', 'الطاقات المتجددة',

    # المنظمات العالمية والمؤسسات
    'صندوق النقد الدولي', 'البنك الدولي', 'منظمة التجارة العالمية', 'منظمة التعاون الاقتصادي والتنمية', 'البنك المركزي الأوروبي', 'الاحتياطي الفيدرالي', 'منطقة اليورو', 'الأسواق الناشئة', 'المتقدمين',
    'الشركات الصغيرة والمتوسطة', 'المشروعات الصغيرة والمتوسطة', 'المؤسسة', 'ريادة الأعمال', 'شركة ناشئة', 'مخاطرة',

    # المصطلحات الواقعية المتفرقة
    'ركود', 'ازدهار', 'انهيار', 'أزمة', 'إنقاذ', 'الإسكان', 'فقاعة', 'هاوية مالية',
    'الدين السيادي', 'فشل السوق', 'الآثار الجانبية', 'تكلفة الفرصة البديلة', 'الطاقات'
],


            
    'education': [
    # Core concepts
    'education', 'educational', 'teaching', 'learning', 'instruction', 'school', 'schooling',
    'pedagogy', 'didactic', 'training', 'lesson', 'curriculum', 'syllabus', 'student',
    'students', 'pupil', 'classroom', 'teacher', 'teachers', 'instructor', 'lecturer',
    'professor', 'professors', 'faculty', 'learner',

    # Educational levels and types
    'kindergarten', 'preschool', 'primary', 'elementary', 'secondary', 'high school',
    'higher education', 'tertiary', 'university', 'college', 'graduate school', 'postgraduate',
    'vocational', 'technical school', 'adult education', 'lifelong learning', 'continuing education',
    'special education', 'inclusive education', 'remedial education', 'homeschooling',

    # Learning methods and theories
    'active learning', 'experiential learning', 'project-based learning', 'inquiry-based learning',
    'flipped classroom', 'blended learning', 'constructivism', 'behaviorism', 'scaffolding',
    'cognitive load', 'self-directed learning', 'peer learning', 'differentiated instruction',
    'learning theory', 'learning strategy', 'competency-based',

    # Assessment and evaluation
    'assessment', 'grading', 'evaluation', 'rubric', 'formative', 'summative', 'examination',
    'quiz', 'test', 'standardized test', 'performance', 'learning outcome', 'feedback',
    'benchmark', 'accreditation', 'scores',

    # Technology and digital learning
    'elearning', 'e-learning', 'online learning', 'remote learning', 'virtual classroom',
    'distance learning', 'digital learning', 'MOOC', 'lms', 'canvas', 'moodle', 'blackboard',
    'edtech', 'educational technology', 'ai in education', 'gamification', 'interactive learning',
    'mobile learning', 'flipped learning', 'technology-enhanced learning',

    # Literacy and 21st-century skills
    'literacy', 'numeracy', 'digital literacy', 'media literacy', 'scientific literacy',
    'civic literacy', 'financial literacy', 'critical thinking', 'problem solving',
    'creativity', 'communication skills', 'collaboration', 'emotional intelligence',
    'soft skills', 'life skills', 'resilience', 'intercultural competence',

    # Teachers and professional development
    'teacher training', 'teacher preparation', 'teacher education', 'in-service training',
    'faculty development', 'teacher certification', 'mentorship', 'instructional design',
    'professional development', 'pedagogical skills', 'educator',

    # Education policy and leadership
    'education policy', 'school policy', 'curriculum reform', 'school reform', 'school leadership',
    'principal', 'educational administration', 'school governance', 'school board',
    'ministry of education', 'department of education', 'education funding', 'school finance',
    'school autonomy', 'school management',

    # Socio-psychological aspects
    'student engagement', 'motivation', 'academic achievement', 'learning disabilities',
    'inclusive pedagogy', 'dropout', 'retention', 'bullying', 'discipline', 'classroom climate',
    'peer pressure', 'identity development', 'gender gap in education',

    # Institutions and global education bodies
    'university', 'college', 'school', 'academy', 'department of education', 'board of education',
    'teacher union', 'education NGO', 'education ministry', 'OECD education', 'PISA', 'UNESCO',
    'international baccalaureate', 'SDG 4', 'education for all', 'global competence',

    # Global education topics
    'global education', 'education for sustainable development', 'inclusive education',
    'gender equality in education', 'education access', 'educational equity', 'distance education',
    'educational opportunity', 'learning gap', 'global learning', 'international schooling'

    'exam', 'credential', 'certificate', 'credentialing', 'learning style', 'learning pace', 'education outcome',
    'MOOCs', 'coursework', 'edutainment', 'digital pedagogy',

    # Conceptos básicos
    'educación', 'educativo', 'enseñanza', 'aprendizaje', 'instrucción', 'escuela', 'educación escolar',
    'pedagogía', 'didáctico', 'formación', 'lección', 'currículo', 'programa de estudios', 'estudiante',
    'estudiantes', 'alumno', 'aula', 'maestro', 'maestros', 'instructor', 'profesor',
    'profesores', 'facultad', 'aprendiz',

    # Niveles y tipos de educación
    'jardín de infancia', 'preescolar', 'primaria', 'escuela primaria', 'secundaria', 'secundaria superior',
    'educación superior', 'terciaria', 'universidad', 'colegio', 'escuela de posgrado', 'posgrado',
    'formación profesional', 'escuela técnica', 'educación para adultos', 'aprendizaje a lo largo de la vida', 'educación continua',
    'educación especial', 'educación inclusiva', 'educación remedial', 'educación en casa',

    # Métodos de aprendizaje y teorías
    'aprendizaje activo', 'aprendizaje experiencial', 'aprendizaje basado en proyectos', 'aprendizaje basado en la indagación',
    'aula invertida', 'aprendizaje combinado', 'constructivismo', 'conductismo', 'andamiaje',
    'carga cognitiva', 'aprendizaje autodirigido', 'aprendizaje entre pares', 'instrucción diferenciada',
    'teoría del aprendizaje', 'estrategia de aprendizaje', 'basado en competencias',

    # Evaluación y valoración
    'evaluación', 'calificación', 'valoración', 'rúbrica', 'formativa', 'sumativa', 'examen',
    'cuestionario', 'prueba', 'prueba estandarizada', 'rendimiento', 'resultado del aprendizaje', 'retroalimentación',
    'referencia', 'acreditación', 'calificaciones',

    # Tecnología y aprendizaje digital
    'aprendizaje electrónico', 'e-learning', 'aprendizaje en línea', 'aprendizaje a distancia', 'aula virtual',
    'aprendizaje a distancia', 'aprendizaje digital', 'MOOC', 'lms', 'canvas', 'moodle', 'blackboard',
    'tecnología educativa', 'educación tecnológica', 'IA en la educación', 'gamificación', 'aprendizaje interactivo',
    'aprendizaje móvil', 'aprendizaje invertido', 'aprendizaje mejorado por tecnología',

    # Alfabetización y habilidades del siglo XXI
    'alfabetización', 'numeración', 'alfabetización digital', 'alfabetización mediática', 'alfabetización científica',
    'alfabetización cívica', 'alfabetización financiera', 'pensamiento crítico', 'resolución de problemas',
    'creatividad', 'habilidades de comunicación', 'colaboración', 'inteligencia emocional',
    'habilidades blandas', 'habilidades para la vida', 'resiliencia', 'competencia intercultural',

    # Maestros y desarrollo profesional
    'formación de maestros', 'preparación de maestros', 'educación de maestros', 'capacitación en servicio',
    'desarrollo de la facultad', 'certificación de maestros', 'mentoría', 'diseño instruccional',
    'desarrollo profesional', 'habilidades pedagógicas', 'educador',

    # Política educativa y liderazgo
    'política educativa', 'política escolar', 'reforma curricular', 'reforma escolar', 'liderazgo escolar',
    'director', 'administración educativa', 'gobernanza escolar', 'junta escolar',
    'ministerio de educación', 'departamento de educación', 'financiamiento educativo', 'finanzas escolares',
    'autonomía escolar', 'gestión escolar',

    # Aspectos socio-psicológicos
    'compromiso estudiantil', 'motivación', 'logro académico', 'trastornos del aprendizaje',
    'pedagogía inclusiva', 'deserción', 'retención', 'acoso escolar', 'disciplina', 'clima en el aula',
    'presión de grupo', 'desarrollo de la identidad', 'brecha de género en la educación',

    # Instituciones y organismos educativos globales
    'universidad', 'colegio', 'escuela', 'academia', 'departamento de educación', 'junta de educación',
    'sindicato de maestros', 'ONG educativa', 'ministerio de educación', 'educación OCDE', 'PISA', 'UNESCO',
    'bachillerato internacional', 'ODS 4', 'educación para todos', 'competencia global',

    # Temas educativos globales
    'educación global', 'educación para el desarrollo sostenible', 'educación inclusiva',
    'igualdad de género en la educación', 'acceso a la educación', 'equidad educativa', 'educación a distancia',
    'oportunidad educativa', 'brecha de aprendizaje', 'aprendizaje global', 'escuelas internacionales',

    'examen', 'certificado', 'credencial', 'credencialización', 'estilo de aprendizaje', 'ritmo de aprendizaje', 'resultado educativo',
    'MOOCs', 'trabajo de curso', 'edutainment', 'pedagogía digital',

    # Conceitos principais
    'educação', 'educacional', 'ensino', 'aprendizagem', 'instrução', 'escola', 'escolaridade',
    'pedagogia', 'didática', 'treinamento', 'aula', 'currículo', 'programa de ensino', 'estudante',
    'estudantes', 'aluno', 'sala de aula', 'professor', 'professores', 'instrutor', 'palestrante',
    'professor universitário', 'professores universitários', 'faculdade', 'aprendiz',

    # Níveis e tipos de educação
    'jardim de infância', 'pré-escola', 'primário', 'ensino fundamental', 'secundário', 'ensino médio',
    'educação superior', 'terciário', 'universidade', 'faculdade', 'pós-graduação', 'pós-graduação',
    'educação profissional', 'escola técnica', 'educação de adultos', 'aprendizagem ao longo da vida', 'educação contínua',
    'educação especial', 'educação inclusiva', 'educação de recuperação', 'ensino domiciliar',

    # Métodos e teorias de aprendizagem
    'aprendizagem ativa', 'aprendizagem experiencial', 'aprendizagem baseada em projetos', 'aprendizagem baseada em investigação',
    'sala de aula invertida', 'aprendizagem híbrida', 'construtivismo', 'behaviorismo', 'scaffolding',
    'carga cognitiva', 'aprendizagem autodirigida', 'aprendizagem entre pares', 'instrução diferenciada',
    'teoria da aprendizagem', 'estratégia de aprendizagem', 'baseada em competências',

    # Avaliação e avaliação
    'avaliação', 'notação', 'avaliação', 'rubrica', 'formativa', 'somativa', 'exame',
    'teste', 'prova', 'teste padronizado', 'desempenho', 'resultado de aprendizagem', 'feedback',
    'padrão', 'acreditação', 'pontuação',

    # Tecnologia e aprendizagem digital
    'e-learning', 'ensino a distância', 'aprendizagem online', 'aprendizagem remota', 'sala de aula virtual',
    'ensino a distância', 'aprendizagem digital', 'MOOC', 'lms', 'canvas', 'moodle', 'blackboard',
    'edtech', 'tecnologia educacional', 'ia na educação', 'gamificação', 'aprendizagem interativa',
    'aprendizagem móvel', 'aprendizagem invertida', 'aprendizagem aprimorada por tecnologia',

    # Alfabetização e habilidades do século XXI
    'alfabetização', 'numeracia', 'alfabetização digital', 'alfabetização midiática', 'alfabetização científica',
    'alfabetização cívica', 'alfabetização financeira', 'pensamento crítico', 'resolução de problemas',
    'criatividade', 'habilidades de comunicação', 'colaboração', 'inteligência emocional',
    'habilidades interpessoais', 'habilidades para a vida', 'resiliência', 'competência intercultural',

    # Formação de professores e desenvolvimento profissional
    'formação de professores', 'preparação de professores', 'educação de professores', 'treinamento em serviço',
    'desenvolvimento de faculdade', 'certificação de professores', 'mentoria', 'design instrucional',
    'desenvolvimento profissional', 'habilidades pedagógicas', 'educador',

    # Política educacional e liderança
    'política educacional', 'política escolar', 'reforma curricular', 'reforma escolar', 'liderança escolar',
    'diretor', 'administração educacional', 'governança escolar', 'conselho escolar',
    'ministério da educação', 'departamento de educação', 'financiamento da educação', 'finanças escolares',
    'autonomia escolar', 'gestão escolar',

    # Aspectos socio-psicológicos
    'engajamento estudantil', 'motivação', 'realização acadêmica', 'deficiência de aprendizagem',
    'pedagogia inclusiva', 'evasão escolar', 'retenção', 'bullying', 'disciplina', 'clima de sala de aula',
    'pressão dos colegas', 'desenvolvimento da identidade', 'desigualdade de gênero na educação',

    # Instituições e órgãos globais de educação
    'universidade', 'faculdade', 'escola', 'academia', 'departamento de educação', 'conselho de educação',
    'sindicato de professores', 'ONG educacional', 'ministério da educação', 'educação da OCDE', 'PISA', 'UNESCO',
    'baccalauréat internacional', 'ODS 4', 'educação para todos', 'competência global',

    # Tópicos globais de educação
    'educação global', 'educação para o desenvolvimento sustentável', 'educação inclusiva',
    'igualdade de gênero na educação', 'acesso à educação', 'equidade educacional', 'educação a distância',
    'oportunidade educacional', 'lacuna de aprendizagem', 'aprendizagem global', 'educação internacional',

    'exame', 'credencial', 'certificado', 'credenciamento', 'estilo de aprendizagem', 'ritmo de aprendizagem', 'resultado educacional',
    'MOOCs', 'trabalhos de curso', 'edutainment', 'pedagogia digital',

    # Concetti di base
    'istruzione', 'educativo', 'insegnamento', 'apprendimento', 'istruzione', 'scuola', 'scuolamento',
    'pedagogia', 'didattica', 'formazione', 'lezione', 'curriculum', 'programma', 'studente',
    'studenti', 'alunno', 'aula', 'insegnante', 'insegnanti', 'istruttore', 'docente',
    'professore', 'professori', 'facoltà', 'apprendente',

    # Livelli e tipi di istruzione
    'asilo', 'scuola dell\'infanzia', 'primaria', 'elementare', 'secondaria', 'scuola superiore',
    'istruzione superiore', 'terziaria', 'università', 'college', 'scuola di specializzazione', 'post-laurea',
    'professionale', 'scuola tecnica', 'educazione degli adulti', 'apprendimento continuo', 'istruzione continua',
    'educazione speciale', 'istruzione inclusiva', 'istruzione di recupero', 'homeschooling',

    # Metodi di apprendimento e teorie
    'apprendimento attivo', 'apprendimento esperienziale', 'apprendimento basato su progetti', 'apprendimento basato sull\'inchiesta',
    'classe capovolta', 'apprendimento misto', 'costruttivismo', 'comportamentismo', 'scaffolding',
    'carico cognitivo', 'apprendimento autodiretto', 'apprendimento tra pari', 'istruzione differenziata',
    'teoria dell\'apprendimento', 'strategia di apprendimento', 'basato su competenze',

    # Valutazione e valutazione
    'valutazione', 'votazione', 'esame', 'rubrica', 'formativo', 'summativo', 'esame',
    'quiz', 'test', 'test standardizzati', 'performance', 'risultato di apprendimento', 'feedback',
    'benchmark', 'accreditamento', 'punteggi',

    # Tecnologia e apprendimento digitale
    'apprendimento elettronico', 'e-learning', 'apprendimento online', 'apprendimento remoto', 'classe virtuale',
    'apprendimento a distanza', 'apprendimento digitale', 'MOOC', 'lms', 'canvas', 'moodle', 'blackboard',
    'edtech', 'tecnologia educativa', 'ai nell\'istruzione', 'gamificazione', 'apprendimento interattivo',
    'apprendimento mobile', 'apprendimento capovolto', 'apprendimento potenziato dalla tecnologia',

    # Alfabetizzazione e competenze del 21° secolo
    'alfabetizzazione', 'competenza numerica', 'alfabetizzazione digitale', 'alfabetizzazione mediatica', 'alfabetizzazione scientifica',
    'alfabetizzazione civica', 'alfabetizzazione finanziaria', 'pensiero critico', 'risoluzione dei problemi',
    'creatività', 'competenze comunicative', 'collaborazione', 'intelligenza emotiva',
    'soft skills', 'competenze per la vita', 'resilienza', 'competenza interculturale',

    # Insegnanti e sviluppo professionale
    'formazione insegnanti', 'preparazione insegnanti', 'educazione degli insegnanti', 'formazione in servizio',
    'sviluppo della facoltà', 'certificazione degli insegnanti', 'mentorship', 'progettazione didattica',
    'sviluppo professionale', 'competenze pedagogiche', 'educatore',

    # Politica educativa e leadership
    'politica educativa', 'politica scolastica', 'riforma del curriculum', 'riforma scolastica', 'leadership scolastica',
    'preside', 'amministrazione educativa', 'governance scolastica', 'consiglio scolastico',
    'ministero dell\'istruzione', 'dipartimento dell\'istruzione', 'finanziamento dell\'istruzione', 'finanza scolastica',
    'autonomia scolastica', 'gestione scolastica',

    # Aspetti socio-psicologici
    'coinvolgimento degli studenti', 'motivazione', 'successo accademico', 'disturbi dell\'apprendimento',
    'pedagogia inclusiva', 'abbandono', 'ritenzione', 'bullismo', 'disciplina', 'clima in aula',
    'pressione dei pari', 'sviluppo dell\'identità', 'disuguaglianza di genere nell\'istruzione',

    # Istituzioni e organismi globali dell'istruzione
    'università', 'college', 'scuola', 'accademia', 'dipartimento dell\'istruzione', 'consiglio dell\'istruzione',
    'sindacato degli insegnanti', 'ong per l\'istruzione', 'ministero dell\'istruzione', 'istruzione OCSE', 'PISA', 'UNESCO',
    'baccalaureato internazionale', 'SDG 4', 'istruzione per tutti', 'competenza globale',

    # Temi globali dell'istruzione
    'istruzione globale', 'istruzione per lo sviluppo sostenibile', 'istruzione inclusiva',
    'uguaglianza di genere nell\'istruzione', 'accesso all\'istruzione', 'equità educativa', 'istruzione a distanza',
    'opportunità educative', 'gap di apprendimento', 'apprendimento globale', 'scuole internazionali',

    'esame', 'qualifica', 'certificato', 'certificazione', 'stile di apprendimento', 'velocità di apprendimento', 'risultato educativo',
    'MOOC', 'lavoro di corso', 'edutainment', 'pedagogia digitale',

    # Concepts de base
    'éducation', 'éducatif', 'enseignement', 'apprentissage', 'instruction', 'école', 'scolarité',
    'pédagogie', 'didactique', 'formation', 'leçon', 'programme', 'syllabus', 'étudiant',
    'étudiants', 'élève', 'salle de classe', 'enseignant', 'enseignants', 'instructeur', 'conférencier',
    'professeur', 'professeurs', 'faculté', 'apprenant',

    # Niveaux et types d'éducation
    'maternelle', 'école primaire', 'primaire', 'élémentaire', 'secondaire', 'lycée',
    'enseignement supérieur', 'tertiaire', 'université', 'collège', 'école supérieure', 'postgradué',
    'formation professionnelle', 'école technique', 'éducation des adultes', 'apprentissage tout au long de la vie', 'éducation continue',
    'éducation spécialisée', 'éducation inclusive', 'éducation de rattrapage', 'école à domicile',

    # Méthodes et théories d'apprentissage
    'apprentissage actif', 'apprentissage expérientiel', 'apprentissage basé sur les projets', 'apprentissage par enquête',
    'classe inversée', 'apprentissage mixte', 'constructivisme', 'comportementalisme', 'échafaudage',
    'charge cognitive', 'apprentissage autodirigé', 'apprentissage entre pairs', 'instruction différenciée',
    'théorie de l\'apprentissage', 'stratégie d\'apprentissage', 'basé sur les compétences',

    # Évaluation et évaluation
    'évaluation', 'notation', 'évaluation', 'grille', 'formatif', 'sommatif', 'examen',
    'quiz', 'test', 'test standardisé', 'performance', 'résultat d\'apprentissage', 'retour d\'information',
    'référence', 'accréditation', 'scores',

    # Technologie et apprentissage numérique
    'apprentissage en ligne', 'apprentissage en ligne', 'apprentissage en ligne', 'apprentissage à distance', 'classe virtuelle',
    'apprentissage à distance', 'apprentissage numérique', 'MOOC', 'LMS', 'canvas', 'moodle', 'blackboard',
    'edtech', 'technologie éducative', 'IA en éducation', 'gamification', 'apprentissage interactif',
    'apprentissage mobile', 'apprentissage inversé', 'apprentissage amélioré par la technologie',

    # Littératie et compétences du 21e siècle
    'littératie', 'numératie', 'littératie numérique', 'littératie médiatique', 'littératie scientifique',
    'littératie civique', 'littératie financière', 'pensée critique', 'résolution de problèmes',
    'créativité', 'compétences en communication', 'collaboration', 'intelligence émotionnelle',
    'compétences sociales', 'compétences pour la vie', 'résilience', 'compétence interculturelle',

    # Formation des enseignants et développement professionnel
    'formation des enseignants', 'préparation des enseignants', 'éducation des enseignants', 'formation continue',
    'développement de la faculté', 'certification des enseignants', 'mentorat', 'conception pédagogique',
    'développement professionnel', 'compétences pédagogiques', 'éducateur',

    # Politique éducative et leadership
    'politique éducative', 'politique scolaire', 'réforme des programmes', 'réforme scolaire', 'leadership scolaire',
    'directeur', 'administration éducative', 'gouvernance scolaire', 'conseil scolaire',
    'ministère de l\'éducation', 'département de l\'éducation', 'financement de l\'éducation', 'finances scolaires',
    'autonomie scolaire', 'gestion scolaire',

    # Aspects socio-psychologiques
    'engagement des étudiants', 'motivation', 'réussite académique', 'troubles de l\'apprentissage',
    'pédagogie inclusive', 'abandon scolaire', 'rétention', 'intimidation', 'discipline', 'climat de classe',
    'pression des pairs', 'développement de l\'identité', 'écart entre les sexes dans l\'éducation',

    # Institutions et organismes éducatifs mondiaux
    'université', 'collège', 'école', 'académie', 'département de l\'éducation', 'conseil de l\'éducation',
    'syndicat des enseignants', 'ONG éducative', 'ministère de l\'éducation', 'éducation de l\'OCDE', 'PISA', 'UNESCO',
    'baccalauréat international', 'ODD 4', 'éducation pour tous', 'compétence mondiale',

    # Thèmes mondiaux de l'éducation
    'éducation mondiale', 'éducation pour le développement durable', 'éducation inclusive',
    'égalité des sexes dans l\'éducation', 'accès à l\'éducation', 'équité en éducation', 'éducation à distance',
    'opportunité éducative', 'écart d\'apprentissage', 'apprentissage mondial', 'écoles internationales',

    'examen', 'diplôme', 'certificat', 'accréditation', 'style d\'apprentissage', 'rythme d\'apprentissage', 'résultat de l\'éducation',
    'MOOCs', 'travaux de cours', 'éducation divertissante', 'pédagogie numérique',

    # 核心概念
    '教育', '教育的', '教学', '学习', '指导', '学校', '教育过程',
    '教育学', '教学法', '培训', '课程', '课程大纲', '教学大纲', '学生',
    '学生们', '学员', '教室', '教师', '教师们', '讲师', '讲师',
    '教授', '教授们', '学院', '学习者',

    # 教育水平和类型
    '幼儿园', '学前教育', '小学', '初级', '中学', '高中',
    '高等教育', '高等教育', '大学', '学院', '研究生院', '研究生',
    '职业教育', '技术学校', '成人教育', '终身学习', '继续教育',
    '特殊教育', '包容性教育', '补救教育', '家庭教育',

    # 学习方法和理论
    '主动学习', '体验式学习', '基于项目的学习', '探究式学习',
    '翻转课堂', '混合学习', '建构主义', '行为主义', '支架式教学',
    '认知负荷', '自主学习', '同伴学习', '差异化教学',
    '学习理论', '学习策略', '基于能力的',

    # 评估与评价
    '评估', '评分', '评价', '评分标准', '形成性评估', '总结性评估', '考试',
    '测验', '测试', '标准化测试', '表现', '学习成果', '反馈',
    '基准', '认证', '分数',

    # 技术和数字学习
    '电子学习', '在线学习', '远程学习', '虚拟课堂',
    '远程教育', '数字学习', 'MOOC', '学习管理系统', 'Canvas', 'Moodle', '黑板',
    '教育技术', '教育技术', '人工智能在教育中的应用', '游戏化', '互动学习',
    '移动学习', '翻转学习', '技术增强学习',

    # 识字和21世纪技能
    '识字', '算术', '数字素养', '媒体素养', '科学素养',
    '公民素养', '财务素养', '批判性思维', '问题解决',
    '创造力', '沟通技巧', '协作', '情商',
    '软技能', '生活技能', '复原力', '跨文化能力',

    # 教师和职业发展
    '教师培训', '教师准备', '教师教育', '在职培训',
    '教师发展', '教师认证', '导师制度', '教学设计',
    '职业发展', '教学技能', '教育者',

    # 教育政策和领导力
    '教育政策', '学校政策', '课程改革', '学校改革', '学校领导',
    '校长', '教育管理', '学校治理', '学校委员会',
    '教育部', '教育局', '教育资金', '学校财政',
    '学校自治', '学校管理',

    # 社会心理学方面
    '学生参与', '动机', '学术成就', '学习障碍',
    '包容性教学法', '辍学', '留存率', '欺凌', '纪律', '课堂氛围',
    '同伴压力', '身份发展', '教育中的性别差距',

    # 机构和全球教育机构
    '大学', '学院', '学校', '学术机构', '教育部门', '教育委员会',
    '教师工会', '教育非政府组织', '教育部', '经合组织教育', '国际学生评估项目', '联合国教科文组织',
    '国际学士学位', '可持续发展目标4', '普及教育', '全球能力',

    # 全球教育话题
    '全球教育', '可持续发展教育', '包容性教育',
    '教育中的性别平等', '教育机会', '教育公平', '远程教育',
    '教育机会', '学习差距', '全球学习', '国际学校',

    '考试', '证书', '认证', '认证过程', '学习风格', '学习进度', '教育成果',
    '大规模在线课程', '课程作业', '教育娱乐', '数字教学法',

    # Основные концепции
    'образование', 'образовательный', 'обучение', 'учебный процесс', 'инструкция', 'школа', 'образование (обучение)',
    'педагогика', 'дидактика', 'обучение (подготовка)', 'урок', 'учебный план', 'программа курса', 'студент',
    'студенты', 'ученик', 'класс', 'учитель', 'учителя', 'инструктор', 'лектор',
    'профессор', 'профессора', 'факультет', 'обучающийся',

    # Уровни и типы образования
    'детский сад', 'дошкольное образование', 'начальная школа', 'начальное образование', 'среднее', 'старшая школа',
    'высшее образование', 'третичный уровень', 'университет', 'колледж', 'аспирантура', 'послевузовское образование',
    'профессиональное образование', 'техникум', 'образование для взрослых', 'обучение на протяжении всей жизни', 'непрерывное образование',
    'специальное образование', 'инклюзивное образование', 'восстановительное образование', 'домашнее обучение',

    # Методы обучения и теории
    'активное обучение', 'экспериентальное обучение', 'обучение на основе проектов', 'обучение через исследование',
    'перевернутый класс', 'смешанное обучение', 'конструктивизм', 'поведенческая теория', 'поддержка',
    'когнитивная нагрузка', 'самостоятельное обучение', 'обучение в группе', 'дифференцированное обучение',
    'теория обучения', 'стратегия обучения', 'обучение на основе компетенций',

    # Оценка и оценивание
    'оценка', 'оценивание', 'экзамен', 'критерии оценки', 'формативное', 'суммативное', 'экзамен',
    'викторина', 'тест', 'стандартизированный тест', 'производительность', 'учебный результат', 'обратная связь',
    'показатели', 'аккредитация', 'оценки',

    # Технологии и цифровое обучение
    'электронное обучение', 'e-learning', 'онлайн обучение', 'дистанционное обучение', 'виртуальный класс',
    'дистанционное обучение', 'цифровое обучение', 'МОК', 'LMS', 'канвас', 'мудл', 'черная доска',
    'образовательные технологии', 'технологии в образовании', 'ИИ в образовании', 'геймификация', 'интерактивное обучение',
    'мобильное обучение', 'перевернутое обучение', 'обучение с использованием технологий',

    # Грамотность и навыки 21 века
    'грамотность', 'цифровая грамотность', 'медийная грамотность', 'научная грамотность',
    'гражданская грамотность', 'финансовая грамотность', 'критическое мышление', 'решение проблем',
    'креативность', 'коммуникативные навыки', 'сотрудничество', 'эмоциональный интеллект',
    'мягкие навыки', 'жизненные навыки', 'устойчивость', 'межкультурная компетенция',

    # Учителя и профессиональное развитие
    'обучение учителей', 'подготовка учителей', 'образование учителей', 'обучение в процессе работы',
    'развитие преподавателей', 'сертификация учителей', 'менторство', 'дизайн обучения',
    'профессиональное развитие', 'педагогические навыки', 'педагог',

    # Политика образования и лидерство
    'образовательная политика', 'политика школы', 'реформа учебных программ', 'реформа школ', 'руководство школой',
    'директор', 'образовательное управление', 'управление школой', 'школьная доска',
    'министерство образования', 'департамент образования', 'финансирование образования', 'финансы школ',
    'автономия школ', 'управление школой',

    # Социально-психологические аспекты
    'вовлеченность студентов', 'мотивация', 'академические достижения', 'обучающие расстройства',
    'инклюзивная педагогика', 'выбросы из системы', 'удержание', 'буллинг', 'дисциплина', 'климат в классе',
    'давление со стороны сверстников', 'развитие идентичности', 'гендерный разрыв в образовании',

    # Учреждения и мировые образовательные организации
    'университет', 'колледж', 'школа', 'академия', 'департамент образования', 'образовательный совет',
    'профсоюз учителей', 'образовательная НПО', 'министерство образования', 'образование по версии ОЭСР', 'PISA', 'ЮНЕСКО',
    'международный бакалавриат', 'ЦУР 4', 'образование для всех', 'глобальная компетентность',

    # Глобальные темы образования
    'глобальное образование', 'образование для устойчивого развития', 'инклюзивное образование',
    'гендерное равенство в образовании', 'доступ к образованию', 'образовательное равенство', 'дистанционное образование',
    'образовательные возможности', 'разрыв в обучении', 'глобальное обучение', 'международное образование',

    'экзамен', 'квалификация', 'сертификат', 'квалификация', 'стиль обучения', 'темп обучения', 'образовательный результат',
    'МОК', 'курсовая работа', 'образовательное развлечение', 'цифровая педагогика',

    # 基本的な概念
    '教育', '教育的', '教授', '学習', '指導', '学校', '教育課程',
    '教育学', '教授法', 'トレーニング', '授業', 'カリキュラム', 'シラバス', '学生',
    '学生たち', '生徒', '教室', '教師', '教師たち', 'インストラクター', '講師',
    '教授', '教授たち', '学部', '学習者',

    # 教育のレベルと種類
    '幼稚園', '保育園', '初等教育', '小学校', '中等教育', '高校',
    '高等教育', '大学教育', '大学', 'カレッジ', '大学院', '大学院後',
    '職業教育', '技術学校', '成人教育', '生涯学習', '継続教育',
    '特別支援教育', '包括的教育', '補習教育', '家庭学習',

    # 学習方法と理論
    'アクティブラーニング', '体験学習', 'プロジェクトベース学習', '探求ベース学習',
    '反転授業', 'ブレンド学習', '構成主義', '行動主義', '支援',
    '認知負荷', '自己主導学習', 'ピアラーニング', '差別化された指導',
    '学習理論', '学習戦略', '能力ベース',

    # 評価と査定
    '評価', '採点', '査定', 'ルーブリック', '形成的評価', '総括的評価', '試験',
    'クイズ', 'テスト', '標準化されたテスト', 'パフォーマンス', '学習成果', 'フィードバック',
    'ベンチマーク', '認証', 'スコア',

    # テクノロジーとデジタル学習
    'eラーニング', 'オンライン学習', '遠隔学習', '仮想教室',
    'ディスタンスラーニング', 'デジタル学習', 'MOOC', 'LMS', 'Canvas', 'Moodle', 'Blackboard',
    'EdTech', '教育技術', '教育におけるAI', 'ゲーミフィケーション', 'インタラクティブラーニング',
    'モバイル学習', '反転学習', 'テクノロジー強化学習',

    # リテラシーと21世紀スキル
    'リテラシー', '算数能力', 'デジタルリテラシー', 'メディアリテラシー', '科学的リテラシー',
    '市民リテラシー', '金融リテラシー', '批判的思考', '問題解決',
    '創造性', 'コミュニケーション能力', '協力', '感情知能',
    'ソフトスキル', 'ライフスキル', 'レジリエンス', '異文化能力',

    # 教師と専門的な成長
    '教師の研修', '教師の準備', '教師教育', '現職研修',
    '学部の発展', '教師資格', 'メンタリング', '指導設計',
    '専門的な成長', '教育学的スキル', '教育者',

    # 教育政策とリーダーシップ
    '教育政策', '学校の政策', 'カリキュラム改革', '学校改革', '学校のリーダーシップ',
    '校長', '教育管理', '学校のガバナンス', '学校の理事会',
    '教育省', '教育部', '教育資金', '学校財政',
    '学校の自律性', '学校管理',

    # 社会心理学的側面
    '学生の関与', '動機', '学業成績', '学習障害',
    '包括的教育学', '中退', '定着率', 'いじめ', '規律', '教室の雰囲気',
    '仲間からの圧力', 'アイデンティティの発展', '教育における性別格差',

    # 教育機関と国際教育機関
    '大学', 'カレッジ', '学校', 'アカデミー', '教育部門', '教育委員会',
    '教師組合', '教育NGO', '教育省', 'OECD教育', 'PISA', 'ユネスコ',
    '国際バカロレア', 'SDG 4', 'すべての教育', 'グローバルコンピテンス',

    # 世界の教育のトピック
    'グローバル教育', '持続可能な開発のための教育', '包括的教育',
    '教育における性別平等', '教育へのアクセス', '教育の公平性', '遠隔教育',
    '教育機会', '学習ギャップ', 'グローバル学習', '国際的な学校教育',

    '試験', '資格', '証明書', '資格取得', '学習スタイル', '学習速度', '教育成果',
    'MOOC', 'コースワーク', 'エデュテイメント', 'デジタル教育法',

    # 핵심 개념
    '교육', '교육적인', '교육', '학습', '지도', '학교', '학교 교육',
    '교육학', '교수법', '훈련', '수업', '교육과정', '강의계획서', '학생',
    '학생들', '학생(초등학생)', '교실', '교사', '교사들', '강사', '강의자',
    '교수', '교수들', '학부', '학습자',

    # 교육 수준 및 유형
    '유치원', '미취학 아동', '초등', '초등학교', '중등', '고등학교',
    '고등 교육', '대학', '대학교', '대학원', '대학원생', '대학원 과정',
    '직업 교육', '기술학교', '성인 교육', '평생 학습', '지속적 교육',
    '특수 교육', '포용 교육', '보충 교육', '홈스쿨링',

    # 학습 방법 및 이론
    '능동 학습', '경험 학습', '프로젝트 기반 학습', '탐구 기반 학습',
    '뒤집힌 교실', '혼합 학습', '구성주의', '행동주의', '발판 이론',
    '인지 부하', '자기주도 학습', '또래 학습', '차별화된 지도법',
    '학습 이론', '학습 전략', '역량 기반 학습',

    # 평가 및 평가
    '평가', '채점', '평가', '루브릭', '형성 평가', '총합 평가', '시험',
    '퀴즈', '테스트', '표준화된 시험', '성과', '학습 결과', '피드백',
    '벤치마크', '인증', '점수',

    # 기술 및 디지털 학습
    '전자학습', '이러닝', '온라인 학습', '원격 학습', '가상 교실',
    '원거리 학습', '디지털 학습', 'MOOC', 'LMS', '캔버스', '무들', '블랙보드',
    '교육 기술', '교육 기술', 'AI 교육', '게임화', '인터랙티브 학습',
    '모바일 학습', '뒤집힌 학습', '기술 향상 학습',

    # 문해력 및 21세기 기술
    '문해력', '수리력', '디지털 문해력', '미디어 문해력', '과학 문해력',
    '시민 문해력', '재정 문해력', '비판적 사고', '문제 해결',
    '창의성', '의사소통 능력', '협업', '정서 지능',
    '소프트 스킬', '생활 기술', '회복력', '문화간 역량',

    # 교사 및 전문성 개발
    '교사 훈련', '교사 준비', '교사 교육', '재직 중 훈련',
    '학부 개발', '교사 인증', '멘토링', '교수 설계',
    '전문성 개발', '교수 기술', '교육자',

    # 교육 정책 및 리더십
    '교육 정책', '학교 정책', '교육과정 개혁', '학교 개혁', '학교 리더십',
    '교장', '교육 행정', '학교 관리', '학교 이사회',
    '교육부', '교육부서', '교육 자금', '학교 재정',
    '학교 자율성', '학교 경영',

    # 사회심리적 측면
    '학생 참여', '동기 부여', '학업 성취', '학습 장애',
    '포용적 교수법', '학교 중퇴', '유지', '괴롭힘', '징계', '교실 분위기',
    '또래 압력', '정체성 개발', '성별 격차',

    # 기관 및 글로벌 교육 기관
    '대학교', '대학', '학교', '아카데미', '교육부', '교육위원회',
    '교사 노조', '교육 NGO', '교육부', 'OECD 교육', 'PISA', '유네스코',
    '국제 바칼로레아', 'SDG 4', '모두를 위한 교육', '글로벌 역량',

    # 글로벌 교육 주제
    '글로벌 교육', '지속 가능한 개발을 위한 교육', '포용적 교육',
    '교육에서의 성평등', '교육 접근', '교육 형평성', '원격 교육',
    '교육 기회', '학습 격차', '글로벌 학습', '국제 학교 교육',

    '시험', '자격 증명', '증명서', '자격 인증', '학습 스타일', '학습 속도', '교육 결과',
    'MOOC', '과제', '교육적 오락', '디지털 교수법',

    # 核心概念
    '教育', '教育的', '教學', '學習', '指導', '學校', '教育過程',
    '教育學', '教學法', '訓練', '課程', '課程大綱', '大綱', '學生',
    '學生們', '學童', '教室', '老師', '老師們', '指導員', '講師',
    '教授', '教授們', '教職員', '學習者',

    # 教育水平與類型
    '幼兒園', '學前教育', '小學', '初等教育', '中等教育', '高中',
    '高等教育', '大學教育', '大學', '學院', '研究生院', '研究所',
    '職業教育', '技術學校', '成人教育', '終身學習', '繼續教育',
    '特殊教育', '包容性教育', '補救教育', '居家教育',

    # 學習方法與理論
    '主動學習', '經驗學習', '基於項目的學習', '探究式學習',
    '翻轉課堂', '混合學習', '建構主義', '行為主義', '支架式學習',
    '認知負荷', '自主學習', '同儕學習', '差異化教學',
    '學習理論', '學習策略', '能力基礎',

    # 評估與評價
    '評估', '打分', '評價', '評分標準', '形成性評估', '總結性評估', '考試',
    '小測驗', '測試', '標準化測試', '表現', '學習成果', '回饋',
    '基準', '認證', '分數',

    # 技術與數位學習
    '電子學習', '網路學習', '線上學習', '遠程學習', '虛擬教室',
    '遠距學習', '數位學習', 'MOOC', '學習管理系統', 'Canvas', 'Moodle', 'Blackboard',
    '教育科技', '教學科技', '人工智慧在教育中的應用', '遊戲化', '互動學習',
    '移動學習', '翻轉學習', '技術增能學習',

    # 讀寫能力與21世紀技能
    '識字能力', '算數能力', '數位識字', '媒體識字', '科學識字',
    '公民識字', '金融識字', '批判性思維', '問題解決',
    '創造力', '溝通技巧', '合作', '情商',
    '軟技能', '生活技能', '韌性', '跨文化能力',

    # 教師與專業發展
    '教師培訓', '教師準備', '教師教育', '在職培訓',
    '教職員發展', '教師認證', '導師制度', '教學設計',
    '專業發展', '教學技能', '教育者',

    # 教育政策與領導
    '教育政策', '學校政策', '課程改革', '學校改革', '學校領導',
    '校長', '教育管理', '學校治理', '學校董事會',
    '教育部', '教育部門', '教育資金', '學校財務',
    '學校自治', '學校管理',

    # 社會心理學方面
    '學生參與', '動機', '學業成就', '學習障礙',
    '包容性教育', '輟學', '留校', '欺凌', '紀律', '教室氣候',
    '同儕壓力', '身份發展', '性別差距在教育中',

    # 教育機構與全球教育機構
    '大學', '學院', '學校', '學院', '教育部門', '教育委員會',
    '教師工會', '教育非政府組織', '教育部', '經濟合作與發展組織教育', 'PISA', '聯合國教科文組織',
    '國際文憑', '可持續發展目標4', '全民教育', '全球競爭力',

    # 全球教育議題
    '全球教育', '可持續發展教育', '包容性教育',
    '教育中的性別平等', '教育機會', '教育公平', '遠程教育',
    '教育機會', '學習差距', '全球學習', '國際學校',

    '考試', '證書', '證明', '學習風格', '學習進度', '教育成果',
    'MOOCs', '課程作業', '教育娛樂', '數位教學法',

    # Základní pojmy
    'vzdělání', 'vzdělávací', 'výuka', 'učení', 'instrukce', 'škola', 'vzdělávání',
    'pedagogika', 'didaktika', 'školení', 'lekce', 'kurikulum', 'učební plán', 'student',
    'studenti', 'žák', 'třída', 'učitel', 'učitelé', 'instruktor', 'lektor',
    'profesor', 'profesoři', 'fakulta', 'učící se',

    # Vzdělávací úrovně a typy
    'mateřská škola', 'předškolní', 'základní', 'základní škola', 'střední', 'střední škola',
    'vysokoškolské vzdělání', 'terciární', 'univerzita', 'kolegium', 'postgraduální škola', 'doktorské studium',
    'odborné', 'technická škola', 'vzdělávání dospělých', 'celostní učení', 'kontinuální vzdělávání',
    'speciální vzdělávání', 'inkluzivní vzdělávání', 'remediální vzdělávání', 'domácí vzdělávání',

    # Metody a teorie učení
    'aktivní učení', 'experienciální učení', 'projektově orientované učení', 'inquiry-based learning',
    'obrácená třída', 'kombinované učení', 'konstruktivismus', 'behaviorismus', 'scaffolding',
    'kognitivní zátěž', 'sebeřízené učení', 'učení od vrstevníků', 'diferencované instrukce',
    'teorie učení', 'strategie učení', 'kompetenčně orientované',

    # Hodnocení a evaluace
    'hodnocení', 'známkování', 'evaluace', 'rubrika', 'formativní', 'sumativní', 'zkouška',
    'kvíz', 'test', 'standardizovaný test', 'výkon', 'výsledky učení', 'zpětná vazba',
    'referenční hodnota', 'akreditace', 'skóre',

    # Technologie a digitální učení
    'elearning', 'e-learning', 'online učení', 'vzdálené učení', 'virtuální třída',
    'distanční učení', 'digitální učení', 'MOOC', 'lms', 'canvas', 'moodle', 'blackboard',
    'edtech', 'vzdělávací technologie', 'ai ve vzdělávání', 'gamifikace', 'interaktivní učení',
    'mobilní učení', 'obrácené učení', 'technologiemi vylepšené učení',

    # Gramotnost a dovednosti 21. století
    'gramotnost', 'numerické dovednosti', 'digitální gramotnost', 'mediální gramotnost', 'vědecká gramotnost',
    'občanská gramotnost', 'finanční gramotnost', 'kritické myšlení', 'řešení problémů',
    'kreativita', 'komunikační dovednosti', 'spolupráce', 'emoční inteligence',
    'měkké dovednosti', 'životní dovednosti', 'odolnost', 'interkulturní kompetence',

    # Učitelé a profesní rozvoj
    'školení učitelů', 'příprava učitelů', 'vzdělávání učitelů', 'školení v průběhu služby',
    'rozvoj fakulty', 'certifikace učitelů', 'mentorství', 'návrh výuky',
    'profesní rozvoj', 'pedagogické dovednosti', 'vzdělavatel',

    # Politika vzdělávání a vedení
    'politika vzdělávání', 'politika školy', 'reforma kurikula', 'reforma školy', 'vedení školy',
    'ředitel', 'vzdělávací správa', 'správa školy', 'školní rada',
    'ministerstvo školství', 'oddělení školství', 'financování vzdělávání', 'škola a finance',
    'autonomie školy', 'řízení školy',

    # Sociálně-psychologické aspekty
    'zapojení studentů', 'motivace', 'akademický úspěch', 'učení s poruchami',
    'inkluzivní pedagogika', 'vypadnutí', 'udržení', 'šikana', 'disciplína', 'klima ve třídě',
    'tlak vrstevníků', 'rozvoj identity', 'genderová mezera ve vzdělávání',

    # Instituce a globální vzdělávací orgány
    'univerzita', 'kolegium', 'škola', 'akademie', 'oddělení školství', 'rada pro školství',
    'syndikát učitelů', 'vzdělávací nezisková organizace', 'ministerstvo školství', 'OECD vzdělávání', 'PISA', 'UNESCO',
    'mezinárodní maturita', 'SDG 4', 'vzdělání pro všechny', 'globální kompetence',

    # Globální vzdělávací témata
    'globální vzdělávání', 'vzdělávání pro udržitelný rozvoj', 'inkluzivní vzdělávání',
    'genderová rovnost ve vzdělávání', 'přístup k vzdělání', 'vzdělávací rovnost', 'distanční vzdělávání',
    'vzdělávací příležitosti', 'výuková mezera', 'globální učení', 'mezinárodní školy',

    'zkouška', 'kredenciál', 'certifikát', 'kredencování', 'styl učení', 'tempo učení', 'výsledky vzdělávání',
    'MOOC', 'kurzová práce', 'edutainment', 'digitální pedagogika',

    # المفاهيم الأساسية
    'التعليم', 'تعليمي', 'التدريس', 'التعلم', 'الإرشاد', 'المدرسة', 'التعليم المدرسي',
    'البيداغوجيا', 'تعليمي', 'التدريب', 'الدرس', 'المنهج الدراسي', 'البرنامج الدراسي', 'الطالب',
    'الطلاب', 'التلميذ', 'الفصل الدراسي', 'المعلم', 'المعلمون', 'المدرب', 'المحاضر',
    'الأستاذ', 'الأساتذة', 'أعضاء هيئة التدريس', 'المتعلم',

    # المستويات التعليمية والأنواع
    'رياض الأطفال', 'مرحلة ما قبل المدرسة', 'المرحلة الابتدائية', 'المرحلة الأساسية', 'المرحلة الثانوية', 'المدرسة الثانوية',
    'التعليم العالي', 'التعليم الجامعي', 'الجامعة', 'الكليات', 'دراسات ما بعد التخرج', 'دراسات عُليا',
    'التعليم المهني', 'المدرسة التقنية', 'التعليم للكبار', 'التعلم مدى الحياة', 'التعليم المستمر',
    'التعليم الخاص', 'التعليم الشامل', 'التعليم العلاجي', 'التعليم المنزلي',

    # طرق التعلم والنظريات
    'التعلم النشط', 'التعلم التجريبي', 'التعلم القائم على المشاريع', 'التعلم القائم على الاستفسار',
    'الفصل المقلوب', 'التعلم المدمج', 'البنيوية', 'السلوكية', 'الدعامات',
    'التحميل المعرفي', 'التعلم الذاتي', 'التعلم بين الأقران', 'التعليم الموجه',
    'نظرية التعلم', 'استراتيجية التعلم', 'القائم على الكفاءات',

    # التقييم والتقويم
    'التقييم', 'التقدير', 'التقويم', 'مقياس', 'تكويني', 'نهائي', 'الاختبار',
    'اختبار', 'امتحان', 'الاختبارات الموحدة', 'الأداء', 'نتيجة التعلم', 'التعليقات',
    'مؤشر مرجعي', 'الاعتماد', 'الدرجات',

    # التكنولوجيا والتعلم الرقمي
    'التعلم الإلكتروني', 'التعليم الإلكتروني', 'التعلم عبر الإنترنت', 'التعلم عن بعد', 'الفصل الدراسي الافتراضي',
    'التعلم عن بُعد', 'التعلم الرقمي', 'الدورات الدراسية الجماعية عبر الإنترنت', 'نظام إدارة التعلم', 'كانفاس', 'مودل', 'بلاكبورد',
    'تقنية التعليم', 'التكنولوجيا التعليمية', 'الذكاء الاصطناعي في التعليم', 'التلعيب', 'التعلم التفاعلي',
    'التعلم عبر الهاتف المحمول', 'التعلم المقلوب', 'التعلم المدعوم بالتكنولوجيا',

    # الثقافة والمهارات القرن الواحد والعشرون
    'التمكين القرائي', 'المهارات الحسابية', 'المهارات الرقمية', 'الثقافة الإعلامية', 'الثقافة العلمية',
    'الثقافة المدنية', 'الثقافة المالية', 'التفكير النقدي', 'حل المشكلات',
    'الإبداع', 'مهارات التواصل', 'التعاون', 'الذكاء العاطفي',
    'المهارات الناعمة', 'مهارات الحياة', 'المرونة', 'الكفاءة بين الثقافات',

    # المعلمون والتطوير المهني
    'تدريب المعلمين', 'إعداد المعلمين', 'تعليم المعلمين', 'التدريب أثناء الخدمة',
    'تطوير أعضاء هيئة التدريس', 'شهادة المعلم', 'الإرشاد', 'تصميم المناهج',
    'التطوير المهني', 'المهارات التربوية', 'المعلم',

    # سياسة التعليم والقيادة
    'سياسة التعليم', 'سياسة المدرسة', 'إصلاح المناهج الدراسية', 'إصلاح المدارس', 'القيادة المدرسية',
    'المدير', 'الإدارة التعليمية', 'حوكمة المدارس', 'مجلس المدرسة',
    'وزارة التعليم', 'إدارة التعليم', 'تمويل التعليم', 'مالية المدارس',
    'استقلالية المدارس', 'إدارة المدارس',

    # الجوانب النفسية الاجتماعية
    'مشاركة الطالب', 'الدافع', 'الإنجاز الأكاديمي', 'اضطرابات التعلم',
    'البيداغوجيا الشاملة', 'التسرب المدرسي', 'الاحتفاظ', 'التنمر', 'الانضباط', 'مناخ الفصل الدراسي',
    'الضغط الاجتماعي', 'تطور الهوية', 'الفجوة بين الجنسين في التعليم',

    # المؤسسات والهيئات التعليمية العالمية
    'الجامعة', 'الكليات', 'المدرسة', 'الأكاديمية', 'إدارة التعليم', 'مجلس التعليم',
    'اتحاد المعلمين', 'منظمة غير حكومية تعليمية', 'وزارة التعليم', 'التعليم في منظمة التعاون الاقتصادي والتنمية', 'برنامج التقييم الدولي للطلاب', 'اليونسكو',
    'البكالوريا الدولية', 'الهدف 4 من أهداف التنمية المستدامة', 'التعليم للجميع', 'الكفاءة العالمية',

    # مواضيع التعليم العالمي
    'التعليم العالمي', 'التعليم من أجل التنمية المستدامة', 'التعليم الشامل',
    'المساواة بين الجنسين في التعليم', 'إمكانية الوصول إلى التعليم', 'عدالة التعليم', 'التعليم عن بُعد',
    'فرص التعليم', 'الفجوة التعليمية', 'التعلم العالمي', 'المدارس الدولية',

    'امتحان', 'شهادة', 'دبلوم', 'تصديق', 'أسلوب التعلم', 'سرعة التعلم', 'نتيجة التعليم',
    'الدورات الدراسية الجماعية عبر الإنترنت', 'المقررات الدراسية', 'التعليم الترفيهي', 'التدريس الرقمي'

],

    'engineering': [
    # General concepts
    'engineer', 'engineering', 'engineered', 'design', 'designs', 'designed', 'technical', 'technician',
    'system design', 'modelling', 'modeling', 'model', 'models', 'simulation', 'simulate', 'simulated',
    'analysis', 'analyze', 'analysed', 'analyzed', 'optimization', 'optimize', 'efficiency', 'efficient',
    'integration', 'integrate', 'blueprint', 'calculation', 'calculations', 'schematic', 'drawing', 'drawings', 'energies'

    # Disciplines (add missing forms)
    'mechanical engineering', 'electrical engineering', 'civil engineering', 'chemical engineering',
    'aerospace engineering', 'industrial engineering', 'biomedical engineering', 'structural engineering',
    'geotechnical engineering', 'nuclear engineering', 'automotive engineering',
    'marine engineering', 'software engineering', 'computer engineering', 'systems engineering',
    'optical engineering', 'environmental engineering', 'materials engineering',

    # Physical principles
    'load', 'loads', 'force', 'forces', 'torque', 'momentum', 'stress', 'strain',
    'tension', 'compression', 'elasticity', 'plasticity', 'friction', 'inertia', 'vibration', 'resonance',

    # Materials
    'material', 'materials', 'metal', 'metals', 'alloy', 'alloys', 'composite', 'ceramic', 'ceramics',
    'polymer', 'polymers', 'fatigue', 'fracture', 'crack', 'cracks', 'corrosion',
    'thermal conductivity', 'electrical conductivity', 'nanomaterials', 'nano-composites',

    # Robotics & Control
    'robot', 'robots', 'robotics', 'automation', 'automated', 'autonomous', 'servo motor', 'servo motors',
    'trajectory', 'path planning', 'SLAM', 'PID control', 'actuator', 'actuators', 'kinematics', 'inverse kinematics',

    # Electronics
    'circuit', 'circuits', 'pcb', 'microcontroller', 'microcontrollers', 'sensor', 'sensors', 'transistor',
    'resistor', 'capacitor', 'inductor', 'diode', 'signal', 'signals', 'amplifier', 'oscilloscope', 'voltage', 'current',

    # Simulation/Tools
    'CAD', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', 'finite element',
    'FEM', 'FEA', '3D printing', 'digital twin', 'computational mechanics', 'numerical methods',

    # Energy
    'renewable energy', 'energy storage', 'thermal system', 'combustion', 'heat exchanger', 'solar panel',
    'wind turbine', 'fluid dynamics', 'hydraulics', 'power system', 'power grid', 'smart grid', 'battery', 'batteries',

    # Infrastructure
    'construction', 'infrastructure', 'structural integrity', 'bridge', 'bridges', 'tunnel', 'dam', 'highway',
    'load-bearing', 'surveying', 'soil mechanics', 'urban planning', 'concrete', 'reinforced concrete', 'asphalt',

    # Transportation
    'vehicle', 'vehicles', 'mobility', 'engine', 'engines', 'turbo', 'aerodynamics',
    'electric vehicle', 'hybrid vehicle', 'autonomous vehicle', 'fuel efficiency',

    # Safety & QA
    'quality control', 'quality assurance', 'failure analysis', 'fault detection', 'lifecycle management',
    'compliance', 'ISO', 'standard', 'testing', 'certification', 'risk assessment', 'reliability engineering',

    # Emerging Tech
    'cyber-physical systems', 'sensor fusion', 'AI in engineering', 'digitalization',
    'smart materials', 'embedded intelligence', 'edge computing', 'IoT', 'blockchain engineering',
    'green technology', 'sustainable engineering', 'resilient systems', 'interdisciplinary design',

    # Conceptos generales
    'ingeniero', 'ingeniería', 'diseñada', 'diseño', 'diseños', 'diseñada', 'técnico', 'técnico',
    'diseño de sistemas', 'modelado', 'modelización', 'modelo', 'modelos', 'simulación', 'simular', 'simulada',
    'análisis', 'analizar', 'analizado', 'optimizacion', 'optimizar', 'eficiencia', 'eficiente',
    'integración', 'integrar', 'plano', 'cálculo', 'cálculos', 'esquemático', 'dibujo', 'dibujos', 'energías',

    # Disciplina (agregar formas faltantes)
    'ingeniería mecánica', 'ingeniería eléctrica', 'ingeniería civil', 'ingeniería química',
    'ingeniería aeroespacial', 'ingeniería industrial', 'ingeniería biomédica', 'ingeniería estructural',
    'ingeniería geotécnica', 'ingeniería nuclear', 'ingeniería automotriz',
    'ingeniería marina', 'ingeniería de software', 'ingeniería informática', 'ingeniería de sistemas',
    'ingeniería óptica', 'ingeniería ambiental', 'ingeniería de materiales',

    # Principios físicos
    'carga', 'cargas', 'fuerza', 'fuerzas', 'par', 'momentum', 'estrés', 'deformación',
    'tensión', 'compresión', 'elasticidad', 'plasticidad', 'fricción', 'inercia', 'vibración', 'resonancia',

    # Materiales
    'material', 'materiales', 'metal', 'metales', 'aleación', 'aleaciones', 'compuesto', 'cerámica', 'cerámicas',
    'polímero', 'polímeros', 'fatiga', 'fractura', 'grieta', 'grietas', 'corrosión',
    'conductividad térmica', 'conductividad eléctrica', 'nanomateriales', 'nano-compuestos',

    # Robótica y Control
    'robot', 'robots', 'robótica', 'automatización', 'automático', 'autónomo', 'motor servo', 'motores servo',
    'trayectoria', 'planificación de trayectorias', 'SLAM', 'control PID', 'actuador', 'actuadores', 'cinemática', 'cinemática inversa',

    # Electrónica
    'circuito', 'circuitos', 'placa de circuito impreso', 'microcontrolador', 'microcontroladores', 'sensor', 'sensores', 'transistor',
    'resistor', 'capacitor', 'inductor', 'diodo', 'señal', 'señales', 'amplificador', 'osciloscopio', 'voltaje', 'corriente',

    # Simulación/Herramientas
    'CAD', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', 'elemento finito',
    'FEM', 'FEA', 'impresión 3D', 'gemelo digital', 'mecánica computacional', 'métodos numéricos',

    # Energía
    'energía renovable', 'almacenamiento de energía', 'sistema térmico', 'combustión', 'intercambiador de calor', 'panel solar',
    'turbina eólica', 'dinámica de fluidos', 'hidráulica', 'sistema de energía', 'red eléctrica', 'red inteligente', 'batería', 'baterías',

    # Infraestructura
    'construcción', 'infraestructura', 'integridad estructural', 'puente', 'puentes', 'túnel', 'presa', 'carretera',
    'soporte de carga', 'topografía', 'mecánica de suelos', 'planificación urbana', 'concreto', 'concreto reforzado', 'asfalto',

    # Transporte
    'vehículo', 'vehículos', 'movilidad', 'motor', 'motores', 'turbo', 'aerodinámica',
    'vehículo eléctrico', 'vehículo híbrido', 'vehículo autónomo', 'eficiencia de combustible',

    # Seguridad y QA
    'control de calidad', 'aseguramiento de calidad', 'análisis de fallos', 'detección de fallos', 'gestión del ciclo de vida',
    'cumplimiento', 'ISO', 'estándar', 'pruebas', 'certificación', 'evaluación de riesgos', 'ingeniería de fiabilidad',

    # Tecnologías emergentes
    'sistemas ciberfísicos', 'fusión de sensores', 'IA en ingeniería', 'digitalización',
    'materiales inteligentes', 'inteligencia embebida', 'computación en el borde', 'IoT', 'ingeniería blockchain',
    'tecnología verde', 'ingeniería sostenible', 'sistemas resilientes', 'diseño interdisciplinario',

    # Conceitos gerais
    'engenheiro', 'engenharia', 'engenheirado', 'design', 'designs', 'projetado', 'técnico', 'técnico',
    'projeto de sistema', 'modelagem', 'modelagem', 'modelo', 'modelos', 'simulação', 'simular', 'simulado',
    'análise', 'analisar', 'analisado', 'analisado', 'otimização', 'otimizar', 'eficiência', 'eficiente',
    'integração', 'integrar', 'plano', 'cálculo', 'cálculos', 'esquemático', 'desenho', 'desenhos', 'energias',

    # Disciplinas (adicionar formas faltantes)
    'engenharia mecânica', 'engenharia elétrica', 'engenharia civil', 'engenharia química',
    'engenharia aeroespacial', 'engenharia industrial', 'engenharia biomédica', 'engenharia estrutural',
    'engenharia geotécnica', 'engenharia nuclear', 'engenharia automotiva',
    'engenharia naval', 'engenharia de software', 'engenharia da computação', 'engenharia de sistemas',
    'engenharia óptica', 'engenharia ambiental', 'engenharia de materiais',

    # Princípios físicos
    'carga', 'cargas', 'força', 'forças', 'torque', 'momento', 'tensão', 'deformação',
    'compressão', 'elasticidade', 'plasticidade', 'fricção', 'inércia', 'vibração', 'ressonância',

    # Materiais
    'material', 'materiais', 'metal', 'metais', 'liga', 'ligas', 'compósito', 'cerâmica', 'cerâmicas',
    'polímero', 'polímeros', 'fadiga', 'fratura', 'rachadura', 'rachaduras', 'corrosão',
    'condutividade térmica', 'condutividade elétrica', 'nanomateriais', 'nano-compósitos',

    # Robótica e Controle
    'robô', 'robôs', 'robótica', 'automação', 'automatizado', 'autônomo', 'motor servo', 'motores servo',
    'trajetória', 'planejamento de trajetória', 'SLAM', 'controle PID', 'atuador', 'atuadores', 'cinemática', 'cinemática inversa',

    # Eletrônica
    'circuito', 'circuitos', 'placa de circuito impresso', 'microcontrolador', 'microcontroladores', 'sensor', 'sensores', 'transistor',
    'resistor', 'capacitor', 'indutor', 'diodo', 'sinal', 'sinais', 'amplificador', 'osciloscópio', 'voltagem', 'corrente',

    # Simulação/Ferramentas
    'CAD', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', 'elemento finito',
    'FEM', 'FEA', 'impressão 3D', 'gêmeo digital', 'mecânica computacional', 'métodos numéricos',

    # Energia
    'energia renovável', 'armazenamento de energia', 'sistema térmico', 'combustão', 'trocador de calor', 'painel solar',
    'turbina eólica', 'dinâmica dos fluidos', 'hidráulica', 'sistema de energia', 'rede de energia', 'rede inteligente', 'bateria', 'baterias',

    # Infraestrutura
    'construção', 'infraestrutura', 'integridade estrutural', 'ponte', 'pontes', 'túnel', 'barragem', 'rodovia',
    'suporte de carga', 'topografia', 'mecânica do solo', 'planejamento urbano', 'concreto', 'concreto armado', 'asfalto',

    # Transporte
    'veículo', 'veículos', 'mobilidade', 'motor', 'motores', 'turbo', 'aerodinâmica',
    'veículo elétrico', 'veículo híbrido', 'veículo autônomo', 'eficiência de combustível',

    # Segurança e QA
    'controle de qualidade', 'garantia de qualidade', 'análise de falha', 'detecção de falhas', 'gestão do ciclo de vida',
    'conformidade', 'ISO', 'padrão', 'teste', 'certificação', 'avaliação de risco', 'engenharia de confiabilidade',

    # Tecnologias Emergentes
    'sistemas ciberfísicos', 'fusão de sensores', 'IA na engenharia', 'digitalização',
    'materiais inteligentes', 'inteligência incorporada', 'computação de borda', 'IoT', 'engenharia de blockchain',
    'tecnologia verde', 'engenharia sustentável', 'sistemas resilientes', 'design interdisciplinar',

    # Concetti generali
    'ingegnere', 'ingegneria', 'progettato', 'design', 'progetti', 'progettato', 'tecnico', 'tecnico',
    'progettazione del sistema', 'modellizzazione', 'modellazione', 'modello', 'modelli', 'simulazione', 'simulare', 'simulato',
    'analisi', 'analizzare', 'analizzato', 'ottimizzazione', 'ottimizzare', 'efficienza', 'efficiente',
    'integrazione', 'integrare', 'progetto', 'calcolo', 'calcoli', 'schematico', 'disegno', 'disegni', 'energie',

    # Discipline (aggiungi le forme mancanti)
    'ingegneria meccanica', 'ingegneria elettrica', 'ingegneria civile', 'ingegneria chimica',
    'ingegneria aerospaziale', 'ingegneria industriale', 'ingegneria biomedica', 'ingegneria strutturale',
    'ingegneria geotecnica', 'ingegneria nucleare', 'ingegneria automobilistica',
    'ingegneria marina', 'ingegneria del software', 'ingegneria informatica', 'ingegneria dei sistemi',
    'ingegneria ottica', 'ingegneria ambientale', 'ingegneria dei materiali',

    # Principi fisici
    'carico', 'carichi', 'forza', 'forze', 'coppia', 'momento', 'stress', 'deformazione',
    'tensione', 'compressione', 'elasticità', 'plasticità', 'attrito', 'inerzia', 'vibrazione', 'risonanza',

    # Materiali
    'materiale', 'materiali', 'metallo', 'metalli', 'lega', 'leghe', 'composito', 'ceramica', 'ceramiche',
    'polimero', 'polimeri', 'fatica', 'frattura', 'crepa', 'crepe', 'corrosione',
    'conduttività termica', 'conduttività elettrica', 'nanomateriali', 'nano-compositi',

    # Robotica e Controllo
    'robot', 'robot', 'robotica', 'automazione', 'automatizzato', 'autonomo', 'motore servo', 'motori servo',
    'traiettoria', 'pianificazione del percorso', 'SLAM', 'controllo PID', 'attuatore', 'attuatori', 'cinematica', 'cinematica inversa',

    # Elettronica
    'circuito', 'circuiti', 'pcb', 'microcontrollore', 'microcontrollori', 'sensore', 'sensori', 'transistor',
    'resistore', 'condensatore', 'induttore', 'diode', 'segnale', 'segnali', 'amplificatore', 'oscilloscopio', 'tensione', 'corrente',

    # Simulazione / Strumenti
    'CAD', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', 'elemento finito',
    'FEM', 'FEA', 'stampa 3D', 'gemello digitale', 'meccanica computazionale', 'metodi numerici',

    # Energia
    'energia rinnovabile', 'accumulo di energia', 'sistema termico', 'combustione', 'scambiatore di calore', 'pannello solare',
    'turbina eolica', 'dinamica dei fluidi', 'idraulica', 'sistema di alimentazione', 'rete elettrica', 'smart grid', 'batteria', 'batterie',

    # Infrastruttura
    'costruzione', 'infrastruttura', 'integrità strutturale', 'ponte', 'ponti', 'galleria', 'diga', 'autostrada',
    'portante', 'topografia', 'meccanica del suolo', 'pianificazione urbana', 'cemento', 'cemento armato', 'asfalto',

    # Trasporti
    'veicolo', 'veicoli', 'mobilità', 'motore', 'motori', 'turbo', 'aerodinamica',
    'veicolo elettrico', 'veicolo ibrido', 'veicolo autonomo', 'efficienza del carburante',

    # Sicurezza e QA
    'controllo qualità', 'garanzia di qualità', 'analisi dei guasti', 'rilevamento dei guasti', 'gestione del ciclo di vita',
    'conformità', 'ISO', 'standard', 'test', 'certificazione', 'valutazione dei rischi', 'ingegneria dell\'affidabilità',

    # Tecnologie emergenti
    'sistemi cibernetico-fisici', 'fusione dei sensori', 'AI in ingegneria', 'digitalizzazione',
    'materiali intelligenti', 'intelligenza incorporata', 'edge computing', 'IoT', 'ingegneria blockchain',
    'tecnologia verde', 'ingegneria sostenibile', 'sistemi resilienti', 'progettazione interdisciplinare',

    # Concepts généraux
    'ingénieur', 'génie', 'conçu', 'conception', 'conceptions', 'conçu', 'technique', 'technicien',
    'conception du système', 'modélisation', 'modélisation', 'modèle', 'modèles', 'simulation', 'simuler', 'simulé',
    'analyse', 'analyser', 'analysé', 'optimisation', 'optimiser', 'efficacité', 'efficace',
    'intégration', 'intégrer', 'plan', 'calcul', 'calculs', 'schématique', 'dessin', 'dessins', 'énergies',

    # Disciplines (ajouter les formes manquantes)
    'génie mécanique', 'génie électrique', 'génie civil', 'génie chimique',
    'génie aérospatial', 'génie industriel', 'génie biomédical', 'génie structurel',
    'génie géotechnique', 'génie nucléaire', 'génie automobile',
    'génie marin', 'génie logiciel', 'génie informatique', 'génie des systèmes',
    'génie optique', 'génie environnemental', 'génie des matériaux',

    # Principes physiques
    'charge', 'charges', 'force', 'forces', 'couple', 'momentum', 'contrainte', 'déformation',
    'tension', 'compression', 'élasticité', 'plasticité', 'friction', 'inertie', 'vibration', 'résonance',

    # Matériaux
    'matériau', 'matériaux', 'métal', 'métaux', 'alliage', 'alliages', 'composite', 'céramique', 'céramiques',
    'polymère', 'polymères', 'fatigue', 'fracture', 'fissure', 'fissures', 'corrosion',
    'conductivité thermique', 'conductivité électrique', 'nanomatériaux', 'nano-composites',

    # Robotique et contrôle
    'robot', 'robots', 'robotique', 'automatisation', 'automatisé', 'autonome', 'moteur à servomoteur', 'moteurs à servomoteur',
    'trajectoire', 'planification de trajectoire', 'SLAM', 'contrôle PID', 'actionneur', 'actionneurs', 'cinématique', 'cinématique inverse',

    # Électronique
    'circuit', 'circuits', 'pcb', 'microcontrôleur', 'microcontrôleurs', 'capteur', 'capteurs', 'transistor',
    'résistance', 'condensateur', 'inducteur', 'diode', 'signal', 'signaux', 'amplificateur', 'oscilloscope', 'tension', 'courant',

    # Simulation/Outils
    'CAO', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', 'élément fini',
    'FEM', 'FEA', 'impression 3D', 'jumeau numérique', 'mécanique computationnelle', 'méthodes numériques',

    # Énergie
    'énergie renouvelable', 'stockage d\'énergie', 'système thermique', 'combustion', 'échangeur de chaleur', 'panneau solaire',
    'éolienne', 'dynamique des fluides', 'hydraulique', 'système d\'énergie', 'réseau électrique', 'réseau intelligent', 'batterie', 'batteries',

    # Infrastructure
    'construction', 'infrastructure', 'intégrité structurelle', 'pont', 'ponts', 'tunnel', 'barrage', 'autoroute',
    'support de charge', 'arpentage', 'mécanique des sols', 'planification urbaine', 'béton', 'béton armé', 'asphalte',

    # Transport
    'véhicule', 'véhicules', 'mobilité', 'moteur', 'moteurs', 'turbo', 'aérodynamique',
    'véhicule électrique', 'véhicule hybride', 'véhicule autonome', 'efficacité énergétique',

    # Sécurité et assurance qualité
    'contrôle de la qualité', 'assurance qualité', 'analyse des défaillances', 'détection des pannes', 'gestion du cycle de vie',
    'conformité', 'ISO', 'norme', 'tests', 'certification', 'évaluation des risques', 'ingénierie de la fiabilité',

    # Technologies émergentes
    'systèmes cyber-physiques', 'fusion de capteurs', 'IA en ingénierie', 'numérisation',
    'matériaux intelligents', 'intelligence intégrée', 'informatique en périphérie', 'IoT', 'ingénierie de la blockchain',
    'technologie verte', 'ingénierie durable', 'systèmes résilients', 'conception interdisciplinaire',

    # 一般概念
    '工程师', '工程', '设计过的', '设计', '设计们', '设计的', '技术的', '技术员',
    '系统设计', '建模', '建模', '模型', '模型们', '仿真', '模拟', '模拟的',
    '分析', '分析', '已分析', '已分析过', '优化', '优化过', '效率', '高效的',
    '集成', '集成过', '蓝图', '计算', '计算们', '原理图', '图纸', '图纸们', '能量',

    # 学科（添加缺失形式）
    '机械工程', '电气工程', '土木工程', '化学工程', '航空航天工程', '工业工程', '生物医学工程', '结构工程',
    '岩土工程', '核工程', '汽车工程', '海洋工程', '软件工程', '计算机工程', '系统工程',
    '光学工程', '环境工程', '材料工程',

    # 物理原理
    '载荷', '载荷们', '力', '力们', '扭矩', '动量', '应力', '应变',
    '张力', '压缩', '弹性', '塑性', '摩擦', '惯性', '振动', '共振',

    # 材料
    '材料', '材料们', '金属', '金属们', '合金', '合金们', '复合材料', '陶瓷', '陶瓷们',
    '聚合物', '聚合物们', '疲劳', '断裂', '裂纹', '裂纹们', '腐蚀',
    '热导率', '电导率', '纳米材料', '纳米复合材料',

    # 机器人与控制
    '机器人', '机器人们', '机器人技术', '自动化', '自动化的', '自主的', '伺服电机', '伺服电机们',
    '轨迹', '路径规划', 'SLAM', 'PID 控制', '执行器', '执行器们', '运动学', '反向运动学',

    # 电子学
    '电路', '电路们', '印刷电路板', '微控制器', '微控制器们', '传感器', '传感器们', '晶体管',
    '电阻器', '电容器', '电感器', '二极管', '信号', '信号们', '放大器', '示波器', '电压', '电流',

    # 仿真/工具
    'CAD', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', '有限元',
    'FEM', 'FEA', '3D 打印', '数字孪生', '计算力学', '数值方法',

    # 能源
    '可再生能源', '能源储存', '热系统', '燃烧', '热交换器', '太阳能电池板',
    '风力涡轮机', '流体动力学', '水力学', '电力系统', '电网', '智能电网', '电池', '电池们',

    # 基础设施
    '建筑', '基础设施', '结构完整性', '桥梁', '桥梁们', '隧道', '水坝', '高速公路',
    '承载', '测量', '土力学', '城市规划', '混凝土', '钢筋混凝土', '沥青',

    # 运输
    '车辆', '车辆们', '流动性', '发动机', '发动机们', '涡轮', '空气动力学',
    '电动汽车', '混合动力车', '自动驾驶汽车', '燃油效率',

    # 安全与质量保证
    '质量控制', '质量保证', '故障分析', '故障检测', '生命周期管理',
    '合规性', 'ISO', '标准', '测试', '认证', '风险评估', '可靠性工程',

    # 新兴技术
    '赛博物理系统', '传感器融合', '工程中的AI', '数字化',
    '智能材料', '嵌入式智能', '边缘计算', '物联网', '区块链工程',
    '绿色技术', '可持续工程', '弹性系统', '跨学科设计',

    # Общие концепции
    'инженер', 'инженерия', 'спроектировано', 'дизайн', 'дизайны', 'спроектирован', 'технический', 'техник',
    'проектирование системы', 'моделирование', 'моделирование', 'модель', 'модели', 'симуляция', 'симулировать', 'смоделирован',
    'анализ', 'анализировать', 'проанализирован', 'оптимизация', 'оптимизировать', 'эффективность', 'эффективный',
    'интеграция', 'интегрировать', 'чертеж', 'расчёт', 'расчёты', 'схема', 'чертёж', 'чертежи', 'энергия',

    # Дисциплины (добавьте недостающие формы)
    'механическая инженерия', 'электрическая инженерия', 'гражданская инженерия', 'химическая инженерия',
    'авиакосмическая инженерия', 'промышленная инженерия', 'биомедицинская инженерия', 'строительная инженерия',
    'геотехническая инженерия', 'ядерная инженерия', 'автомобильная инженерия',
    'морская инженерия', 'программная инженерия', 'компьютерная инженерия', 'системная инженерия',
    'оптическая инженерия', 'экологическая инженерия', 'материальная инженерия',

    # Физические принципы
    'нагрузка', 'нагрузки', 'сила', 'силы', 'крутящий момент', 'импульс', 'напряжение', 'деформация',
    'натяжение', 'сжатие', 'упругость', 'пластичность', 'трение', 'инерция', 'вибрация', 'резонанс',

    # Материалы
    'материал', 'материалы', 'металл', 'металлы', 'сплав', 'сплавы', 'композит', 'керамика', 'керамические',
    'полимер', 'полимеры', 'усталость', 'трещина', 'трещины', 'коррозия',
    'теплопроводность', 'электрическая проводимость', 'нано-материалы', 'нано-композиты',

    # Робототехника и управление
    'робот', 'роботы', 'робототехника', 'автоматизация', 'автоматизированный', 'автономный', 'серводвигатель', 'серводвигатели',
    'траектория', 'планирование пути', 'SLAM', 'PID управление', 'привод', 'приводы', 'кинематика', 'обратная кинематика',

    # Электроника
    'схема', 'схемы', 'плата', 'микроконтроллер', 'микроконтроллеры', 'датчик', 'датчики', 'транзистор',
    'резистор', 'конденсатор', 'индуктор', 'диод', 'сигнал', 'сигналы', 'усилитель', 'осциллограф', 'напряжение', 'ток',

    # Симуляция/Инструменты
    'CAD', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', 'конечный элемент',
    'FEM', 'FEA', '3D печать', 'цифровой двойник', 'вычислительная механика', 'численные методы',

    # Энергия
    'возобновляемая энергия', 'хранение энергии', 'тепловая система', 'сгорание', 'теплообменник', 'солнечная панель',
    'ветровая турбина', 'динамика жидкостей', 'гидравлика', 'энергетическая система', 'энергетическая сеть', 'умная сеть', 'аккумулятор', 'аккумуляторы',

    # Инфраструктура
    'строительство', 'инфраструктура', 'структурная целостность', 'мост', 'мосты', 'туннель', 'дамба', 'автодорога',
    'нагрузка', 'землемерные работы', 'механика грунтов', 'городское планирование', 'бетон', 'армированный бетон', 'асфальт',

    # Транспорт
    'транспортное средство', 'транспортные средства', 'мобильность', 'двигатель', 'двигатели', 'турбина', 'аэродинамика',
    'электрический автомобиль', 'гибридный автомобиль', 'автономное транспортное средство', 'топливная эффективность',

    # Безопасность и контроль качества
    'контроль качества', 'гарантия качества', 'анализ сбоев', 'обнаружение неисправностей', 'управление жизненным циклом',
    'соответствие', 'ISO', 'стандарт', 'тестирование', 'сертификация', 'оценка рисков', 'инжиниринг надежности',

    # Новые технологии
    'кибер-физические системы', 'слияние датчиков', 'ИИ в инженерии', 'цифровизация',
    'умные материалы', 'встроенный интеллект', 'периферийные вычисления', 'IoT', 'инженерия блокчейн',
    'зеленые технологии', 'устойчивая инженерия', 'устойчивые системы', 'междисциплинарный дизайн',

    # 一般的な概念
    'エンジニア', 'エンジニアリング', '設計された', '設計', '設計図', 'デザイン', '技術的', '技術者',
    'システム設計', 'モデリング', 'モデル化', 'モデル', 'モデル（複数形）', 'シミュレーション', 'シミュレート', 'シミュレートされた',
    '分析', '分析する', '分析された', '分析済み', '最適化', '最適化する', '効率', '効率的な',
    '統合', '統合する', '青写真', '計算', '計算（複数形）', '回路図', '図面', '図面（複数形）', 'エネルギー',

    # 専門分野（欠落している形を追加）
    '機械工学', '電気工学', '土木工学', '化学工学', '航空宇宙工学', '産業工学', '生物医学工学', '構造工学',
    '地盤工学', '原子力工学', '自動車工学', '海洋工学', 'ソフトウェア工学', 'コンピュータ工学', 'システム工学',
    '光学工学', '環境工学', '材料工学',

    # 物理的な原則
    '荷重', '荷重（複数形）', '力', '力（複数形）', 'トルク', '運動量', '応力', 'ひずみ',
    '引張', '圧縮', '弾性', '塑性', '摩擦', '慣性', '振動', '共鳴',

    # 材料
    '材料', '材料（複数形）', '金属', '金属（複数形）', '合金', '合金（複数形）', '複合材料', 'セラミック', 'セラミックス',
    'ポリマー', 'ポリマー（複数形）', '疲労', '破断', '亀裂', '亀裂（複数形）', '腐食',
    '熱伝導率', '電気伝導率', 'ナノ材料', 'ナノ複合材料',

    # ロボット工学と制御
    'ロボット', 'ロボット（複数形）', 'ロボット工学', '自動化', '自動化された', '自律的', 'サーボモーター', 'サーボモーター（複数形）',
    '軌道', '経路計画', 'SLAM', 'PID制御', 'アクチュエータ', 'アクチュエータ（複数形）', '運動学', '逆運動学',

    # 電子工学
    '回路', '回路（複数形）', 'プリント基板', 'マイクロコントローラー', 'マイクロコントローラー（複数形）', 'センサー', 'センサー（複数形）', 'トランジスタ',
    '抵抗器', 'コンデンサ', 'インダクタ', 'ダイオード', '信号', '信号（複数形）', 'アンプ', 'オシロスコープ', '電圧', '電流',

    # シミュレーション/ツール
    'CAD', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', '有限要素',
    'FEM', 'FEA', '3Dプリンティング', 'デジタルツイン', '計算力学', '数値解析',

    # エネルギー
    '再生可能エネルギー', 'エネルギー貯蔵', '熱システム', '燃焼', '熱交換器', '太陽光パネル',
    '風力タービン', '流体力学', '水力学', '電力システム', '電力網', 'スマートグリッド', 'バッテリー', 'バッテリー（複数形）',

    # インフラ
    '建設', 'インフラ', '構造的整合性', '橋', '橋（複数形）', 'トンネル', 'ダム', '高速道路',
    '荷重支持', '測量', '土壌力学', '都市計画', 'コンクリート', '鉄筋コンクリート', 'アスファルト',

    # 輸送
    '車両', '車両（複数形）', '移動性', 'エンジン', 'エンジン（複数形）', 'ターボ', '空力学',
    '電気自動車', 'ハイブリッド車', '自律走行車', '燃費',

    # 安全性と品質管理
    '品質管理', '品質保証', '故障分析', '故障検出', 'ライフサイクル管理',
    'コンプライアンス', 'ISO', '標準', 'テスト', '認証', 'リスク評価', '信頼性工学',

    # 新興技術
    'サイバーフィジカルシステム', 'センサーフュージョン', 'エンジニアリングにおけるAI', 'デジタル化',
    'スマート材料', '組み込み型知能', 'エッジコンピューティング', 'IoT', 'ブロックチェーン工学',
    'グリーンテクノロジー', '持続可能な工学', 'レジリエントシステム', '学際的設計',

    # 일반 개념
    '엔지니어', '공학', '엔지니어링', '설계', '설계들', '설계된', '기술적인', '기술자',
    '시스템 설계', '모델링', '모델링', '모델', '모델들', '시뮬레이션', '시뮬레이트', '시뮬레이션된',
    '분석', '분석하다', '분석됨', '분석됨 (영어식)', '최적화', '최적화하다', '효율성', '효율적인',
    '통합', '통합하다', '청사진', '계산', '계산들', '도식', '도면', '도면들', '에너지',

    # 학문 분야 (누락된 형태 추가)
    '기계 공학', '전기 공학', '토목 공학', '화학 공학',
    '항공 우주 공학', '산업 공학', '생체 의학 공학', '구조 공학',
    '지반 공학', '핵 공학', '자동차 공학',
    '해양 공학', '소프트웨어 공학', '컴퓨터 공학', '시스템 공학',
    '광학 공학', '환경 공학', '재료 공학',

    # 물리적 원리
    '하중', '하중들', '힘', '힘들', '토크', '모멘텀', '응력', '변형',
    '장력', '압축', '탄성', '소성', '마찰', '관성', '진동', '공명',

    # 재료
    '재료', '재료들', '금속', '금속들', '합금', '합금들', '복합재', '세라믹', '세라믹들',
    '폴리머', '폴리머들', '피로', '파손', '균열', '균열들', '부식',
    '열전도율', '전기전도율', '나노재료', '나노복합재',

    # 로보틱스 & 제어
    '로봇', '로봇들', '로보틱스', '자동화', '자동화된', '자율', '서보모터', '서보모터들',
    '경로', '경로 계획', 'SLAM', 'PID 제어', '액추에이터', '액추에이터들', '운동학', '역운동학',

    # 전자공학
    '회로', '회로들', 'PCB', '마이크로컨트롤러', '마이크로컨트롤러들', '센서', '센서들', '트랜지스터',
    '저항기', '커패시터', '인덕터', '다이오드', '신호', '신호들', '증폭기', '오실로스코프', '전압', '전류',

    # 시뮬레이션/도구
    'CAD', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', '유한 요소법',
    'FEM', 'FEA', '3D 프린팅', '디지털 트윈', '컴퓨터 역학', '수치 방법',

    # 에너지
    '재생 가능 에너지', '에너지 저장', '열 시스템', '연소', '열교환기', '태양광 패널',
    '풍력 터빈', '유체 역학', '수리학', '전력 시스템', '전력망', '스마트 그리드', '배터리', '배터리들',

    # 인프라
    '건설', '인프라', '구조적 무결성', '다리', '다리들', '터널', '댐', '고속도로',
    '하중 지지', '측량', '토양 역학', '도시 계획', '콘크리트', '강화 콘크리트', '아스팔트',

    # 교통
    '차량', '차량들', '모빌리티', '엔진', '엔진들', '터보', '공기역학',
    '전기차', '하이브리드 차량', '자율주행차', '연료 효율',

    # 안전 & 품질 관리
    '품질 관리', '품질 보증', '고장 분석', '결함 탐지', '생애 주기 관리',
    '규정 준수', 'ISO', '표준', '테스트', '인증', '위험 평가', '신뢰성 공학',

    # 신기술
    '사이버-물리 시스템', '센서 융합', '엔지니어링에서의 AI', '디지털화',
    '스마트 재료', '내장 지능', '엣지 컴퓨팅', 'IoT', '블록체인 공학',
    '그린 기술', '지속 가능한 공학', '회복력 있는 시스템', '학제 간 설계',

    # 一般概念
    '工程師', '工程學', '已工程化', '設計', '設計們', '已設計', '技術的', '技術員',
    '系統設計', '建模', '建模', '模型', '模型們', '模擬', '模擬', '已模擬',
    '分析', '分析', '已分析', '已分析', '優化', '優化', '效率', '有效率',
    '整合', '整合', '藍圖', '計算', '計算們', '原理圖', '繪圖', '圖紙', '能源',

    # 學科（添加缺失的形式）
    '機械工程', '電氣工程', '土木工程', '化學工程',
    '航空工程', '工業工程', '生物醫學工程', '結構工程',
    '岩土工程', '核能工程', '汽車工程',
    '海洋工程', '軟體工程', '計算機工程', '系統工程',
    '光學工程', '環境工程', '材料工程',

    # 物理原則
    '負載', '負載們', '力', '力量', '扭矩', '動量', '應力', '應變',
    '張力', '壓縮', '彈性', '塑性', '摩擦', '慣性', '振動', '共振',

    # 材料
    '材料', '材料們', '金屬', '金屬們', '合金', '合金們', '複合材料', '陶瓷', '陶瓷們',
    '聚合物', '聚合物們', '疲勞', '斷裂', '裂縫', '裂縫們', '腐蝕',
    '熱傳導率', '電導率', '納米材料', '納米複合材料',

    # 機器人與控制
    '機器人', '機器人們', '機器人學', '自動化', '自動化的', '自律', '伺服馬達', '伺服馬達們',
    '軌跡', '路徑規劃', 'SLAM', 'PID控制', '執行器', '執行器們', '運動學', '逆運動學',

    # 電子學
    '電路', '電路們', '印刷電路板', '微控制器', '微控制器們', '感應器', '感應器們', '晶體管',
    '電阻器', '電容器', '電感器', '二極管', '信號', '信號們', '放大器', '示波器', '電壓', '電流',

    # 模擬/工具
    'CAD', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', '有限元素',
    'FEM', 'FEA', '3D列印', '數位雙胞胎', '計算力學', '數值方法',

    # 能源
    '可再生能源', '能源儲存', '熱系統', '燃燒', '熱交換器', '太陽能板',
    '風力渦輪機', '流體動力學', '水力學', '電力系統', '電力網', '智能電網', '電池', '電池們',

    # 基礎設施
    '建設', '基礎設施', '結構完整性', '橋梁', '橋梁們', '隧道', '水壩', '高速公路',
    '承載', '測量', '土壤力學', '城市規劃', '混凝土', '鋼筋混凝土', '瀝青',

    # 交通運輸
    '車輛', '車輛們', '流動性', '引擎', '引擎們', '渦輪', '空氣動力學',
    '電動車', '混合動力車', '自駕車', '燃油效率',

    # 安全與品質保證
    '品質控制', '品質保證', '故障分析', '故障檢測', '生命周期管理',
    '合規性', 'ISO', '標準', '測試', '認證', '風險評估', '可靠性工程',

    # 新興技術
    '物理-網絡系統', '感測融合', '工程中的AI', '數位化',
    '智能材料', '嵌入式智慧', '邊緣計算', '物聯網', '區塊鏈工程',
    '綠色技術', '可持續工程', '彈性系統', '跨學科設計',

    # Obecné pojmy
    'inženýr', 'inženýrství', 'navrženo', 'návrh', 'návrhy', 'navržený', 'technický', 'technik',
    'návrh systému', 'modelování', 'modelování', 'model', 'modely', 'simulace', 'simulovat', 'simulovaný',
    'analýza', 'analyzovat', 'analyzováno', 'analyzováno', 'optimalizace', 'optimalizovat', 'efektivita', 'efektivní',
    'integrace', 'integrovat', 'plán', 'výpočet', 'výpočty', 'schéma', 'kreslení', 'kresby', 'energie',

    # Disciplíny (přidání chybějících tvarů)
    'mechanické inženýrství', 'elektrické inženýrství', 'stavební inženýrství', 'chemické inženýrství',
    'letecké inženýrství', 'průmyslové inženýrství', 'biomedicínské inženýrství', 'strukturní inženýrství',
    'geotechnické inženýrství', 'jaderné inženýrství', 'automobilové inženýrství',
    'námořní inženýrství', 'softwarové inženýrství', 'počítačové inženýrství', 'systémové inženýrství',
    'optické inženýrství', 'environmentální inženýrství', 'materiálové inženýrství',

    # Fyzikální principy
    'zátěž', 'zátěže', 'síla', 'síly', 'moment', 'hybnost', 'napětí', 'deformace',
    'tah', 'stlačení', 'elasticita', 'plasticita', 'tření', 'setrvačnost', 'vibrace', 'rezonance',

    # Materiály
    'materiál', 'materiály', 'kov', 'kovy', 'slitina', 'slitiny', 'kompozit', 'keramika', 'keramiky',
    'polymer', 'polymerní', 'únava', 'zlom', 'trhlina', 'trhliny', 'koroze',
    'teplotní vodivost', 'elektrická vodivost', 'nanomateriály', 'nano-kompozity',

    # Robotika a řízení
    'robot', 'roboti', 'robotika', 'automatizace', 'automatizovaný', 'autonomní', 'servomotor', 'servomotory',
    'trajektorie', 'plánování cesty', 'SLAM', 'PID řízení', 'aktuátor', 'aktuátory', 'kinematika', 'inverzní kinematika',

    # Elektronika
    'obvod', 'obvody', 'PCB', 'mikrořadič', 'mikrořadiče', 'senzor', 'senzory', 'tranzistor',
    'rezistor', 'kondenzátor', 'induktor', 'dioda', 'signál', 'signály', 'zesilovač', 'osciloskop', 'napětí', 'proud',

    # Simulace/nástroje
    'CAD', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', 'metoda konečných prvků',
    'FEM', 'FEA', '3D tisk', 'digitální dvojče', 'výpočetní mechanika', 'numerické metody',

    # Energie
    'obnovitelná energie', 'úložiště energie', 'tepelné systémy', 'spalování', 'tepelné výměníky', 'solární panel',
    'větrná turbína', 'fluidní dynamika', 'hydraulika', 'energetický systém', 'energetická síť', 'inteligentní síť', 'baterie', 'baterie',

    # Infrastruktura
    'stavba', 'infrastruktura', 'strukturní integrita', 'most', 'mosty', 'tunel', 'přehrada', 'dálnice',
    'nosnost', 'geodézie', 'mechanika půdy', 'městské plánování', 'beton', 'armovaný beton', 'asfalt',

    # Doprava
    'vozidlo', 'vozidla', 'mobilita', 'motor', 'motory', 'turbo', 'aerodynamika',
    'elektrické vozidlo', 'hybridní vozidlo', 'autonomní vozidlo', 'palivová účinnost',

    # Bezpečnost a QA
    'kontrola kvality', 'zajištění kvality', 'analýza poruchy', 'detekce poruch', 'řízení životního cyklu',
    'shoda', 'ISO', 'standard', 'testování', 'certifikace', 'hodnocení rizik', 'inženýrství spolehlivosti',

    # Vznikající technologie
    'kyber-fyzikální systémy', 'fúze senzorů', 'AI v inženýrství', 'digitalizace',
    'inteligentní materiály', 'vestavěná inteligence', 'edge computing', 'IoT', 'blockchain inženýrství',
    'zelená technologie', 'udržitelné inženýrství', 'odolné systémy', 'interdisciplinární návrh',

    # المفاهيم العامة
    'مهندس', 'الهندسة', 'مُصمم', 'تصميم', 'تصاميم', 'مصمم', 'تقني', 'فني',
    'تصميم النظام', 'النمذجة', 'النمذجة', 'نموذج', 'نماذج', 'محاكاة', 'يحاكي', 'محاكاة',
    'تحليل', 'يحلل', 'محلل', 'محللة', 'تحسين', 'يحسن', 'كفاءة', 'فعال',
    'تكامل', 'يُدمج', 'مخطط', 'حساب', 'حسابات', 'مخطط تفصيلي', 'رسم', 'رسومات', 'طاقات',

    # التخصصات (إضافة الأشكال المفقودة)
    'الهندسة الميكانيكية', 'الهندسة الكهربائية', 'الهندسة المدنية', 'الهندسة الكيميائية',
    'الهندسة الجوية', 'الهندسة الصناعية', 'الهندسة الطبية الحيوية', 'الهندسة الإنشائية',
    'الهندسة الجيوتقنية', 'الهندسة النووية', 'الهندسة السيارات',
    'الهندسة البحرية', 'الهندسة البرمجية', 'هندسة الكمبيوتر', 'هندسة النظم',
    'الهندسة البصرية', 'الهندسة البيئية', 'هندسة المواد',

    # المبادئ الفيزيائية
    'حمل', 'أحمال', 'قوة', 'قوى', 'عزم', 'زخم', 'إجهاد', 'تمدد',
    'توتر', 'ضغط', 'مرونة', 'لدونة', 'احتكاك', 'قصور', 'اهتزاز', 'رنين',

    # المواد
    'مادة', 'مواد', 'معدن', 'معادن', 'سبيكة', 'سبائك', 'مركب', 'سيراميك', 'سيراميك',
    'بوليمر', 'بوليمرات', 'إرهاق', 'كسر', 'شَق', 'شقوق', 'تآكل',
    'التوصيل الحراري', 'التوصيل الكهربائي', 'مواد نانوية', 'مركبات نانوية',

    # الروبوتات والتحكم
    'روبوت', 'روبوتات', 'الروبوتات', 'أتمتة', 'أوتوماتيكي', 'ذاتي', 'محرك سيرفو', 'محركات سيرفو',
    'مسار', 'تخطيط المسار', 'SLAM', 'التحكم PID', 'محرك تنشيط', 'محركات تنشيط', 'الحركيات', 'الحركيات العكسية',

    # الإلكترونيات
    'دائرة', 'دوائر', 'دائرة مطبوعة', 'متحكم دقيق', 'متحكمات دقيقة', 'مستشعر', 'مستشعرات', 'ترانزستور',
    'مقاوم', 'مكثف', 'ملف', 'دايود', 'إشارة', 'إشارات', 'مكبر صوت', 'مذبذب', 'جهد', 'تيار',

    # المحاكاة/الأدوات
    'CAD', 'AutoCAD', 'SolidWorks', 'ANSYS', 'Simulink', 'MATLAB', 'LabVIEW', 'CFD', 'عنصر منتهي',
    'FEM', 'FEA', 'الطباعة ثلاثية الأبعاد', 'التوأم الرقمي', 'الميكانيكا الحاسوبية', 'الطرق العددية',

    # الطاقة
    'الطاقة المتجددة', 'تخزين الطاقة', 'النظام الحراري', 'الاحتراق', 'مبادل حراري', 'لوحة شمسية',
    'توربينات الرياح', 'ديناميكا السوائل', 'هيدروليكا', 'نظام الطاقة', 'شبكة الطاقة', 'الشبكة الذكية', 'بطارية', 'بطاريات',

    # البنية التحتية
    'البناء', 'البنية التحتية', 'سلامة هيكلية', 'جسر', 'جسور', 'نفق', 'سد', 'طريق سريع',
    'داعم للحمل', 'مسح', 'ميكانيكا التربة', 'التخطيط الحضري', 'خرسانة', 'خرسانة مسلحة', 'أسفلت',

    # النقل
    'مركبة', 'مركبات', 'تنقل', 'محرك', 'محركات', 'توربو', 'ديناميكا هوائية',
    'مركبة كهربائية', 'مركبة هجينة', 'مركبة ذاتية القيادة', 'كفاءة الوقود',

    # السلامة وضمان الجودة
    'مراقبة الجودة', 'ضمان الجودة', 'تحليل الفشل', 'الكشف عن الأخطاء', 'إدارة دورة الحياة',
    'الامتثال', 'ISO', 'معيار', 'اختبار', 'شهادة', 'تقييم المخاطر', 'هندسة الموثوقية',

    # التقنيات الناشئة
    'أنظمة السيبرانية الفيزيائية', 'دمج المستشعرات', 'الذكاء الاصطناعي في الهندسة', 'الرقمنة',
    'المواد الذكية', 'الذكاء المدمج', 'الحوسبة الحدية', 'إنترنت الأشياء', 'هندسة البلوكشين',
    'التكنولوجيا الخضراء', 'الهندسة المستدامة', 'الأنظمة القوية', 'التصميم متعدد التخصصات'

],

    'biology': [
    # General biology
    'biology', 'biological', 'biologist', 'biologists', 'bioscience', 'biosciences', 'life science', 'life sciences',

    # Organisms and taxonomy
    'organism', 'organisms', 'species', 'genus', 'taxon', 'taxa', 'taxonomy', 'phylogeny', 'evolution', 'evolutionary',
    'biodiversity', 'population', 'populations', 'ecosystem', 'ecosystems', 'biome', 'biomes',

    # Subfields of biology
    'genetics', 'genetic', 'genomic', 'genomics', 'epigenetic', 'epigenetics',
    'molecular biology', 'molecular', 'cell biology', 'cytology',
    'neurobiology', 'neuroscience', 'zoology', 'botany', 'microbiology', 'mycology',
    'parasitology', 'virology', 'immunology', 'anatomy', 'physiology', 'ethology',
    'embryology', 'developmental biology', 'paleontology', 'biochemistry', 'bioorganic',

    # Molecular and cellular biology
    'gene', 'genes', 'genome', 'genomes', 'epigenome', 'DNA', 'RNA', 'mRNA', 'tRNA', 'protein', 'proteins',
    'enzyme', 'enzymes', 'transcription', 'translation', 'replication', 'nucleotide', 'nucleotides',
    'mutation', 'mutations', 'chromosome', 'chromosomes', 'organelle', 'organelles',
    'mitochondrion', 'mitochondria', 'chloroplast', 'ribosome', 'ribosomes', 'lysosome', 'cell', 'cells',
    'cell wall', 'cell membrane', 'plasma membrane', 'nucleus', 'nuclei',

    # Organism-level and functional biology
    'reproduction', 'metabolism', 'homeostasis', 'adaptation', 'natural selection',
    'inheritance', 'phenotype', 'phenotypes', 'genotype', 'genotypes',
    'photosynthesis', 'respiration', 'autotroph', 'heterotroph',

    # Techniques and experimental tools
    'PCR', 'qPCR', 'RT-PCR', 'gel electrophoresis', 'cloning', 'microscopy', 'confocal', 'fluorescence',
    'flow cytometry', 'spectrophotometry', 'immunostaining', 'western blot', 'ELISA', 'assay', 'assays',
    'sequencing', 'NGS', 'CRISPR', 'genome editing', 'transfection', 'transgenic', 'culture', 'in vitro', 'in vivo',

    # Ecology and environmental biology
    'habitat', 'habitats', 'niche', 'symbiosis', 'predator', 'prey', 'trophic', 'food chain', 'food web',
    'carbon cycle', 'nitrogen cycle', 'pollination', 'ecosystem service', 'ecosystem services',
    'deforestation', 'reforestation', 'bioconservation', 'ecology', 'ecological',

    # Systems and applied biology
    'systems biology', 'synthetic biology', 'bioinformatics', 'biotechnology', 'biomedicine', 'biomedical',
    'agricultural biology', 'forensic biology', 'marine biology', 'environmental biology',

    # Organ systems (animals and humans)
    'circulatory system', 'digestive system', 'nervous system', 'endocrine system',
    'reproductive system', 'muscular system', 'immune system', 'urinary system', 'respiratory system',

    # Neuroscience specific (often overlaps with psychology)
    'neuron', 'neurons', 'glia', 'neurotransmitter', 'neurotransmitters', 'synapse', 'synapses',
    'brain', 'cortex', 'hippocampus', 'spinal cord', 'CNS', 'PNS',

    # Model organisms and life forms
    'bacteria', 'archaea', 'virus', 'viruses', 'fungi', 'fungus', 'yeast',
    'eukaryote', 'eukaryotes', 'prokaryote', 'prokaryotes', 'mouse', 'mice', 'zebrafish', 'fly', 'flies', 'insect', 'insects',

    # Plants and related terms
    'plant', 'plants', 'angiosperm', 'gymnosperm', 'algae', 'plankton', 'photosynthetic',
    'chlorophyll', 'xylem', 'phloem', 'root', 'stem', 'leaf', 'flower', 'pollination', 'seeds',

    # Animals and classification
    'animal', 'animals', 'vertebrate', 'invertebrate', 'mammal', 'mammals', 'bird', 'birds',
    'fish', 'amphibian', 'reptile', 'arthropod', 'insect', 'insects',

    # Research settings
    'lab', 'laboratory', 'petri dish', 'incubator', 'centrifuge', 'biohazard', 'biosafety',
    'biobank', 'bioreactor', 'model organism', 'model organisms',
    ],
    'biology': [
    # Biología general
    'biología', 'biológico', 'biólogo', 'biólogos', 'ciencias biológicas', 'ciencias biológicas', 'ciencias de la vida', 'ciencias de la vida',

    # Organismos y taxonomía
    'organismo', 'organismos', 'especies', 'género', 'taxón', 'taxa', 'taxonomía', 'filogenia', 'evolución', 'evolutiva',
    'biodiversidad', 'población', 'poblaciones', 'ecosistema', 'ecosistemas', 'bioma', 'biomas',

    # Subcampos de la biología
    'genética', 'genético', 'genómico', 'genómica', 'epigenético', 'epigenética',
    'biología molecular', 'molecular', 'biología celular', 'citología',
    'neurobiología', 'neurociencia', 'zoología', 'botánica', 'microbiología', 'micología',
    'parasitología', 'virología', 'inmunología', 'anatomía', 'fisiología', 'etología',
    'embriología', 'biología del desarrollo', 'paleontología', 'bioquímica', 'bioorgánica',

    # Biología molecular y celular
    'gen', 'genes', 'genoma', 'genomas', 'epigenoma', 'ADN', 'ARN', 'ARNm', 'ARNt', 'proteína', 'proteínas',
    'enzima', 'enzimas', 'transcripción', 'traducción', 'replicación', 'nucleótido', 'nucleótidos',
    'mutación', 'mutaciones', 'cromosoma', 'cromosomas', 'organelo', 'organillos',
    'mitocondria', 'mitocondrias', 'cloroplasto', 'ribosoma', 'ribosomas', 'lisosoma', 'célula', 'células',
    'pared celular', 'membrana celular', 'membrana plasmática', 'núcleo', 'núcleos',

    # Biología a nivel de organismo y funcional
    'reproducción', 'metabolismo', 'homeostasis', 'adaptación', 'selección natural',
    'herencia', 'fenotipo', 'fenotipos', 'genotipo', 'genotipos',
    'fotosíntesis', 'respiración', 'autótrofo', 'heterótrofo',

    # Técnicas y herramientas experimentales
    'PCR', 'qPCR', 'RT-PCR', 'electroforesis en gel', 'clonación', 'microscopía', 'confocal', 'fluorescencia',
    'citometría de flujo', 'espectrofotometría', 'inmunotinción', 'western blot', 'ELISA', 'ensayo', 'ensayos',
    'secuenciación', 'NGS', 'CRISPR', 'edición del genoma', 'transfección', 'transgénico', 'cultivo', 'in vitro', 'in vivo',

    # Ecología y biología ambiental
    'hábitat', 'hábitats', 'nicho', 'simbiosis', 'depredador', 'presa', 'trófico', 'cadena alimentaria', 'red trófica',
    'ciclo del carbono', 'ciclo del nitrógeno', 'polinización', 'servicio ecosistémico', 'servicios ecosistémicos',
    'deforestación', 'reforestación', 'bioconservación', 'ecología', 'ecológico',

    # Biología de sistemas y aplicada
    'biología de sistemas', 'biología sintética', 'bioinformática', 'biotecnología', 'biomedicina', 'biomédico',
    'biología agrícola', 'biología forense', 'biología marina', 'biología ambiental',

    # Sistemas orgánicos (animales y humanos)
    'sistema circulatorio', 'sistema digestivo', 'sistema nervioso', 'sistema endocrino',
    'sistema reproductivo', 'sistema muscular', 'sistema inmune', 'sistema urinario', 'sistema respiratorio',

    # Neurociencia específica (a menudo se superpone con psicología)
    'neurona', 'neuronas', 'glía', 'neurotransmisor', 'neurotransmisores', 'sinapsis', 'sinapsis',
    'cerebro', 'corteza', 'hipocampo', 'médula espinal', 'SNC', 'SNP',

    # Organismos modelo y formas de vida
    'bacteria', 'arqueas', 'virus', 'virus (plural)', 'hongos', 'hongo', 'levadura',
    'eucariota', 'eucariotas', 'procariota', 'procariotas', 'ratón', 'ratas', 'pez cebra', 'mosca', 'moscas', 'insecto', 'insectos',

    # Plantas y términos relacionados
    'planta', 'plantas', 'angiosperma', 'gimnosperma', 'alga', 'plancton', 'fotosintético',
    'clorofila', 'xilema', 'floema', 'raíz', 'tallo', 'hoja', 'flor', 'polinización', 'semillas',

    # Animales y clasificación
    'animal', 'animales', 'vertebrado', 'invertebrado', 'mamífero', 'mamíferos', 'ave', 'aves',
    'pez', 'anfibio', 'reptil', 'artrópodo', 'insecto', 'insectos',

    # Entornos de investigación
    'laboratorio', 'laboratorio', 'plato de petri', 'incubadora', 'centrífuga', 'biohazard', 'biosalud',
    'biobanco', 'biorreactor', 'organismo modelo', 'organismos modelo',

    # Biologia geral
    'biologia', 'biológico', 'biólogo', 'biólogos', 'ciências biológicas', 'ciências biológicas', 'ciência da vida', 'ciências da vida',

    # Organismos e taxonomia
    'organismo', 'organismos', 'espécie', 'gênero', 'taxon', 'taxa', 'taxonomia', 'filogenia', 'evolução', 'evolutivo',
    'biodiversidade', 'população', 'populações', 'ecossistema', 'ecossistemas', 'bioma', 'biomas',

    # Subáreas da biologia
    'genética', 'genético', 'genômico', 'genômica', 'epigenético', 'epigenética',
    'biologia molecular', 'molecular', 'biologia celular', 'citologia',
    'neurobiologia', 'neurociência', 'zoologia', 'botânica', 'microbiologia', 'micologia',
    'parasitologia', 'virologia', 'imunologia', 'anatomia', 'fisiologia', 'etologia',
    'embriologia', 'biologia do desenvolvimento', 'paleontologia', 'bioquímica', 'bioorgânica',

    # Biologia molecular e celular
    'gene', 'genes', 'genoma', 'genomas', 'epigenoma', 'DNA', 'RNA', 'mRNA', 'tRNA', 'proteína', 'proteínas',
    'enzima', 'enzimas', 'transcrição', 'tradução', 'replicação', 'nucleotídeo', 'nucleotídeos',
    'mutação', 'mutações', 'cromossomo', 'cromossomos', 'organelas', 'organelas',
    'mitocôndria', 'mitocôndrias', 'cloroplasto', 'ribossomo', 'ribossomos', 'lisossomo', 'célula', 'células',
    'parede celular', 'membrana celular', 'membrana plasmática', 'núcleo', 'núcleos',

    # Biologia de nível de organismo e funcional
    'reprodução', 'metabolismo', 'homeostase', 'adaptação', 'seleção natural',
    'herança', 'fenótipo', 'fenótipos', 'genótipo', 'genótipos',
    'fotossíntese', 'respiração', 'autótrofo', 'heterótrofo',

    # Técnicas e ferramentas experimentais
    'PCR', 'qPCR', 'RT-PCR', 'eletroforese em gel', 'clonagem', 'microscopia', 'confocal', 'fluorescência',
    'citometria de fluxo', 'espectrofotometria', 'imunocoloração', 'Western blot', 'ELISA', 'ensaio', 'ensaios',
    'sequenciamento', 'NGS', 'CRISPR', 'edição genômica', 'transfecção', 'transgênico', 'cultura', 'in vitro', 'in vivo',

    # Ecologia e biologia ambiental
    'habitat', 'habitats', 'nicho', 'simbiose', 'predador', 'presa', 'trófico', 'cadeia alimentar', 'teia alimentar',
    'ciclo do carbono', 'ciclo do nitrogênio', 'polinização', 'serviço ecossistêmico', 'serviços ecossistêmicos',
    'desmatamento', 'reflorestamento', 'bioconservação', 'ecologia', 'ecológico',

    # Sistemas e biologia aplicada
    'biologia de sistemas', 'biologia sintética', 'bioinformática', 'biotecnologia', 'biomedicina', 'biomédico',
    'biologia agrícola', 'biologia forense', 'biologia marinha', 'biologia ambiental',

    # Sistemas orgânicos (animais e humanos)
    'sistema circulatório', 'sistema digestivo', 'sistema nervoso', 'sistema endócrino',
    'sistema reprodutor', 'sistema muscular', 'sistema imunológico', 'sistema urinário', 'sistema respiratório',

    # Neurociência específica (geralmente sobrepõe-se com psicologia)
    'neurônio', 'neurônios', 'glia', 'neurotransmissor', 'neurotransmissores', 'sinapse', 'sinapses',
    'cérebro', 'córtex', 'hipocampo', 'medula espinhal', 'SNC', 'SNP',

    # Organismos modelo e formas de vida
    'bactérias', 'arqueias', 'vírus', 'vírus', 'fungos', 'fungo', 'levedura',
    'eucarioto', 'eucariotos', 'procarioto', 'procariotos', 'camundongo', 'camundongos', 'peixe-zebra', 'mosca', 'moscas', 'inseto', 'insetos',

    # Plantas e termos relacionados
    'planta', 'plantas', 'angiosperma', 'ginospérmica', 'algas', 'plâncton', 'fotossintético',
    'clorofila', 'xilema', 'floema', 'raiz', 'caule', 'folha', 'flor', 'polinização', 'sementes',

    # Animais e classificação
    'animal', 'animais', 'vertebrado', 'invertebrado', 'mamífero', 'mamíferos', 'pássaro', 'pássaros',
    'peixe', 'anfíbio', 'réptil', 'artrópode', 'inseto', 'insetos',

    # Configurações de pesquisa
    'laboratório', 'laboratório', 'câmara de Petri', 'incubadora', 'centrífuga', 'risco biológico', 'biossegurança',
    'biobanco', 'biorreator', 'organismo modelo', 'organismos modelo',

    # Biologia generale
    'biologia', 'biologico', 'biologo', 'biologi', 'bioscienza', 'bioscienze', 'scienze della vita', 'scienze della vita',

    # Organismi e tassonomia
    'organismo', 'organismi', 'specie', 'genere', 'taxon', 'taxa', 'tassonomia', 'filogenesi', 'evoluzione', 'evolutivo',
    'biodiversità', 'popolazione', 'popolazioni', 'ecosistema', 'ecosistemi', 'bioma', 'biomi',

    # Sottocampi della biologia
    'genetica', 'genetico', 'genomico', 'genomica', 'epigenetico', 'epigenetica',
    'biologia molecolare', 'molecolare', 'biologia cellulare', 'citologia',
    'neurobiologia', 'neuroscienze', 'zoologia', 'botanica', 'microbiologia', 'micologia',
    'parassitologia', 'virologia', 'immunologia', 'anatomia', 'fisiologia', 'etologia',
    'embriologia', 'biologia dello sviluppo', 'paleontologia', 'biochimica', 'bioorganica',

    # Biologia molecolare e cellulare
    'gene', 'geni', 'genoma', 'genomi', 'epigenoma', 'DNA', 'RNA', 'mRNA', 'tRNA', 'proteina', 'proteine',
    'enzima', 'enzimi', 'trascrizione', 'traduzione', 'replicazione', 'nucleotide', 'nucleotidi',
    'mutazione', 'mutazioni', 'cromosoma', 'cromosomi', 'organelo', 'organelli',
    'mitocondrio', 'mitocondri', 'cloroplasto', 'ribosoma', 'ribosomi', 'lisosoma', 'cellula', 'cellule',
    'parete cellulare', 'membrana cellulare', 'membrana plasmatica', 'nucleo', 'nuclei',

    # Biologia a livello di organismo e funzionale
    'riproduzione', 'metabolismo', 'omeostasi', 'adattamento', 'selezione naturale',
    'ereditarietà', 'fenotipo', 'fenotipi', 'genotipo', 'genotipi',
    'fotosintesi', 'respirazione', 'autotrofo', 'eterotrofo',

    # Tecniche e strumenti sperimentali
    'PCR', 'qPCR', 'RT-PCR', 'elettroforesi su gel', 'clonazione', 'microscopia', 'confocale', 'fluorescenza',
    'citometria a flusso', 'spettrofotometria', 'immunocolorazione', 'Western blot', 'ELISA', 'saggio', 'saggi',
    'sequenziamento', 'NGS', 'CRISPR', 'editing del genoma', 'trasfezione', 'transgenico', 'coltura', 'in vitro', 'in vivo',

    # Ecologia e biologia ambientale
    'habitat', 'habitat', 'nicchia', 'simbiosi', 'predatore', 'prede', 'trofico', 'catena alimentare', 'rete alimentare',
    'ciclo del carbonio', 'ciclo dell\'azoto', 'impollinazione', 'servizio ecosistemico', 'servizi ecosistemici',
    'deforestazione', 'riforestazione', 'bioconservazione', 'ecologia', 'ecologico',

    # Sistemi e biologia applicata
    'biologia dei sistemi', 'biologia sintetica', 'bioinformatica', 'biotecnologia', 'biomedicina', 'biomedicale',
    'biologia agricola', 'biologia forense', 'biologia marina', 'biologia ambientale',

    # Sistemi degli organi (animali e umani)
    'sistema circolatorio', 'sistema digestivo', 'sistema nervoso', 'sistema endocrino',
    'sistema riproduttivo', 'sistema muscolare', 'sistema immunitario', 'sistema urinario', 'sistema respiratorio',

    # Neuroscienze specifiche (spesso sovrapposte alla psicologia)
    'neuron', 'neuroni', 'glia', 'neurotrasmettitore', 'neurotrasmettitori', 'sinapsi', 'sinapsi',
    'cervello', 'corteccia', 'ippocampo', 'midollo spinale', 'SNC', 'SNP',

    # Organismi modello e forme di vita
    'batteri', 'archea', 'virus', 'virus', 'funghi', 'fungo', 'lievito',
    'eucariote', 'eucarioti', 'procariote', 'procarioti', 'topo', 'topi', 'zebrafish', 'mosca', 'mosche', 'insetto', 'insetti',

    # Piante e termini correlati
    'pianta', 'piante', 'angiosperma', 'gimnosperma', 'alghe', 'fitoplancton', 'fotosintetico',
    'clorofilla', 'xilema', 'floema', 'radice', 'fusto', 'foglia', 'fiore', 'impollinazione', 'semi',

    # Animali e classificazione
    'animale', 'animali', 'vertebrato', 'invertebrato', 'mammifero', 'mammiferi', 'uccello', 'uccelli',
    'pesce', 'anfibio', 'rettile', 'artropode', 'insetto', 'insetti',

    # Impostazioni di ricerca
    'laboratorio', 'laboratorio', 'piatto di petri', 'incubatrice', 'centrifuga', 'rischio biologico', 'biosicurezza',
    'biobanca', 'bioreattore', 'organismo modello', 'organismi modello',

    # Biologie générale
    'biologie', 'biologique', 'biologiste', 'biologistes', 'biosciences', 'sciences biologiques', 'sciences de la vie', 'sciences de la vie',

    # Organismes et taxonomie
    'organisme', 'organismes', 'espèces', 'genre', 'taxon', 'taxons', 'taxonomie', 'phylogénie', 'évolution', 'évolutif',
    'biodiversité', 'population', 'populations', 'écosystème', 'écosystèmes', 'biome', 'biomes',

    # Sous-domaines de la biologie
    'génétique', 'génétique', 'génomique', 'génomique', 'épigénétique', 'épigénétique',
    'biologie moléculaire', 'moléculaire', 'biologie cellulaire', 'cytologie',
    'neurobiologie', 'neurosciences', 'zoologie', 'botanique', 'microbiologie', 'mycologie',
    'parasitologie', 'virologie', 'immunologie', 'anatomie', 'physiologie', 'éthologie',
    'embryologie', 'biologie du développement', 'paléontologie', 'biochimie', 'bioorganique',

    # Biologie moléculaire et cellulaire
    'gène', 'gènes', 'génome', 'génomes', 'épigénome', 'ADN', 'ARN', 'ARNm', 'ARNt', 'protéine', 'protéines',
    'enzyme', 'enzymes', 'transcription', 'traduction', 'réplication', 'nucléotide', 'nucléotides',
    'mutation', 'mutations', 'chromosome', 'chromosomes', 'organite', 'organites',
    'mitochondrie', 'mitochondries', 'chloroplaste', 'ribosome', 'ribosomes', 'lysosome', 'cellule', 'cellules',
    'paroi cellulaire', 'membrane cellulaire', 'membrane plasmique', 'noyau', 'noyaux',

    # Biologie au niveau des organismes et fonctionnelle
    'reproduction', 'métabolisme', 'homéostasie', 'adaptation', 'sélection naturelle',
    'héritage', 'phénotype', 'phénotypes', 'génotype', 'génotypes',
    'photosynthèse', 'respiration', 'autotrophe', 'hétérotrophe',

    # Techniques et outils expérimentaux
    'PCR', 'qPCR', 'RT-PCR', 'électrophorèse sur gel', 'clonage', 'microscopie', 'confocal', 'fluorescence',
    'cytométrie en flux', 'spectrophotométrie', 'immunomarquage', 'western blot', 'ELISA', 'essai', 'essais',
    'séquençage', 'NGS', 'CRISPR', 'édition de génome', 'transfection', 'transgénique', 'culture', 'in vitro', 'in vivo',

    # Écologie et biologie environnementale
    'habitat', 'habitats', 'niche', 'symbiose', 'prédateur', 'proie', 'trophique', 'chaîne alimentaire', 'réseau alimentaire',
    'cycle du carbone', 'cycle de l\'azote', 'pollinisation', 'service écosystémique', 'services écosystémiques',
    'déforestation', 'reforestation', 'bioconservation', 'écologie', 'écologique',

    # Biologie des systèmes et appliquée
    'biologie des systèmes', 'biologie synthétique', 'bioinformatique', 'biotechnologie', 'biomédecine', 'biomédical',
    'biologie agricole', 'biologie médico-légale', 'biologie marine', 'biologie environnementale',

    # Systèmes organiques (animaux et humains)
    'système circulatoire', 'système digestif', 'système nerveux', 'système endocrinien',
    'système reproducteur', 'système musculaire', 'système immunitaire', 'système urinaire', 'système respiratoire',

    # Neurosciences spécifiques (souvent se recoupent avec la psychologie)
    'neurone', 'neurones', 'glie', 'neurotransmetteur', 'neurotransmetteurs', 'synapse', 'synapses',
    'cerveau', 'cortex', 'hippocampe', 'moelle épinière', 'SNC', 'SNP',

    # Organismes modèles et formes de vie
    'bactéries', 'archées', 'virus', 'virus', 'champignons', 'champignon', 'levure',
    'eucaryote', 'eucaryotes', 'procaryote', 'procaryotes', 'souris', 'souris', 'poisson zèbre', 'mouche', 'mouches', 'insecte', 'insectes',

    # Plantes et termes associés
    'plante', 'plantes', 'angiosperme', 'gymnosperme', 'algues', 'plancton', 'photosynthétique',
    'chlorophylle', 'xylème', 'phloème', 'racine', 'tige', 'feuille', 'fleur', 'pollinisation', 'graines',

    # Animaux et classification
    'animal', 'animaux', 'vertébré', 'invertébré', 'mammifère', 'mammifères', 'oiseau', 'oiseaux',
    'poisson', 'amphibien', 'reptile', 'arthropode', 'insecte', 'insectes',

    # Paramètres de recherche
    'laboratoire', 'laboratoire', 'boîte de pétri', 'incubateur', 'centrifugeuse', 'risque biologique', 'sécurité biologique',
    'biobanque', 'bioréacteur', 'organisme modèle', 'organismes modèles',

    # 基本生物学术语
    '生物学', '生物的', '生物学家', '生物学家们', '生物科学', '生物科学们', '生命科学', '生命科学们',

    # 有机体与分类学
    '有机体', '有机体们', '物种', '属', '分类群', '分类群们', '分类学', '系统发育', '进化', '进化的',
    '生物多样性', '种群', '种群们', '生态系统', '生态系统们', '生物群落', '生物群落们',

    # 生物学的子领域
    '遗传学', '遗传的', '基因组学', '基因组学们', '表观遗传学', '表观遗传学们',
    '分子生物学', '分子', '细胞生物学', '细胞学',
    '神经生物学', '神经科学', '动物学', '植物学', '微生物学', '真菌学',
    '寄生虫学', '病毒学', '免疫学', '解剖学', '生理学', '行为学',
    '胚胎学', '发育生物学', '古生物学', '生物化学', '生物有机化学',

    # 分子与细胞生物学
    '基因', '基因们', '基因组', '基因组们', '表观基因组', 'DNA', 'RNA', 'mRNA', 'tRNA', '蛋白质', '蛋白质们',
    '酶', '酶们', '转录', '翻译', '复制', '核苷酸', '核苷酸们',
    '突变', '突变们', '染色体', '染色体们', '细胞器', '细胞器们',
    '线粒体', '线粒体们', '叶绿体', '核糖体', '核糖体们', '溶酶体', '细胞', '细胞们',
    '细胞壁', '细胞膜', '质膜', '细胞核', '细胞核们',

    # 有机体层次与功能生物学
    '生殖', '代谢', '稳态', '适应', '自然选择',
    '遗传', '表型', '表型们', '基因型', '基因型们',
    '光合作用', '呼吸作用', '自养', '异养',

    # 技术与实验工具
    'PCR', 'qPCR', 'RT-PCR', '凝胶电泳', '克隆', '显微镜学', '共聚焦', '荧光',
    '流式细胞术', '分光光度法', '免疫染色', 'Western印迹', 'ELISA', '测定', '测定们',
    '测序', 'NGS', 'CRISPR', '基因组编辑', '转染', '转基因', '培养', '体外', '体内',

    # 生态学与环境生物学
    '栖息地', '栖息地们', '生态位', '共生', '捕食者', '猎物', '营养', '食物链', '食物网',
    '碳循环', '氮循环', '授粉', '生态系统服务', '生态系统服务们',
    '森林砍伐', '再造林', '生物保护', '生态学', '生态的',

    # 系统与应用生物学
    '系统生物学', '合成生物学', '生物信息学', '生物技术', '生物医学', '生物医学的',
    '农业生物学', '法医生物学', '海洋生物学', '环境生物学',

    # 有机系统（动物与人类）
    '循环系统', '消化系统', '神经系统', '内分泌系统',
    '生殖系统', '肌肉系统', '免疫系统', '泌尿系统', '呼吸系统',

    # 神经科学专门术语（通常与心理学交集）
    '神经元', '神经元们', '胶质细胞', '神经递质', '神经递质们', '突触', '突触们',
    '大脑', '皮层', '海马体', '脊髓', '中枢神经系统', '外周神经系统',

    # 模型生物与生命形式
    '细菌', '古菌', '病毒', '病毒们', '真菌', '真菌类', '酵母',
    '真核生物', '真核生物们', '原核生物', '原核生物们', '小鼠', '小鼠们', '斑马鱼', '果蝇', '果蝇们', '昆虫', '昆虫们',

    # 植物与相关术语
    '植物', '植物们', '被子植物', '裸子植物', '藻类', '浮游生物', '光合的',
    '叶绿素', '木质部', '韧皮部', '根', '茎', '叶', '花', '授粉', '种子',

    # 动物与分类
    '动物', '动物们', '脊椎动物', '无脊椎动物', '哺乳动物', '哺乳动物们', '鸟类', '鸟类们',
    '鱼', '两栖动物', '爬行动物', '节肢动物', '昆虫', '昆虫们',

    # 研究设置
    '实验室', '实验室', '培养皿', '培养箱', '离心机', '生物危害', '生物安全',
    '生物样本库', '生物反应器', '模型生物', '模型生物们',

    # Общая биология
    'биология', 'биологический', 'биолог', 'биологи', 'бионаука', 'бионауки', 'наука о жизни', 'науки о жизни',

    # Организмы и таксономия
    'организм', 'организмы', 'вид', 'род', 'таксон', 'таксоны', 'таксономия', 'филогения', 'эволюция', 'эволюционный',
    'биоразнообразие', 'популяция', 'популяции', 'экосистема', 'экосистемы', 'биом', 'биомы',

    # Подобласти биологии
    'генетика', 'генетический', 'геномный', 'геномика', 'эпигенетика', 'эпигенетика',
    'молекулярная биология', 'молекулярный', 'клеточная биология', 'цитология',
    'нейробиология', 'нейронаука', 'зоология', 'ботаника', 'микробиология', 'микология',
    'паразитология', 'вирусология', 'иммунология', 'анатомия', 'физиология', 'этология',
    'эмбриология', 'развивающаяся биология', 'палеонтология', 'биохимия', 'биоорганическая',

    # Молекулярная и клеточная биология
    'ген', 'гены', 'геном', 'геномы', 'эпигеном', 'ДНК', 'РНК', 'мРНК', 'тРНК', 'белок', 'белки',
    'фермент', 'ферменты', 'транскрипция', 'трансляция', 'репликация', 'нуклеотид', 'нуклеотиды',
    'мутация', 'мутации', 'хромосома', 'хромосомы', 'органелла', 'органеллы',
    'митохондрия', 'митохондрии', 'хлоропласт', 'рибосома', 'рибосомы', 'лизосома', 'клетка', 'клетки',
    'клеточная стенка', 'клеточная мембрана', 'плазматическая мембрана', 'ядро', 'ядра',

    # Биология на уровне организма и функциональная биология
    'репродукция', 'метаболизм', 'гомеостаз', 'адаптация', 'естественный отбор',
    'наследование', 'фенотип', 'фенотипы', 'генотип', 'генотипы',
    'фотосинтез', 'дыхание', 'автотроф', 'гетеротроф',

    # Техники и экспериментальные инструменты
    'ПЦР', 'qПЦР', 'РТ-ПЦР', 'гель-электрофорез', 'клонирование', 'микроскопия', 'конфокальная', 'флуоресценция',
    'проточная цитометрия', 'спектрофотометрия', 'иммуностанирование', 'вестерн-блот', 'ЭЛИЗА', 'анализ', 'анализы',
    'секвенирование', 'НГС', 'КРИСПР', 'редактирование генома', 'трансфекция', 'трансгенный', 'культура', 'в пробирке', 'в живом организме',

    # Экология и экологическая биология
    'местообитание', 'места обитания', 'ниша', 'симбиоз', 'хищник', 'добыча', 'трофический', 'пищевой цикл', 'пищевая цепь',
    'углеродный цикл', 'азотный цикл', 'опыление', 'экосистемная услуга', 'экосистемные услуги',
    'вырубка лесов', 'восстановление лесов', 'биоконсервация', 'экология', 'экологический',

    # Системная и прикладная биология
    'системная биология', 'синтетическая биология', 'биоинформатика', 'биотехнология', 'биомедицина', 'биомедицинский',
    'сельскохозяйственная биология', 'судебная биология', 'морская биология', 'экологическая биология',

    # Организменные системы (животные и люди)
    'кровеносная система', 'пищеварительная система', 'нервная система', 'эндокринная система',
    'репродуктивная система', 'мышечная система', 'иммунная система', 'мочевыделительная система', 'дыхательная система',

    # Нейронаука (часто пересекается с психологией)
    'нейрон', 'нейроны', 'глия', 'нейротрансмиттер', 'нейротрансмиттеры', 'синапс', 'синапсы',
    'мозг', 'кора', 'гиппокамп', 'спинной мозг', 'ЦНС', 'ПНС',

    # Модельные организмы и формы жизни
    'бактерии', 'археи', 'вирус', 'вирусы', 'грибы', 'гриб', 'дрожжи',
    'эукариот', 'эукариоты', 'прокариот', 'прокариоты', 'мышь', 'мыши', 'зебрарыба', 'муха', 'мухи', 'насекомое', 'насекомые',

    # Растения и связанные термины
    'растение', 'растения', 'покрытосеменные', 'голосеменные', 'водоросли', 'планктон', 'фотосинтетический',
    'хлорофилл', 'ксилема', 'флоэма', 'корень', 'стебель', 'листва', 'цветок', 'опыление', 'семена',

    # Животные и классификация
    'животное', 'животные', 'позвоночное', 'беспозвоночное', 'млекопитающее', 'млекопитающие', 'птица', 'птицы',
    'рыба', 'амфибия', 'рептилия', 'членистоногое', 'насекомое', 'насекомые',

    # Исследовательские условия
    'лаборатория', 'лаборатория', 'петри', 'инкубатор', 'центрифуга', 'биологическая опасность', 'биобезопасность',
    'биобанк', 'биореактор', 'модельный организм', 'модельные организмы',

    # 一般的な生物学
    '生物学', '生物学的', '生物学者', '生物学者たち', 'バイオサイエンス', 'バイオサイエンス', 'ライフサイエンス', 'ライフサイエンス',

    # 生物と分類学
    '生物', '生物たち', '種', '属', '分類群', '分類群たち', '分類学', '系統発生', '進化', '進化的',
    '生物多様性', '集団', '集団たち', '生態系', '生態系たち', 'バイオーム', 'バイオームたち',

    # 生物学のサブフィールド
    '遺伝学', '遺伝的', 'ゲノム学', 'ゲノム学', 'エピジェネティクス', 'エピジェネティクス',
    '分子生物学', '分子', '細胞生物学', '細胞学',
    '神経生物学', '神経科学', '動物学', '植物学', '微生物学', '菌学',
    '寄生虫学', 'ウイルス学', '免疫学', '解剖学', '生理学', '行動学',
    '胚胎学', '発生生物学', '古生物学', '生化学', '生体有機化学',

    # 分子および細胞生物学
    '遺伝子', '遺伝子たち', 'ゲノム', 'ゲノムたち', 'エピゲノム', 'DNA', 'RNA', 'mRNA', 'tRNA', 'タンパク質', 'タンパク質たち',
    '酵素', '酵素たち', '転写', '翻訳', '複製', 'ヌクレオチド', 'ヌクレオチドたち',
    '突然変異', '突然変異たち', '染色体', '染色体たち', '小器官', '小器官たち',
    'ミトコンドリア', 'ミトコンドリアたち', '葉緑体', 'リボソーム', 'リボソームたち', 'リソソーム', '細胞', '細胞たち',
    '細胞壁', '細胞膜', 'プラズマ膜', '核', '核たち',

    # 生物学的な機能と生物組織
    '繁殖', '代謝', '恒常性', '適応', '自然選択',
    '遺伝', '表現型', '表現型たち', '遺伝子型', '遺伝子型たち',
    '光合成', '呼吸', '従属栄養', '自家栄養',

    # 技術と実験ツール
    'PCR', 'qPCR', 'RT-PCR', 'ゲル電気泳動', 'クローン作成', '顕微鏡', '共焦点', '蛍光',
    'フローサイトメトリー', '分光光度計', '免疫染色', 'ウェスタンブロット', 'ELISA', 'アッセイ', 'アッセイたち',
    'シーケンシング', 'NGS', 'CRISPR', 'ゲノム編集', '転写', '転写遺伝子', '培養', 'インビトロ', 'インビボ',

    # 生態学と環境生物学
    '生息地', '生息地たち', 'ニッチ', '共生', '捕食者', '獲物', '栄養学的', '食物連鎖', '食物網',
    '炭素循環', '窒素循環', '受粉', '生態系サービス', '生態系サービスたち',
    '森林伐採', '再植林', '生物多様性保護', '生態学', '生態学的',

    # システムおよび応用生物学
    'システム生物学', '合成生物学', 'バイオインフォマティクス', 'バイオテクノロジー', 'バイオメディスン', 'バイオメディカル',
    '農業生物学', '法医学生物学', '海洋生物学', '環境生物学',

    # 動物と人間の組織系
    '循環系', '消化系', '神経系', '内分泌系',
    '生殖系', '筋肉系', '免疫系', '泌尿系', '呼吸系',

    # 神経科学（心理学と重なることが多い）
    '神経細胞', '神経細胞たち', 'グリア細胞', '神経伝達物質', '神経伝達物質たち', 'シナプス', 'シナプスたち',
    '脳', '皮質', '海馬', '脊髄', '中枢神経系', '末梢神経系',

    # モデル生物と生命形態
    '細菌', '古細菌', 'ウイルス', 'ウイルスたち', '真菌', '真菌たち', '酵母',
    '真核生物', '真核生物たち', '原核生物', '原核生物たち', 'マウス', 'マウスたち', 'ゼブラフィッシュ', 'ハエ', 'ハエたち', '昆虫', '昆虫たち',

    # 植物と関連用語
    '植物', '植物たち', '被子植物', '裸子植物', '藻類', 'プランクトン', '光合成',
    'クロロフィル', '木部', '師部', '根', '茎', '葉', '花', '受粉', '種',

    # 動物と分類
    '動物', '動物たち', '脊椎動物', '無脊椎動物', '哺乳類', '哺乳類たち', '鳥', '鳥たち',
    '魚', '両生類', '爬虫類', '節足動物', '昆虫', '昆虫たち',

    # 研究設定
    'ラボ', '実験室', 'ペトリ皿', 'インキュベーター', '遠心機', '生物危険', 'バイオセーフティ',
    'バイオバンク', 'バイオリアクター', 'モデル生物', 'モデル生物たち',

    # 일반 생물학
    '생물학', '생물학적', '생물학자', '생물학자들', '생명과학', '생명과학들', '생명과학', '생명과학들',

    # 유기체 및 분류학
    '유기체', '유기체들', '종', '속', '분류군', '분류군들', '분류학', '계통발생', '진화', '진화적',
    '생물 다양성', '인구', '인구들', '생태계', '생태계들', '생물군계', '생물군계들',

    # 생물학의 하위 분야
    '유전학', '유전적', '유전체학', '유전체학들', '후생유전학', '후생유전학들',
    '분자 생물학', '분자', '세포 생물학', '세포학',
    '신경 생물학', '신경과학', '동물학', '식물학', '미생물학', '곰팡이학',
    '기생충학', '바이러스학', '면역학', '해부학', '생리학', '행동학',
    '배아학', '발달 생물학', '고생물학', '생화학', '생유기화학',

    # 분자 및 세포 생물학
    '유전자', '유전자들', '유전체', '유전체들', '후생유전체', 'DNA', 'RNA', 'mRNA', 'tRNA', '단백질', '단백질들',
    '효소', '효소들', '전사', '번역', '복제', '뉴클레오타이드', '뉴클레오타이드들',
    '돌연변이', '돌연변이들', '염색체', '염색체들', '소기관', '소기관들',
    '미토콘드리아', '미토콘드리아들', '엽록체', '리보솜', '리보솜들', '라이소좀', '세포', '세포들',
    '세포벽', '세포막', '원형질막', '핵', '핵들',

    # 유기체 수준 및 기능 생물학
    '번식', '대사', '항상성', '적응', '자연 선택',
    '유전', '표현형', '표현형들', '유전자형', '유전자형들',
    '광합성', '호흡', '자생식물', '이식물',

    # 기술 및 실험 도구
    'PCR', 'qPCR', 'RT-PCR', '겔 전기영동', '클로닝', '현미경', '공초점', '형광',
    '유세포 분석', '분광광도계', '면역염색', '웨스턴 블롯', 'ELISA', '측정', '측정들',
    '염기서열 분석', 'NGS', 'CRISPR', '유전체 편집', '형질전환', '형질전환된', '배양', '시험관 내', '생체 내',

    # 생태학 및 환경 생물학
    '서식지', '서식지들', '니치', '공생', '포식자', '먹이', '영양', '먹이사슬', '먹이망',
    '탄소 순환', '질소 순환', '수정', '생태계 서비스', '생태계 서비스들',
    '삼림 파괴', '재조림', '생물보존', '생태학', '생태적',

    # 시스템 및 응용 생물학
    '시스템 생물학', '합성 생물학', '생물정보학', '생명공학', '생의학', '생의학적',
    '농업 생물학', '법의학 생물학', '해양 생물학', '환경 생물학',

    # 유기체 시스템 (동물 및 인간)
    '순환계', '소화계', '신경계', '내분비계',
    '생식계', '근육계', '면역계', '비뇨계', '호흡계',

    # 신경과학 특화 (심리학과 종종 겹침)
    '뉴런', '뉴런들', '교세포', '신경전달물질', '신경전달물질들', '시냅스', '시냅스들',
    '뇌', '대뇌피질', '해마', '척수', '중추신경계', '말초신경계',

    # 모델 유기체 및 생명체
    '박테리아', '고세균', '바이러스', '바이러스들', '곰팡이', '효모',
    '진핵생물', '진핵생물들', '원핵생물', '원핵생물들', '쥐', '쥐들', '제브라피시', '파리', '파리들', '곤충', '곤충들',

    # 식물 및 관련 용어
    '식물', '식물들', '속씨식물', '겉씨식물', '조류', '플랑크톤', '광합성',
    '엽록소', '물관', '체관', '뿌리', '줄기', '잎', '꽃', '수정', '씨앗',

    # 동물 및 분류
    '동물', '동물들', '척추동물', '무척추동물', '포유류', '포유류들', '새', '새들',
    '물고기', '양서류', '파충류', '절지동물', '곤충', '곤충들',

    # 연구 환경
    '실험실', '실험실', '페트리 접시', '배양기', '원심분리기', '생물학적 위험', '생물안전',
    '생물은행', '생물반응기', '모델 유기체', '모델 유기체들',

    # 一般生物學
    '生物學', '生物學的', '生物學家', '生物學家們', '生命科學', '生命科學們', '生命科學', '生命科學們',

    # 有機體與分類學
    '有機體', '有機體們', '物種', '屬', '分類單元', '分類單元們', '分類學', '系統發生', '演化', '演化的',
    '生物多樣性', '人口', '人口們', '生態系統', '生態系統們', '生物群落', '生物群落們',

    # 生物學的子領域
    '遺傳學', '遺傳的', '基因組學', '基因組學們', '後天遺傳學', '後天遺傳學們',
    '分子生物學', '分子', '細胞生物學', '細胞學',
    '神經生物學', '神經科學', '動物學', '植物學', '微生物學', '真菌學',
    '寄生蟲學', '病毒學', '免疫學', '解剖學', '生理學', '行為學',
    '胚胎學', '發育生物學', '古生物學', '生物化學', '生物有機化學',

    # 分子與細胞生物學
    '基因', '基因們', '基因組', '基因組們', '後天基因組', 'DNA', 'RNA', 'mRNA', 'tRNA', '蛋白質', '蛋白質們',
    '酶', '酶們', '轉錄', '翻譯', '複製', '核苷酸', '核苷酸們',
    '突變', '突變們', '染色體', '染色體們', '細胞器', '細胞器們',
    '線粒體', '線粒體們', '葉綠體', '核糖體', '核糖體們', '溶酶體', '細胞', '細胞們',
    '細胞壁', '細胞膜', '質膜', '細胞核', '細胞核們',

    # 有機體層級與功能生物學
    '繁殖', '新陳代謝', '恆定', '適應', '自然選擇',
    '遺傳', '表型', '表型們', '基因型', '基因型們',
    '光合作用', '呼吸作用', '自養', '異養',

    # 技術與實驗工具
    'PCR', 'qPCR', 'RT-PCR', '凝膠電泳', '克隆', '顯微鏡', '共聚焦', '螢光',
    '流式細胞儀', '分光光度計', '免疫染色', '西方印跡法', 'ELISA', '測定', '測定們',
    '基因排序', 'NGS', 'CRISPR', '基因組編輯', '轉染', '轉基因', '培養', '體外', '體內',

    # 生態學與環境生物學
    '棲息地', '棲息地們', '生態位', '共生', '掠食者', '獵物', '營養', '食物鏈', '食物網',
    '碳循環', '氮循環', '授粉', '生態系統服務', '生態系統服務們',
    '森林砍伐', '再造林', '生物保護', '生態學', '生態的',

    # 系統與應用生物學
    '系統生物學', '合成生物學', '生物信息學', '生物技術', '生物醫學', '生物醫學的',
    '農業生物學', '法醫生物學', '海洋生物學', '環境生物學',

    # 有機體系統（動物與人類）
    '循環系統', '消化系統', '神經系統', '內分泌系統',
    '生殖系統', '肌肉系統', '免疫系統', '泌尿系統', '呼吸系統',

    # 神經科學特定（通常與心理學重疊）
    '神經元', '神經元們', '膠質細胞', '神經傳遞物質', '神經傳遞物質們', '突觸', '突觸們',
    '大腦', '大腦皮層', '海馬體', '脊髓', '中樞神經系統', '外周神經系統',

    # 模型有機體與生命形式
    '細菌', '古細菌', '病毒', '病毒們', '真菌', '酵母',
    '真核生物', '真核生物們', '原核生物', '原核生物們', '老鼠', '老鼠們', '斑馬魚', '蒼蠅', '蒼蠅們', '昆蟲', '昆蟲們',

    # 植物與相關術語
    '植物', '植物們', '被子植物', '裸子植物', '藻類', '浮游生物', '光合作用',
    '葉綠素', '木質部', '韌皮部', '根', '幹', '葉', '花', '授粉', '種子',

    # 動物與分類
    '動物', '動物們', '脊椎動物', '無脊椎動物', '哺乳類', '哺乳類們', '鳥', '鳥類',
    '魚', '兩棲動物', '爬行動物', '節肢動物', '昆蟲', '昆蟲們',

    # 研究設置
    '實驗室', '實驗室', '培養皿', '培養箱', '離心機', '生物危險', '生物安全',
    '生物銀行', '生物反應器', '模型有機體', '模型有機體們',

    # Obecná biologie
    'biologie', 'biologický', 'biolog', 'biologové', 'biovědy', 'biovědy (pl)', 'vědy o životě', 'vědy o živé přírodě',

    # Organizmy a taxonomie
    'organismus', 'organizmy', 'druh', 'rod', 'taxon', 'taxony', 'taxonomie', 'fylogeneze', 'evoluce', 'evoluční',
    'biodiverzita', 'populace', 'populace (pl)', 'ekosystém', 'ekosystémy', 'biom', 'biomy',

    # Podpole biologie
    'genetika', 'genetický', 'genomický', 'genomika', 'epigenetika', 'epigenetické',
    'molekulární biologie', 'molekulární', 'buněčná biologie', 'cytologie',
    'neurobiologie', 'neurovědy', 'zoologie', 'botanika', 'mikrobiologie', 'mykologie',
    'parazitologie', 'virologie', 'imunologie', 'anatomie', 'fyziologie', 'etologie',
    'embryologie', 'vývojová biologie', 'paleontologie', 'biochemie', 'bioorganická',

    # Molekulární a buněčná biologie
    'gen', 'geny', 'genom', 'genomy', 'epigenom', 'DNA', 'RNA', 'mRNA', 'tRNA', 'protein', 'proteiny',
    'enzym', 'enzymy', 'transkripce', 'translace', 'replikace', 'nukleotid', 'nukleotidy',
    'mutace', 'mutace (pl)', 'chromozom', 'chromozomy', 'organela', 'organely',
    'mitochondrie', 'mitochondrie (pl)', 'chloroplast', 'ribozom', 'ribozomy', 'lysozom', 'buňka', 'buňky',
    'buněčná stěna', 'buněčná membrána', 'plazmatická membrána', 'jádro', 'jádra',

    # Biologie na úrovni organizmů a funkční biologie
    'reprodukce', 'metabolismus', 'homeostáza', 'adaptace', 'přirozený výběr',
    'dědičnost', 'fenotyp', 'fenotypy', 'genotyp', 'genotypy',
    'fotosyntéza', 'dýchání', 'autotrof', 'heterotrof',

    # Techniky a experimentální nástroje
    'PCR', 'qPCR', 'RT-PCR', 'gelová elektroforéza', 'klonování', 'mikroskopie', 'konfokální', 'fluorescence',
    'průtoková cytometrie', 'spektrální fotometrie', 'imunostaining', 'western blot', 'ELISA', 'test', 'testy',
    'sekvenování', 'NGS', 'CRISPR', 'úprava genomu', 'transfekce', 'transgenní', 'kultura', 'in vitro', 'in vivo',

    # Ekologie a environmentální biologie
    'habitat', 'habitáty', 'ekologická nika', 'symbióza', 'predátor', 'kořist', 'tropický', 'potravní řetězec', 'potravní síť',
    'uhlíkový cyklus', 'dusíkový cyklus', 'oplodnění', 'ekosystémová služba', 'služby ekosystému',
    'odlesňování', 'zalesňování', 'biokonservace', 'ekologie', 'ekologický',

    # Systémy a aplikovaná biologie
    'systémová biologie', 'syntetická biologie', 'bioinformatika', 'biotechnologie', 'biomedicína', 'biomedicínský',
    'zemědělská biologie', 'forenzní biologie', 'mořská biologie', 'environmentální biologie',

    # Orgánové systémy (zvířata a lidé)
    'kardiovaskulární systém', 'trávicí systém', 'nervový systém', 'endokrinní systém',
    'reprodukční systém', 'svalový systém', 'imunitní systém', 'močový systém', 'dýchací systém',

    # Specifické neurovědy (často se překrývají s psychologií)
    'neurony', 'neurony', 'glie', 'neurotransmiter', 'neurotransmitery', 'synapse', 'synapse',
    'mozek', 'kůra', 'hipokampus', 'mícha', 'CNS', 'PNS',

    # Modeloví organizmy a životní formy
    'bakterie', 'archae', 'virus', 'viry', 'houbové', 'houbová', 'kvasinky',
    'eukaryot', 'eukaryoti', 'prokaryot', 'prokaryoti', 'myš', 'myši', 'zebrafish', 'moucha', 'mouchy', 'hmyz', 'hmyz',

    # Rostliny a související termíny
    'rostlina', 'rostliny', 'angiosperm', 'gymnosperm', 'řasy', 'plankton', 'fotosyntetický',
    'chlorofyl', 'xylém', 'floém', 'kořen', 'stonek', 'list', 'květ', 'oplodnění', 'semínka',

    # Zvířata a klasifikace
    'zvíře', 'zvířata', 'obratlovec', 'neobratlovec', 'savci', 'savci (pl)', 'pták', 'ptáci',
    'ryby', 'obojživelník', 'plaz', 'artropod', 'hmyz', 'hmyzy',

    # Výzkumná prostředí
    'laboratoř', 'laboratoř', 'Petriho miska', 'inkubátor', 'centrifuga', 'biohazard', 'biosafety',
    'biobanka', 'bioreaktor', 'modelový organismus', 'modeloví organizmy',

    # علم الأحياء العام
    'علم الأحياء', 'بيولوجي', 'عالم أحياء', 'علماء الأحياء', 'علوم الحياة', 'العلوم البيولوجية', 'علوم الحياة', 'العلوم الحية',

    # الكائنات والتصنيف
    'كائن', 'كائنات', 'نوع', 'جنس', 'تصنيف', 'تصنيفات', 'تصنيف الكائنات', 'النسالة', 'تطور', 'تطوري',
    'التنوع البيولوجي', 'سكان', 'السكان', 'نظام بيئي', 'أنظمة بيئية', 'حزام حيوي', 'أحزمة حيوية',

    # المجالات الفرعية لعلم الأحياء
    'وراثيات', 'وراثي', 'جينومي', 'جينوميات', 'علم الوراثة التفاعلي', 'الوراثة التفاعلية',
    'علم الأحياء الجزيئي', 'جزيئي', 'علم الأحياء الخلوي', 'علم الخلايا',
    'علم الأعصاب', 'علوم الأعصاب', 'علم الحيوان', 'علم النباتات', 'علم الميكروبات', 'علم الفطريات',
    'علم الطفيليات', 'علم الفيروسات', 'علم المناعة', 'تشريح', 'فسيولوجيا', 'علم السلوك',
    'علم الأجنة', 'علم الأحياء التطوري', 'علم الحفريات', 'الكيمياء الحيوية', 'الكيمياء العضوية الحيوية',

    # علم الأحياء الجزيئي والخلايا
    'جين', 'جينات', 'جينيوم', 'جينيومات', 'إبيجينيوم', 'DNA', 'RNA', 'mRNA', 'tRNA', 'بروتين', 'بروتينات',
    'إنزيم', 'إنزيمات', 'نسخ', 'ترجمة', 'تكرار', 'نوكليوتيد', 'نوكليوتيدات',
    'طفرات', 'طفرة', 'كروموسوم', 'كروموسومات', 'عضية', 'عضيات',
    'ميتوكندريا', 'ميتوكندريات', 'كلوروبلاست', 'ريبوسوم', 'ريبوسومات', 'لايسوسوم', 'خلية', 'خلايا',
    'جدار الخلية', 'غشاء الخلية', 'غشاء البلازما', 'نواة', 'أنوية',

    # علم الأحياء على مستوى الكائنات والوظائف
    'تكاثر', 'أيض', 'توازن داخلي', 'تكيّف', 'انتقاء طبيعي',
    'وراثة', 'الصفات الظاهرة', 'الصفات الظاهرة (جمع)', 'الوراثة الجينية', 'الوراثات الجينية',
    'التمثيل الضوئي', 'تنفس', 'مغذي ذاتي', 'مغذي غير ذاتي',

    # تقنيات وأدوات تجريبية
    'PCR', 'qPCR', 'RT-PCR', 'رحلان الهلام الكهربائي', 'استنساخ', 'مجهرية', 'مجهري ضوئي', 'فلورية',
    'تحليل تدفق الخلايا', 'طيفية', 'تلطيخ مناعي', 'وسترن بلوت', 'ELISA', 'اختبار', 'اختبارات',
    'تسلسل', 'NGS', 'CRISPR', 'تحرير الجينات', 'انتقال', 'انتقال جيني', 'زراعة', 'في المختبر', 'في الجسم الحي',

    # علم البيئة وعلم الأحياء البيئية
    'موطن', 'مواطن', 'فئة بيئية', 'تعاون تكافلي', 'مفترس', 'فريسة', 'سلسلة غذائية', 'شبكة غذائية',
    'دورة الكربون', 'دورة النيتروجين', 'تلقيح', 'خدمة النظام البيئي', 'خدمات النظام البيئي',
    'إزالة الغابات', 'إعادة تشجير', 'حفظ التنوع البيولوجي', 'علم البيئة', 'بيئي',

    # نظم وعلم الأحياء التطبيقي
    'علم الأحياء النظامي', 'علم الأحياء الاصطناعي', 'معلوماتية حيوية', 'تكنولوجيا حيوية', 'طب حيوي', 'الطب البيولوجي',
    'علم الأحياء الزراعي', 'علم الأحياء الجنائي', 'علم الأحياء البحري', 'علم الأحياء البيئي',

    # أنظمة الأعضاء (الحيوانات والبشر)
    'النظام الدوري', 'النظام الهضمي', 'النظام العصبي', 'النظام الغدي',
    'النظام التناسلي', 'النظام العضلي', 'النظام المناعي', 'النظام البولي', 'النظام التنفسي',

    # علوم الأعصاب المحددة (غالبًا تتداخل مع علم النفس)
    'عصبون', 'عصبونات', 'خلايا دُبْقِيَّة', 'ناقل عصبي', 'ناقلات عصبية', 'تشابك عصبي', 'تشابكات عصبية',
    'دماغ', 'قشرة دماغية', 'حصين', 'الحبل الشوكي', 'الجهاز العصبي المركزي', 'الجهاز العصبي المحيطي',

    # الكائنات النموذجية والأشكال الحية
    'بكتيريا', 'أركيا', 'فيروس', 'فيروسات', 'فطريات', 'فطر', 'خميرة',
    'يُوكَارِيُوت', 'يُوكَارِيُوتات', 'بروكاريُوت', 'بروكاريُوتات', 'فأر', 'فئران', 'سمك زيبرا', 'ذبابة', 'ذبابة', 'حشرة', 'حشرات',

    # النباتات والمصطلحات المتعلقة بها
    'نبات', 'نباتات', 'زهري', 'عاري البذور', 'طحالب', 'عوالق', 'مُعَشّب ضوئي',
    'كلوروفيل', 'خشب الوعاء', 'لحاء الوعاء', 'جذر', 'ساق', 'ورقة', 'زهرة', 'تلقيح', 'بذور',

    # الحيوانات والتصنيف
    'حيوان', 'حيوانات', 'فقاري', 'لافقاري', 'ثديي', 'ثدييات', 'طائر', 'طيور',
    'سمك', 'برمائي', 'زاحف', 'مفصلي', 'حشرة', 'حشرات',

    # بيئات البحث
    'مختبر', 'مختبرات', 'طبق بتري', 'حاضنة', 'طرد مركزي', 'خطر بيولوجي', 'سلامة بيولوجية',
    'بنك بيولوجي', 'مفاعل حيوي', 'كائن نموذجي', 'كائنات نموذجية'
    ],

    'law': [
    # General legal terms
    'law', 'laws', 'legal', 'legality', 'lawful', 'unlawful', 'legislation', 'regulation', 'regulations',
    'statute', 'statutes', 'compliance', 'jurisdiction', 'jurisdictions',
    'litigation', 'lawsuit', 'lawsuits', 'plaintiff', 'plaintiffs', 'defendant', 'defendants',
    'precedent', 'precedents', 'remedy', 'remedies', 'damages', 'liability', 'negligence',
    'tort', 'torts', 'contract', 'contracts', 'settlement', 'settlements',

    # Branches of law
    'constitutional law', 'criminal law', 'civil law', 'administrative law', 'international law',
    'human rights law', 'corporate law', 'commercial law', 'family law', 'property law',
    'labor law', 'employment law', 'tax law', 'environmental law', 'immigration law',
    'intellectual property', 'IP law', 'cyber law', 'digital law', 'maritime law',
    'medical law', 'health law', 'military law', 'banking law', 'banking regulation',

    # Legal systems and bodies
    'constitution', 'constitutions', 'judiciary', 'court', 'courts', 'tribunal', 'tribunals',
    'supreme court', 'appeal', 'appeals', 'verdict', 'verdicts', 'sentence', 'sentences',
    'indictment', 'indictments', 'prosecution', 'defense', 'hearing', 'hearings', 'writ', 'writs',

    # Legal professionals and roles
    'lawyer', 'lawyers', 'attorney', 'attorneys', 'counsel', 'legal counsel',
    'barrister', 'barristers', 'solicitor', 'solicitors', 'prosecutor', 'prosecutors',
    'judge', 'judges', 'jury', 'jurors', 'arbitrator', 'arbitrators', 'mediator', 'mediators',
    'notary', 'notaries', 'law clerk', 'legal assistant', 'legal scholar', 'legal expert',

    # Rights and ethics
    'right', 'rights', 'human right', 'human rights', 'civil rights', 'legal rights',
    'freedom', 'freedoms', 'equality', 'justice', 'social justice', 'due process',
    'privacy', 'freedom of speech', 'freedom of religion', 'anti-discrimination', 'non-discrimination',
    'ethics', 'ethical', 'transparency', 'accountability', 'equity',

    # International and EU law
    'treaty', 'treaties', 'convention', 'conventions', 'sovereignty', 'international tribunal',
    'asylum law', 'refugee law', 'humanitarian law', 'UN charter',
    'EU law', 'EU directive', 'EU regulation', 'European Court of Justice',

    # Governance and compliance
    'governance', 'corporate governance', 'good governance', 'regulatory framework',
    'audit', 'audits', 'anti-corruption', 'anti-money laundering', 'AML', 'compliance officer',
    'data protection', 'GDPR', 'whistleblower', 'whistleblowing',

    # Criminal justice and security
    'crime', 'crimes', 'criminal', 'criminality', 'penalty', 'penalties', 'punishment',
    'detention', 'incarceration', 'rehabilitation', 'forensics', 'forensic science',
    'policing', 'police', 'investigation', 'evidence', 'testimony', 'confession', 'criminal record',

    # Technology and law
    'cybersecurity', 'cyber law', 'digital law', 'data privacy', 'digital rights',
    'intellectual property', 'copyright', 'trademark', 'trademarks', 'patent', 'patents',
    'licensing', 'terms of service', 'privacy policy', 'online contracts',

    # Emerging legal areas
    'AI ethics', 'robot law', 'legaltech', 'blockchain law', 'smart contracts', 'digital identity',
    'bioethics', 'genetic privacy', 'environmental justice', 'climate litigation', 'space law',
    'autonomous vehicles law', 'algorithmic transparency', 'platform governance',

    # Academic/legal research terms
    'legal theory', 'legal framework', 'case study', 'case studies', 'jurisprudence',
    'legal reform', 'comparative law', 'law review', 'statutory interpretation',

    # Términos legales generales
    'ley', 'leyes', 'legal', 'legalidad', 'lícito', 'ilícito', 'legislación', 'regulación', 'regulaciones',
    'estatuto', 'estatutos', 'cumplimiento', 'jurisdicción', 'jurisdicciones',
    'litigio', 'demandante', 'demandantes', 'demandado', 'demandados',
    'precedente', 'precedentes', 'remedio', 'remedios', 'daños', 'responsabilidad', 'negligencia',
    'agravio', 'agravios', 'contrato', 'contratos', 'acuerdo', 'acuerdos',

    # Ramas del derecho
    'derecho constitucional', 'derecho penal', 'derecho civil', 'derecho administrativo', 'derecho internacional',
    'derechos humanos', 'derecho corporativo', 'derecho comercial', 'derecho de familia', 'derecho de propiedad',
    'derecho laboral', 'derecho de empleo', 'derecho tributario', 'derecho ambiental', 'derecho de inmigración',
    'propiedad intelectual', 'derecho de propiedad intelectual', 'derecho cibernético', 'derecho digital', 'derecho marítimo',
    'derecho médico', 'derecho de la salud', 'derecho militar', 'derecho bancario', 'regulación bancaria',

    # Sistemas legales y organismos
    'constitución', 'constituciones', 'poder judicial', 'tribunal', 'tribunales', 'tribunal', 'tribunales',
    'corte suprema', 'apelación', 'apelaciones', 'veredicto', 'veredictos', 'sentencia', 'sentencias',
    'acusación', 'acusaciones', 'fiscalía', 'defensa', 'audiencia', 'audiencias', 'auto', 'autos',

    # Profesionales y roles legales
    'abogado', 'abogados', 'abogada', 'abogadas', 'asesor', 'asesoría legal',
    'abogado defensor', 'abogados defensores', 'abogado solicitante', 'abogados solicitantes', 'fiscal', 'fiscales',
    'juez', 'jueces', 'jurado', 'jurados', 'árbitro', 'árbitros', 'mediador', 'mediadores',
    'notario', 'notarios', 'auxiliar legal', 'asistente legal', 'académico legal', 'experto legal',

    # Derechos y ética
    'derecho', 'derechos', 'derecho humano', 'derechos humanos', 'derechos civiles', 'derechos legales',
    'libertad', 'libertades', 'igualdad', 'justicia', 'justicia social', 'debido proceso',
    'privacidad', 'libertad de expresión', 'libertad religiosa', 'anti-discriminación', 'no discriminación',
    'ética', 'ético', 'transparencia', 'responsabilidad', 'equidad',

    # Derecho internacional y de la UE
    'tratado', 'tratados', 'convención', 'convenciones', 'soberanía', 'tribunal internacional',
    'derecho de asilo', 'derecho de los refugiados', 'derecho humanitario', 'carta de la ONU',
    'derecho de la UE', 'directiva de la UE', 'reglamento de la UE', 'Corte de Justicia de la Unión Europea',

    # Gobernanza y cumplimiento
    'gobernanza', 'gobernanza corporativa', 'buena gobernanza', 'marco regulatorio',
    'auditoría', 'auditorías', 'anticorrupción', 'anti-lavado de dinero', 'ALD', 'oficial de cumplimiento',
    'protección de datos', 'GDPR', 'denunciante', 'denuncia',

    # Justicia penal y seguridad
    'crimen', 'crímenes', 'penal', 'criminalidad', 'penalidad', 'penalidades', 'castigo',
    'detención', 'prisión', 'rehabilitación', 'forense', 'ciencia forense',
    'policía', 'policía', 'investigación', 'evidencia', 'testimonio', 'confesión', 'antecedentes penales',

    # Tecnología y derecho
    'ciberseguridad', 'derecho cibernético', 'derecho digital', 'privacidad de datos', 'derechos digitales',
    'propiedad intelectual', 'derechos de autor', 'marca registrada', 'marcas registradas', 'patente', 'patentes',
    'licencias', 'términos de servicio', 'política de privacidad', 'contratos en línea',

    # Áreas legales emergentes
    'ética de la IA', 'derecho de los robots', 'legaltech', 'derecho de blockchain', 'contratos inteligentes', 'identidad digital',
    'bioética', 'privacidad genética', 'justicia ambiental', 'litigación climática', 'derecho espacial',
    'derecho de vehículos autónomos', 'transparencia algorítmica', 'gobernanza de plataformas',

    # Términos de investigación académica/legal
    'teoría legal', 'marco legal', 'estudio de caso', 'estudios de caso', 'jurisprudencia',
    'reforma legal', 'derecho comparado', 'revista jurídica', 'interpretación estatutaria',

    # Termos legais gerais
    'lei', 'leis', 'legal', 'legalidade', 'lícito', 'ilícito', 'legislação', 'regulação', 'regulações',
    'estatuto', 'estatutos', 'conformidade', 'jurisdição', 'jurisdições',
    'litígio', 'processo', 'processos', 'autor', 'autores', 'réu', 'réus',
    'precedente', 'precedentes', 'remédio', 'remédios', 'danos', 'responsabilidade', 'negligência',
    'dano', 'danos', 'contrato', 'contratos', 'acordo', 'acordos',

    # Ramos do direito
    'direito constitucional', 'direito penal', 'direito civil', 'direito administrativo', 'direito internacional',
    'direitos humanos', 'direito corporativo', 'direito comercial', 'direito de família', 'direito de propriedade',
    'direito trabalhista', 'direito do trabalho', 'direito tributário', 'direito ambiental', 'direito de imigração',
    'propriedade intelectual', 'direito de propriedade intelectual', 'direito cibernético', 'direito digital', 'direito marítimo',
    'direito médico', 'direito da saúde', 'direito militar', 'direito bancário', 'regulação bancária',

    # Sistemas legais e organismos
    'constituição', 'constituições', 'judiciário', 'tribunal', 'tribunais', 'tribunal', 'tribunais',
    'supremo tribunal', 'apelação', 'apelações', 'veredicto', 'veredictos', 'sentença', 'sentenças',
    'acusação', 'acusação', 'acusação pública', 'defesa', 'audiência', 'audiências', 'mandado', 'mandados',

    # Profissionais e papéis legais
    'advogado', 'advogados', 'advogada', 'advogadas', 'consultor', 'consultoria jurídica',
    'advogado defensor', 'advogados defensores', 'advogado solicitante', 'advogados solicitantes', 'promotor', 'promotores',
    'juiz', 'juízes', 'jurado', 'jurados', 'árbitro', 'árbitros', 'mediador', 'mediadores',
    'notário', 'notários', 'assistente legal', 'auxiliar legal', 'acadêmico legal', 'especialista jurídico',

    # Direitos e ética
    'direito', 'direitos', 'direito humano', 'direitos humanos', 'direitos civis', 'direitos legais',
    'liberdade', 'liberdades', 'igualdade', 'justiça', 'justiça social', 'devido processo',
    'privacidade', 'liberdade de expressão', 'liberdade religiosa', 'anti-discriminação', 'não discriminação',
    'ética', 'ético', 'transparência', 'responsabilidade', 'equidade',

    # Direito internacional e da UE
    'tratado', 'tratados', 'convenção', 'convenções', 'soberania', 'tribunal internacional',
    'direito de asilo', 'direito dos refugiados', 'direito humanitário', 'carta da ONU',
    'direito da UE', 'diretiva da UE', 'regulamento da UE', 'Corte de Justiça da União Europeia',

    # Governança e conformidade
    'governança', 'governança corporativa', 'boa governança', 'estrutura regulatória',
    'auditoria', 'auditorias', 'anticorrupção', 'anti-lavagem de dinheiro', 'ALD', 'oficial de conformidade',
    'proteção de dados', 'GDPR', 'denunciante', 'denúncia',

    # Justiça criminal e segurança
    'crime', 'crimes', 'criminal', 'criminalidade', 'penalidade', 'penalidades', 'castigo',
    'detenção', 'prisão', 'rehabilitação', 'forense', 'ciência forense',
    'polícia', 'polícia', 'investigação', 'evidência', 'depoimento', 'confissão', 'registro criminal',

    # Tecnologia e direito
    'cibersegurança', 'direito cibernético', 'direito digital', 'privacidade de dados', 'direitos digitais',
    'propriedade intelectual', 'direitos autorais', 'marca registrada', 'marcas registradas', 'patente', 'patentes',
    'licenciamento', 'termos de serviço', 'política de privacidade', 'contratos online',

    # Áreas legais emergentes
    'ética da IA', 'direito dos robôs', 'legaltech', 'direito do blockchain', 'contratos inteligentes', 'identidade digital',
    'bioética', 'privacidade genética', 'justiça ambiental', 'litigação climática', 'direito espacial',
    'direito de veículos autônomos', 'transparência algorítmica', 'governança de plataformas',

    # Termos de pesquisa acadêmica/jurídica
    'teoria jurídica', 'estrutura jurídica', 'estudo de caso', 'estudos de caso', 'jurisprudência',
    'reforma jurídica', 'direito comparado', 'revisão jurídica', 'interpretação estatutária',

    # Termini legali generali
    'legge', 'leggi', 'legale', 'legalità', 'lecito', 'illecito', 'legislazione', 'regolamento', 'regolamenti',
    'statuto', 'statuti', 'conformità', 'giurisdizione', 'giurisdizioni',
    'contenzioso', 'causa legale', 'cause legali', 'attore', 'attori', 'difensore', 'difensori',
    'precedente', 'precedenti', 'rimedio', 'rimedi', 'danni', 'responsabilità', 'negligenza',
    'illecito', 'illeciti', 'contratto', 'contratti', 'accordo', 'accordi',

    # Rami del diritto
    'diritto costituzionale', 'diritto penale', 'diritto civile', 'diritto amministrativo', 'diritto internazionale',
    'diritto dei diritti umani', 'diritto societario', 'diritto commerciale', 'diritto di famiglia', 'diritto di proprietà',
    'diritto del lavoro', 'diritto del lavoro', 'diritto tributario', 'diritto ambientale', 'diritto dell\'immigrazione',
    'proprietà intellettuale', 'diritto di proprietà intellettuale', 'diritto informatico', 'diritto digitale', 'diritto marittimo',
    'diritto sanitario', 'diritto della salute', 'diritto militare', 'diritto bancario', 'regolamentazione bancaria',

    # Sistemi legali e organi
    'costituzione', 'costituzioni', 'giudiziario', 'tribunale', 'tribunali', 'tribunale', 'tribunali',
    'corte suprema', 'appello', 'appelli', 'verdetto', 'verdetti', 'sentenza', 'sentenze',
    'accusa', 'accuse', 'processo', 'difesa', 'udienza', 'udienze', 'atto', 'atti legali',

    # Professionisti legali e ruoli
    'avvocato', 'avvocati', 'difensore', 'difensori', 'consulente legale', 'consulente legale',
    'avvocato', 'avvocati', 'solicitatore', 'solicitori', 'pubblico ministero', 'pubblici ministeri',
    'giudice', 'giudici', 'giuria', 'giurati', 'arbitro', 'arbitri', 'mediatore', 'mediatori',
    'notaio', 'notai', 'segretario legale', 'assistente legale', 'studioso legale', 'esperto legale',

    # Diritti ed etica
    'diritto', 'diritti', 'diritto umano', 'diritti umani', 'diritti civili', 'diritti legali',
    'libertà', 'libertà', 'uguaglianza', 'giustizia', 'giustizia sociale', 'processo dovuto',
    'privacy', 'libertà di parola', 'libertà di religione', 'anti-discriminazione', 'non discriminazione',
    'etica', 'etico', 'trasparenza', 'responsabilità', 'equità',

    # Diritto internazionale e dell'UE
    'trattato', 'trattati', 'convenzione', 'convenzioni', 'sovranità', 'tribunale internazionale',
    'diritto d\'asilo', 'diritto dei rifugiati', 'diritto umanitario', 'Carta delle Nazioni Unite',
    'diritto dell\'UE', 'direttiva dell\'UE', 'regolamento dell\'UE', 'Corte di giustizia europea',

    # Governance e conformità
    'governance', 'governance aziendale', 'buona governance', 'quadro normativo',
    'audit', 'audits', 'anti-corruzione', 'antiriciclaggio', 'AML', 'responsabile della conformità',
    'protezione dei dati', 'GDPR', 'informativa', 'segnalazione',

    # Giustizia penale e sicurezza
    'reato', 'reati', 'penale', 'criminalità', 'penalità', 'penalità', 'punizione',
    'detenzione', 'incarcerazione', 'riabilitazione', 'forense', 'scienza forense',
    'polizia', 'polizia', 'indagine', 'evidenza', 'testimonianza', 'confessione', 'record penale',

    # Tecnologia e diritto
    'sicurezza informatica', 'diritto informatico', 'diritto digitale', 'privacy dei dati', 'diritti digitali',
    'proprietà intellettuale', 'copyright', 'marchio', 'marchi', 'brevetto', 'brevetti',
    'licenze', 'termini di servizio', 'politica sulla privacy', 'contratti online',

    # Nuove aree legali
    'etica dell\'AI', 'diritto dei robot', 'legaltech', 'diritto blockchain', 'contratti intelligenti', 'identità digitale',
    'bioetica', 'privacy genetica', 'giustizia ambientale', 'contenzioso climatico', 'diritto spaziale',
    'veicoli autonomi', 'trasparenza algoritmica', 'governance delle piattaforme',

    # Ricerca accademica/legale
    'teoria legale', 'quadro giuridico', 'studio di caso', 'studi di caso', 'giurisprudenza',
    'riforma legale', 'diritto comparato', 'revista legale', 'interpretazione statutaria',

    # Termes juridiques généraux
    'loi', 'lois', 'légal', 'légalité', 'légitime', 'illégal', 'législation', 'réglementation', 'règlements',
    'statut', 'statuts', 'conformité', 'juridiction', 'juridictions',
    'litige', 'poursuite', 'poursuites', 'demandeur', 'demandeurs', 'défendeur', 'défendeurs',
    'précédent', 'précédents', 'remède', 'remèdes', 'dommages', 'responsabilité', 'négligence',
    'délit', 'délits', 'contrat', 'contrats', 'règlement', 'règlements',

    # Branches du droit
    'droit constitutionnel', 'droit pénal', 'droit civil', 'droit administratif', 'droit international',
    'droit des droits de l\'homme', 'droit des sociétés', 'droit commercial', 'droit de la famille', 'droit de la propriété',
    'droit du travail', 'droit de l\'emploi', 'droit fiscal', 'droit de l\'environnement', 'droit de l\'immigration',
    'propriété intellectuelle', 'droit de la propriété intellectuelle', 'droit informatique', 'droit numérique', 'droit maritime',
    'droit médical', 'droit de la santé', 'droit militaire', 'droit bancaire', 'réglementation bancaire',

    # Systèmes juridiques et organes
    'constitution', 'constitutions', 'système judiciaire', 'tribunal', 'tribunaux', 'tribunal', 'tribunaux',
    'cour suprême', 'appel', 'appels', 'verdict', 'verdicts', 'sentence', 'sentences',
    'acte d\'accusation', 'actes d\'accusation', 'poursuite', 'défense', 'audience', 'audiences', 'ordonnance', 'ordonnances',

    # Professionnels du droit et rôles
    'avocat', 'avocats', 'conseiller juridique', 'conseillers juridiques', 'conseil', 'avocat',
    'avocat', 'avocats', 'solliciteur', 'sollicitateurs', 'procureur', 'procureurs',
    'juge', 'juges', 'jury', 'jurés', 'arbitre', 'arbitres', 'médiateur', 'médiateurs',
    'notaire', 'notaires', 'clerc de notaire', 'assistant juridique', 'chercheur juridique', 'expert juridique',

    # Droits et éthique
    'droit', 'droits', 'droit de l\'homme', 'droits de l\'homme', 'droits civils', 'droits légaux',
    'liberté', 'libertés', 'égalité', 'justice', 'justice sociale', 'processus dû',
    'confidentialité', 'liberté d\'expression', 'liberté de religion', 'anti-discrimination', 'non-discrimination',
    'éthique', 'éthique', 'transparence', 'responsabilité', 'équité',

    # Droit international et de l\'UE
    'traité', 'traités', 'convention', 'conventions', 'souveraineté', 'tribunal international',
    'droit d\'asile', 'droit des réfugiés', 'droit humanitaire', 'charte de l\'ONU',
    'droit de l\'UE', 'directive de l\'UE', 'réglementation de l\'UE', 'Cour de justice de l\'Union européenne',

    # Gouvernance et conformité
    'gouvernance', 'gouvernance d\'entreprise', 'bonne gouvernance', 'cadre réglementaire',
    'audit', 'audits', 'anti-corruption', 'lutte contre le blanchiment d\'argent', 'LBA', 'responsable conformité',
    'protection des données', 'RGPD', 'lanceur d\'alerte', 'signalement',

    # Justice pénale et sécurité
    'crime', 'crimes', 'criminel', 'criminalité', 'peine', 'peines', 'punition',
    'détention', 'incarcération', 'réhabilitation', 'médecine légale', 'sciences forensiques',
    'police', 'police', 'enquête', 'preuve', 'témoignage', 'confession', 'casier judiciaire',

    # Technologie et droit
    'cybersécurité', 'droit informatique', 'droit numérique', 'protection des données', 'droits numériques',
    'propriété intellectuelle', 'droit d\'auteur', 'marque', 'marques', 'brevet', 'brevets',
    'licence', 'conditions d\'utilisation', 'politique de confidentialité', 'contrats en ligne',

    # Nouvelles zones juridiques
    'éthique de l\'IA', 'droit des robots', 'legaltech', 'droit blockchain', 'contrats intelligents', 'identité numérique',
    'bioéthique', 'confidentialité génétique', 'justice environnementale', 'contentieux climatique', 'droit spatial',
    'véhicules autonomes', 'transparence algorithmique', 'gouvernance des plateformes',

    # Recherche académique/légale
    'théorie du droit', 'cadre juridique', 'étude de cas', 'études de cas', 'jurisprudence',
    'réforme juridique', 'droit comparé', 'revue juridique', 'interprétation statutaire',

    # 一般法律术语
    '法律', '法律（复数）', '法律的', '合法性', '合法', '非法', '立法', '法规', '规章',
    '法令', '法令（复数）', '合规', '管辖权', '管辖区',
    '诉讼', '诉讼案件', '诉讼案件（复数）', '原告', '原告们', '被告', '被告们',
    '判例', '判例（复数）', '救济', '救济措施（复数）', '损害赔偿', '责任', '疏忽',
    '侵权行为', '侵权行为（复数）', '合同', '合同（复数）', '和解', '和解协议（复数）',

    # 法律分支
    '宪法法', '刑法', '民法', '行政法', '国际法',
    '人权法', '公司法', '商业法', '家庭法', '财产法',
    '劳动法', '就业法', '税法', '环境法', '移民法',
    '知识产权', '知识产权法', '网络法', '数字法', '海事法',
    '医学法', '健康法', '军事法', '银行法', '银行监管',

    # 法律系统和机构
    '宪法', '宪法（复数）', '司法', '法院', '法院（复数）', '法庭', '法庭（复数）',
    '最高法院', '上诉', '上诉（复数）', '判决', '判决（复数）', '判刑', '判刑（复数）',
    '起诉书', '起诉书（复数）', '公诉', '辩护', '听证会', '听证会（复数）', '传票', '传票（复数）',

    # 法律专业人士和角色
    '律师', '律师们', '代理人', '代理人（复数）', '顾问', '法律顾问',
    '辩护律师', '辩护律师（复数）', '公证人', '公证人（复数）', '检察官', '检察官们',
    '法官', '法官们', '陪审团', '陪审员们', '仲裁员', '仲裁员们', '调解员', '调解员们',
    '公证员', '公证员们', '法律助理', '法律助理（复数）', '法学者', '法律专家',

    # 权利与伦理
    '权利', '权利（复数）', '人权', '人权（复数）', '公民权利', '法律权利',
    '自由', '自由（复数）', '平等', '正义', '社会正义', '正当程序',
    '隐私', '言论自由', '宗教自由', '反歧视', '非歧视',
    '伦理', '伦理的', '透明度', '问责制', '公平',

    # 国际法和欧盟法
    '条约', '条约（复数）', '公约', '公约（复数）', '主权', '国际法庭',
    '庇护法', '难民法', '人道法', '联合国宪章',
    '欧盟法', '欧盟指令', '欧盟规章', '欧洲法院',

    # 治理与合规
    '治理', '公司治理', '良好治理', '监管框架',
    '审计', '审计（复数）', '反腐败', '反洗钱', 'AML', '合规官',
    '数据保护', 'GDPR', '举报人', '举报',

    # 刑事司法与安全
    '犯罪', '犯罪（复数）', '刑事', '刑事行为', '惩罚', '惩罚（复数）', '处罚',
    '拘留', '监禁', '康复', '法医学', '法医学科学',
    '警务', '警察', '调查', '证据', '证词', '供认', '犯罪记录',

    # 技术与法律
    '网络安全', '网络法', '数字法', '数据隐私', '数字权利',
    '知识产权', '版权', '商标', '商标（复数）', '专利', '专利（复数）',
    '许可', '服务条款', '隐私政策', '在线合同',

    # 新兴法律领域
    '人工智能伦理', '机器人法', '法律技术', '区块链法', '智能合约', '数字身份',
    '生物伦理', '基因隐私', '环境正义', '气候诉讼', '太空法',
    '自动驾驶车辆法', '算法透明度', '平台治理',

    # 学术/法律研究术语
    '法律理论', '法律框架', '案例研究', '案例研究（复数）', '法学',
    '法律改革', '比较法', '法律评论', '法定解释',

    # Общие юридические термины
    'право', 'законы', 'юридический', 'законность', 'законный', 'незаконный', 'законодательство', 'регулирование', 'регламенты',
    'статут', 'статуты', 'соблюдение', 'юрисдикция', 'юрисдикции',
    'судебный процесс', 'иск', 'исковые дела', 'истец', 'истцы', 'ответчик', 'ответчики',
    'прецедент', 'прецеденты', 'ремедиум', 'ремедии', 'ущерб', 'ответственность', 'небрежность',
    'деликт', 'деликты', 'контракт', 'контракты', 'урегулирование', 'урегулирования',

    # Отрасли права
    'конституционное право', 'уголовное право', 'гражданское право', 'административное право', 'международное право',
    'право человека', 'корпоративное право', 'коммерческое право', 'семейное право', 'право собственности',
    'трудовое право', 'трудовое право', 'налоговое право', 'экологическое право', 'иммиграционное право',
    'интеллектуальная собственность', 'право интеллектуальной собственности', 'киберправо', 'цифровое право', 'морское право',
    'медицинское право', 'здравоохранительное право', 'военное право', 'банковское право', 'банковская регуляция',

    # Юридические системы и органы
    'конституция', 'конституции', 'судебная власть', 'суд', 'суды', 'трибунал', 'трибуналы',
    'верховный суд', 'апелляция', 'апелляции', 'приговор', 'приговоры', 'приговор', 'приговоры',
    'обвинительное заключение', 'обвинения', 'обвинение', 'защита', 'слушание', 'слушания', 'письмо', 'письма',

    # Юридические профессии и роли
    'юрист', 'юристы', 'адвокат', 'адвокаты', 'консультант', 'юридический консультант',
    'барристер', 'барристеры', 'солиситор', 'солиситоры', 'прокурор', 'прокуроры',
    'судья', 'судьи', 'присяжные', 'присяжные заседатели', 'арбитр', 'арбитры', 'медиатор', 'медиаторы',
    'нотариус', 'нотариусы', 'юридический помощник', 'юридический асистент', 'юридический ученый', 'юридический эксперт',

    # Права и этика
    'право', 'права', 'права человека', 'права человека', 'гражданские права', 'юридические права',
    'свобода', 'свободы', 'равенство', 'справедливость', 'социальная справедливость', 'надлежащая процедура',
    'конфиденциальность', 'свобода слова', 'свобода вероисповедания', 'антидискриминация', 'недискриминация',
    'этика', 'этический', 'прозрачность', 'подотчетность', 'справедливость',

    # Международное право и право ЕС
    'договор', 'договоры', 'конвенция', 'конвенции', 'суверенитет', 'международный трибунал',
    'закон о убежище', 'закон о беженцах', 'гуманитарное право', 'Хартия ООН',
    'право ЕС', 'директива ЕС', 'регламент ЕС', 'Европейский суд справедливости',

    # Управление и соблюдение
    'управление', 'корпоративное управление', 'хорошее управление', 'регуляторная структура',
    'аудит', 'аудиты', 'антикоррупция', 'антиотмывание денег', 'AML', 'комплаенс-офицер',
    'защита данных', 'GDPR', 'осведомитель', 'осведомительство',

    # Уголовное правосудие и безопасность
    'преступление', 'преступления', 'уголовный', 'уголовность', 'наказание', 'наказания', 'покарание',
    'задержание', 'тюремное заключение', 'реабилитация', 'криминалистика', 'криминалистическая наука',
    'полиция', 'полиция', 'расследование', 'доказательства', 'свидетельство', 'признание', 'уголовная запись',

    # Технологии и право
    'кибербезопасность', 'киберправо', 'цифровое право', 'конфиденциальность данных', 'цифровые права',
    'интеллектуальная собственность', 'авторское право', 'торговая марка', 'торговые марки', 'производственный патент', 'производственные патенты',
    'лицензирование', 'условия использования', 'политика конфиденциальности', 'онлайн контракты',

    # Новые области права
    'этика ИИ', 'право роботов', 'право технологий', 'право блокчейна', 'умные контракты', 'цифровая идентификация',
    'биоэтика', 'генетическая конфиденциальность', 'экологическая справедливость', 'климатические иски', 'космическое право',
    'право автономных транспортных средств', 'прозрачность алгоритмов', 'управление платформами',

    # Академические/юридические исследовательские термины
    'юридическая теория', 'юридическая структура', 'кейс-стадия', 'кейс-стадии', 'юриспруденция',
    'реформа права', 'сравнительное право', 'правовой обзор', 'толкование законов',

    # 一般的な法的用語
    '法律', '法', '法的', '合法性', '合法', '違法', '立法', '規制', '規則',
    '法令', '法令集', '遵守', '管轄権', '管轄区域',
    '訴訟', '訴訟案件', '訴訟', '原告', '原告たち', '被告', '被告たち',
    '判例', '判例集', '救済', '救済措置', '損害賠償', '責任', '過失',
    '不法行為', '不法行為（複数）', '契約', '契約（複数）', '和解', '和解（複数）',

    # 法律の分野
    '憲法', '刑法', '民法', '行政法', '国際法',
    '人権法', '企業法', '商法', '家族法', '財産法',
    '労働法', '雇用法', '税法', '環境法', '移民法',
    '知的財産', '知的財産法', 'サイバー法', 'デジタル法', '海事法',
    '医療法', '健康法', '軍法', '銀行法', '銀行規制',

    # 法的システムと機関
    '憲法', '憲法（複数）', '司法', '裁判所', '裁判所（複数）', '法廷', '法廷（複数）',
    '最高裁判所', '控訴', '控訴（複数）', '判決', '判決（複数）', '刑罰', '刑罰（複数）',
    '起訴状', '起訴状（複数）', '起訴', '弁護', '公聴会', '公聴会（複数）', '命令', '命令書',

    # 法的専門家と役割
    '弁護士', '弁護士たち', '弁護人', '弁護人たち', '顧問', '法的顧問',
    '弁護士（上級）', '弁護士（複数）', '弁護士（下級）', '弁護士（下級）', '検察官', '検察官たち',
    '裁判官', '裁判官たち', '陪審員', '陪審員たち', '仲裁人', '仲裁人たち', '調停者', '調停者たち',
    '公証人', '公証人（複数）', '法務助手', '法的助手', '法学者', '法的専門家',

    # 権利と倫理
    '権利', '権利（複数）', '人権', '人権（複数）', '市民権', '法的権利',
    '自由', '自由（複数）', '平等', '正義', '社会正義', '適正手続き',
    'プライバシー', '言論の自由', '宗教の自由', '反差別', '非差別',
    '倫理', '倫理的', '透明性', '説明責任', '公平',

    # 国際法およびEU法
    '条約', '条約（複数）', '協定', '協定（複数）', '主権', '国際裁判所',
    '亡命法', '難民法', '人道法', '国連憲章',
    'EU法', 'EU指令', 'EU規則', '欧州司法裁判所',

    # ガバナンスとコンプライアンス
    'ガバナンス', '企業ガバナンス', '良好なガバナンス', '規制枠組み',
    '監査', '監査（複数）', '反腐敗', 'マネーロンダリング防止', 'AML', 'コンプライアンス担当者',
    'データ保護', 'GDPR', '内部告発者', '内部告発',

    # 刑事司法とセキュリティ
    '犯罪', '犯罪（複数）', '刑事', '犯罪性', '罰', '罰（複数）', '刑罰',
    '拘留', '収監', 'リハビリテーション', '法医学', '法医学の科学',
    '警察', '警察（複数）', '調査', '証拠', '証言', '自白', '犯罪記録',

    # テクノロジーと法
    'サイバーセキュリティ', 'サイバー法', 'デジタル法', 'データプライバシー', 'デジタル権利',
    '知的財産', '著作権', '商標', '商標（複数）', '特許', '特許（複数）',
    'ライセンス', 'サービス利用規約', 'プライバシーポリシー', 'オンライン契約',

    # 新興法分野
    'AI倫理', 'ロボット法', '法テクノロジー', 'ブロックチェーン法', 'スマート契約', 'デジタルID',
    'バイオ倫理', '遺伝子プライバシー', '環境正義', '気候訴訟', '宇宙法',
    '自律走行車両法', 'アルゴリズムの透明性', 'プラットフォームガバナンス',

    # 学術/法的研究用語
    '法理論', '法的枠組み', 'ケーススタディ', 'ケーススタディ（複数）', '判例法',
    '法改革', '比較法', '法学レビュー', '法定解釈',

    # 일반 법률 용어
    '법', '법률', '법적', '합법성', '합법', '불법', '입법', '규제', '규정',
    '법령', '법령들', '준수', '관할권', '관할 지역',
    '소송', '소송 사건', '소송들', '원고', '원고들', '피고', '피고들',
    '판례', '판례들', '구제', '구제 방법들', '손해배상', '책임', '과실',
    '불법 행위', '불법 행위들', '계약', '계약들', '합의', '합의들',

    # 법의 분야들
    '헌법', '형법', '민법', '행정법', '국제법',
    '인권법', '기업법', '상법', '가족법', '재산법',
    '노동법', '고용법', '세법', '환경법', '이민법',
    '지적재산권', '지적재산권법', '사이버법', '디지털법', '해양법',
    '의료법', '건강법', '군사법', '은행법', '은행 규제',

    # 법률 시스템과 기관들
    '헌법', '헌법들', '사법부', '법원', '법원들', '법정', '법정들',
    '대법원', '항소', '항소들', '판결', '판결들', '형벌', '형벌들',
    '기소장', '기소장들', '기소', '변호', '청문회', '청문회들', '명령', '명령서들',

    # 법률 전문가와 역할들
    '변호사', '변호사들', '법률 대리인', '법률 대리인들', '고문', '법률 고문',
    '법정 변호사', '법정 변호사들', '솔리시터', '솔리시터들', '검사', '검사들',
    '판사', '판사들', '배심원', '배심원들', '중재인', '중재인들', '조정자', '조정자들',
    '공증인', '공증인들', '법률 보조원', '법률 조수', '법학자', '법률 전문가',

    # 권리와 윤리
    '권리', '권리들', '인권', '인권들', '시민권', '법적 권리',
    '자유', '자유들', '평등', '정의', '사회 정의', '적법 절차',
    '개인 정보 보호', '언론의 자유', '종교의 자유', '반차별', '비차별',
    '윤리', '윤리적', '투명성', '책임', '공평',

    # 국제법 및 EU법
    '조약', '조약들', '협약', '협약들', '주권', '국제재판소',
    '망명법', '난민법', '인도적 법', '유엔 헌장',
    'EU법', 'EU 지침', 'EU 규제', '유럽연합 사법재판소',

    # 거버넌스와 준수
    '거버넌스', '기업 거버넌스', '좋은 거버넌스', '규제 프레임워크',
    '감사', '감사들', '반부패', '자금 세탁 방지', 'AML', '준수 담당자',
    '데이터 보호', 'GDPR', '내부 고발자', '내부 고발',

    # 범죄 사법과 보안
    '범죄', '범죄들', '형사', '범죄성', '형벌', '형벌들', '처벌',
    '구금', '수감', '재활', '법의학', '법의학 과학',
    '경찰', '경찰들', '조사', '증거', '증언', '자백', '범죄 기록',

    # 기술과 법
    '사이버 보안', '사이버 법', '디지털 법', '데이터 프라이버시', '디지털 권리',
    '지적 재산', '저작권', '상표', '상표들', '특허', '특허들',
    '라이선스', '서비스 이용약관', '개인정보 처리방침', '온라인 계약',

    # 새로운 법적 분야들
    'AI 윤리', '로봇 법', '법률기술', '블록체인 법', '스마트 계약', '디지털 신원',
    '생명 윤리', '유전자 개인정보 보호', '환경 정의', '기후 소송', '우주법',
    '자율 주행 차량 법', '알고리즘 투명성', '플랫폼 거버넌스',

    # 학문적/법적 연구 용어들
    '법 이론', '법적 프레임워크', '사례 연구', '사례 연구들', '판례법',
    '법 개혁', '비교법', '법학 리뷰', '법 해석',

    # 一般法律術語
    '法律', '法律 (複數)', '合法的', '合法性', '合法', '非法', '立法', '規範', '法規',
    '法典', '法典 (複數)', '遵守', '司法管轄區', '司法管轄區域',
    '訴訟', '訴訟案', '訴訟案件', '原告', '原告們', '被告', '被告們',
    '先例', '先例們', '補救', '補救措施', '損害賠償', '責任', '疏忽',
    '侵權行為', '侵權行為們', '契約', '契約們', '和解', '和解協議',

    # 法律分支
    '憲法法', '刑法', '民法', '行政法', '國際法',
    '人權法', '公司法', '商業法', '家庭法', '財產法',
    '勞動法', '就業法', '稅法', '環境法', '移民法',
    '智慧財產權', '智慧財產法', '網路法', '數位法', '海事法',
    '醫療法', '健康法', '軍事法', '銀行法', '銀行規範',

    # 法律系統與機構
    '憲法', '憲法們', '司法機構', '法院', '法院們', '法庭', '法庭們',
    '最高法院', '上訴', '上訴們', '判決', '判決們', '刑罰', '刑罰們',
    '起訴書', '起訴書們', '檢控', '辯護', '聽證', '聽證會', '令狀', '令狀們',

    # 法律專業與角色
    '律師', '律師們', '律師事務所', '律師事務所們', '顧問', '法律顧問',
    '大律師', '大律師們', '律師事務所律師', '律師事務所律師們', '檢察官', '檢察官們',
    '法官', '法官們', '陪審團', '陪審員們', '仲裁人', '仲裁人們', '調解人', '調解人們',
    '公證人', '公證人們', '法律助理', '法律助理們', '法律學者', '法律專家',

    # 權利與道德
    '權利', '權利們', '人權', '人權們', '公民權', '法律權利',
    '自由', '自由們', '平等', '正義', '社會正義', '正當程序',
    '隱私', '言論自由', '宗教自由', '反歧視', '非歧視',
    '道德', '道德的', '透明', '問責制', '公平',

    # 國際與歐盟法
    '條約', '條約們', '公約', '公約們', '主權', '國際法庭',
    '庇護法', '難民法', '人道法', '聯合國憲章',
    '歐盟法', '歐盟指令', '歐盟法規', '歐洲法院',

    # 治理與合規
    '治理', '企業治理', '良好治理', '監管框架',
    '審計', '審計們', '反貪污', '反洗錢', 'AML', '合規官',
    '資料保護', 'GDPR', '告密者', '舉報',

    # 刑事司法與安全
    '犯罪', '犯罪們', '刑事的', '刑事行為', '懲罰', '懲罰們', '處罰',
    '拘留', '監禁', '康復', '法醫', '法醫科學',
    '警察工作', '警察', '調查', '證據', '證詞', '供認', '刑事紀錄',

    # 科技與法律
    '網絡安全', '網絡法', '數位法', '資料隱私', '數位權利',
    '智慧財產權', '版權', '商標', '商標們', '專利', '專利們',
    '授權', '服務條款', '隱私政策', '線上契約',

    # 新興法律領域
    'AI道德', '機器人法', '法律科技', '區塊鏈法', '智能合約', '數位身份',
    '生物倫理學', '基因隱私', '環境正義', '氣候訴訟', '太空法',
    '自駕車法', '算法透明度', '平台治理',

    # 學術/法律研究術語
    '法律理論', '法律框架', '案例研究', '案例研究們', '法理學',
    '法律改革', '比較法', '法律評論', '法規解釋',

    # Obecné právní termíny
    'právo', 'zákony', 'právní', 'legalita', 'zákonný', 'nelegální', 'legislativa', 'regulace', 'předpisy',
    'statut', 'statuty', 'dodržování', 'jurisdikce', 'jurisdikce (pl)',
    'spor', 'žaloba', 'žaloby', 'žalobce', 'žalobci', 'žalovaný', 'žalovaní',
    'precedens', 'precedensy', 'náprava', 'nápravy', 'škoda', 'odpovědnost', 'nedbalost',
    'tort', 'torty', 'smlouva', 'smlouvy', 'smíření', 'smíření',
    
    # Právní oblasti
    'ústavní právo', 'trestní právo', 'občanské právo', 'správní právo', 'mezinárodní právo',
    'právo lidských práv', 'korporátní právo', 'obchodní právo', 'rodinné právo', 'právo vlastnictví',
    'právo práce', 'zaměstnanecké právo', 'daňové právo', 'ochranné právo', 'imigrační právo',
    'duševní vlastnictví', 'právo duševního vlastnictví', 'kyberprávo', 'digitální právo', 'námořní právo',
    'lékařské právo', 'zdravotní právo', 'vojenské právo', 'bankovní právo', 'bankovní regulace',

    # Právní systémy a orgány
    'ústava', 'ústavy', 'soudnictví', 'soud', 'soudy', 'tribunál', 'tribunály',
    'nejvyšší soud', 'odvolání', 'odvolání (pl)', 'verdikt', 'verdikty', 'trest', 'tresty',
    'obvinění', 'obvinění (pl)', 'stíhání', 'obhajoba', 'výslech', 'výslechy', 'příkaz', 'příkazy',

    # Právní profese a role
    'advokát', 'advokáti', 'právník', 'právníci', 'poradce', 'právní poradce',
    'barrister', 'barristers', 'solicitor', 'solicitors', 'prokurátor', 'prokurátoři',
    'soudce', 'soudci', 'porota', 'porotci', 'rozhodce', 'rozhodci', 'mediátor', 'mediátoři',
    'notář', 'notáři', 'právní tajemník', 'právní asistent', 'právní vědec', 'právní expert',

    # Práva a etika
    'právo', 'práva', 'lidská práva', 'lidská práva', 'občanská práva', 'právní práva',
    'svoboda', 'svobody', 'rovnost', 'spravedlnost', 'sociální spravedlnost', 'řádný proces',
    'souhlas', 'svoboda projevu', 'svoboda náboženského vyznání', 'proti diskriminaci', 'nediskriminace',
    'etika', 'etický', 'transparentnost', 'zodpovědnost', 'spravedlnost',

    # Mezinárodní a EU právo
    'smlouva', 'smlouvy', 'úmluva', 'úmluvy', 'suverenita', 'mezinárodní tribunál',
    'právo na azyl', 'právo uprchlíků', 'humanitární právo', 'Charta OSN',
    'právo EU', 'směrnice EU', 'nařízení EU', 'Soudní dvůr EU',

    # Governance a shoda
    'správa', 'firemní správa', 'dobrá správa', 'regulační rámec',
    'audit', 'audity', 'protikorupce', 'proti praní špinavých peněz', 'AML', 'oficer pro shodu',
    'ochrana údajů', 'GDPR', 'whistleblower', 'whistleblowing',

    # Trestní spravedlnost a bezpečnost
    'zločin', 'zločiny', 'kriminální', 'trestná činnost', 'trest', 'tresty', 'trestání',
    'zadržení', 'uvěznění', 'rehabilitace', 'forenzní', 'forenzní vědy',
    'policie', 'policie', 'vyšetřování', 'důkazy', 'svědectví', 'přiznání', 'trestní záznam',

    # Technologie a právo
    'kybernetická bezpečnost', 'kyberprávo', 'digitální právo', 'ochrana údajů', 'digitální práva',
    'duševní vlastnictví', 'autorské právo', 'obchodní značka', 'obchodní značky', 'patent', 'patenty',
    'licencování', 'podmínky služby', 'zásady ochrany osobních údajů', 'online smlouvy',

    # Nové oblasti práva
    'AI etika', 'robotické právo', 'právní technologie', 'blockchain právo', 'inteligentní smlouvy', 'digitální identita',
    'bioetika', 'genetická ochrana', 'environmentální spravedlnost', 'klimatické soudní řízení', 'kosmické právo',
    'autonomní vozidla', 'transparentnost algoritmů', 'správa platforem',

    # Akademický/výzkumný právní termíny
    'právní teorie', 'právní rámec', 'případová studie', 'případové studie', 'jurisprudence',
    'právní reforma', 'komparativní právo', 'právní recenze', 'statutární výklad',

    # المصطلحات القانونية العامة
    'قانون', 'قوانين', 'قانوني', 'الشرعية', 'قانوني', 'غير قانوني', 'تشريع', 'تنظيم', 'أنظمة',
    'قانون', 'قوانين', 'الامتثال', 'الاختصاص القضائي', 'الاختصاصات القضائية',
    'تقاضي', 'دعوى قضائية', 'الدعاوى القضائية', 'مدعي', 'المدعين', 'مدعى عليه', 'المدعى عليهم',
    'سابقة', 'سابِقَات', 'علاج', 'علاجات', 'تعويضات', 'مسؤولية', 'إهمال',
    'خطأ', 'أخطاء', 'عقد', 'عقود', 'تسوية', 'تسويات',

    # فروع القانون
    'القانون الدستوري', 'القانون الجنائي', 'القانون المدني', 'القانون الإداري', 'القانون الدولي',
    'قانون حقوق الإنسان', 'القانون التجاري', 'القانون العائلي', 'قانون الملكية', 'قانون العمل',
    'قانون التوظيف', 'قانون الضرائب', 'قانون البيئة', 'قانون الهجرة',
    'حقوق الملكية الفكرية', 'قانون الملكية الفكرية', 'القانون الإلكتروني', 'القانون الرقمي', 'القانون البحري',
    'القانون الطبي', 'قانون الصحة', 'القانون العسكري', 'قانون البنوك', 'تنظيم البنوك',

    # الأنظمة القانونية والهيئات
    'دستور', 'دساتير', 'السلطة القضائية', 'محكمة', 'محاكم', 'محكمة', 'محاكم',
    'المحكمة العليا', 'استئناف', 'استئنافات', 'حكم', 'أحكام', 'حكم', 'أحكام',
    'اتهام', 'اتهامات', 'مقاضاة', 'دفاع', 'جلسة', 'جلسات', 'أمر قضائي', 'أوامر قضائية',

    # المحامون والأدوار القانونية
    'محامي', 'محامون', 'محامي دفاع', 'محامون دفاع', 'مستشار قانوني', 'مستشار قانوني',
    'محامي مدني', 'محامو الدفاع', 'محامي شركات', 'محامو الشركات', 'مدعٍ عام', 'مدعون عامون',
    'قاضي', 'قضاة', 'هيئة محلفين', 'أعضاء هيئة المحلفين', 'محكم', 'محكمون', 'وسيط', 'وسطاء',
    'موثق', 'موثقون', 'كاتب عدل', 'مساعد قانوني', 'باحث قانوني', 'خبير قانوني',

    # الحقوق والأخلاقيات
    'حق', 'حقوق', 'حق الإنسان', 'حقوق الإنسان', 'حقوق مدنية', 'حقوق قانونية',
    'حرية', 'حريات', 'مساواة', 'عدالة', 'عدالة اجتماعية', 'الحق في الإجراءات القانونية',
    'خصوصية', 'حرية التعبير', 'حرية الدين', 'مكافحة التمييز', 'عدم التمييز',
    'أخلاقيات', 'أخلاقي', 'شفافية', 'مساءلة', 'إنصاف',

    # القانون الدولي وقانون الاتحاد الأوروبي
    'معاهدة', 'معاهدات', 'اتفاقية', 'اتفاقيات', 'سيادة', 'محكمة دولية',
    'قانون اللجوء', 'قانون اللاجئين', 'القانون الإنساني', 'ميثاق الأمم المتحدة',
    'قانون الاتحاد الأوروبي', 'توجيه الاتحاد الأوروبي', 'أنظمة الاتحاد الأوروبي', 'محكمة العدل الأوروبية',

    # الحوكمة والامتثال
    'حوكمة', 'حوكمة الشركات', 'حوكمة جيدة', 'إطار تنظيمي',
    'تدقيق', 'تدقيقات', 'مكافحة الفساد', 'مكافحة غسل الأموال', 'AML', 'مسؤول الامتثال',
    'حماية البيانات', 'GDPR', 'الملّوح', 'التبليغ',

    # العدالة الجنائية والأمن
    'جريمة', 'جرائم', 'جنائي', 'إجرام', 'عقوبة', 'عقوبات', 'معاقبة',
    'احتجاز', 'سجن', 'إعادة تأهيل', 'علم الطب الشرعي', 'علم الطب الشرعي',
    'شرطة', 'الشرطة', 'تحقيق', 'دليل', 'شهادة', 'اعتراف', 'سجل جنائي',

    # التكنولوجيا والقانون
    'الأمن السيبراني', 'القانون السيبراني', 'القانون الرقمي', 'خصوصية البيانات', 'الحقوق الرقمية',
    'الملكية الفكرية', 'حقوق الطبع والنشر', 'علامة تجارية', 'علامات تجارية', 'براءة اختراع', 'براءات اختراع',
    'ترخيص', 'شروط الخدمة', 'سياسة الخصوصية', 'العقود عبر الإنترنت',

    # المجالات القانونية الناشئة
    'أخلاقيات الذكاء الاصطناعي', 'قانون الروبوتات', 'التقنية القانونية', 'قانون البلوكشين', 'العقود الذكية', 'الهوية الرقمية',
    'الأخلاقيات البيولوجية', 'خصوصية الجينات', 'العدالة البيئية', 'قضايا المناخ', 'قانون الفضاء',
    'قانون المركبات الذاتية القيادة', 'الشفافية الخوارزمية', 'حوكمة المنصات',

    # مصطلحات البحث الأكاديمي/القانوني
    'نظرية قانونية', 'إطار قانوني', 'دراسة حالة', 'دراسات حالة', 'فقه القانون',
    'إصلاحات قانونية', 'القانون المقارن', 'مراجعة قانونية', 'تفسير الأنظمة'
],

    'arts_humanities': [
    # General fields
    'philosophy', 'philosophies', 'history', 'histories', 'language', 'languages',
    'literature', 'literatures', 'art', 'arts', 'culture', 'cultures', 'aesthetic', 'aesthetics',
    'music', 'musics', 'theatre', 'theater', 'paintings', 'painting', 'sculpture', 'sculptures',
    'architecture', 'architectures', 'visual art', 'visual arts',

    # Literature & language
    'poetry', 'poem', 'poems', 'prose', 'novel', 'novels', 'drama', 'narrative', 'narratives',
    'fiction', 'nonfiction', 'linguistics', 'semantic', 'semantics', 'syntax', 'morphology',
    'phonetic', 'phonetics', 'philology', 'translation', 'translations', 'rhetoric',
    'discourses', 'discourse', 'comparative literature',

    # History
    'medieval', 'renaissance', 'modern history', 'ancient history', 'world war', 'world wars',
    'colonialism', 'imperialism', 'revolution', 'revolutions', 'historical memory', 'memory',
    'archives', 'archival', 'archivist',

    # Philosophy
    'ethic', 'ethics', 'metaphysics', 'epistemology', 'epistemologies', 'logic', 'logics',
    'aesthetic', 'aesthetics', 'existentialism', 'phenomenology', 'morality', 'moralities',
    'political philosophy', 'critical theory',

    # Arts
    'drawing', 'drawings', 'fine art', 'fine arts', 'cinema', 'film', 'films', 'film studies',
    'photograph', 'photography', 'performance', 'performances', 'installation', 'installations',
    'art theory', 'design', 'designs', 'fashion', 'graphic design', 'animation', 'animations',

    # Culture
    'cultural studies', 'anthropology', 'mythology', 'heritage', 'ritual', 'rituals',
    'symbolism', 'folklore', 'ethnography', 'ethnographies', 'identity', 'identities',
    'tradition', 'traditions', 'intercultural', 'semiotic', 'semiotics',

    # Music & performing arts
    'composition', 'compositions', 'orchestra', 'orchestras', 'opera', 'operas', 'dance',
    'dances', 'choral', 'instrumental', 'jazz', 'sound study', 'sound studies',
    'ethnomusicology', 'musicology', 'notation', 'rhythm', 'rhythms',

    # Theatre & performance
    'acting', 'actor', 'actors', 'actress', 'actresses', 'playwriting', 'stage', 'stages',
    'drama', 'dramas', 'directing', 'direction', 'director', 'script', 'scripts',
    'costume', 'costumes', 'scene', 'scenes',

    # Visual communication
    'visual', 'illustration', 'illustrations', 'iconography', 'symbol', 'symbols', 'typography',

    # Digital humanities
    'digital humanities', 'computational humanities', 'archival digitization', 'text mining',

    # Campos generales
    'filosofía', 'filosofías', 'historia', 'historias', 'lengua', 'lenguas',
    'literatura', 'literaturas', 'arte', 'artes', 'cultura', 'culturas', 'estética', 'estéticas',
    'música', 'músicas', 'teatro', 'teatro', 'pinturas', 'pintura', 'escultura', 'esculturas',
    'arquitectura', 'arquitecturas', 'arte visual', 'artes visuales',

    # Literatura y lenguaje
    'poesía', 'poema', 'poemas', 'prosa', 'novela', 'novelas', 'drama', 'narrativa', 'narrativas',
    'ficción', 'no ficción', 'lingüística', 'semántica', 'semánticas', 'sintaxis', 'morfología',
    'fonética', 'fonéticas', 'filología', 'traducción', 'traducciones', 'retórica',
    'discursos', 'discurso', 'literatura comparada',

    # Historia
    'medieval', 'renacimiento', 'historia moderna', 'historia antigua', 'guerra mundial', 'guerras mundiales',
    'colonialismo', 'imperialismo', 'revolución', 'revoluciones', 'memoria histórica', 'memoria',
    'archivos', 'archivística', 'archivista',

    # Filosofía
    'ética', 'éticas', 'metafísica', 'epistemología', 'epistemologías', 'lógica', 'lógicas',
    'estética', 'estéticas', 'existencialismo', 'fenomenología', 'moralidad', 'moralidades',
    'filosofía política', 'teoría crítica',

    # Artes
    'dibujo', 'dibujos', 'bellas artes', 'bellas artes', 'cine', 'película', 'películas', 'estudios cinematográficos',
    'fotografía', 'fotografía', 'performance', 'performances', 'instalación', 'instalaciones',
    'teoría del arte', 'diseño', 'diseños', 'moda', 'diseño gráfico', 'animación', 'animaciones',

    # Cultura
    'estudios culturales', 'antropología', 'mitología', 'patrimonio', 'ritual', 'rituales',
    'simbolismo', 'folklore', 'etnografía', 'etnografías', 'identidad', 'identidades',
    'tradición', 'tradiciones', 'intercultural', 'semiótica', 'semióticas',

    # Música y artes escénicas
    'composición', 'composiciones', 'orquesta', 'orquestas', 'ópera', 'óperas', 'danza',
    'danzas', 'coral', 'instrumental', 'jazz', 'estudio del sonido', 'estudios del sonido',
    'etnomusicología', 'musicología', 'notación', 'ritmo', 'ritmos',

    # Teatro y performance
    'actuación', 'actor', 'actores', 'actriz', 'actrices', 'escritura dramática', 'escenario', 'escenarios',
    'drama', 'dramas', 'dirección', 'dirección', 'director', 'guion', 'guiones',
    'traje', 'trajes', 'escena', 'escenas',

    # Comunicación visual
    'visual', 'ilustración', 'ilustraciones', 'iconografía', 'símbolo', 'símbolos', 'tipografía',

    # Humanidades digitales
    'humanidades digitales', 'humanidades computacionales', 'digitalización archivística', 'minería de textos',

    # Campos gerais
    'filosofia', 'filosofias', 'história', 'histórias', 'língua', 'línguas',
    'literatura', 'literaturas', 'arte', 'artes', 'cultura', 'culturas', 'estética', 'estéticas',
    'música', 'músicas', 'teatro', 'teatro', 'pinturas', 'pintura', 'escultura', 'esculturas',
    'arquitetura', 'arquiteturas', 'arte visual', 'artes visuais',

    # Literatura e linguagem
    'poesia', 'poema', 'poemas', 'prosa', 'romance', 'romances', 'drama', 'narrativa', 'narrativas',
    'ficção', 'não-ficção', 'linguística', 'semântica', 'semânticas', 'sintaxe', 'morfologia',
    'fonética', 'fonéticas', 'filologia', 'tradução', 'traduções', 'retórica',
    'discursos', 'discurso', 'literatura comparada',

    # História
    'medieval', 'renascimento', 'história moderna', 'história antiga', 'guerra mundial', 'guerras mundiais',
    'colonialismo', 'imperialismo', 'revolução', 'revoluções', 'memória histórica', 'memória',
    'arquivos', 'arquivística', 'arquivista',

    # Filosofia
    'ética', 'éticas', 'metafísica', 'epistemologia', 'epistemologias', 'lógica', 'lógicas',
    'estética', 'estéticas', 'existencialismo', 'fenomenologia', 'moralidade', 'moralidades',
    'filosofia política', 'teoria crítica',

    # Artes
    'desenho', 'desenhos', 'belas artes', 'belas artes', 'cinema', 'filme', 'filmes', 'estudos de cinema',
    'fotografia', 'fotografia', 'performance', 'performances', 'instalação', 'instalações',
    'teoria da arte', 'design', 'designs', 'moda', 'design gráfico', 'animação', 'animações',

    # Cultura
    'estudos culturais', 'antropologia', 'mitologia', 'patrimônio', 'ritual', 'rituais',
    'simbolismo', 'folclore', 'etnografia', 'etnografias', 'identidade', 'identidades',
    'tradição', 'tradições', 'intercultural', 'semiótica', 'semióticas',

    # Música e artes performáticas
    'composição', 'composições', 'orquestra', 'orquestras', 'ópera', 'óperas', 'dança',
    'danças', 'coral', 'instrumental', 'jazz', 'estudo de som', 'estudos de som',
    'etnomusicologia', 'musicologia', 'notação', 'ritmo', 'ritmos',

    # Teatro e performance
    'ato', 'ator', 'atores', 'atriz', 'atrizes', 'roteiro teatral', 'palco', 'palcos',
    'drama', 'dramas', 'direção', 'direção', 'diretor', 'roteiro', 'roteiros',
    'fantasia', 'fantasias', 'cena', 'cenas',

    # Comunicação visual
    'visual', 'ilustração', 'ilustrações', 'iconografia', 'símbolo', 'símbolos', 'tipografia',

    # Humanidades digitais
    'humanidades digitais', 'humanidades computacionais', 'digitalização de arquivos', 'mineração de texto',

    # Aree generali
    'filosofia', 'filosofie', 'storia', 'storie', 'lingua', 'lingue',
    'letteratura', 'letterature', 'arte', 'arti', 'cultura', 'culture', 'estetica', 'estetiche',
    'musica', 'musiche', 'teatro', 'teatro', 'pitture', 'pittura', 'scultura', 'sculture',
    'architettura', 'architetture', 'arte visiva', 'arti visive',

    # Letteratura e linguistica
    'poesia', 'poesia', 'poemi', 'prosa', 'romanzo', 'romanzi', 'dramma', 'narrativa', 'narrazioni',
    'fiction', 'non fiction', 'linguistica', 'semantica', 'semantiche', 'sintassi', 'morfologia',
    'fonetica', 'fonetiche', 'filologia', 'traduzione', 'traduzioni', 'retorica',
    'discorsi', 'discorso', 'letteratura comparata',

    # Storia
    'medievale', 'rinascimento', 'storia moderna', 'storia antica', 'guerra mondiale', 'guerre mondiali',
    'colonialismo', 'imperialismo', 'rivoluzione', 'rivoluzioni', 'memoria storica', 'memoria',
    'archivi', 'archivistica', 'archivista',

    # Filosofia
    'etica', 'etiche', 'metafisica', 'epistemologia', 'epistemologie', 'logica', 'logiche',
    'estetica', 'estetiche', 'esistenzialismo', 'fenomenologia', 'moralità', 'moralità',
    'filosofia politica', 'teoria critica',

    # Arti
    'disegno', 'disegni', 'belle arti', 'belle arti', 'cinema', 'film', 'film', 'studi cinematografici',
    'fotografia', 'fotografia', 'performance', 'performances', 'installazione', 'installazioni',
    'teoria dell\'arte', 'design', 'design', 'moda', 'design grafico', 'animazione', 'animazioni',

    # Cultura
    'studi culturali', 'antropologia', 'mitologia', 'patrimonio', 'rito', 'riti',
    'simbolismo', 'folclore', 'etnografia', 'etnografie', 'identità', 'identità',
    'tradizione', 'tradizioni', 'interculturale', 'semiotica', 'semiotiche',

    # Musica e arti performative
    'composizione', 'composizioni', 'orchestra', 'orchestre', 'opera', 'opere', 'danza',
    'danze', 'corale', 'strumentale', 'jazz', 'studio del suono', 'studi del suono',
    'etnomusicologia', 'musicologia', 'notazione', 'ritmo', 'ritmi',

    # Teatro e performance
    'recitazione', 'attore', 'attori', 'attrice', 'attrici', 'scrittura teatrale', 'palco', 'palchi',
    'dramma', 'drami', 'direzione', 'direzione', 'direttore', 'copione', 'copioni',
    'costume', 'costumi', 'scena', 'scene',

    # Comunicazione visiva
    'visuale', 'illustrazione', 'illustrazioni', 'iconografia', 'simbolo', 'simboli', 'tipografia',

    # Umanistica digitale
    'umanistica digitale', 'umanistica computazionale', 'digitalizzazione archivistica', 'mining di testo',

    # Domaines généraux
    'philosophie', 'philosophies', 'histoire', 'histoires', 'langue', 'langues',
    'littérature', 'littératures', 'art', 'arts', 'culture', 'cultures', 'esthétique', 'esthétiques',
    'musique', 'musiques', 'théâtre', 'théâtre', 'peintures', 'peinture', 'sculpture', 'sculptures',
    'architecture', 'architectures', 'art visuel', 'arts visuels',

    # Littérature et langue
    'poésie', 'poème', 'poèmes', 'prose', 'roman', 'romans', 'drame', 'narration', 'narrations',
    'fiction', 'non-fiction', 'linguistique', 'sémantique', 'sémantique', 'syntaxe', 'morphologie',
    'phonétique', 'phonétiques', 'philatologie', 'traduction', 'traductions', 'rhétorique',
    'discours', 'discours', 'littérature comparée',

    # Histoire
    'médiéval', 'Renaissance', 'histoire moderne', 'histoire ancienne', 'guerre mondiale', 'guerres mondiales',
    'colonialisme', 'impérialisme', 'révolution', 'révolutions', 'mémoire historique', 'mémoire',
    'archives', 'archivistique', 'archiviste',

    # Philosophie
    'éthique', 'éthiques', 'métaphysique', 'épistémologie', 'épistémologies', 'logique', 'logiques',
    'esthétique', 'esthétiques', 'existentialisme', 'phénoménologie', 'moralité', 'moralités',
    'philosophie politique', 'théorie critique',

    # Arts
    'dessin', 'dessins', 'beaux-arts', 'beaux-arts', 'cinéma', 'film', 'films', 'études cinématographiques',
    'photographie', 'photographie', 'performance', 'performances', 'installation', 'installations',
    'théorie de l’art', 'design', 'designs', 'mode', 'design graphique', 'animation', 'animations',

    # Culture
    'études culturelles', 'anthropologie', 'mythologie', 'patrimoine', 'rituel', 'rituels',
    'symbolisme', 'folklore', 'ethnographie', 'ethnographies', 'identité', 'identités',
    'tradition', 'traditions', 'interculturel', 'sémiotique', 'sémiotiques',

    # Musique et arts du spectacle
    'composition', 'compositions', 'orchestre', 'orchestres', 'opéra', 'opéras', 'danse',
    'danses', 'chorale', 'instrumental', 'jazz', 'étude sonore', 'études sonores',
    'ethnomusicologie', 'musicologie', 'notation', 'rythme', 'rythmes',

    # Théâtre et performance
    'jeu', 'acteur', 'acteurs', 'actrice', 'actrices', 'écriture dramatique', 'scène', 'scènes',
    'drame', 'drames', 'réalisation', 'direction', 'réalisateur', 'scénario', 'scénarios',
    'costume', 'costumes', 'scène', 'scènes',

    # Communication visuelle
    'visuel', 'illustration', 'illustrations', 'iconographie', 'symbole', 'symboles', 'typographie',

    # Humanités numériques
    'humanités numériques', 'humanités computationnelles', 'numérisation des archives', 'text mining',

    # 一般领域
    '哲学', '哲学（复数）', '历史', '历史（复数）', '语言', '语言（复数）',
    '文学', '文学（复数）', '艺术', '艺术（复数）', '文化', '文化（复数）', '美学', '美学（复数）',
    '音乐', '音乐（复数）', '戏剧', '剧院', '绘画', '绘画（复数）', '雕塑', '雕塑（复数）',
    '建筑', '建筑（复数）', '视觉艺术', '视觉艺术（复数）',

    # 文学与语言
    '诗歌', '诗', '诗（复数）', '散文', '小说', '小说（复数）', '戏剧', '叙述', '叙述（复数）',
    '小说', '非虚构', '语言学', '语义', '语义学', '句法', '形态学',
    '语音学', '语音学（复数）', '语文学', '翻译', '翻译（复数）', '修辞',
    '话语', '话语（复数）', '比较文学',

    # 历史
    '中世纪', '文艺复兴', '现代历史', '古代历史', '世界大战', '世界大战（复数）',
    '殖民主义', '帝国主义', '革命', '革命（复数）', '历史记忆', '记忆',
    '档案', '档案学', '档案管理员',

    # 哲学
    '伦理', '伦理学', '形而上学', '认识论', '认识论（复数）', '逻辑', '逻辑（复数）',
    '美学', '美学（复数）', '存在主义', '现象学', '道德', '道德观',
    '政治哲学', '批判理论',

    # 艺术
    '绘画', '绘画（复数）', '美术', '美术（复数）', '电影', '电影（复数）', '电影研究',
    '摄影', '摄影（复数）', '表演', '表演（复数）', '装置艺术', '装置（复数）',
    '艺术理论', '设计', '设计（复数）', '时尚', '平面设计', '动画', '动画（复数）',

    # 文化
    '文化研究', '人类学', '神话学', '遗产', '仪式', '仪式（复数）',
    '符号学', '民俗学', '民族志', '民族志（复数）', '身份', '身份（复数）',
    '传统', '传统（复数）', '跨文化', '符号学', '符号学（复数）',

    # 音乐与表演艺术
    '作曲', '作曲（复数）', '交响乐', '交响乐团（复数）', '歌剧', '歌剧（复数）', '舞蹈',
    '舞蹈（复数）', '合唱', '器乐', '爵士乐', '声音研究', '声音研究（复数）',
    '民族音乐学', '音乐学', '音符', '节奏', '节奏（复数）',

    # 戏剧与表演
    '表演', '演员', '演员们', '女演员', '女演员们', '剧本创作', '舞台', '舞台（复数）',
    '戏剧', '戏剧（复数）', '导演', '导演（复数）', '导演（指人）', '剧本', '剧本（复数）',
    '服装', '服装（复数）', '场景', '场景（复数）',

    # 视觉传达
    '视觉', '插图', '插图（复数）', '图像学', '符号', '符号（复数）', '排版',

    # 数字人文学科
    '数字人文学科', '计算人文学科', '档案数字化', '文本挖掘',

    # Общие области
    'философия', 'философии', 'история', 'истории', 'язык', 'языки',
    'литература', 'литературы', 'искусство', 'искусства', 'культура', 'культуры', 'эстетика', 'эстетики',
    'музыка', 'музыки', 'театр', 'театры', 'картины', 'картина', 'скульптура', 'скульптуры',
    'архитектура', 'архитектуры', 'визуальное искусство', 'визуальные искусства',

    # Литература и язык
    'поэзия', 'стихотворение', 'стихи', 'проза', 'роман', 'романы', 'драма', 'повествование', 'повествования',
    'художественная литература', 'ненаучная литература', 'лингвистика', 'семантика', 'семантика', 'синтаксис', 'морфология',
    'фонетика', 'фонетика (множественное число)', 'филология', 'перевод', 'переводы', 'риторика',
    'дискурсы', 'дискурс', 'сравнительная литература',

    # История
    'средневековье', 'Ренессанс', 'современная история', 'древняя история', 'мировая война', 'мировые войны',
    'колониализм', 'империализм', 'революция', 'революции', 'историческая память', 'память',
    'архивы', 'архивирование', 'архивист',

    # Философия
    'этика', 'этики', 'метафизика', 'эпистемология', 'эпистемологии', 'логика', 'логики',
    'эстетика', 'эстетики', 'экзистенциализм', 'феноменология', 'мораль', 'морали',
    'политическая философия', 'критическая теория',

    # Искусства
    'рисование', 'рисунки', 'изобразительное искусство', 'изобразительные искусства', 'кино', 'фильм', 'фильмы', 'киноискусство',
    'фотография', 'фотография (множественное число)', 'перформанс', 'перформансы', 'инсталляция', 'инсталляции',
    'теория искусства', 'дизайн', 'дизайны', 'мода', 'графический дизайн', 'аниматика', 'аниматика (множественное число)',

    # Культура
    'культурология', 'антропология', 'мифология', 'наследие', 'ритуал', 'ритуалы',
    'символизм', 'фольклор', 'этнография', 'этнографии', 'идентичность', 'идентичности',
    'традиция', 'традиции', 'межкультурный', 'семиотика', 'семиотика (множественное число)',

    # Музыка и исполнительские искусства
    'композиция', 'композиции', 'оркестр', 'оркестры', 'опера', 'оперы', 'танец',
    'танцы', 'хоровой', 'инструментальный', 'джаз', 'звуковое исследование', 'звуковые исследования',
    'этномузыковедение', 'музыковедение', 'нотация', 'ритм', 'ритмы',

    # Театр и исполнительские искусства
    'актерское мастерство', 'актер', 'актеры', 'актриса', 'актрисы', 'сценарий', 'сцена', 'сцены',
    'драма', 'драмы', 'режиссура', 'направление', 'режиссер', 'сценарий', 'сценарии',
    'костюм', 'костюмы', 'сцена', 'сцены',

    # Визуальная коммуникация
    'визуальный', 'иллюстрация', 'иллюстрации', 'иконография', 'символ', 'символы', 'типография',

    # Цифровые гуманитарные науки
    'цифровые гуманитарные науки', 'вычислительные гуманитарные науки', 'цифровизация архивов', 'добыча текста',

    # 一般分野
    '哲学', '哲学 (複数)', '歴史', '歴史 (複数)', '言語', '言語 (複数)',
    '文学', '文学 (複数)', '芸術', '芸術 (複数)', '文化', '文化 (複数)', '美学', '美学 (複数)',
    '音楽', '音楽 (複数)', '演劇', '劇場', '絵画', '絵画 (複数)', '彫刻', '彫刻 (複数)',
    '建築', '建築 (複数)', '視覚芸術', '視覚芸術 (複数)',

    # 文学と言語
    '詩', '詩 (複数)', '詩歌', '散文', '小説', '小説 (複数)', 'ドラマ', '物語', '物語 (複数)',
    'フィクション', 'ノンフィクション', '言語学', '意味論', '意味論 (複数)', '構文論', '形態論',
    '音声学', '音声学 (複数)', '文献学', '翻訳', '翻訳 (複数)', '修辞学',
    '談話', '談話 (複数)', '比較文学',

    # 歴史
    '中世', 'ルネサンス', '近代史', '古代史', '世界大戦', '世界大戦 (複数)',
    '植民地主義', '帝国主義', '革命', '革命 (複数)', '歴史的記憶', '記憶',
    'アーカイブ', 'アーカイブの', 'アーカイブ管理者',

    # 哲学
    '倫理', '倫理学', '形而上学', '認識論', '認識論 (複数)', '論理学', '論理',
    '美学', '美学 (複数)', '実存主義', '現象学', '道徳', '道徳 (複数)',
    '政治哲学', '批判理論',

    # 芸術
    'ドローイング', 'ドローイング (複数)', '美術', '美術 (複数)', '映画', '映画 (複数)', '映画研究',
    '写真', '写真 (複数)', 'パフォーマンス', 'パフォーマンス (複数)', 'インスタレーション', 'インスタレーション (複数)',
    '芸術理論', 'デザイン', 'デザイン (複数)', 'ファッション', 'グラフィックデザイン', 'アニメーション', 'アニメーション (複数)',

    # 文化
    '文化研究', '人類学', '神話学', '遺産', '儀式', '儀式 (複数)',
    '象徴主義', '民俗学', '民族誌', '民族誌 (複数)', 'アイデンティティ', 'アイデンティティ (複数)',
    '伝統', '伝統 (複数)', '異文化間', '記号論', '記号論 (複数)',

    # 音楽と舞台芸術
    '作曲', '作曲 (複数)', 'オーケストラ', 'オーケストラ (複数)', 'オペラ', 'オペラ (複数)', 'ダンス',
    'ダンス (複数)', '合唱', '器楽', 'ジャズ', '音響学', '音響学 (複数)',
    '民族音楽学', '音楽学', '楽譜', 'リズム', 'リズム (複数)',

    # 演劇とパフォーマンス
    '演技', '俳優', '俳優たち', '女優', '女優たち', '脚本執筆', 'ステージ', 'ステージ (複数)',
    'ドラマ', 'ドラマ (複数)', '演出', '指揮', '監督', '脚本', '脚本 (複数)',
    '衣装', '衣装 (複数)', 'シーン', 'シーン (複数)',

    # 視覚コミュニケーション
    '視覚', 'イラスト', 'イラスト (複数)', 'アイコノグラフィー', 'シンボル', 'シンボル (複数)', 'タイポグラフィ',

    # デジタル人文学
    'デジタル人文学', '計算人文学', 'アーカイブのデジタル化', 'テキストマイニング',

    # 일반 분야
    '철학', '철학들', '역사', '역사들', '언어', '언어들',
    '문학', '문학들', '예술', '예술들', '문화', '문화들', '미학', '미학들',
    '음악', '음악들', '연극', '극장', '회화', '회화들', '조각', '조각들',
    '건축', '건축들', '시각 예술', '시각 예술들',

    # 문학과 언어
    '시', '시들', '시가', '산문', '소설', '소설들', '드라마', '서사', '서사들',
    '소설', '논픽션', '언어학', '의미론', '의미론들', '구문론', '형태론',
    '음성학', '음성학들', '문헌학', '번역', '번역들', '수사학',
    '담론', '담론들', '비교 문학',

    # 역사
    '중세', '르네상스', '근대 역사', '고대 역사', '세계 대전', '세계 대전들',
    '식민주의', '제국주의', '혁명', '혁명들', '역사적 기억', '기억',
    '아카이브', '아카이브의', '아카이브 관리자',

    # 철학
    '윤리', '윤리학', '형이상학', '인식론', '인식론들', '논리학', '논리들',
    '미학', '미학들', '실존주의', '현상학', '도덕', '도덕들',
    '정치철학', '비판이론',

    # 예술
    '드로잉', '드로잉들', '미술', '미술들', '영화', '영화들', '영화 연구',
    '사진', '사진들', '퍼포먼스', '퍼포먼스들', '설치미술', '설치미술들',
    '예술 이론', '디자인', '디자인들', '패션', '그래픽 디자인', '애니메이션', '애니메이션들',

    # 문화
    '문화 연구', '인류학', '신화학', '유산', '의식', '의식들',
    '상징주의', '민속학', '민족학', '민족학들', '정체성', '정체성들',
    '전통', '전통들', '문화 간', '기호학', '기호학들',

    # 음악 및 공연 예술
    '작곡', '작곡들', '오케스트라', '오케스트라들', '오페라', '오페라들', '댄스',
    '댄스들', '합창', '기악', '재즈', '음향학', '음향학들',
    '민족 음악학', '음악학', '악보', '리듬', '리듬들',

    # 연극 및 공연
    '연기', '배우', '배우들', '여배우', '여배우들', '극본 작성', '무대', '무대들',
    '드라마', '드라마들', '연출', '연출들', '감독', '스크립트', '스크립트들',
    '의상', '의상들', '장면', '장면들',

    # 시각 커뮤니케이션
    '시각', '일러스트', '일러스트들', '아이콘화', '기호', '기호들', '타이포그래피',

    # 디지털 인문학
    '디지털 인문학', '계산 인문학', '아카이브 디지털화', '텍스트 마이닝',

    # 一般領域
    '哲學', '哲學 (複數)', '歷史', '歷史 (複數)', '語言', '語言 (複數)',
    '文學', '文學 (複數)', '藝術', '藝術 (複數)', '文化', '文化 (複數)', '美學', '美學 (複數)',
    '音樂', '音樂 (複數)', '劇場', '劇院', '繪畫', '繪畫 (複數)', '雕塑', '雕塑 (複數)',
    '建築', '建築 (複數)', '視覺藝術', '視覺藝術 (複數)',

    # 文學與語言
    '詩', '詩篇', '詩歌', '散文', '小說', '小說 (複數)', '戲劇', '敘事', '敘事 (複數)',
    '小說', '非小說', '語言學', '語意學', '語意學 (複數)', '語法', '形態學',
    '語音學', '語音學 (複數)', '文獻學', '翻譯', '翻譯 (複數)', '修辭學',
    '言論', '言論 (複數)', '比較文學',

    # 歷史
    '中世紀', '文藝復興', '現代歷史', '古代歷史', '世界大戰', '世界大戰 (複數)',
    '殖民主義', '帝國主義', '革命', '革命 (複數)', '歷史記憶', '記憶',
    '檔案', '檔案的', '檔案管理員',

    # 哲學
    '倫理', '倫理學', '形而上學', '認識論', '認識論 (複數)', '邏輯學', '邏輯',
    '美學', '美學 (複數)', '存在主義', '現象學', '道德', '道德 (複數)',
    '政治哲學', '批判理論',

    # 藝術
    '繪畫', '繪畫 (複數)', '美術', '美術 (複數)', '電影', '電影 (複數)', '電影研究',
    '攝影', '攝影 (複數)', '表演', '表演 (複數)', '裝置藝術', '裝置藝術 (複數)',
    '藝術理論', '設計', '設計 (複數)', '時尚', '平面設計', '動畫', '動畫 (複數)',

    # 文化
    '文化研究', '人類學', '神話學', '遺產', '儀式', '儀式 (複數)',
    '象徵主義', '民俗學', '民族誌', '民族誌 (複數)', '身份', '身份 (複數)',
    '傳統', '傳統 (複數)', '跨文化', '符號學', '符號學 (複數)',

    # 音樂與表演藝術
    '作曲', '作曲 (複數)', '管弦樂', '管弦樂 (複數)', '歌劇', '歌劇 (複數)', '舞蹈',
    '舞蹈 (複數)', '合唱', '器樂', '爵士樂', '聲音學', '聲音學 (複數)',
    '民族音樂學', '音樂學', '音符', '節奏', '節奏 (複數)',

    # 戲劇與表演
    '演技', '演員', '演員們', '女演員', '女演員們', '劇本寫作', '舞台', '舞台 (複數)',
    '戲劇', '戲劇 (複數)', '導演', '導演們', '導演', '劇本', '劇本 (複數)',
    '服裝', '服裝 (複數)', '場景', '場景 (複數)',

    # 視覺傳播
    '視覺', '插畫', '插畫 (複數)', '圖像學', '符號', '符號 (複數)', '字體排版',

    # 數位人文學
    '數位人文學', '計算人文學', '檔案數位化', '文本挖掘',


    # Obecné oblasti
    'filosofie', 'filosofie (množné)', 'historie', 'historie (množné)', 'jazyk', 'jazyky',
    'literatura', 'literatury', 'umění', 'umění (množné)', 'kultura', 'kultury', 'estetika', 'estetika (množné)',
    'hudba', 'hudby', 'divadlo', 'teátr', 'obrazy', 'obraz', 'socha', 'sochy',
    'architektura', 'architektury', 'vizuální umění', 'vizuální umění (množné)',

    # Literatura a jazyk
    'poezie', 'básně', 'básně (množné)', 'proza', 'román', 'romány', 'drama', 'narativ', 'narativy',
    'fikce', 'non-fikce', 'lingvistika', 'sémantika', 'sémantiky', 'syntaxe', 'morfologie',
    'fonetika', 'fonetika (množné)', 'filologie', 'překlad', 'překlady', 'rétorika',
    'diskurzy', 'diskurz', 'komparativní literatura',

    # Historie
    'středověk', 'renesance', 'moderní historie', 'starověká historie', 'světová válka', 'světové války',
    'kolonialismus', 'imperialismus', 'revoluce', 'revoluce (množné)', 'historická paměť', 'paměť',
    'archivy', 'archivní', 'archivář',

    # Filosofie
    'etika', 'etika (množné)', 'metafyzika', 'epistemologie', 'epistemologie (množné)', 'logika', 'logiky',
    'estetika', 'estetika (množné)', 'existencialismus', 'fenomenologie', 'moralita', 'morality',
    'politická filozofie', 'kritická teorie',

    # Umění
    'kreslení', 'kresby', 'výtvarné umění', 'výtvarná umění', 'kinematografie', 'film', 'filmy', 'filmologie',
    'fotografie', 'fotografie (množné)', 'představení', 'představení (množné)', 'instalace', 'instalace (množné)',
    'teorie umění', 'design', 'designy', 'móda', 'grafický design', 'animace', 'animace (množné)',

    # Kultura
    'kulturní studie', 'antropologie', 'mytologie', 'dědictví', 'rituál', 'rituály',
    'symbolismus', 'folklór', 'etnografie', 'etnografie (množné)', 'identita', 'identity',
    'tradice', 'tradice (množné)', 'interkulturní', 'semiotika', 'semiotika (množné)',

    # Hudba a performativní umění
    'kompozice', 'kompozice (množné)', 'orchestr', 'orchestry', 'opera', 'opery', 'tanec',
    'tance', 'sbor', 'instrumentální', 'jazz', 'studium zvuku', 'studie zvuku',
    'etnomuzikologie', 'muzikologie', 'notace', 'rytmus', 'rytmus (množné)',

    # Divadlo a performance
    'herectví', 'herec', 'herci', 'herečka', 'herečky', 'scénář', 'scény',
    'drama', 'drama (množné)', 'režie', 'režie (množné)', 'režisér', 'scénář', 'scénáře',
    'kostým', 'kostýmy', 'scéna', 'scény',

    # Vizuální komunikace
    'vizuální', 'ilustrace', 'ilustrace (množné)', 'ikonografie', 'symbol', 'symboly', 'typografie',

    # Digitální humanitní vědy
    'digitální humanitní vědy', 'výpočetní humanitní vědy', 'digitální archivace', 'text mining',

    # المجالات العامة
    'الفلسفة', 'الفلسفات', 'التاريخ', 'التواريخ', 'اللغة', 'اللغات',
    'الأدب', 'الأدبيات', 'الفن', 'الفنون', 'الثقافة', 'الثقافات', 'الجماليات', 'الجماليات (جمع)',
    'الموسيقى', 'الموسيقى (جمع)', 'المسرح', 'المسرحيات', 'اللوحات', 'اللوحة', 'التمثال', 'التماثيل',
    'العمارة', 'العمارات', 'الفن البصري', 'الفنون البصرية',

    # الأدب واللغة
    'الشعر', 'القصيدة', 'القصائد', 'النثر', 'الرواية', 'الروايات', 'الدراما', 'السرد', 'السرديات',
    'الخيال', 'غير الخيال', 'اللغويات', 'الدلالة', 'الدلالات', 'النحو', 'التركيب',
    'علم الأصوات', 'علم الأصوات (جمع)', 'الفيلولوجيا', 'الترجمة', 'الترجمات', 'البلاغة',
    'الخطابات', 'الخطاب', 'الأدب المقارن',

    # التاريخ
    'العصور الوسطى', 'عصر النهضة', 'التاريخ الحديث', 'التاريخ القديم', 'الحرب العالمية', 'الحروب العالمية',
    'الاستعمار', 'الإمبريالية', 'الثورة', 'الثورات', 'الذاكرة التاريخية', 'الذاكرة',
    'الأرشيف', 'الأرشيفي', 'الأرشيفي',

    # الفلسفة
    'الأخلاق', 'الأخلاقيات', 'الماورائيات', 'نظرية المعرفة', 'نظريات المعرفة', 'المنطق', 'المنطقيات',
    'الجماليات', 'الجماليات (جمع)', 'الوجودية', 'الفينومينولوجيا', 'الأخلاقية', 'الأخلاقيات',
    'الفلسفة السياسية', 'نظرية النقد',

    # الفن
    'الرسم', 'الرسوم', 'الفن الجميل', 'الفنون الجميلة', 'السينما', 'الفيلم', 'الأفلام', 'دراسات الفيلم',
    'التصوير الفوتوغرافي', 'التصوير الفوتوغرافي', 'الأداء', 'الأداءات', 'التركيب الفني', 'التركيبات الفنية',
    'نظرية الفن', 'التصميم', 'التصاميم', 'الموضة', 'تصميم الجرافيك', 'الأنيميشن', 'الأنيميشنات',

    # الثقافة
    'دراسات ثقافية', 'الأنثروبولوجيا', 'الأساطير', 'التراث', 'الطقوس', 'الطقوسات',
    'الرمزية', 'الفلكلور', 'الإثنوجرافيا', 'الإثنوجرافيا (جمع)', 'الهوية', 'الهوات',
    'التقاليد', 'التقاليد (جمع)', 'التداخل الثقافي', 'السيميائية', 'السيميائيات',

    # الموسيقى وفنون الأداء
    'التأليف', 'التأليفات', 'الأوركسترا', 'الأوركسترات', 'الأوبرا', 'الأوبراهات', 'الرقص',
    'الرقصات', 'الفرقة', 'العزف', 'الجاز', 'دراسة الصوت', 'دراسات الصوت',
    'علم الموسيقى الإثني', 'علم الموسيقى', 'التدوين', 'الإيقاع', 'الإيقاعات',

    # المسرح وفنون الأداء
    'التمثيل', 'الممثل', 'الممثلون', 'المثلة', 'المثيلات', 'كتابة المسرح', 'المسرح', 'المسارح',
    'الدراما', 'الدرامات', 'الإخراج', 'التوجيه', 'المخرج', 'السيناريو', 'السيناريوهات',
    'الزي', 'الملابس', 'المشهد', 'المشاهد',

    # التواصل البصري
    'البصري', 'الرسوم التوضيحية', 'الرسوم التوضيحية (جمع)', 'الرمزية التصويرية', 'الرمز', 'الرموز', 'الطباعة',

    # العلوم الإنسانية الرقمية
    'العلوم الإنسانية الرقمية', 'العلوم الإنسانية الحسابية', 'الرقمنة الأرشيفية', 'تنقيب النصوص'
],

    'business': [
    # Core concepts
    'business', 'businesses', 'management', 'manager', 'managers', 'leadership', 'strategy',
    'strategies', 'operations', 'operation', 'entrepreneurship', 'entrepreneur', 'entrepreneurs',
    'organization', 'organizations', 'organizational behavior', 'corporate', 'enterprise', 'enterprises',

    # Finance & accounting
    'finance', 'financial', 'accounting', 'auditing', 'audit', 'investment', 'investments',
    'financial reporting', 'mergers', 'acquisitions', 'valuation', 'portfolio', 'portfolios',
    'equity', 'debt', 'risk management', 'risks', 'hedging', 'financing',

    # Marketing & sales
    'marketing', 'branding', 'brand', 'advertising', 'advertisement', 'promotion',
    'promotions', 'consumer behavior', 'consumers', 'digital marketing', 'pricing',
    'retail', 'retailing', 'segmentation', 'loyalty', 'e-commerce', 'sales', 'selling',

    # Human resources & workforce
    'human resources', 'hr', 'recruitment', 'hiring', 'talent management',
    'talent', 'compensation', 'labor', 'employee engagement', 'training',
    'development', 'workforce development', 'employees', 'staff',

    # Supply chain & logistics
    'supply chain', 'logistics', 'procurement', 'inventory', 'inventories',
    'distribution', 'warehousing', 'warehouse', 'transportation', 'sourcing',
    'demand planning', 'supply', 'suppliers',

    # International business
    'globalization', 'international trade', 'trade', 'export', 'exports',
    'import', 'imports', 'foreign direct investment', 'fdi', 'cross-cultural',
    'multinational', 'geopolitics',

    # Innovation & technology in business
    'innovation', 'innovations', 'startups', 'startup', 'digital transformation',
    'business analytics', 'analytics', 'erp', 'crm', 'blockchain',
    'business intelligence', 'agile', 'lean',

    # Business ethics and law
    'corporate governance', 'compliance', 'ethics', 'csr', 'corporate social responsibility',
    'sustainability', 'regulation', 'anti-corruption', 'transparency',

    # Economics-related
    'market', 'markets', 'competition', 'pricing strategy', 'monopoly', 'oligopoly',

    # Strategy & consulting
    'business model', 'business models', 'swot', 'benchmarking', 'consulting',
    'consultant', 'consultants', 'change management', 'organizational change',

    # Miscellaneous
    'kpi', 'kpis', 'roi', 'return on investment', 'stakeholders', 'stakeholder',
    'scalability', 'scalable', 'growth strategy', 'value chain',

    # Conceptos básicos
    'negocio', 'negocios', 'gestión', 'gerente', 'gerentes', 'liderazgo', 'estrategia',
    'estrategias', 'operaciones', 'operación', 'emprendimiento', 'emprendedor', 'emprendedores',
    'organización', 'organizaciones', 'comportamiento organizacional', 'corporativo', 'empresa', 'empresas',

    # Finanzas y contabilidad
    'finanzas', 'financiero', 'contabilidad', 'auditoría', 'auditar', 'inversión', 'inversiones',
    'informes financieros', 'fusiones', 'adquisiciones', 'valoración', 'cartera', 'carteras',
    'capital', 'deuda', 'gestión de riesgos', 'riesgos', 'cobertura', 'financiación',

    # Marketing y ventas
    'marketing', 'marca', 'marca', 'publicidad', 'anuncio', 'promoción',
    'promociones', 'comportamiento del consumidor', 'consumidores', 'marketing digital', 'precios',
    'venta al por menor', 'comercio minorista', 'segmentación', 'lealtad', 'comercio electrónico', 'ventas', 'venta',

    # Recursos humanos y fuerza laboral
    'recursos humanos', 'rrhh', 'reclutamiento', 'contratación', 'gestión del talento',
    'talento', 'compensación', 'trabajo', 'compromiso del empleado', 'capacitación',
    'desarrollo', 'desarrollo de la fuerza laboral', 'empleados', 'personal',

    # Cadena de suministro y logística
    'cadena de suministro', 'logística', 'adquisición', 'inventario', 'inventarios',
    'distribución', 'almacenaje', 'almacén', 'transporte', 'abastecimiento',
    'planificación de la demanda', 'suministro', 'proveedores',

    # Negocios internacionales
    'globalización', 'comercio internacional', 'comercio', 'exportación', 'exportaciones',
    'importación', 'importaciones', 'inversión extranjera directa', 'ied', 'intercultural',
    'multinacional', 'geopolítica',

    # Innovación y tecnología en los negocios
    'innovación', 'innovaciones', 'startups', 'startup', 'transformación digital',
    'analítica empresarial', 'analítica', 'erp', 'crm', 'blockchain',
    'inteligencia empresarial', 'ágil', 'lean',

    # Ética empresarial y derecho
    'gobernanza corporativa', 'cumplimiento', 'ética', 'rse', 'responsabilidad social empresarial',
    'sostenibilidad', 'regulación', 'anticorrupción', 'transparencia',

    # Relacionado con la economía
    'mercado', 'mercados', 'competencia', 'estrategia de precios', 'monopolio', 'oligopolio',

    # Estrategia y consultoría
    'modelo de negocio', 'modelos de negocio', 'foda', 'benchmarking', 'consultoría',
    'consultor', 'consultores', 'gestión del cambio', 'cambio organizacional',

    # Varios
    'kpi', 'kpis', 'roi', 'retorno de la inversión', 'partes interesadas', 'parte interesada',
    'escalabilidad', 'escalable', 'estrategia de crecimiento', 'cadena de valor',

    # Conceitos principais
    'negócio', 'negócios', 'gestão', 'gerente', 'gerentes', 'liderança', 'estratégia',
    'estratégias', 'operações', 'operação', 'empreendedorismo', 'empreendedor', 'empreendedores',
    'organização', 'organizações', 'comportamento organizacional', 'corporativo', 'empresa', 'empresas',

    # Finanças e contabilidade
    'finanças', 'financeiro', 'contabilidade', 'auditoria', 'auditar', 'investimento', 'investimentos',
    'relatórios financeiros', 'fusões', 'aquisições', 'avaliação', 'carteira', 'carteiras',
    'capital', 'dívida', 'gestão de riscos', 'riscos', 'hedge', 'financiamento',

    # Marketing e vendas
    'marketing', 'marca', 'marca', 'publicidade', 'anúncio', 'promoção',
    'promoções', 'comportamento do consumidor', 'consumidores', 'marketing digital', 'preço',
    'varejo', 'vendas', 'segmentação', 'lealdade', 'e-commerce', 'vendas', 'venda',

    # Recursos humanos e força de trabalho
    'recursos humanos', 'rh', 'recrutamento', 'contratação', 'gestão de talentos',
    'talento', 'compensação', 'trabalho', 'engajamento dos funcionários', 'treinamento',
    'desenvolvimento', 'desenvolvimento da força de trabalho', 'funcionários', 'equipe',

    # Cadeia de suprimentos e logística
    'cadeia de suprimentos', 'logística', 'aquisição', 'inventário', 'estoques',
    'distribuição', 'armazenamento', 'armazém', 'transporte', 'abastecimento',
    'planejamento de demanda', 'suprimento', 'fornecedores',

    # Negócios internacionais
    'globalização', 'comércio internacional', 'comércio', 'exportação', 'exportações',
    'importação', 'importações', 'investimento estrangeiro direto', 'ied', 'intercultural',
    'multinacional', 'geopolítica',

    # Inovação e tecnologia nos negócios
    'inovação', 'inovações', 'startups', 'startup', 'transformação digital',
    'análise de negócios', 'análise', 'erp', 'crm', 'blockchain',
    'inteligência de negócios', 'ágil', 'lean',

    # Ética empresarial e lei
    'governança corporativa', 'conformidade', 'ética', 'rse', 'responsabilidade social corporativa',
    'sustentabilidade', 'regulação', 'anticorrupção', 'transparência',

    # Relacionado à economia
    'mercado', 'mercados', 'concorrência', 'estratégia de preços', 'monopólio', 'oligopólio',

    # Estratégia e consultoria
    'modelo de negócio', 'modelos de negócio', 'swot', 'benchmarking', 'consultoria',
    'consultor', 'consultores', 'gestão de mudanças', 'mudança organizacional',

    # Diversos
    'kpi', 'kpis', 'roi', 'retorno sobre investimento', 'partes interessadas', 'parte interessada',
    'escalabilidade', 'escalável', 'estratégia de crescimento', 'cadeia de valor',

    # Concetti principali
    'business', 'aziende', 'gestione', 'manager', 'manager', 'leadership', 'strategia',
    'strategie', 'operazioni', 'operazione', 'imprenditorialità', 'imprenditore', 'imprenditori',
    'organizzazione', 'organizzazioni', 'comportamento organizzativo', 'corporativo', 'impresa', 'imprese',

    # Finanza e contabilità
    'finanza', 'finanziario', 'contabilità', 'auditing', 'audit', 'investimento', 'investimenti',
    'reporting finanziario', 'fusioni', 'acquisizioni', 'valutazione', 'portafoglio', 'portafogli',
    'capitale', 'debito', 'gestione dei rischi', 'rischi', 'copertura', 'finanziamento',

    # Marketing e vendite
    'marketing', 'brand', 'marca', 'pubblicità', 'annuncio', 'promozione',
    'promozioni', 'comportamento del consumatore', 'consumatori', 'marketing digitale', 'prezzo',
    'retail', 'vendita al dettaglio', 'segmentazione', 'fedeltà', 'e-commerce', 'vendite', 'vendita',

    # Risorse umane e forza lavoro
    'risorse umane', 'hr', 'reclutamento', 'assunzione', 'gestione dei talenti',
    'talento', 'compensazione', 'lavoro', 'coinvolgimento dei dipendenti', 'formazione',
    'sviluppo', 'sviluppo della forza lavoro', 'dipendenti', 'personale',

    # Catena di approvvigionamento e logistica
    'catena di approvvigionamento', 'logistica', 'approvvigionamento', 'inventario', 'scorte',
    'distribuzione', 'magazzinaggio', 'magazzino', 'trasporto', 'approvvigionamento',
    'pianificazione della domanda', 'approvvigionamento', 'fornitori',

    # Affari internazionali
    'globalizzazione', 'commercio internazionale', 'commercio', 'esportazione', 'esportazioni',
    'importazione', 'importazioni', 'investimento diretto estero', 'fdi', 'interculturale',
    'multinazionale', 'geopolitica',

    # Innovazione e tecnologia negli affari
    'innovazione', 'innovazioni', 'startup', 'startup', 'trasformazione digitale',
    'analisi aziendale', 'analisi', 'erp', 'crm', 'blockchain',
    'intelligenza aziendale', 'agile', 'lean',

    # Etica aziendale e diritto
    'governance aziendale', 'conformità', 'etica', 'csr', 'responsabilità sociale d\'impresa',
    'sostenibilità', 'regolamentazione', 'anticorruzione', 'trasparenza',

    # Relazioni con l'economia
    'mercato', 'mercati', 'concorrenza', 'strategia dei prezzi', 'monopolio', 'oligopolio',

    # Strategia e consulenza
    'modello di business', 'modelli di business', 'swot', 'benchmarking', 'consulenza',
    'consulente', 'consulenti', 'gestione del cambiamento', 'cambiamento organizzativo',

    # Varie
    'kpi', 'kpis', 'roi', 'ritorno sugli investimenti', 'stakeholder', 'parte interessata',
    'scalabilità', 'scalabile', 'strategia di crescita', 'catena del valore',

    # Concepts de base
    'entreprise', 'entreprises', 'gestion', 'gestionnaire', 'gestionnaires', 'leadership', 'stratégie',
    'stratégies', 'opérations', 'opération', 'entrepreneuriat', 'entrepreneur', 'entrepreneurs',
    'organisation', 'organisations', 'comportement organisationnel', 'entreprise', 'entreprise', 'entreprises',

    # Finance & comptabilité
    'finance', 'financier', 'comptabilité', 'audit', 'auditer', 'investissement', 'investissements',
    'rapport financier', 'fusions', 'acquisitions', 'évaluation', 'portefeuille', 'portefeuilles',
    'équité', 'dette', 'gestion des risques', 'risques', 'couverture', 'financement',

    # Marketing & ventes
    'marketing', 'branding', 'marque', 'publicité', 'annonce', 'promotion',
    'promotions', 'comportement des consommateurs', 'consommateurs', 'marketing digital', 'tarification',
    'commerce de détail', 'vente au détail', 'segmentation', 'fidélité', 'commerce électronique', 'ventes', 'vente',

    # Ressources humaines & main-d'œuvre
    'ressources humaines', 'rh', 'recrutement', 'embauche', 'gestion des talents',
    'talent', 'compensation', 'travail', 'engagement des employés', 'formation',
    'développement', 'développement de la main-d\'œuvre', 'employés', 'personnel',

    # Chaîne d'approvisionnement & logistique
    'chaîne d\'approvisionnement', 'logistique', 'approvisionnement', 'inventaire', 'inventaires',
    'distribution', 'entreposage', 'entrepôt', 'transport', 'approvisionnement',
    'planification de la demande', 'offre', 'fournisseurs',

    # Commerce international
    'mondialisation', 'commerce international', 'commerce', 'exportation', 'exportations',
    'importation', 'importations', 'investissement direct étranger', 'fdi', 'cross-culturel',
    'multinationale', 'géopolitique',

    # Innovation & technologie en affaires
    'innovation', 'innovations', 'startups', 'startup', 'transformation numérique',
    'analytique des affaires', 'analytique', 'erp', 'crm', 'blockchain',
    'intelligence d’affaires', 'agile', 'lean',

    # Éthique des affaires et droit
    'gouvernance d’entreprise', 'conformité', 'éthique', 'rsr', 'responsabilité sociale des entreprises',
    'durabilité', 'régulation', 'anticorruption', 'transparence',

    # Lié à l’économie
    'marché', 'marchés', 'concurrence', 'stratégie de prix', 'monopole', 'oligopole',

    # Stratégie & conseil
    'modèle d’entreprise', 'modèles d’entreprise', 'swot', 'étalonnage', 'conseil',
    'consultant', 'consultants', 'gestion du changement', 'changement organisationnel',

    # Divers
    'kpi', 'kpis', 'roi', 'retour sur investissement', 'parties prenantes', 'partie prenante',
    'évolutivité', 'scalable', 'stratégie de croissance', 'chaîne de valeur',

    # 核心概念
    '商业', '企业', '管理', '经理', '经理们', '领导力', '战略',
    '战略们', '运营', '操作', '创业', '创业者', '企业家们',
    '组织', '组织们', '组织行为', '企业', '公司', '企业们',

    # 财务与会计
    '财务', '金融', '会计', '审计', '审查', '投资', '投资们',
    '财务报告', '并购', '收购', '估值', '投资组合', '投资组合们',
    '股权', '债务', '风险管理', '风险们', '对冲', '融资',

    # 市场营销与销售
    '市场营销', '品牌塑造', '品牌', '广告', '广告宣传', '促销',
    '促销活动', '消费者行为', '消费者们', '数字营销', '定价',
    '零售', '零售业', '市场细分', '忠诚度', '电子商务', '销售', '售卖',

    # 人力资源与劳动力
    '人力资源', 'hr', '招聘', '雇佣', '人才管理',
    '人才', '薪酬', '劳动', '员工参与', '培训',
    '发展', '劳动力发展', '员工们', '员工',

    # 供应链与物流
    '供应链', '物流', '采购', '库存', '库存们',
    '分销', '仓储', '仓库', '运输', '采购',
    '需求规划', '供应', '供应商们',

    # 国际商业
    '全球化', '国际贸易', '贸易', '出口', '出口们',
    '进口', '进口们', '外国直接投资', 'fdi', '跨文化',
    '跨国公司', '地缘政治',

    # 商业中的创新与技术
    '创新', '创新们', '初创公司', '初创企业', '数字化转型',
    '商业分析', '分析', 'erp', 'crm', '区块链',
    '商业智能', '敏捷', '精益',

    # 商业伦理与法律
    '公司治理', '合规', '伦理', '企业社会责任', '企业社会责任',
    '可持续性', '监管', '反腐败', '透明度',

    # 经济相关
    '市场', '市场们', '竞争', '定价策略', '垄断', '寡头市场',

    # 战略与咨询
    '商业模式', '商业模式们', 'swot', '基准测试', '咨询',
    '顾问', '顾问们', '变革管理', '组织变革',

    # 其他
    'kpi', 'kpis', 'roi', '投资回报率', '利益相关者', '利益相关者们',
    '可扩展性', '可扩展的', '增长战略', '价值链',

    # Основные концепции
    'бизнес', 'предприятия', 'менеджмент', 'менеджер', 'менеджеры', 'лидерство', 'стратегия',
    'стратегии', 'операции', 'операция', 'предпринимательство', 'предприниматель', 'предприниматели',
    'организация', 'организации', 'организационное поведение', 'корпоративный', 'предприятие', 'предприятия',

    # Финансы и бухгалтерия
    'финансы', 'финансовый', 'бухгалтерия', 'аудит', 'аудирование', 'инвестиции', 'инвестиции',
    'финансовая отчетность', 'слияния', 'поглощения', 'оценка', 'портфель', 'портфели',
    'акции', 'долг', 'управление рисками', 'риски', 'хеджирование', 'финансирование',

    # Маркетинг и продажи
    'маркетинг', 'брендинг', 'бренд', 'реклама', 'рекламное объявление', 'продвижение',
    'продажи', 'поведение потребителей', 'потребители', 'цифровой маркетинг', 'ценообразование',
    'розничная торговля', 'ретейлинг', 'сегментация', 'лояльность', 'электронная торговля', 'продажа', 'продажа',

    # Человеческие ресурсы и рабочая сила
    'человеческие ресурсы', 'hr', 'набор', 'нанимать', 'управление талантами',
    'талант', 'компенсация', 'труд', 'вовлеченность сотрудников', 'обучение',
    'развитие', 'развитие рабочей силы', 'сотрудники', 'персонал',

    # Цепочка поставок и логистика
    'цепочка поставок', 'логистика', 'закупки', 'инвентаризация', 'инвентаризация',
    'распределение', 'складирование', 'склад', 'транспортировка', 'источник',
    'планирование спроса', 'поставки', 'поставщики',

    # Международный бизнес
    'глобализация', 'международная торговля', 'торговля', 'экспорт', 'экспорт',
    'импорт', 'импорт', 'прямые иностранные инвестиции', 'fdi', 'кросс-культурный',
    'многонациональный', 'геополитика',

    # Инновации и технологии в бизнесе
    'инновации', 'инновации', 'стартапы', 'стартап', 'цифровая трансформация',
    'бизнес-аналитика', 'аналитика', 'erp', 'crm', 'блокчейн',
    'бизнес-аналитика', 'гибкость', 'бережливое',

    # Этика бизнеса и право
    'корпоративное управление', 'соответствие', 'этика', 'csr', 'корпоративная социальная ответственность',
    'устойчивость', 'регулирование', 'антикоррупция', 'прозрачность',

    # Связано с экономикой
    'рынок', 'рынки', 'конкуренция', 'ценовая стратегия', 'монополия', 'олигополия',

    # Стратегия и консалтинг
    'бизнес-модель', 'бизнес-модели', 'swot', 'бенчмаркинг', 'консалтинг',
    'консультант', 'консультанты', 'управление изменениями', 'организационные изменения',

    # Разное
    'kpi', 'kpis', 'roi', 'рентабельность инвестиций', 'заинтересованные стороны', 'заинтересованная сторона',
    'масштабируемость', 'масштабируемый', 'стратегия роста', 'цепочка ценностей',

    # コア概念
    'ビジネス', '企業', '経営', 'マネージャー', 'マネージャーたち', 'リーダーシップ', '戦略',
    '戦略たち', '運営', '運営の', '起業家精神', '起業家', '起業家たち',
    '組織', '組織たち', '組織行動', '企業の', '企業', '企業たち',

    # 財務と会計
    '財務', '財務の', '会計', '監査', '監査を行う', '投資', '投資たち',
    '財務報告', '合併', '買収', '評価', 'ポートフォリオ', 'ポートフォリオたち',
    '資本', '負債', 'リスク管理', 'リスク', 'ヘッジング', '資金調達',

    # マーケティングと販売
    'マーケティング', 'ブランディング', 'ブランド', '広告', '広告物', 'プロモーション',
    'プロモーションたち', '消費者行動', '消費者たち', 'デジタルマーケティング', '価格設定',
    '小売', '小売業', 'セグメンテーション', 'ロイヤリティ', '電子商取引', '販売', '販売する',

    # 人事と労働力
    '人事', 'HR', '採用', '雇用', '人材管理',
    '人材', '報酬', '労働', '従業員エンゲージメント', 'トレーニング',
    '開発', '労働力開発', '従業員たち', 'スタッフ',

    # サプライチェーンとロジスティクス
    'サプライチェーン', 'ロジスティクス', '調達', '在庫', '在庫たち',
    '流通', '倉庫業', '倉庫', '輸送', '調達先',
    '需要予測', '供給', '供給者たち',

    # 国際ビジネス
    'グローバリゼーション', '国際貿易', '貿易', '輸出', '輸出品',
    '輸入', '輸入品', '外国直接投資', 'FDI', '異文化間', 
    '多国籍', '地政学',

    # ビジネスにおける革新と技術
    '革新', '革新たち', 'スタートアップ', 'スタートアップ企業', 'デジタルトランスフォーメーション',
    'ビジネス分析', '分析', 'ERP', 'CRM', 'ブロックチェーン',
    'ビジネスインテリジェンス', 'アジャイル', 'リーン',

    # ビジネス倫理と法律
    'コーポレートガバナンス', 'コンプライアンス', '倫理', 'CSR', '企業の社会的責任',
    '持続可能性', '規制', '反腐敗', '透明性',

    # 経済関連
    '市場', '市場たち', '競争', '価格戦略', '独占', '寡占',

    # 戦略とコンサルティング
    'ビジネスモデル', 'ビジネスモデルたち', 'SWOT', 'ベンチマーキング', 'コンサルティング',
    'コンサルタント', 'コンサルタントたち', '変革管理', '組織変革',

    # その他
    'KPI', 'KPIたち', 'ROI', '投資収益率', '利害関係者たち', '利害関係者',
    'スケーラビリティ', 'スケーラブル', '成長戦略', 'バリューチェーン',

    # 핵심 개념
    '비즈니스', '기업', '경영', '매니저', '매니저들', '리더십', '전략',
    '전략들', '운영', '운영의', '기업가정신', '기업가', '기업가들',
    '조직', '조직들', '조직행동', '기업의', '기업', '기업들',

    # 재무 및 회계
    '재무', '재무의', '회계', '감사', '감사를 하다', '투자', '투자들',
    '재무보고', '합병', '인수', '평가', '포트폴리오', '포트폴리오들',
    '자본', '부채', '위험 관리', '위험들', '헤징', '자금 조달',

    # 마케팅 및 판매
    '마케팅', '브랜딩', '브랜드', '광고', '광고물', '프로모션',
    '프로모션들', '소비자 행동', '소비자들', '디지털 마케팅', '가격 책정',
    '소매', '소매업', '세분화', '충성도', '전자 상거래', '판매', '판매하기',

    # 인사 및 인력
    '인사', 'HR', '채용', '고용', '인재 관리',
    '인재', '보상', '노동', '직원 참여', '훈련',
    '개발', '인력 개발', '직원들', '직원',

    # 공급망 및 물류
    '공급망', '물류', '조달', '재고', '재고들',
    '유통', '창고업', '창고', '운송', '조달원',
    '수요 계획', '공급', '공급자들',

    # 국제 비즈니스
    '글로벌화', '국제 무역', '무역', '수출', '수출품',
    '수입', '수입품', '외국인 직접 투자', 'FDI', '문화 간',
    '다국적', '지정학',

    # 비즈니스 혁신 및 기술
    '혁신', '혁신들', '스타트업', '스타트업 기업', '디지털 전환',
    '비즈니스 분석', '분석', 'ERP', 'CRM', '블록체인',
    '비즈니스 인텔리전스', '애자일', '린',

    # 비즈니스 윤리 및 법률
    '기업 거버넌스', '컴플라이언스', '윤리', 'CSR', '기업의 사회적 책임',
    '지속 가능성', '규제', '반 부패', '투명성',

    # 경제학 관련
    '시장', '시장들', '경쟁', '가격 전략', '독점', '과점',

    # 전략 및 컨설팅
    '비즈니스 모델', '비즈니스 모델들', 'SWOT', '벤치마킹', '컨설팅',
    '컨설턴트', '컨설턴트들', '변화 관리', '조직 변화',

    # 기타
    'KPI', 'KPI들', 'ROI', '투자 수익률', '이해관계자들', '이해관계자',
    '확장성', '확장 가능한', '성장 전략', '가치 사슬',

    # 核心概念
    '商業', '企業', '管理', '經理', '經理們', '領導力', '策略',
    '策略們', '運營', '運營的', '創業精神', '創業者', '創業者們',
    '組織', '組織們', '組織行為', '企業的', '企業', '企業們',

    # 財務與會計
    '財務', '財務的', '會計', '審計', '審計進行', '投資', '投資們',
    '財務報告', '合併', '收購', '估值', '投資組合', '投資組合們',
    '股本', '負債', '風險管理', '風險們', '對沖', '資金籌集',

    # 行銷與銷售
    '行銷', '品牌化', '品牌', '廣告', '廣告物', '促銷',
    '促銷們', '消費者行為', '消費者們', '數位行銷', '定價',
    '零售', '零售業', '市場區隔', '忠誠度', '電子商務', '銷售', '銷售中',

    # 人力資源與員工
    '人力資源', 'HR', '招聘', '雇用', '人才管理',
    '人才', '薪酬', '勞動', '員工參與', '訓練',
    '發展', '員工發展', '員工們', '員工',

    # 供應鏈與物流
    '供應鏈', '物流', '採購', '庫存', '庫存們',
    '分銷', '倉儲業務', '倉庫', '運輸', '採購來源',
    '需求計劃', '供應', '供應商們',

    # 國際商業
    '全球化', '國際貿易', '貿易', '出口', '出口品',
    '進口', '進口品', '外國直接投資', 'FDI', '跨文化',
    '跨國', '地緣政治',

    # 商業創新與技術
    '創新', '創新們', '新創企業', '新創公司', '數位轉型',
    '商業分析', '分析', 'ERP', 'CRM', '區塊鏈',
    '商業智慧', '敏捷', '精益',

    # 商業倫理與法律
    '企業治理', '合規性', '倫理', '企業社會責任', 'CSR',
    '可持續性', '規範', '反貪腐', '透明性',

    # 經濟學相關
    '市場', '市場們', '競爭', '定價策略', '壟斷', '寡頭市場',

    # 策略與諮詢
    '商業模式', '商業模式們', 'SWOT', '基準測試', '諮詢',
    '顧問', '顧問們', '變革管理', '組織變革',

    # 其他
    'KPI', 'KPI們', 'ROI', '投資回報率', '利益相關者們', '利益相關者',
    '可擴展性', '可擴展的', '增長策略', '價值鏈',

    # Klíčové pojmy
    'byznys', 'firmy', 'management', 'manažer', 'manažeři', 'vedení', 'strategie',
    'strategie', 'operace', 'operace', 'podnikání', 'podnikatel', 'podnikatelé',
    'organizace', 'organizace', 'organizační chování', 'korporátní', 'podnik', 'podniky',

    # Finance & účetnictví
    'finance', 'finanční', 'účetnictví', 'audit', 'auditing', 'investice', 'investice',
    'finanční reportování', 'fúze', 'akvizice', 'ocenění', 'portfolia', 'portfolia',
    'kapitál', 'dluh', 'řízení rizik', 'rizika', 'hedging', 'financování',

    # Marketing & prodej
    'marketing', 'branding', 'značka', 'reklama', 'reklama', 'propagace',
    'propagace', 'chování spotřebitelů', 'spotřebitelé', 'digitální marketing', 'cenotvorba',
    'maloobchod', 'maloobchod', 'segmentace', 'věrnost', 'e-commerce', 'prodej', 'prodej',

    # Lidské zdroje & pracovní síla
    'lidské zdroje', 'HR', 'rekrutování', 'najímání', 'řízení talentů',
    'talent', 'kompenzace', 'práce', 'angažovanost zaměstnanců', 'školení',
    'rozvoj', 'rozvoj pracovní síly', 'zaměstnanci', 'personál',

    # Dodavatelský řetězec & logistika
    'dodavatelský řetězec', 'logistika', 'nákup', 'skladování', 'sklady',
    'distribuce', 'skladování', 'sklad', 'doprava', 'sourcing',
    'plánování poptávky', 'nabídka', 'dodavatelé',

    # Mezinárodní obchod
    'globalizace', 'mezinárodní obchod', 'obchod', 'export', 'exporty',
    'import', 'importy', 'přímé zahraniční investice', 'FDI', 'mezikulturní',
    'nadnárodní', 'geopolitika',

    # Inovace & technologie v byznysu
    'inovace', 'inovace', 'startupy', 'startup', 'digitální transformace',
    'byznysová analýza', 'analýza', 'ERP', 'CRM', 'blockchain',
    'byznysová inteligence', 'agilní', 'štíhlý',

    # Byznysové etiky a právo
    'korporátní governance', 'shoda', 'etika', 'CSR', 'společenská odpovědnost firem',
    'udržitelnost', 'regulace', 'protikorupce', 'transparentnost',

    # Ekonomika
    'trh', 'trhy', 'konkurence', 'cenová strategie', 'monopol', 'oligopol',

    # Strategie & poradenství
    'byznys model', 'byznys modely', 'SWOT', 'benchmarking', 'poradenství',
    'konzultant', 'konzultanti', 'řízení změn', 'organizační změna',

    # Různé
    'KPI', 'KPI', 'ROI', 'návratnost investice', 'stakeholdeři', 'stakeholder',
    'škálovatelnost', 'škálovatelné', 'strategie růstu', 'hodnotový řetězec',

    # المفاهيم الأساسية
    'العمل', 'الشركات', 'الإدارة', 'المدير', 'المديرين', 'القيادة', 'الاستراتيجية',
    'الاستراتيجيات', 'العمليات', 'العملية', 'ريادة الأعمال', 'رائد الأعمال', 'رواد الأعمال',
    'التنظيم', 'التنظيمات', 'السلوك التنظيمي', 'الشركات', 'المؤسسة', 'المؤسسات',

    # المالية والمحاسبة
    'المالية', 'المالي', 'المحاسبة', 'التدقيق', 'التدقيقات', 'الاستثمار', 'الاستثمارات',
    'التقارير المالية', 'الاندماجات', 'الاستحواذات', 'التقييم', 'المحفظة', 'المحافظ',
    'رأس المال', 'الدين', 'إدارة المخاطر', 'المخاطر', 'التحوط', 'التمويل',

    # التسويق والمبيعات
    'التسويق', 'العلامة التجارية', 'العلامة', 'الإعلانات', 'الإعلان', 'الترويج',
    'الترويجيات', 'سلوك المستهلك', 'المستهلكين', 'التسويق الرقمي', 'التسعير',
    'التجزئة', 'التجزئة', 'التقسيم', 'الولاء', 'التجارة الإلكترونية', 'المبيعات', 'البيع',

    # الموارد البشرية والقوى العاملة
    'الموارد البشرية', 'HR', 'التوظيف', 'الاستعانة', 'إدارة المواهب',
    'المواهب', 'التعويضات', 'العمل', 'مشاركة الموظفين', 'التدريب',
    'التطوير', 'تطوير القوى العاملة', 'الموظفين', 'الطاقم',

    # سلسلة الإمداد واللوجستيات
    'سلسلة الإمداد', 'اللوجستيات', 'الشراء', 'المخزون', 'المخزونات',
    'التوزيع', 'التخزين', 'المستودع', 'النقل', 'المصادر',
    'تخطيط الطلب', 'الإمداد', 'الموردين',

    # الأعمال الدولية
    'العولمة', 'التجارة الدولية', 'التجارة', 'التصدير', 'الصادرات',
    'الاستيراد', 'الواردات', 'الاستثمار المباشر الأجنبي', 'FDI', 'عبر الثقافات',
    'متعدد الجنسيات', 'الجغرافيا السياسية',

    # الابتكار والتكنولوجيا في الأعمال
    'الابتكار', 'الابتكارات', 'الشركات الناشئة', 'الشركة الناشئة', 'التحول الرقمي',
    'تحليلات الأعمال', 'التحليل', 'ERP', 'CRM', 'البلوكشين',
    'ذكاء الأعمال', 'أجايل', 'لين',

    # أخلاقيات الأعمال والقانون
    'حوكمة الشركات', 'الامتثال', 'الأخلاقيات', 'CSR', 'المسؤولية الاجتماعية للشركات',
    'الاستدامة', 'التنظيم', 'مكافحة الفساد', 'الشفافية',

    # الاقتصاديات ذات الصلة
    'السوق', 'الأسواق', 'المنافسة', 'استراتيجية التسعير', 'الاحتكار', 'الاحتكار القليل',

    # الاستراتيجية والاستشارات
    'نموذج العمل', 'نماذج العمل', 'SWOT', 'التحليل المرجعي', 'الاستشارات',
    'استشاري', 'استشاريون', 'إدارة التغيير', 'تغيير التنظيم',

    # متفرقات
    'KPI', 'KPI', 'ROI', 'عائد الاستثمار', 'المعنيون', 'المعني',
    'قابلية التوسع', 'قابل للتوسع', 'استراتيجية النمو', 'سلسلة القيمة'
],

    'history': [
    # General terms
    'history', 'histories', 'historical', 'chronology', 'chronologies', 'timeline', 'timelines', 'epoch', 'epochs',
    'era', 'eras', 'periodization', 'period', 'periods',

    # Time periods
    'ancient', 'medieval', 'renaissance', 'early modern', 'modern', 'contemporary', 'prehistoric', 'classical',

    # Historical themes and events
    'revolution', 'revolutions', 'empire', 'empires', 'colonialism', 'colonization', 'war', 'wars', 'treaty', 'treaties',
    'independence', 'decolonization', 'migration', 'migrations', 'trade route', 'trade routes',
    'civilization', 'civilizations', 'dynasty', 'dynasties', 'industrialization', 'industrial revolutions',

    # Key global regions
    'european history', 'asian history', 'african history', 'american history',
    'middle east history', 'latin american history', 'world war', 'world wars',
    'ww1', 'ww2', 'cold war', 'global history', 'transatlantic',

    # Disciplines and approaches
    'archaeology', 'archaeological', 'oral history', 'oral histories', 'historiography', 'primary source', 'primary sources',
    'secondary source', 'secondary sources', 'cultural history', 'political history', 'economic history',
    'social history', 'military history', 'intellectual history', 'gender history', 'environmental history',
    'digital history', 'comparative history', 'microhistory',

    # Key figures
    'monarch', 'monarchs', 'emperor', 'emperors', 'king', 'kings', 'queen', 'queens',
    'pharaoh', 'pharaohs', 'president', 'presidents', 'dictator', 'dictators', 'general', 'generals',
    'historical figure', 'leaders', 'rulers',

    # Institutions and sources
    'archive', 'archives', 'manuscript', 'manuscripts', 'document', 'documents',
    'chronicle', 'chronicles', 'biography', 'biographies', 'memoir', 'memoirs', 'record', 'records',

    # Concepts
    'heritage', 'legacies', 'legacy', 'tradition', 'traditions', 'memory', 'memories', 'identity', 'identities',
    'transformation', 'transition', 'continuity', 'change',

    # Conflicts and struggles
    'civil war', 'civil wars', 'conflict', 'conflicts', 'resistance', 'occupation', 'occupations',
    'slavery', 'enslavement', 'genocide', 'massacre', 'uprising', 'rebellion',

    # Historical places and artifacts
    'museum', 'museums', 'artifact', 'artifacts', 'ruins', 'monument', 'monuments', 'site', 'sites', 'excavation', 'excavations',

    # Political movements
    'nationalism', 'fascism', 'communism', 'liberalism', 'feudalism', 'capitalism',
    'socialism', 'anarchism', 'imperialism',

    # Miscellaneous
    'historian', 'historians', 'timeline analysis', 'period study', 'document analysis',
    'source analysis', 'archival research', 'public history', 'historical methodology',

    # Términos generales
    'historia', 'historias', 'histórico', 'cronología', 'cronologías', 'línea de tiempo', 'líneas de tiempo', 'época', 'épocas',
    'era', 'eras', 'periodización', 'periodo', 'periodos',

    # Periodos de tiempo
    'antiguo', 'medieval', 'renacimiento', 'moderno temprano', 'moderno', 'contemporáneo', 'prehistórico', 'clásico',

    # Temas históricos y eventos
    'revolución', 'revoluciones', 'imperio', 'imperios', 'colonialismo', 'colonización', 'guerra', 'guerras', 'tratado', 'tratados',
    'independencia', 'descolonización', 'migración', 'migraciones', 'ruta comercial', 'rutas comerciales',
    'civilización', 'civilizaciones', 'dinastía', 'dinastías', 'industrialización', 'revoluciones industriales',

    # Regiones globales clave
    'historia europea', 'historia asiática', 'historia africana', 'historia americana',
    'historia de Oriente Medio', 'historia de América Latina', 'guerra mundial', 'guerras mundiales',
    'ww1', 'ww2', 'guerra fría', 'historia global', 'transatlántico',

    # Disciplinas y enfoques
    'arqueología', 'arqueológico', 'historia oral', 'historias orales', 'historiografía', 'fuente primaria', 'fuentes primarias',
    'fuente secundaria', 'fuentes secundarias', 'historia cultural', 'historia política', 'historia económica',
    'historia social', 'historia militar', 'historia intelectual', 'historia de género', 'historia ambiental',
    'historia digital', 'historia comparada', 'microhistoria',

    # Figuras clave
    'monarca', 'monarcas', 'emperador', 'emperadores', 'rey', 'reyes', 'reina', 'reinas',
    'faraón', 'faraones', 'presidente', 'presidentes', 'dictador', 'dictadores', 'general', 'generales',
    'figura histórica', 'líderes', 'gobernantes',

    # Instituciones y fuentes
    'archivo', 'archivos', 'manuscrito', 'manuscritos', 'documento', 'documentos',
    'crónica', 'crónicas', 'biografía', 'biografías', 'memorias', 'memorias', 'registro', 'registros',

    # Conceptos
    'patrimonio', 'legados', 'legado', 'tradición', 'tradiciones', 'memoria', 'memorias', 'identidad', 'identidades',
    'transformación', 'transición', 'continuidad', 'cambio',

    # Conflictos y luchas
    'guerra civil', 'guerras civiles', 'conflicto', 'conflictos', 'resistencia', 'ocupación', 'ocupaciones',
    'esclavitud', 'esclavización', 'genocidio', 'masacre', 'levantamiento', 'rebeldía',

    # Lugares históricos y artefactos
    'museo', 'museos', 'artefacto', 'artefactos', 'ruinas', 'monumento', 'monumentos', 'sitio', 'sitios', 'excavación', 'excavaciones',

    # Movimientos políticos
    'nacionalismo', 'fascismo', 'comunismo', 'liberalismo', 'feudalismo', 'capitalismo',
    'socialismo', 'anarquismo', 'imperialismo',

    # Varios
    'historiador', 'historiadores', 'análisis de línea de tiempo', 'estudio de períodos', 'análisis de documentos',
    'análisis de fuentes', 'investigación archivística', 'historia pública', 'metodología histórica',

    # Termos gerais
    'história', 'histórias', 'histórico', 'cronologia', 'cronologias', 'linha do tempo', 'linhas do tempo', 'época', 'épocas',
    'era', 'eras', 'periodização', 'período', 'períodos',

    # Períodos de tempo
    'antigo', 'medieval', 'renascimento', 'moderno inicial', 'moderno', 'contemporâneo', 'pré-histórico', 'clássico',

    # Temas históricos e eventos
    'revolução', 'revoluções', 'império', 'impérios', 'colonialismo', 'colonização', 'guerra', 'guerras', 'tratado', 'tratados',
    'independência', 'descolonização', 'migração', 'migrações', 'rota comercial', 'rotas comerciais',
    'civilização', 'civilizações', 'dinastia', 'dinastias', 'industrialização', 'revoluções industriais',

    # Regiões globais chave
    'história europeia', 'história asiática', 'história africana', 'história americana',
    'história do Oriente Médio', 'história da América Latina', 'primeira guerra mundial', 'guerras mundiais',
    'ww1', 'ww2', 'guerra fria', 'história global', 'transatlântico',

    # Disciplinas e abordagens
    'arqueologia', 'arqueológico', 'história oral', 'histórias orais', 'historiografia', 'fonte primária', 'fontes primárias',
    'fonte secundária', 'fontes secundárias', 'história cultural', 'história política', 'história econômica',
    'história social', 'história militar', 'história intelectual', 'história de gênero', 'história ambiental',
    'história digital', 'história comparativa', 'microhistória',

    # Figuras chave
    'monarca', 'monarcas', 'imperador', 'imperadores', 'rei', 'reis', 'rainha', 'rainhas',
    'faraó', 'faraós', 'presidente', 'presidentes', 'ditador', 'ditadores', 'general', 'generais',
    'figura histórica', 'líderes', 'governantes',

    # Instituições e fontes
    'arquivo', 'arquivos', 'manuscrito', 'manuscritos', 'documento', 'documentos',
    'crônica', 'crônicas', 'biografia', 'biografias', 'memórias', 'memórias', 'registro', 'registros',

    # Conceitos
    'patrimônio', 'legados', 'legado', 'tradição', 'tradições', 'memória', 'memórias', 'identidade', 'identidades',
    'transformação', 'transição', 'continuidade', 'mudança',

    # Conflitos e lutas
    'guerra civil', 'guerras civis', 'conflito', 'conflitos', 'resistência', 'ocupação', 'ocupações',
    'escravidão', 'escravização', 'genocídio', 'massacre', 'levantamento', 'rebelião',

    # Locais históricos e artefatos
    'museu', 'museus', 'artefato', 'artefatos', 'ruínas', 'monumento', 'monumentos', 'site', 'sites', 'escavação', 'escavações',

    # Movimentos políticos
    'nacionalismo', 'fascismo', 'comunismo', 'liberalismo', 'feudalismo', 'capitalismo',
    'socialismo', 'anarquismo', 'imperialismo',

    # Diversos
    'historiador', 'historiadores', 'análise da linha do tempo', 'estudo de períodos', 'análise de documentos',
    'análise de fontes', 'pesquisa arquivística', 'história pública', 'metodologia histórica',

    # Termini generali
    'storia', 'storie', 'storico', 'cronologia', 'cronologie', 'linea del tempo', 'linee del tempo', 'epoca', 'epoche',
    'era', 'ere', 'periodizzazione', 'periodo', 'periodi',

    # Periodi di tempo
    'antico', 'medievale', 'rinascimento', 'moderno iniziale', 'moderno', 'contemporaneo', 'preistorico', 'classico',

    # Temi storici ed eventi
    'rivoluzione', 'rivoluzioni', 'imperio', 'imperi', 'colonialismo', 'colonizzazione', 'guerra', 'guerre', 'trattato', 'trattati',
    'indipendenza', 'decolonizzazione', 'migrazione', 'migrazioni', 'rotta commerciale', 'rotte commerciali',
    'civiltà', 'civiltà', 'dinastia', 'dinastie', 'industrializzazione', 'rivoluzioni industriali',

    # Regioni globali chiave
    'storia europea', 'storia asiatica', 'storia africana', 'storia americana',
    'storia del Medio Oriente', 'storia dell\'America Latina', 'prima guerra mondiale', 'guerre mondiali',
    'ww1', 'ww2', 'guerra fredda', 'storia globale', 'transatlantico',

    # Discipline e approcci
    'arqueologia', 'arqueologico', 'storia orale', 'storie orali', 'storiografia', 'fonte primaria', 'fonti primarie',
    'fonte secondaria', 'fonti secondarie', 'storia culturale', 'storia politica', 'storia economica',
    'storia sociale', 'storia militare', 'storia intellettuale', 'storia di genere', 'storia ambientale',
    'storia digitale', 'storia comparata', 'microstoria',

    # Figure chiave
    'monarca', 'monarchi', 'imperatore', 'imperatori', 're', 're', 'regina', 'regine',
    'faraone', 'faraoni', 'presidente', 'presidenti', 'dittatore', 'dittatori', 'generale', 'generali',
    'figura storica', 'leader', 'governanti',

    # Istituzioni e fonti
    'archivio', 'archivi', 'manoscritto', 'manoscritti', 'documento', 'documenti',
    'cronaca', 'cronache', 'biografia', 'biografie', 'memorie', 'memorie', 'registro', 'registrazioni',

    # Concetti
    'patrimonio', 'lasciati', 'legato', 'tradizione', 'tradizioni', 'memoria', 'memorie', 'identità', 'identità',
    'trasformazione', 'transizione', 'continuità', 'cambiamento',

    # Conflitti e lotte
    'guerra civile', 'guerre civili', 'conflitto', 'conflitti', 'resistenza', 'occupazione', 'occupazioni',
    'schiavitù', 'schiavizzazione', 'genocidio', 'massacro', 'sollevamento', 'ribellione',

    # Luoghi storici e artefatti
    'museo', 'musei', 'artefatto', 'artefatti', 'rovine', 'monumento', 'monumenti', 'sito', 'siti', 'scavo', 'scavi',

    # Movimenti politici
    'nazionalismo', 'fascismo', 'comunismo', 'liberalismo', 'feudalesimo', 'capitalismo',
    'socialismo', 'anarchismo', 'imperialismo',

    # Varie
    'storico', 'storici', 'analisi della linea temporale', 'studio del periodo', 'analisi del documento',
    'analisi delle fonti', 'ricerca archivistica', 'storia pubblica', 'metodologia storica',

    # Termes généraux
    'histoire', 'histoires', 'historique', 'chronologie', 'chronologies', 'ligne du temps', 'lignes du temps', 'époque', 'époques',
    'ère', 'ères', 'périodisation', 'période', 'périodes',

    # Périodes de temps
    'ancien', 'médiéval', 'renaissance', 'moderne précoce', 'moderne', 'contemporain', 'préhistorique', 'classique',

    # Thèmes historiques et événements
    'révolution', 'révolutions', 'empire', 'empires', 'colonialisme', 'colonisation', 'guerre', 'guerres', 'traité', 'traités',
    'indépendance', 'décolonisation', 'migration', 'migrations', 'route commerciale', 'routes commerciales',
    'civilisation', 'civilisations', 'dynastie', 'dynasties', 'industrialisation', 'révolutions industrielles',

    # Régions mondiales clés
    'histoire européenne', 'histoire asiatique', 'histoire africaine', 'histoire américaine',
    'histoire du Moyen-Orient', 'histoire de l\'Amérique latine', 'première guerre mondiale', 'guerres mondiales',
    'ww1', 'ww2', 'guerre froide', 'histoire mondiale', 'transatlantique',

    # Disciplines et approches
    'archéologie', 'archéologique', 'histoire orale', 'histoires orales', 'historiographie', 'source primaire', 'sources primaires',
    'source secondaire', 'sources secondaires', 'histoire culturelle', 'histoire politique', 'histoire économique',
    'histoire sociale', 'histoire militaire', 'histoire intellectuelle', 'histoire du genre', 'histoire environnementale',
    'histoire numérique', 'histoire comparée', 'microhistoire',

    # Figures clés
    'monarque', 'monarques', 'empereur', 'empereurs', 'roi', 'rois', 'reine', 'reines',
    'pharaon', 'pharaons', 'président', 'présidents', 'dictateur', 'dictateurs', 'général', 'généraux',
    'personnage historique', 'leaders', 'dirigeants',

    # Institutions et sources
    'archive', 'archives', 'manuscrit', 'manuscrits', 'document', 'documents',
    'chronique', 'chroniques', 'biographie', 'biographies', 'mémoires', 'mémoires', 'enregistrement', 'enregistrements',

    # Concepts
    'patrimoine', 'héritages', 'héritage', 'tradition', 'traditions', 'mémoire', 'mémoires', 'identité', 'identités',
    'transformation', 'transition', 'continuité', 'changement',

    # Conflits et luttes
    'guerre civile', 'guerres civiles', 'conflit', 'conflits', 'résistance', 'occupation', 'occupations',
    'esclavage', 'esclavage', 'génocide', 'massacre', 'soulèvement', 'rébellion',

    # Lieux historiques et artefacts
    'musée', 'musées', 'artefact', 'artefacts', 'ruines', 'monument', 'monuments', 'site', 'sites', 'excavation', 'excavations',

    # Mouvements politiques
    'nationalisme', 'fascisme', 'communisme', 'libéralisme', 'féodalisme', 'capitalisme',
    'socialisme', 'anarchisme', 'impérialisme',

    # Divers
    'historien', 'historiens', 'analyse de ligne du temps', 'étude de période', 'analyse de document',
    'analyse de sources', 'recherche archivistique', 'histoire publique', 'méthodologie historique',

    # 一般术语
    '历史', '历史学', '历史的', '年表', '年表们', '时间线', '时间线们', '时代', '时代们',
    '纪元', '纪元们', '时间段划分', '时期', '时期们',

    # 时间段
    '古代', '中世纪', '文艺复兴', '早期现代', '现代', '当代', '史前', '经典',

    # 历史主题和事件
    '革命', '革命们', '帝国', '帝国们', '殖民主义', '殖民化', '战争', '战争们', '条约', '条约们',
    '独立', '去殖民化', '迁移', '迁移们', '贸易路线', '贸易路线们',
    '文明', '文明们', '王朝', '王朝们', '工业化', '工业革命们',

    # 主要全球区域
    '欧洲历史', '亚洲历史', '非洲历史', '美洲历史',
    '中东历史', '拉丁美洲历史', '第一次世界大战', '世界大战们',
    '第一次世界大战', '第二次世界大战', '冷战', '全球历史', '跨大西洋',

    # 学科和方法
    '考古学', '考古学的', '口述历史', '口述历史们', '历史学', '原始来源', '原始来源们',
    '二手来源', '二手来源们', '文化历史', '政治历史', '经济历史',
    '社会历史', '军事历史', '知识分子历史', '性别历史', '环境历史',
    '数字历史', '比较历史', '微观历史',

    # 关键人物
    '君主', '君主们', '皇帝', '皇帝们', '国王', '国王们', '女王', '女王们',
    '法老', '法老们', '总统', '总统们', '独裁者', '独裁者们', '将军', '将军们',
    '历史人物', '领导者们', '统治者们',

    # 机构和来源
    '档案馆', '档案馆们', '手稿', '手稿们', '文件', '文件们',
    '编年史', '编年史们', '传记', '传记们', '回忆录', '回忆录们', '记录', '记录们',

    # 概念
    '遗产', '遗产们', '传统', '传统们', '记忆', '记忆们', '身份', '身份们',
    '转型', '过渡', '连续性', '变化',

    # 冲突与斗争
    '内战', '内战们', '冲突', '冲突们', '抵抗', '占领', '占领们',
    '奴隶制', '奴役', '种族灭绝', '大屠杀', '起义', '反叛',

    # 历史地点和文物
    '博物馆', '博物馆们', '文物', '文物们', '废墟', '纪念碑', '纪念碑们', '遗址', '遗址们', '挖掘', '挖掘们',

    # 政治运动
    '民族主义', '法西斯主义', '共产主义', '自由主义', '封建主义', '资本主义',
    '社会主义', '无政府主义', '帝国主义',

    # 其他
    '历史学家', '历史学者们', '时间线分析', '时期研究', '文件分析',
    '来源分析', '档案研究', '公共历史', '历史方法学',

    # Общие термины
    'история', 'истории', 'исторический', 'хронология', 'хронологии', 'временная линия', 'временные линии', 'эпоха', 'эпохи',
    'эра', 'эры', 'периодизация', 'период', 'периоды',

    # Периоды времени
    'древний', 'средневековый', 'Ренессанс', 'раннее современное', 'современный', 'современность', 'дописьменный', 'классический',

    # Исторические темы и события
    'революция', 'революции', 'империя', 'империи', 'колониализм', 'колонизация', 'война', 'войны', 'договор', 'договоры',
    'независимость', 'деоколоизация', 'миграция', 'миграции', 'торговый маршрут', 'торговые маршруты',
    'цивилизация', 'цивилизации', 'династия', 'династии', 'индустриализация', 'промышленные революции',

    # Основные глобальные регионы
    'европейская история', 'азиатская история', 'африканская история', 'американская история',
    'история Ближнего Востока', 'история Латинской Америки', 'первая мировая война', 'мировые войны',
    'первая мировая война', 'вторая мировая война', 'холодная война', 'глобальная история', 'трансатлантическая',

    # Дисциплины и подходы
    'археология', 'археологический', 'устная история', 'устные истории', 'историография', 'первичный источник', 'первичные источники',
    'вторичный источник', 'вторичные источники', 'культурная история', 'политическая история', 'экономическая история',
    'социальная история', 'военная история', 'интеллектуальная история', 'история пола', 'экологическая история',
    'цифровая история', 'сравнительная история', 'микроистория',

    # Ключевые фигуры
    'монарх', 'монархи', 'император', 'императоры', 'король', 'короли', 'королева', 'королевы',
    'фараон', 'фараоны', 'президент', 'президенты', 'диктатор', 'диктаторы', 'генерал', 'генералы',
    'историческая фигура', 'лидеры', 'правители',

    # Учреждения и источники
    'архив', 'архивы', 'рукопись', 'рукописи', 'документ', 'документы',
    'хроника', 'хроники', 'биография', 'биографии', 'мемуары', 'мемуары', 'запись', 'записи',

    # Концепции
    'наследие', 'наследия', 'наследие', 'традиция', 'традиции', 'память', 'памяти', 'идентичность', 'идентичности',
    'трансформация', 'переход', 'непрерывность', 'изменение',

    # Конфликты и борьбы
    'гражданская война', 'гражданские войны', 'конфликт', 'конфликты', 'сопротивление', 'оккупация', 'оккупации',
    'рабство', 'порабощение', 'геноцид', 'резня', 'восстание', 'мятеж',

    # Исторические места и артефакты
    'музей', 'музеи', 'артефакт', 'артефакты', 'руины', 'памятник', 'памятники', 'сайт', 'сайты', 'раскопки', 'раскопки',

    # Политические движения
    'национализм', 'фашизм', 'коммунизм', 'либерализм', 'феодализм', 'капитализм',
    'социализм', 'анархизм', 'империализм',

    # Разное
    'историк', 'историки', 'анализ временной линии', 'изучение периодов', 'анализ документов',
    'анализ источников', 'архивные исследования', 'публичная история', 'историческая методология',

    # 一般的な用語
    '歴史', '歴史学', '歴史的', '年表', '年表たち', 'タイムライン', 'タイムラインたち', '時代', '時代たち',
    '時代区分', '時期', '時期たち',

    # 時代
    '古代', '中世', 'ルネサンス', '初期近代', '近代', '現代', '先史', '古典',

    # 歴史的なテーマと出来事
    '革命', '革命たち', '帝国', '帝国たち', '植民地主義', '植民地化', '戦争', '戦争たち', '条約', '条約たち',
    '独立', '脱植民地化', '移住', '移住たち', '貿易路', '貿易路たち',
    '文明', '文明たち', '王朝', '王朝たち', '工業化', '産業革命たち',

    # 主要な世界地域
    'ヨーロッパの歴史', 'アジアの歴史', 'アフリカの歴史', 'アメリカの歴史',
    '中東の歴史', 'ラテンアメリカの歴史', '第一次世界大戦', '世界大戦たち',
    '第一次世界大戦', '第二次世界大戦', '冷戦', '世界の歴史', '大西洋横断',

    # 学問分野とアプローチ
    '考古学', '考古学的', '口述歴史', '口述歴史たち', '歴史学', '一次資料', '一次資料たち',
    '二次資料', '二次資料たち', '文化歴史', '政治歴史', '経済歴史',
    '社会歴史', '軍事歴史', '知識人歴史', '性別歴史', '環境歴史',
    'デジタル歴史', '比較歴史', 'ミクロ歴史',

    # 重要な人物
    '君主', '君主たち', '皇帝', '皇帝たち', '王', '王たち', '女王', '女王たち',
    'ファラオ', 'ファラオたち', '大統領', '大統領たち', '独裁者', '独裁者たち', '将軍', '将軍たち',
    '歴史的人物', 'リーダーたち', '支配者たち',

    # 機関と資料
    'アーカイブ', 'アーカイブたち', '手稿', '手稿たち', '文書', '文書たち',
    '年代記', '年代記たち', '伝記', '伝記たち', '回想録', '回想録たち', '記録', '記録たち',

    # 概念
    '遺産', '遺産たち', '伝統', '伝統たち', '記憶', '記憶たち', 'アイデンティティ', 'アイデンティティたち',
    '変革', '過渡', '連続性', '変化',

    # 紛争と闘争
    '内戦', '内戦たち', '紛争', '紛争たち', '抵抗', '占領', '占領たち',
    '奴隷制', '奴隷化', 'ジェノサイド', '虐殺', '蜂起', '反乱',

    # 歴史的な場所と遺物
    '博物館', '博物館たち', '遺物', '遺物たち', '遺跡', '記念碑', '記念碑たち', 'サイト', 'サイトたち', '発掘', '発掘たち',

    # 政治運動
    '国粋主義', 'ファシズム', '共産主義', 'リベラリズム', '封建主義', '資本主義',
    '社会主義', '無政府主義', '帝国主義',

    # その他
    '歴史学者', '歴史学者たち', 'タイムライン分析', '時期研究', '文書分析',
    '資料分析', 'アーカイブ研究', '公共歴史', '歴史的方法論',

    # 일반 용어
    '역사', '역사학', '역사적', '연대기', '연대기들', '타임라인', '타임라인들', '시대', '시대들',
    '시대 구분', '기간', '기간들',

    # 시간 기간
    '고대', '중세', '르네상스', '초기 근대', '근대', '현대', '선사', '고전',

    # 역사적 주제와 사건
    '혁명', '혁명들', '제국', '제국들', '식민주의', '식민지화', '전쟁', '전쟁들', '조약', '조약들',
    '독립', '탈식민화', '이주', '이주들', '무역로', '무역로들',
    '문명', '문명들', '왕조', '왕조들', '산업화', '산업 혁명들',

    # 주요 글로벌 지역
    '유럽 역사', '아시아 역사', '아프리카 역사', '미국 역사',
    '중동 역사', '라틴 아메리카 역사', '제1차 세계 대전', '세계 대전들',
    '제1차 세계 대전', '제2차 세계 대전', '냉전', '세계 역사', '대서양 횡단',

    # 학문 분야와 접근법
    '고고학', '고고학적', '구술 역사', '구술 역사들', '역사학', '1차 자료', '1차 자료들',
    '2차 자료', '2차 자료들', '문화 역사', '정치 역사', '경제 역사',
    '사회 역사', '군사 역사', '지적 역사', '성별 역사', '환경 역사',
    '디지털 역사', '비교 역사', '미시 역사',

    # 주요 인물
    '군주', '군주들', '황제', '황제들', '왕', '왕들', '여왕', '여왕들',
    '파라오', '파라오들', '대통령', '대통령들', '독재자', '독재자들', '장군', '장군들',
    '역사적 인물', '리더들', '지배자들',

    # 기관과 출처
    '아카이브', '아카이브들', '원고', '원고들', '문서', '문서들',
    '연대기', '연대기들', '전기', '전기들', '회고록', '회고록들', '기록', '기록들',

    # 개념
    '유산', '유산들', '전통', '전통들', '기억', '기억들', '아이덴티티', '아이덴티티들',
    '변형', '전환', '연속성', '변화',

    # 갈등과 투쟁
    '내전', '내전들', '갈등', '갈등들', '저항', '점령', '점령들',
    '노예제', '노예화', '집단학살', '학살', '봉기', '반란',

    # 역사적 장소와 유물
    '박물관', '박물관들', '유물', '유물들', '유적', '기념비', '기념비들', '사이트', '사이트들', '발굴', '발굴들',

    # 정치적 운동
    '민족주의', '파시즘', '공산주의', '자유주의', '봉건주의', '자본주의',
    '사회주의', '무정부주의', '제국주의',

    # 기타
    '역사학자', '역사학자들', '타임라인 분석', '기간 연구', '문서 분석',
    '자료 분석', '아카이브 연구', '공공 역사', '역사 방법론',

    # 一般術語
    '歷史', '歷史學', '歷史的', '年表', '年表們', '時間線', '時間線們', '時代', '時代們',
    '時代劃分', '時期', '時期們',

    # 時間段
    '古代', '中世紀', '文藝復興', '早期現代', '現代', '當代', '史前', '經典',

    # 歷史主題和事件
    '革命', '革命們', '帝國', '帝國們', '殖民主義', '殖民化', '戰爭', '戰爭們', '條約', '條約們',
    '獨立', '去殖民化', '遷徙', '遷徙們', '貿易路線', '貿易路線們',
    '文明', '文明們', '王朝', '王朝們', '工業化', '工業革命們',

    # 主要全球區域
    '歐洲歷史', '亞洲歷史', '非洲歷史', '美洲歷史',
    '中東歷史', '拉丁美洲歷史', '第一次世界大戰', '世界大戰們',
    '第一次世界大戰', '第二次世界大戰', '冷戰', '全球歷史', '跨大西洋',

    # 學科和方法
    '考古學', '考古學的', '口述歷史', '口述歷史們', '歷史學', '原始資料', '原始資料們',
    '二手資料', '二手資料們', '文化歷史', '政治歷史', '經濟歷史',
    '社會歷史', '軍事歷史', '知識分子歷史', '性別歷史', '環境歷史',
    '數位歷史', '比較歷史', '微觀歷史',

    # 重要人物
    '君主', '君主們', '皇帝', '皇帝們', '國王', '國王們', '女王', '女王們',
    '法老', '法老們', '總統', '總統們', '獨裁者', '獨裁者們', '將軍', '將軍們',
    '歷史人物', '領袖們', '統治者們',

    # 機構和來源
    '檔案館', '檔案館們', '手稿', '手稿們', '文件', '文件們',
    '編年史', '編年史們', '傳記', '傳記們', '回憶錄', '回憶錄們', '記錄', '記錄們',

    # 概念
    '遺產', '遺產們', '傳統', '傳統們', '記憶', '記憶們', '身份', '身份們',
    '轉變', '過渡', '連續性', '變化',

    # 衝突與鬥爭
    '內戰', '內戰們', '衝突', '衝突們', '抵抗', '佔領', '佔領們',
    '奴隸制', '奴役', '種族滅絕', '大屠殺', '起義', '叛亂',

    # 歷史性地點和文物
    '博物館', '博物館們', '文物', '文物們', '遺址', '紀念碑', '紀念碑們', '網站', '網站們', '挖掘', '挖掘們',

    # 政治運動
    '民族主義', '法西斯主義', '共產主義', '自由主義', '封建主義', '資本主義',
    '社會主義', '無政府主義', '帝國主義',

    # 其他
    '歷史學家', '歷史學者們', '時間線分析', '時期研究', '文件分析',
    '資料分析', '檔案研究', '公共歷史', '歷史方法論',

    # Obecné pojmy
    'historie', 'historie vědy', 'historický', 'kronika', 'kroniky', 'časová osa', 'časové osy', 'éra', 'éry',
    'periodizace', 'perioda', 'periody',

    # Časové období
    'starověk', 'středověk', 'renesance', 'raný novověk', 'moderní', 'současnost', 'předhistorie', 'klasický',

    # Historické témata a události
    'revoluce', 'revoluce', 'říše', 'říše', 'kolonialismus', 'kolonizace', 'válka', 'války', 'smlouva', 'smlouvy',
    'nezávislost', 'dekolonizace', 'migrace', 'migrace', 'obchodní trasa', 'obchodní trasy',
    'civilizace', 'civilizace', 'dynastie', 'dynastie', 'industrializace', 'průmyslové revoluce',

    # Klíčové globální oblasti
    'evropská historie', 'asijská historie', 'africká historie', 'americká historie',
    'historie Blízkého východu', 'historie Latinské Ameriky', 'první světová válka', 'světové války',
    'první světová válka', 'druhá světová válka', 'studená válka', 'globální historie', 'transatlantické',

    # Disciplíny a přístupy
    'archaeologie', 'archaeologický', 'ústní historie', 'ústní historie', 'historiografie', 'primární zdroj', 'primární zdroje',
    'sekundární zdroj', 'sekundární zdroje', 'kulturní historie', 'politická historie', 'ekonomická historie',
    'sociální historie', 'vojenská historie', 'intellectuální historie', 'historie pohlaví', 'historie životního prostředí',
    'digitální historie', 'komparativní historie', 'mikro historie',

    # Klíčové postavy
    'monarcha', 'monarchové', 'císař', 'císaři', 'král', 'králové', 'královna', 'královny',
    'faraon', 'faraoni', 'prezident', 'prezidenti', 'diktátor', 'diktátoři', 'generál', 'generálové',
    'historická postava', 'vedoucí', 'vládci',

    # Instituce a zdroje
    'archiv', 'archivy', 'rukopis', 'rukopisy', 'dokument', 'dokumenty',
    'kronika', 'kroniky', 'biografie', 'biografie', 'paměť', 'paměti', 'záznam', 'záznamy',

    # Koncepty
    'dědictví', 'dědictví', 'tradice', 'tradice', 'paměť', 'paměť', 'identita', 'identity',
    'transformace', 'přechod', 'kontinuita', 'změna',

    # Konflikty a boje
    'občanská válka', 'občanské války', 'konflikt', 'konflikty', 'odpor', 'okupace', 'okupace',
    'otroctví', 'otrocké', 'genocida', 'masakr', 'povstání', 'rebélie',

    # Historická místa a artefakty
    'muzeum', 'muzea', 'artefakt', 'artefakty', 'ruiny', 'pomník', 'pomníky', 'místo', 'místa', 'výkop', 'výkopy',

    # Politické pohyby
    'nacionalismus', 'fašismus', 'komunismus', 'liberalismus', 'feudalismus', 'kapitalismus',
    'socialismus', 'anarchismus', 'imperialismus',

    # Různé
    'historik', 'historici', 'analýza časové osy', 'studie období', 'analýza dokumentů',
    'analýza zdrojů', 'archivní výzkum', 'veřejná historie', 'historická metodologie',

    # المصطلحات العامة
    'التاريخ', 'علم التاريخ', 'تاريخي', 'الكرونولوجيا', 'الكرونولوجيات', 'الخط الزمني', 'الخطوط الزمنية', 'العصر', 'العصور',
    'التقسيم الزمني', 'الفترة', 'الفترات',

    # الفترات الزمنية
    'العصر القديم', 'العصور الوسطى', 'عصر النهضة', 'العصر الحديث المبكر', 'الحديث', 'المعاصر', 'ما قبل التاريخ', 'الكلاسيكي',

    # الموضوعات والأحداث التاريخية
    'ثورة', 'ثورات', 'إمبراطورية', 'إمبراطوريات', 'استعمار', 'استعمارية', 'حرب', 'حروب', 'معاهدة', 'معاهدات',
    'استقلال', 'مناهضة الاستعمار', 'هجرة', 'هجرات', 'طريق تجاري', 'طرق تجارية',
    'حضارة', 'حضارات', 'سلالة', 'سلالات', 'تصنيع', 'الثورات الصناعية',

    # المناطق العالمية الرئيسية
    'تاريخ أوروبا', 'تاريخ آسيا', 'تاريخ أفريقيا', 'تاريخ أمريكا',
    'تاريخ الشرق الأوسط', 'تاريخ أمريكا اللاتينية', 'الحرب العالمية الأولى', 'الحروب العالمية',
    'الحرب العالمية الأولى', 'الحرب العالمية الثانية', 'الحرب الباردة', 'التاريخ العالمي', 'عبر الأطلسي',

    # التخصصات والمناهج
    'علم الآثار', 'آثاري', 'التاريخ الشفوي', 'التواريخ الشفوية', 'كتابة التاريخ', 'المصادر الأولية', 'المصادر الأولية',
    'المصادر الثانوية', 'المصادر الثانوية', 'تاريخ ثقافي', 'تاريخ سياسي', 'تاريخ اقتصادي',
    'تاريخ اجتماعي', 'تاريخ عسكري', 'تاريخ فكري', 'تاريخ الجندر', 'تاريخ بيئي',
    'تاريخ رقمي', 'تاريخ مقارن', 'تاريخ دقيق',

    # الشخصيات الرئيسية
    'ملك', 'ملوك', 'إمبراطور', 'أباطرة', 'ملك', 'ملوك', 'ملكة', 'ملكات',
    'فرعون', 'فراعنة', 'رئيس', 'رؤساء', 'ديكتاتور', 'ديكتاتوريات', 'جنرال', 'جنرالات',
    'شخصية تاريخية', 'قادة', 'حكام',

    # المؤسسات والمصادر
    'أرشيف', 'أرشيفات', 'مخطوطة', 'مخطوطات', 'وثيقة', 'وثائق',
    'سجل', 'سجلات', 'سيرة', 'سير', 'مذكرات', 'مذكرات', 'تسجيل', 'تسجيلات',

    # المفاهيم
    'تراث', 'تراثات', 'تقليد', 'تقليدات', 'ذاكرة', 'ذكريات', 'هوية', 'هويات',
    'تحول', 'انتقال', 'استمرارية', 'تغيير',

    # الصراعات والنضال
    'حرب أهلية', 'حروب أهلية', 'صراع', 'صراعات', 'مقاومة', 'احتلال', 'احتلالات',
    'عبودية', 'استعباد', 'إبادة جماعية', 'مجزرة', 'انتفاضة', 'تمرد',

    # الأماكن التاريخية والآثار
    'متحف', 'متاحف', 'أثر', 'آثار', 'أنقاض', 'نصب تذكاري', 'نصب تذكارية', 'موقع', 'مواقع', 'تنقيب', 'تنقيبات',

    # الحركات السياسية
    'القومية', 'الفاشية', 'الشيوعية', 'الليبرالية', 'الإقطاعية', 'الرأسمالية',
    'الاشتراكية', 'الفوضوية', 'الإمبريالية',

    # متفرقات
    'مؤرخ', 'مؤرخون', 'تحليل الخط الزمني', 'دراسة الفترات', 'تحليل الوثائق',
    'تحليل المصادر', 'بحث أرشيفي', 'تاريخ عام', 'منهجية تاريخية'

    ],


    'sociology': [
    # Core concepts
    'sociology', 'society', 'societies', 'social', 'sociality', 'culture', 'cultures',
    'identity', 'identities', 'norm', 'norms', 'value', 'values',
    'institution', 'institutions', 'socialization', 'resocialization', 'interaction',
    'group dynamic', 'group dynamics', 'status', 'statuses', 'role', 'roles',

    # Subfields
    'urban sociology', 'rural sociology', 'medical sociology', 'sociology of education',
    'sociology of religion', 'economic sociology', 'political sociology',
    'criminology', 'gender studies', 'migration studies', 'family sociology',
    'environmental sociology', 'organizational sociology', 'cultural sociology',

    # Theories & paradigms
    'functionalism', 'conflict theory', 'symbolic interactionism', 'structuralism',
    'post-structuralism', 'critical theory', 'social constructivism', 'social constructionism',
    'feminist theory', 'queer theory', 'rational choice theory', 'world-systems theory',

    # Social issues
    'inequality', 'inequalities', 'racism', 'sexism', 'ageism', 'ableism',
    'discrimination', 'poverty', 'marginalization', 'unemployment', 'underemployment',
    'homelessness', 'crime', 'deviance', 'deviant behavior', 'violence', 'addiction',
    'inequity', 'prejudice', 'oppression', 'victimization', 'stigmatization',

    # Methodologies
    'qualitative', 'quantitative', 'ethnography', 'ethnographies', 'survey', 'surveys',
    'interview', 'interviews', 'case study', 'case studies', 'participant observation',
    'observation', 'fieldwork', 'longitudinal study', 'cross-sectional study',
    'mixed methods', 'content analysis', 'discourse analysis',

    # Social structures & systems
    'class', 'classes', 'gender', 'race', 'ethnicity', 'religion', 'religions',
    'family', 'families', 'education', 'bureaucracy', 'bureaucracies', 'government',
    'governments', 'economy', 'media', 'technology', 'technologies', 'healthcare',
    'social institution', 'social structure', 'structural system', 'normative order',

    # Global and current topics
    'globalization', 'migration', 'immigration', 'emigration', 'diaspora',
    'social media', 'digital society', 'online communities', 'environmental justice',
    'climate justice', 'activism', 'protest', 'social change', 'civic engagement',
    'intersectionality', 'identity politics', 'public sociology',

    # Notable terms
    'social capital', 'social mobility', 'upward mobility', 'downward mobility',
    'social stratification', 'agency', 'structure', 'structuration', 'power',
    'hegemony', 'ideology', 'ideologies', 'alienation', 'anomie', 'modernity',
    'postmodernity', 'rationalization', 'secularization', 'collective behavior',

    # Influential thinkers
    'marx', 'karl marx', 'durkheim', 'emile durkheim', 'weber', 'max weber',
    'goffman', 'erving goffman', 'bourdieu', 'pierre bourdieu',
    'foucault', 'michel foucault', 'giddens', 'anthony giddens',
    'habermas', 'juergen habermas', 'comte', 'auguste comte', 'mills', 'wright mills',

    # Conceptos principales
    'sociología', 'sociedad', 'sociedades', 'social', 'socialidad', 'cultura', 'culturas',
    'identidad', 'identidades', 'norma', 'normas', 'valor', 'valores',
    'institución', 'instituciones', 'socialización', 'resocialización', 'interacción',
    'dinámica de grupo', 'dinámicas de grupo', 'estatus', 'estatuses', 'rol', 'roles',

    # Subcampos
    'sociología urbana', 'sociología rural', 'sociología médica', 'sociología de la educación',
    'sociología de la religión', 'sociología económica', 'sociología política',
    'criminología', 'estudios de género', 'estudios de migración', 'sociología de la familia',
    'sociología ambiental', 'sociología organizacional', 'sociología cultural',

    # Teorías y paradigmas
    'funcionalismo', 'teoría del conflicto', 'interaccionismo simbólico', 'estructuralismo',
    'posestructuralismo', 'teoría crítica', 'constructivismo social', 'constructivismo social',
    'teoría feminista', 'teoría queer', 'teoría de la elección racional', 'teoría de los sistemas-mundo',

    # Problemas sociales
    'desigualdad', 'desigualdades', 'racismo', 'sexismo', 'edadismo', 'habilismo',
    'discriminación', 'pobreza', 'marginación', 'desempleo', 'subempleo',
    'personas sin hogar', 'delincuencia', 'desviación', 'comportamiento desviado', 'violencia', 'adicción',
    'inequidad', 'prejuicio', 'opresión', 'victimización', 'estigmatización',

    # Metodologías
    'cualitativa', 'cuantitativa', 'etnografía', 'etnografías', 'encuesta', 'encuestas',
    'entrevista', 'entrevistas', 'estudio de caso', 'estudios de caso', 'observación participante',
    'observación', 'trabajo de campo', 'estudio longitudinal', 'estudio transversal',
    'métodos mixtos', 'análisis de contenido', 'análisis del discurso',

    # Estructuras y sistemas sociales
    'clase', 'clases', 'género', 'raza', 'etnicidad', 'religión', 'religiones',
    'familia', 'familias', 'educación', 'burocracia', 'burocracias', 'gobierno',
    'gobiernos', 'economía', 'medios', 'tecnología', 'tecnologías', 'atención médica',
    'institución social', 'estructura social', 'sistema estructural', 'orden normativo',

    # Temas globales y actuales
    'globalización', 'migración', 'inmigración', 'emigración', 'diáspora',
    'redes sociales', 'sociedad digital', 'comunidades en línea', 'justicia ambiental',
    'justicia climática', 'activismo', 'protesta', 'cambio social', 'participación cívica',
    'interseccionalidad', 'política de identidad', 'sociología pública',

    # Términos notables
    'capital social', 'movilidad social', 'movilidad ascendente', 'movilidad descendente',
    'estratificación social', 'agencia', 'estructura', 'estructuración', 'poder',
    'hegemonía', 'ideología', 'ideologías', 'alienación', 'anomia', 'modernidad',
    'posmodernidad', 'racionalización', 'secularización', 'comportamiento colectivo',

    # Pensadores influyentes
    'marx', 'karl marx', 'durkheim', 'emile durkheim', 'weber', 'max weber',
    'goffman', 'erving goffman', 'bourdieu', 'pierre bourdieu',
    'foucault', 'michel foucault', 'giddens', 'anthony giddens',
    'habermas', 'juergen habermas', 'comte', 'auguste comte', 'mills', 'wright mills',

    # Conceitos principais
    'sociologia', 'sociedade', 'sociedades', 'social', 'socialidade', 'cultura', 'culturas',
    'identidade', 'identidades', 'norma', 'normas', 'valor', 'valores',
    'instituição', 'instituições', 'socialização', 'resocialização', 'interação',
    'dinâmica de grupo', 'dinâmicas de grupo', 'status', 'status', 'papel', 'papéis',

    # Subáreas
    'sociologia urbana', 'sociologia rural', 'sociologia médica', 'sociologia da educação',
    'sociologia da religião', 'sociologia econômica', 'sociologia política',
    'criminologia', 'estudos de gênero', 'estudos de migração', 'sociologia da família',
    'sociologia ambiental', 'sociologia organizacional', 'sociologia cultural',

    # Teorias e paradigmas
    'funcionalismo', 'teoria do conflito', 'interacionismo simbólico', 'estruturalismo',
    'pós-estruturalismo', 'teoria crítica', 'construtivismo social', 'construccionismo social',
    'teoria feminista', 'teoria queer', 'teoria da escolha racional', 'teoria dos sistemas-mundo',

    # Problemas sociais
    'desigualdade', 'desigualdades', 'racismo', 'sexismo', 'idadeismo', 'habilismo',
    'discriminação', 'pobreza', 'marginalização', 'desemprego', 'subemprego',
    'falta de moradia', 'crime', 'desvio', 'comportamento desviado', 'violência', 'dependência',
    'inequidade', 'preconceito', 'opressão', 'vitimização', 'estigmatização',

    # Metodologias
    'qualitativo', 'quantitativo', 'etnografia', 'etnografias', 'pesquisa', 'pesquisas',
    'entrevista', 'entrevistas', 'estudo de caso', 'estudos de caso', 'observação participante',
    'observação', 'trabalho de campo', 'estudo longitudinal', 'estudo transversal',
    'métodos mistos', 'análise de conteúdo', 'análise do discurso',

    # Estruturas sociais e sistemas
    'classe', 'classes', 'gênero', 'raça', 'etnia', 'religião', 'religiões',
    'família', 'famílias', 'educação', 'burocracia', 'burocracias', 'governo',
    'governos', 'economia', 'mídia', 'tecnologia', 'tecnologias', 'cuidados de saúde',
    'instituição social', 'estrutura social', 'sistema estrutural', 'ordem normativa',

    # Temas globais e atuais
    'globalização', 'migração', 'imigração', 'emigração', 'diáspora',
    'mídias sociais', 'sociedade digital', 'comunidades online', 'justiça ambiental',
    'justiça climática', 'ativismo', 'protesto', 'mudança social', 'engajamento cívico',
    'interseccionalidade', 'política de identidade', 'sociologia pública',

    # Termos notáveis
    'capital social', 'mobilidade social', 'mobilidade ascendente', 'mobilidade descendente',
    'estratificação social', 'agência', 'estrutura', 'estruturação', 'poder',
    'hegemonia', 'ideologia', 'ideologias', 'alienação', 'anomalia', 'modernidade',
    'pós-modernidade', 'racionalização', 'secularização', 'comportamento coletivo',

    # Pensadores influentes
    'marx', 'karl marx', 'durkheim', 'emile durkheim', 'weber', 'max weber',
    'goffman', 'erving goffman', 'bourdieu', 'pierre bourdieu',
    'foucault', 'michel foucault', 'giddens', 'anthony giddens',
    'habermas', 'juergen habermas', 'comte', 'auguste comte', 'mills', 'wright mills',

    # Concetti principali
    'sociologia', 'società', 'società', 'sociale', 'socialità', 'cultura', 'culture',
    'identità', 'identità', 'norma', 'norme', 'valore', 'valori',
    'istituzione', 'istituzioni', 'socializzazione', 'resocializzazione', 'interazione',
    'dinamica di gruppo', 'dinamiche di gruppo', 'stato', 'stati', 'ruolo', 'ruoli',

    # Sottocampi
    'sociologia urbana', 'sociologia rurale', 'sociologia medica', 'sociologia dell\'educazione',
    'sociologia della religione', 'sociologia economica', 'sociologia politica',
    'criminologia', 'studi di genere', 'studi migratori', 'sociologia della famiglia',
    'sociologia ambientale', 'sociologia organizzativa', 'sociologia culturale',

    # Teorie e paradigmi
    'funzionalismo', 'teoria del conflitto', 'interazionismo simbolico', 'strutturalismo',
    'post-strutturalismo', 'teoria critica', 'costruttivismo sociale', 'costruttivismo sociale',
    'teoria femminista', 'teoria queer', 'teoria della scelta razionale', 'teoria dei sistemi-mondo',

    # Problemi sociali
    'disuguaglianza', 'disuguaglianze', 'razzismo', 'sessismo', 'ageismo', 'abilismo',
    'discriminazione', 'povertà', 'emarginazione', 'disoccupazione', 'sottoccupazione',
    'senza tetto', 'criminalità', 'devianza', 'comportamento deviante', 'violenza', 'dipendenza',
    'inequità', 'pregiudizio', 'oppressione', 'vittimizzazione', 'stigmatizzazione',

    # Metodologie
    'qualitativo', 'quantitativo', 'etnografia', 'etnografie', 'sondaggio', 'sondaggi',
    'intervista', 'interviste', 'studio di caso', 'studi di caso', 'osservazione partecipante',
    'osservazione', 'lavoro sul campo', 'studio longitudinale', 'studio trasversale',
    'metodi misti', 'analisi del contenuto', 'analisi del discorso',

    # Strutture sociali e sistemi
    'classe', 'classi', 'genere', 'razza', 'etnia', 'religione', 'religioni',
    'famiglia', 'famiglie', 'educazione', 'burocrazia', 'burocrazia', 'governo',
    'governi', 'economia', 'media', 'tecnologia', 'tecnologie', 'sanità',
    'istituzione sociale', 'struttura sociale', 'sistema strutturale', 'ordine normativo',

    # Temi globali e attuali
    'globalizzazione', 'migrazione', 'immigrazione', 'emigrazione', 'diaspora',
    'social media', 'società digitale', 'comunità online', 'giustizia ambientale',
    'giustizia climatica', 'attivismo', 'protesta', 'cambiamento sociale', 'partecipazione civica',
    'intersezionalità', 'politiche identitarie', 'sociologia pubblica',

    # Termini notabili
    'capitale sociale', 'mobilità sociale', 'mobilità ascendente', 'mobilità discendente',
    'stratificazione sociale', 'agenzia', 'struttura', 'strutturazione', 'potere',
    'egemonia', 'ideologia', 'ideologie', 'alienazione', 'anomalia', 'modernità',
    'post-modernità', 'razionalizzazione', 'secolarizzazione', 'comportamento collettivo',

    # Pensatori influenti
    'marx', 'karl marx', 'durkheim', 'emile durkheim', 'weber', 'max weber',
    'goffman', 'erving goffman', 'bourdieu', 'pierre bourdieu',
    'foucault', 'michel foucault', 'giddens', 'anthony giddens',
    'habermas', 'juergen habermas', 'comte', 'auguste comte', 'mills', 'wright mills',

    # Concepts de base
    'sociologie', 'société', 'sociétés', 'social', 'socialité', 'culture', 'cultures',
    'identité', 'identités', 'norme', 'normes', 'valeur', 'valeurs',
    'institution', 'institutions', 'socialisation', 'resocialisation', 'interaction',
    'dynamique de groupe', 'dynamiques de groupe', 'statut', 'statuts', 'rôle', 'rôles',

    # Sous-champs
    'sociologie urbaine', 'sociologie rurale', 'sociologie médicale', 'sociologie de l\'éducation',
    'sociologie de la religion', 'sociologie économique', 'sociologie politique',
    'criminologie', 'études de genre', 'études migratoires', 'sociologie de la famille',
    'sociologie environnementale', 'sociologie organisationnelle', 'sociologie culturelle',

    # Théories et paradigmes
    'fonctionnalisme', 'théorie du conflit', 'interactionnisme symbolique', 'structuralisme',
    'post-structuralisme', 'théorie critique', 'constructivisme social', 'constructionnisme social',
    'théorie féministe', 'théorie queer', 'théorie du choix rationnel', 'théorie des systèmes-monde',

    # Problèmes sociaux
    'inégalité', 'inégalités', 'racisme', 'sexisme', 'âgisme', 'capacitisme',
    'discrimination', 'pauvreté', 'marginalisation', 'chômage', 'sous-emploi',
    'sans-abrisme', 'criminalité', 'déviance', 'comportement déviant', 'violence', 'dépendance',
    'inequité', 'préjugé', 'oppression', 'victimisation', 'stigmatisation',

    # Méthodologies
    'qualitatif', 'quantitatif', 'ethnographie', 'ethnographies', 'enquête', 'enquêtes',
    'entretien', 'entretiens', 'étude de cas', 'études de cas', 'observation participante',
    'observation', 'travail de terrain', 'étude longitudinale', 'étude transversale',
    'méthodes mixtes', 'analyse de contenu', 'analyse du discours',

    # Structures sociales et systèmes
    'classe', 'classes', 'genre', 'race', 'ethnie', 'religion', 'religions',
    'famille', 'familles', 'éducation', 'bureaucratie', 'bureaucraties', 'gouvernement',
    'gouvernements', 'économie', 'médias', 'technologie', 'technologies', 'soins de santé',
    'institution sociale', 'structure sociale', 'système structurel', 'ordre normatif',

    # Thèmes mondiaux et actuels
    'mondialisation', 'migration', 'immigration', 'émigration', 'diaspora',
    'réseaux sociaux', 'société numérique', 'communautés en ligne', 'justice environnementale',
    'justice climatique', 'activisme', 'protestation', 'changement social', 'engagement civique',
    'intersectionnalité', 'politique identitaire', 'sociologie publique',

    # Termes notables
    'capital social', 'mobilité sociale', 'mobilité ascendante', 'mobilité descendante',
    'stratification sociale', 'agence', 'structure', 'structuration', 'pouvoir',
    'hégémonie', 'idéologie', 'idéologies', 'aliénation', 'anomie', 'modernité',
    'postmodernité', 'rationalisation', 'sécularisation', 'comportement collectif',

    # Penseurs influents
    'marx', 'karl marx', 'durkheim', 'émile durkheim', 'weber', 'max weber',
    'goffman', 'erving goffman', 'bourdieu', 'pierre bourdieu',
    'foucault', 'michel foucault', 'giddens', 'anthony giddens',
    'habermas', 'jürgen habermas', 'comte', 'auguste comte', 'mills', 'wright mills',

    # 核心概念
    '社会学', '社会', '社会群体', '社会的', '社会性', '文化', '文化群体',
    '身份', '身份群体', '规范', '规范群体', '价值', '价值观',
    '制度', '社会制度', '社会化', '再社会化', '互动',
    '群体动力学', '群体动力学理论', '地位', '地位群体', '角色', '角色群体',

    # 分支领域
    '城市社会学', '农村社会学', '医学社会学', '教育社会学',
    '宗教社会学', '经济社会学', '政治社会学',
    '犯罪学', '性别研究', '迁移研究', '家庭社会学',
    '环境社会学', '组织社会学', '文化社会学',

    # 理论与范式
    '功能主义', '冲突理论', '符号互动主义', '结构主义',
    '后结构主义', '批判理论', '社会建构主义', '社会构造主义',
    '女性主义理论', '酷儿理论', '理性选择理论', '世界体系理论',

    # 社会问题
    '不平等', '不平等现象', '种族主义', '性别歧视', '年龄歧视', '能力歧视',
    '歧视', '贫困', '边缘化', '失业', '过度就业',
    '无家可归', '犯罪', '越轨', '越轨行为', '暴力', '成瘾',
    '不公', '偏见', '压迫', '受害', '污名化',

    # 方法论
    '定性', '定量', '民族志', '民族志研究', '调查', '调查研究',
    '访谈', '访谈研究', '案例研究', '案例分析', '参与观察',
    '观察', '田野调查', '纵向研究', '横向研究',
    '混合方法', '内容分析', '话语分析',

    # 社会结构与系统
    '阶级', '阶层', '性别', '种族', '民族', '宗教', '宗教群体',
    '家庭', '家庭群体', '教育', '官僚制度', '官僚群体', '政府',
    '政府群体', '经济', '媒体', '技术', '科技', '医疗保健',
    '社会制度', '社会结构', '结构系统', '规范秩序',

    # 全球与当前主题
    '全球化', '迁移', '移民', '外移', '侨民',
    '社交媒体', '数字社会', '在线社区', '环境正义',
    '气候正义', '激进主义', '抗议', '社会变革', '公民参与',
    '交集性', '身份政治', '公共社会学',

    # 重要术语
    '社会资本', '社会流动性', '向上流动性', '向下流动性',
    '社会分层', '代理', '结构', '结构化', '权力',
    '霸权', '意识形态', '意识形态群体', '疏离', '无序', '现代性',
    '后现代性', '理性化', '世俗化', '集体行为',

    # 影响力思想家
    '马克思', '卡尔·马克思', '杜尔凯姆', '埃米尔·杜尔凯姆', '韦伯', '马克斯·韦伯',
    '戈夫曼', '厄文·戈夫曼', '布尔迪厄', '皮埃尔·布尔迪厄',
    '福柯', '米歇尔·福柯', '吉登斯', '安东尼·吉登斯',
    '哈贝马斯', '尤尔根·哈贝马斯', '孔德', '奥古斯特·孔德', '米尔斯', '赖特·米尔斯',

    # Основные концепции
    'социология', 'общество', 'общества', 'социальный', 'социальность', 'культура', 'культуры',
    'идентичность', 'идентичности', 'норма', 'нормы', 'ценность', 'ценности',
    'институт', 'институты', 'социализация', 'ресоциализация', 'взаимодействие',
    'групповая динамика', 'групповая динамика', 'статус', 'статусы', 'роль', 'роли',

    # Подразделы
    'городская социология', 'сельская социология', 'медицинская социология', 'социология образования',
    'социология религии', 'экономическая социология', 'политическая социология',
    'криминология', 'гендерные исследования', 'миграционные исследования', 'социология семьи',
    'экологическая социология', 'организационная социология', 'культурная социология',

    # Теории и парадигмы
    'функционализм', 'теория конфликта', 'символический интеракционизм', 'структурализм',
    'постструктурализм', 'критическая теория', 'социальный конструктивизм', 'социальный конструкционизм',
    'феминистская теория', 'квир-тегория', 'теория рационального выбора', 'теория мировых систем',

    # Социальные проблемы
    'неравенство', 'неравенства', 'расизм', 'сексизм', 'эйджизм', 'абилитизм',
    'дискриминация', 'бедность', 'маргинализация', 'безработица', 'недоиспользование труда',
    'бездомность', 'преступление', 'девиация', 'девиантное поведение', 'насилие', 'зависимость',
    'неравенство', 'предвзятость', 'угнетение', 'виктимизация', 'стигматизация',

    # Методологии
    'качественные', 'количественные', 'этнография', 'этнографии', 'опрос', 'опросы',
    'интервью', 'интервью', 'кейс-стадии', 'кейс-стадии', 'участие в наблюдениях',
    'наблюдение', 'полевые работы', 'долгосрочные исследования', 'кросс-секционные исследования',
    'смешанные методы', 'анализ содержания', 'анализ дискурса',

    # Социальные структуры и системы
    'класс', 'классы', 'пол', 'раса', 'этническая группа', 'религия', 'религии',
    'семья', 'семьи', 'образование', 'бюрократия', 'бюрократии', 'правительство',
    'правительства', 'экономика', 'медиа', 'технология', 'технологии', 'здравоохранение',
    'социальный институт', 'социальная структура', 'структурная система', 'нормативный порядок',

    # Глобальные и текущие темы
    'глобализация', 'миграция', 'иммиграция', 'эмиграция', 'диаспора',
    'социальные сети', 'цифровое общество', 'онлайн-сообщества', 'экологическая справедливость',
    'климатическая справедливость', 'активизм', 'протест', 'социальные изменения', 'гражданское участие',
    'пересечение идентичности', 'политика идентичности', 'публичная социология',

    # Значимые термины
    'социальный капитал', 'социальная мобильность', 'восходящая мобильность', 'нисходящая мобильность',
    'социальная стратификация', 'агентство', 'структура', 'структурирование', 'власть',
    'гегемония', 'идеология', 'идеологии', 'ализация', 'аномия', 'современность',
    'постмодернизм', 'рационализация', 'секуляризация', 'коллективное поведение',

    # Влиятельные мыслители
    'маркс', 'карл маркс', 'дуркгейм', 'эмиль дуркгейм', 'вебер', 'макс вебер',
    'гоффман', 'эрвинг гоффман', 'бурдье', 'пьер бурдье',
    'фуко', 'мишель фуко', 'гидденс', 'энтони гидденс',
    'хабермас', 'юрген хабермас', 'комт', 'огюст комт', 'миллс', 'райт миллс',

    # 基本概念
    '社会学', '社会', '社会群体', '社会的', '社会性', '文化', '文化群体',
    'アイデンティティ', 'アイデンティティ群体', '規範', '規範群体', '価値', '価値観',
    '制度', '社会制度', '社会化', '再社会化', '相互作用',
    'グループダイナミクス', 'グループダイナミクス理論', '地位', '地位群体', '役割', '役割群体',

    # 分野
    '都市社会学', '農村社会学', '医学社会学', '教育社会学',
    '宗教社会学', '経済社会学', '政治社会学',
    '犯罪学', 'ジェンダー研究', '移民研究', '家族社会学',
    '環境社会学', '組織社会学', '文化社会学',

    # 理論とパラダイム
    '機能主義', '対立理論', '記号的相互作用論', '構造主義',
    'ポスト構造主義', '批判理論', '社会構築主義', '社会構造主義',
    'フェミニスト理論', 'クィア理論', '合理的選択理論', '世界システム理論',

    # 社会問題
    '不平等', '不平等現象', '人種差別', '性差別', '年齢差別', '能力差別',
    '差別', '貧困', '周縁化', '失業', '過剰就業',
    'ホームレス', '犯罪', '逸脱', '逸脱行動', '暴力', '依存',
    '不公正', '偏見', '抑圧', '被害者化', 'スティグマ化',

    # 方法論
    '定性的', '定量的', '民族誌', '民族誌研究', '調査', '調査研究',
    'インタビュー', 'インタビュー研究', 'ケーススタディ', 'ケーススタディ研究', '参加型観察',
    '観察', 'フィールドワーク', '縦断的研究', '横断的研究',
    '混合方法', '内容分析', '談話分析',

    # 社会構造とシステム
    '階級', '階層', '性別', '人種', '民族', '宗教', '宗教群体',
    '家族', '家族群体', '教育', '官僚制', '官僚群体', '政府',
    '政府群体', '経済', 'メディア', '技術', 'テクノロジー', '医療保険',
    '社会制度', '社会構造', '構造システム', '規範秩序',

    # グローバルおよび現代のテーマ
    'グローバル化', '移住', '移民', '出国', 'ディアスポラ',
    'ソーシャルメディア', 'デジタル社会', 'オンラインコミュニティ', '環境的正義',
    '気候的正義', '活動家', '抗議', '社会的変化', '市民参加',
    '交差性', 'アイデンティティ政治', '公共社会学',

    # 重要な用語
    '社会的資本', '社会的流動性', '上昇流動性', '下降流動性',
    '社会的階層化', 'エージェンシー', '構造', '構造化', '権力',
    'ヘゲモニー', 'イデオロギー', 'イデオロギー群体', '疎外', 'アノミー', '現代性',
    'ポスト現代性', '合理化', '世俗化', '集団行動',

    # 影響力のある思想家
    'マルクス', 'カール・マルクス', 'デュルケーム', 'エミール・デュルケーム', 'ウェーバー', 'マックス・ウェーバー',
    'ゴフマン', 'アーヴィング・ゴフマン', 'ブールディウ', 'ピエール・ブールディウ',
    'フーコー', 'ミシェル・フーコー', 'ギデンズ', 'アンソニー・ギデンズ',
    'ハーバーマス', 'ユルゲン・ハーバーマス', 'コント', 'オーギュスト・コント', 'ミルズ', 'ライト・ミルズ',

    # 핵심 개념
    '사회학', '사회', '사회 집단', '사회적', '사회성', '문화', '문화 집단',
    '정체성', '정체성 집단', '규범', '규범 집단', '가치', '가치관',
    '제도', '사회 제도', '사회화', '재사회화', '상호작용',
    '집단 역학', '집단 역학 이론', '지위', '지위 집단', '역할', '역할 집단',

    # 세부 분야
    '도시 사회학', '농촌 사회학', '의학 사회학', '교육 사회학',
    '종교 사회학', '경제 사회학', '정치 사회학',
    '범죄학', '성별 연구', '이주 연구', '가족 사회학',
    '환경 사회학', '조직 사회학', '문화 사회학',

    # 이론 및 패러다임
    '기능주의', '갈등 이론', '상징적 상호작용주의', '구조주의',
    '후구조주의', '비판 이론', '사회 구성주의', '사회 건설주의',
    '페미니스트 이론', '퀴어 이론', '합리적 선택 이론', '세계 시스템 이론',

    # 사회 문제
    '불평등', '불평등 현상', '인종 차별', '성 차별', '연령 차별', '능력 차별',
    '차별', '빈곤', '소외', '실업', '과잉 고용',
    '노숙자', '범죄', '일탈', '일탈 행동', '폭력', '중독',
    '불공정', '편견', '억압', '피해자화', '낙인 찍기',

    # 방법론
    '질적', '양적', '민족지', '민족지 연구', '조사', '조사 연구',
    '인터뷰', '인터뷰 연구', '사례 연구', '사례 연구 연구', '참여 관찰',
    '관찰', '현장 연구', '종단 연구', '횡단 연구',
    '혼합 방법', '내용 분석', '담론 분석',

    # 사회 구조 및 시스템
    '계급', '계층', '성별', '인종', '민족', '종교', '종교 집단',
    '가족', '가족 집단', '교육', '관료제', '관료 집단', '정부',
    '정부 집단', '경제', '미디어', '기술', '기술들', '보건',
    '사회 제도', '사회 구조', '구조적 시스템', '규범 질서',

    # 글로벌 및 현재 주제
    '세계화', '이주', '이민', '출국', '디아스포라',
    '소셜 미디어', '디지털 사회', '온라인 커뮤니티', '환경 정의',
    '기후 정의', '활동주의', '항의', '사회적 변화', '시민 참여',
    '교차성', '정체성 정치', '공공 사회학',

    # 주요 용어
    '사회적 자본', '사회적 이동성', '상승 이동성', '하강 이동성',
    '사회적 계층화', '대리인', '구조', '구조화', '권력',
    '헤게모니', '이데올로기', '이데올로기들', '소외', '무질서', '현대성',
    '포스트 현대성', '합리화', '세속화', '집단 행동',

    # 영향력 있는 사상가들
    '마르크스', '칼 마르크스', '뒤르케임', '에밀 뒤르케임', '베버', '막스 베버',
    '고프만', '어빙 고프만', '부르디외', '피에르 부르디외',
    '푸코', '미셸 푸코', '기든스', '앤서니 기든스',
    '하버마스', '위르겐 하버마스', '콩트', '오귀스트 콩트', '밀스', '라이트 밀스',

    # 核心概念
    '社會學', '社會', '社會群體', '社會的', '社會性', '文化', '文化群體',
    '身份', '身份群體', '規範', '規範群體', '價值', '價值觀',
    '制度', '社會制度', '社會化', '再社會化', '互動',
    '群體動態', '群體動態理論', '地位', '地位群體', '角色', '角色群體',

    # 分支領域
    '都市社會學', '農村社會學', '醫學社會學', '教育社會學',
    '宗教社會學', '經濟社會學', '政治社會學',
    '犯罪學', '性別研究', '移民研究', '家庭社會學',
    '環境社會學', '組織社會學', '文化社會學',

    # 理論與範式
    '功能主義', '衝突理論', '符號互動主義', '結構主義',
    '後結構主義', '批判理論', '社會建構主義', '社會結構主義',
    '女性主義理論', '酷兒理論', '理性選擇理論', '世界系統理論',

    # 社會問題
    '不平等', '不平等現象', '種族主義', '性別歧視', '年齡歧視', '能力歧視',
    '歧視', '貧困', '邊緣化', '失業', '過度就業',
    '無家可歸', '犯罪', '越軌', '越軌行為', '暴力', '成癮',
    '不公', '偏見', '壓迫', '受害者化', '污名化',

    # 方法論
    '定性', '定量', '民族誌', '民族誌研究', '調查', '調查研究',
    '訪談', '訪談研究', '案例研究', '案例研究研究', '參與觀察',
    '觀察', '田野研究', '縱向研究', '橫向研究',
    '混合方法', '內容分析', '話語分析',

    # 社會結構與系統
    '階級', '階層', '性別', '種族', '民族', '宗教', '宗教群體',
    '家庭', '家庭群體', '教育', '官僚制', '官僚群體', '政府',
    '政府群體', '經濟', '媒體', '技術', '科技', '醫療保健',
    '社會制度', '社會結構', '結構系統', '規範秩序',

    # 全球與當前主題
    '全球化', '遷移', '移民', '外移', '侨民',
    '社交媒體', '數字社會', '線上社群', '環境正義',
    '氣候正義', '激進主義', '抗議', '社會變革', '公民參與',
    '交集性', '身份政治', '公共社會學',

    # 重要術語
    '社會資本', '社會流動性', '上升流動性', '下降流動性',
    '社會分層', '代理人', '結構', '結構化', '權力',
    '霸權', '意識形態', '意識形態群體', '疏離', '無序', '現代性',
    '後現代性', '理性化', '世俗化', '集體行為',

    # 影響力的思想家
    '馬克思', '卡爾·馬克思', '杜爾克海姆', '埃米爾·杜爾克海姆', '韋伯', '馬克斯·韋伯',
    '高夫曼', '厄文·高夫曼', '布爾迪厄', '皮埃爾·布爾迪厄',
    '福柯', '米歇爾·福柯', '吉登斯', '安東尼·吉登斯',
    '哈貝馬斯', '尤爾根·哈貝馬斯', '孔德', '奧古斯特·孔德', '米爾斯', '賴特·米爾斯',

    # Základní koncepty
    'sociologie', 'společnost', 'společnosti', 'sociální', 'sociálnost', 'kultura', 'kultury',
    'identita', 'identity', 'norma', 'normy', 'hodnota', 'hodnoty',
    'instituce', 'instituce', 'socializace', 'resocializace', 'interakce',
    'skupinová dynamika', 'dynamika skupin', 'status', 'statusy', 'role', 'role',

    # Podobory
    'městská sociologie', 'venkovská sociologie', 'lékařská sociologie', 'sociologie vzdělání',
    'sociologie náboženství', 'ekonomická sociologie', 'politická sociologie',
    'kriminologie', 'genderová studia', 'studie migrace', 'sociologie rodiny',
    'environmentální sociologie', 'organizační sociologie', 'kulturní sociologie',

    # Teorie a paradigmata
    'funkcionalismus', 'teorie konfliktu', 'symbolický interakcionismus', 'strukturálníismus',
    'poststrukturálníismus', 'kritická teorie', 'sociální konstruktivismus', 'sociální konstrukcionismus',
    'feministická teorie', 'queer teorie', 'teorie racionálního výběru', 'teorie světového systému',

    # Sociální problémy
    'nerovnost', 'nerovnosti', 'rasismus', 'sexismus', 'věkový diskriminace', 'ableismus',
    'diskriminace', 'chudoba', 'marginalizace', 'nezaměstnanost', 'podzaměstnanost',
    'bezdomovectví', 'kriminalita', 'odchylka', 'odchylkové chování', 'násilí', 'závislost',
    'nefér', 'předpojatost', 'útisk', 'victimizace', 'stigmatizace',

    # Metodologie
    'kvalitativní', 'kvantitativní', 'etnografie', 'etnografické studie', 'anketa', 'průzkumy',
    'rozhovor', 'rozhovory', 'případová studie', 'případové studie', 'účastnické pozorování',
    'pozorování', 'terénní výzkum', 'longitudinální studie', 'průřezová studie',
    'smíšené metody', 'analýza obsahu', 'analýza diskurzu',

    # Sociální struktury a systémy
    'třída', 'třídy', 'pohlaví', 'rasu', 'etnicitu', 'náboženství', 'náboženství',
    'rodina', 'rodiny', 'vzdělání', 'byrokracie', 'byrokracie', 'vláda',
    'vlády', 'ekonomika', 'media', 'technologie', 'technologie', 'zdravotní péče',
    'sociální instituce', 'sociální struktura', 'strukturální systém', 'normativní řád',

    # Globální a současná témata
    'globalizace', 'migrace', 'imigrace', 'emigrace', 'diaspora',
    'sociální média', 'digitální společnost', 'online komunity', 'environmentální spravedlnost',
    'klimatická spravedlnost', 'aktivismus', 'protest', 'sociální změna', 'občanský engagement',
    'intersekcionalita', 'politika identity', 'veřejná sociologie',

    # Významné termíny
    'sociální kapitál', 'sociální mobilita', 'vstupní mobilita', 'sestupná mobilita',
    'sociální stratifikace', 'agentura', 'struktura', 'strukturace', 'moc',
    'hegemonie', 'ideologie', 'ideologie', 'odcizení', 'anomie', 'modernita',
    'postmodernita', 'racionalizace', 'sekularizace', 'kolektivní chování',

    # Vlivní myslitelé
    'Marx', 'Karl Marx', 'Durkheim', 'Émile Durkheim', 'Weber', 'Max Weber',
    'Goffman', 'Erving Goffman', 'Bourdieu', 'Pierre Bourdieu',
    'Foucault', 'Michel Foucault', 'Giddens', 'Anthony Giddens',
    'Habermas', 'Jürgen Habermas', 'Comte', 'Auguste Comte', 'Mills', 'Wright Mills',

    # المفاهيم الأساسية
    'علم الاجتماع', 'المجتمع', 'المجتمعات', 'اجتماعي', 'الاجتماعية', 'ثقافة', 'ثقافات',
    'هوية', 'هويات', 'قيد', 'قيود', 'قيمة', 'قيم',
    'مؤسسة', 'مؤسسات', 'التنشئة الاجتماعية', 'إعادة التنشئة الاجتماعية', 'تفاعل',
    'ديناميكية المجموعة', 'ديناميكيات المجموعة', 'وضع', 'أوضاع', 'دور', 'أدوار',

    # الفروع
    'علم الاجتماع الحضري', 'علم الاجتماع الريفي', 'علم الاجتماع الطبي', 'علم الاجتماع التربوي',
    'علم اجتماع الدين', 'علم الاجتماع الاقتصادي', 'علم الاجتماع السياسي',
    'علم الجريمة', 'دراسات النوع الاجتماعي', 'دراسات الهجرة', 'علم اجتماع الأسرة',
    'علم الاجتماع البيئي', 'علم الاجتماع التنظيمي', 'علم الاجتماع الثقافي',

    # النظريات والنماذج
    'الوظيفية', 'نظرية الصراع', 'التفاعل الرمزي', 'البنيوية',
    'ما بعد البنيوية', 'النظرية النقدية', 'البنائية الاجتماعية', 'التكوين الاجتماعي',
    'نظرية النسوية', 'نظرية الكوير', 'نظرية الاختيار العقلاني', 'نظرية الأنظمة العالمية',

    # القضايا الاجتماعية
    'اللامساواة', 'اللامساواة', 'التمييز العنصري', 'التمييز الجنسي', 'التمييز حسب العمر', 'التمييز ضد المعاقين',
    'التمييز', 'الفقر', 'التهميش', 'البطالة', 'تحت البطالة',
    'التشرد', 'الجريمة', 'الانحراف', 'السلوك المنحرف', 'العنف', 'الإدمان',
    'عدم الإنصاف', 'التحيز', 'الاضطهاد', 'الضحية', 'الوصمة',

    # المنهجيات
    'نوعي', 'كمّي', 'الإثنوغرافيا', 'الإثنوغرافيات', 'استبيان', 'استطلاعات',
    'مقابلة', 'مقابلات', 'دراسة حالة', 'دراسات حالة', 'ملاحظة مشاركة',
    'ملاحظة', 'بحث ميداني', 'دراسة طولية', 'دراسة مقطعية',
    'الطرق المختلطة', 'تحليل المحتوى', 'تحليل الخطاب',

    # الهياكل الاجتماعية والأنظمة
    'الطبقة', 'الطبقات', 'الجنس', 'العرق', 'العرقية', 'الدين', 'الأديان',
    'الأسرة', 'الأسر', 'التعليم', 'البيروقراطية', 'البيروقراطيات', 'الحكومة',
    'الحكومات', 'الاقتصاد', 'الإعلام', 'التكنولوجيا', 'التقنيات', 'الرعاية الصحية',
    'المؤسسة الاجتماعية', 'الهيكل الاجتماعي', 'النظام الهيكلي', 'الترتيب النمطي',

    # المواضيع العالمية والحالية
    'العولمة', 'الهجرة', 'ال immigration', 'الانتقال', 'الشتات',
    'وسائل التواصل الاجتماعي', 'المجتمع الرقمي', 'المجتمعات عبر الإنترنت', 'العدالة البيئية',
    'العدالة المناخية', 'الناشطية', 'الاحتجاج', 'التغيير الاجتماعي', 'المشاركة المدنية',
    'التقاطع', 'سياسة الهوية', 'علم الاجتماع العام',

    # المصطلحات البارزة
    'رأس المال الاجتماعي', 'الحركة الاجتماعية', 'الحركة الاجتماعية الصاعدة', 'الحركة الاجتماعية الهابطة',
    'الطبقية الاجتماعية', 'الوكالة', 'الهيكل', 'الهيكلة', 'القوة',
    'الهيمنة', 'الإيديولوجيا', 'الإيديولوجيات', 'الاغتراب', 'الأنومي', 'الحداثة',
    'ما بعد الحداثة', 'التعقل', 'العلمانية', 'السلوك الجماعي',

    # المفكرون المؤثرون
    'ماركس', 'كارل ماركس', 'دوركهايم', 'إميل دوركهايم', 'ويبر', 'ماكس ويبر',
    'غوفمان', 'إرفينغ غوفمان', 'بورديو', 'بيير بورديو',
    'فوكو', 'ميشيل فوكو', 'كيدينز', 'أنتوني كيدينز',
    'هابرماس', 'يورغن هابرماس', 'كونت', 'أوغست كونت', 'ميلز', 'رايت ميلز'
]

}


In [11]:
!pip install tqdm


In [14]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
import re

# === Load dataset ===
df = pd.read_csv(r"C:\Users\ICSA\eleni\ORCID\pythonProject11\data\merged_affil_journals.csv")

# === Initialize tqdm progress bar for pandas apply ===
tqdm.pandas(desc="Scoring paper titles")

# === Preprocess domain keywords to lowercase sets ===
domain_keywords_lower = {
    domain: set(map(str.lower, keywords))
    for domain, keywords in domain_keywords.items()
}

# === Define keyword-based scoring function ===
def fast_score(text):
    words = set(re.findall(r'\w+', str(text).lower()))
    return {
        domain: len(words & keywords)
        for domain, keywords in domain_keywords_lower.items()
    }

# === Apply scoring function to each title ===
df["domain_scores"] = df["Jtitle"].progress_apply(fast_score)

# === Extract top-3 scoring domains (with non-zero score) ===
def extract_top_n(score_dict, n=3):
    sorted_scores = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
    top_items = [item for item in sorted_scores if item[1] > 0][:n]
    while len(top_items) < n:
        top_items.append((None, None))
    return top_items

# Apply to extract domain names and scores
top3 = df["domain_scores"].apply(lambda d: extract_top_n(d, n=3))
df["domain_1"] = top3.apply(lambda x: x[0][0])
df["score_1"] = top3.apply(lambda x: x[0][1])
df["domain_2"] = top3.apply(lambda x: x[1][0])
df["score_2"] = top3.apply(lambda x: x[1][1])
df["domain_3"] = top3.apply(lambda x: x[2][0])
df["score_3"] = top3.apply(lambda x: x[2][1])

# === Select only the required columns ===
output_cols = [
    "OID", "Fname", "Gname", "locale", "Role", "Org",
    "Jtitle", "Jdate", "domain_1", "score_1", "domain_2", "score_2", "domain_3", "score_3"
]

# === Save to CSV ===
df[output_cols].to_csv("paper_top3_domains_clean.csv", index=False)

print(" Saved top-3 domain classification per paper to 'paper_top3_domains_clean.csv'")


Scoring paper titles: 100%|██████████| 16162862/16162862 [01:34<00:00, 171311.18it/s]


 Saved top-3 domain classification per paper to 'paper_top3_domains_clean.csv'


In [21]:
import pandas as pd

# Load the processed dataset
df = pd.read_csv("paper_top3_domains_clean.csv")

# Display 5 random rows
print(df.sample(20))


                          OID                     Fname          Gname locale  \
7098214   0000-0002-4355-8114                     Patel          Kiran     en   
5161828   0000-0001-7205-8584                Kabaklarlı           Esra     en   
7459555   0000-0002-9689-543X                Papamarkou       Theodore     en   
7661499   0000-0001-5403-0035          Mc Sween-Cadieux         Esther     en   
14800292  0000-0003-2596-6461                    angius         andrea     en   
911325    0000-0003-2836-2962                       Ali         Shahid     en   
11076393  0000-0001-5148-7689                     Foley          Kevin     en   
12238591  0000-0001-8977-6144                  Sharwood           Lisa     en   
3165021   0000-0003-1293-8613                    Nguyen         Loi T.     en   
15911968  0000-0002-7212-8367  Caçador Martins-Ferreira  Marco Antonio     en   
4310054   0000-0003-1007-8939                    Hassan      Dr. Warda     en   
10407762  0000-0002-5707-639

In [15]:
# Load the CSV
df = pd.read_csv(r"C:\Users\ICSA\eleni\ORCID\paper_top3_domains_clean.csv")

# Remove rows where Jtitle is NaN
df_cleaned = df.dropna(subset=["Jtitle"])

# Save to a new CSV file
df_cleaned.to_csv("paper_top3_domains_clean_no_missing_titles.csv", index=False)

print("Saved cleaned file to 'paper_top3_domains_clean_no_missing_titles.csv'")



Saved cleaned file to 'paper_top3_domains_clean_no_missing_titles.csv'


In [22]:
import pandas as pd

# Load the processed dataset
df = pd.read_csv("paper_top3_domains_clean_no_missing_titles.csv")

# Display 5 random rows
print(df.sample(20))

                          OID            Fname             Gname locale  \
1408778   0000-0002-7885-3201          Barakat             Assem     en   
5587179   0000-0003-0819-5933            Velez           Gabriel     en   
638710    0000-0001-5224-7046       Pennington           William     en   
1595010   0000-0001-6911-9243          Chaubey          Narendra     en   
3712802   0000-0002-3453-1825             Wong          Wing-tak     en   
9904298   0000-0002-5819-0754               Lu          ChengKai     en   
6398057   0000-0003-3852-4523       Del Gaudio          Pasquale     it   
7187843   0000-0003-4304-7653          Bradley         Alexander     en   
12570606  0000-0001-5856-2813           Dalmau             Josep     en   
11406849  0000-0003-4464-1195  VASQUEZ PALACIO  GONZALO DE JESUS     es   
7123413   0000-0002-5832-402X           Lawton           Rebecca     en   
3602117   0000-0003-4403-4790            Busto           Jesús H     es   
1405385   0000-0002-8769-

In [25]:
import pandas as pd

# Load your merged dataset
df = pd.read_csv("pythonProject11/data/merged_affil_journals.csv")

dept_to_category = {
    # Medicine
    "cardiology": "medicine",
    "center of medical genetics": "medicine",
    "gastroenterology": "medicine",
    "gynecology and obstetrics": "medicine",
    "public health sciences": "medicine",
    "patologia experimental": "medicine",
    "midwest orthopaedics at rush": "medicine",
    "outcomes research": "medicine",
    "medicine": "medicine",

    # Biology
    "biology": "biology",
    "biochemistry": "biology",
    "molecular biology and genetics": "biology",
    "genetics": "biology",
    "chemical and biological sciences": "biology",

    # Engineering
    "civil enginnering": "engineering",
    "computer enggineering": "engineering",
    "computer engineering": "engineering",
    "computer engineering department": "engineering",
    "electrical engineering": "engineering",
    "electronics and communication engineering": "engineering",
    "engineering and natural sciences": "engineering",
    "optoelectronic engineering and instrumentation science": "engineering",

    # Informatics
    "analysis and systems development": "informatics",
    "library science": "informatics",

    # Psychology
    "psychology": "psychology",

    # Environment
    "environmental sciences and policy": "environment",
    "porter environmental studies": "environment",
    "water, energy and environment": "environment",
    "sustainable development board": "environment",

    # Economics / Business
    "economics": "economics",
    "mathematics and economics": "economics",
    "majoring in business administration": "business",

    # Education
    "education": "education",
    "graduate program - master and doctorate in education": "education",
    "master's graduate program and ph.d.": "education",
    "specific teaching": "education",

    # Law
    "law institute": "law",
    "patents": "law",

    # Arts & Humanities
    "classics": "arts_humanities",
    "classics & classical association of wellington": "arts_humanities",
    "communication, film & popular culture": "arts_humanities",
    "cultural studies": "arts_humanities",
    "english": "arts_humanities",
    "letters": "arts_humanities",
    "pharmacy": "arts_humanities",  # borderline – can also be "medicine"
    "fassgrad conference": "arts_humanities",
    "visual": "arts_humanities",

    # History
    "history": "history",
    "history, classics & archaeology": "history",

    # Sociology
    "social sciences": "sociology",
    "social and communication sciences": "sociology",

    # General Science
    "science": "science",
    "statistics": "science",
    "earth sciences": "science",
    "geosciences": "science",
    "paleontology": "science",
    "applied chemistry research centre": "science",
    "mrc lmb": "science",
    "chemistry": "science",

    # Ambiguous / Could Map Later
    "atdc": None,
    "cities research institute": None,
    "lysholm neuroradiology": "medicine",  # assumed
    "university linking and technology transfer center": None,
}

# Normalize and apply mapping
df["clean_dept"] = df["clean_dept"].str.lower()
df["dept_domain"] = df["clean_dept"].map(dept_to_category)

# Select relevant fields and drop duplicates
output_dept = df[["OID", "clean_dept", "dept_domain"]].drop_duplicates()

# Save to CSV
output_dept.to_csv("researcher_department_domains.csv", index=False)


In [31]:
import pandas as pd

# Load both datasets
df_scores = pd.read_csv(r"C:\\Users\\ICSA\\eleni\\ORCID\\paper_top3_domains_clean.csv")
df_dept = pd.read_csv(r"C:\\Users\\ICSA\\eleni\\ORCID\\researcher_department_domains.csv")


# Merge on OID
df_merged = pd.merge(df_scores, df_dept, on="OID", how="left")


# Save for future steps
df_merged.to_csv("merged_researcher_domains.csv", index=False)


In [38]:
import pandas as pd

# Load merged dataset
df = pd.read_csv(r"C:\\Users\\ICSA\\eleni\\ORCID\\merged_researcher_domains.csv")

# Define weight parameters
weights = {
    "dept_domain": 0.5,
    "domain_1": 0.3,
    "domain_2": 0.15,
    "domain_3": 0.05,
}

# Fill NaNs with 'unknown' for safety
df[["dept_domain", "domain_1", "domain_2", "domain_3"]] = df[
    ["dept_domain", "domain_1", "domain_2", "domain_3"]
].fillna("unknown")

# Compute weighted domain affinity per paper
def compute_domain_affinity(row):
    affinity = {}
    for col, weight in weights.items():
        domain = row[col]
        if domain and domain != "unknown":
            affinity[domain] = affinity.get(domain, 0) + weight

    # Normalize
    total = sum(affinity.values())
    if total > 0:
        affinity = {k: round(v / total, 6) for k, v in affinity.items()}
    return affinity

# Apply to each row
df["domain_affinity"] = df.apply(compute_domain_affinity, axis=1)

# Save cleaned affinity per paper with extra metadata
cols_to_keep = [
    "OID", "Fname", "Gname", "locale", "Role", "Org", 
    "Jtitle", "Jdate", "domain_affinity"
]
df[cols_to_keep].to_csv("researcher_domain_affinity.csv", index=False)

print("Saved domain affinities with metadata to researcher_domain_affinity.csv")


Saved domain affinities with metadata to researcher_domain_affinity.csv


Encode target labels and generate TF-IDF features (Term Frequency – Inverse Document Frequency)- to convert text into numbers

In [17]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Encode target labels (department)
label_encoder = LabelEncoder()
grouped["dept_label"] = label_encoder.fit_transform(grouped["clean_dept"])

# TF-IDF vectorization of journal titles
vectorizer = TfidfVectorizer(max_features=3000, stop_words="english")
X = vectorizer.fit_transform(grouped["Jtitle"])
y = grouped["dept_label"]

# Display result shapes
print("TF-IDF feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)


TF-IDF feature matrix shape: (415315, 3000)
Target vector shape: (415315,)


415315: the number of researcher entries (one per row, probably per OID + journal title combination).

3000: the number of TF-IDF features, we've limited the vectorizer to the top 3000 most informative words (using max_features=3000).

The target vector has shape (415315,), one label (discipline, like clean_dept) per researcher-title pair.